## Librerias


In [1]:
import pandas as pd
import numpy as np      # Para np.nan, np.where, etc.
import re
import unicodedata
from datetime import datetime  # Solo si usas datetime.strptime o similar


In [2]:
df = pd.read_excel(
    "Matriz Viabilidad Juridica DAT-2.xlsx",
    sheet_name="VIABILIDAD JURÍDICA",
    header=1,
    dtype={"ID": "string"},
)

c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## ID

In [3]:
import pandas as pd
import numpy as np
import re
import unicodedata

# 1) Cargar archivo base
df = pd.read_excel(
    "Matriz Viabilidad Juridica DAT-2.xlsx",
    sheet_name="VIABILIDAD JURÍDICA",
    header=1,
    dtype={"ID": "string"},
)

# 2) Columnas base para revisión manual
df["ID_RAW"] = df["ID"].astype("string")
df["Fila_Excel"] = df.index + 2  # (se deja tal cual indicas)

# 3) Normalizar ID → ID_LIMPIO
df["ID_LIMPIO"] = (
    df["ID_RAW"]
      .astype("string")
      .str.replace("\u00A0", " ", regex=False)  # NBSP → espacio normal
      .str.replace(r"\s+", " ", regex=True)    # espacios múltiples → uno
      .str.strip()
)

# 4) Detectar IDs vacíos (después de limpiar)
mask_vacios = df["ID_LIMPIO"].isna() | (df["ID_LIMPIO"] == "")

# 5) Detectar duplicados (solo considerando IDs no vacíos)
mask_no_vacios = ~mask_vacios
mask_dups = mask_no_vacios & df["ID_LIMPIO"].duplicated(keep=False)

# 6) Columna final para migración: ID_SQL
#    - IDs válidos: se usa ID_LIMPIO
#    - IDs vacíos: "SIN REGISTRO 1", "SIN REGISTRO 2", ...
df["ID_SQL"] = df["ID_LIMPIO"].astype("string")

n_vacios = int(mask_vacios.sum())
if n_vacios > 0:
    secuencia = np.arange(1, n_vacios + 1).astype(str)
    df.loc[mask_vacios, "ID_SQL"] = "SIN REGISTRO " + secuencia

# (Opcional recomendado) Validar duplicados en el ID final (ID_SQL)
mask_dups_sql = df["ID_SQL"].duplicated(keep=False)

# 7) Reordenar columnas: dejar las nuevas junto a la original
cols = list(df.columns)
grupo = ["ID", "ID_RAW", "ID_LIMPIO", "ID_SQL", "Fila_Excel"]
grupo = [c for c in grupo if c in df.columns]
otras = [c for c in cols if c not in grupo]
df = df[grupo + otras]

# 8) Resumen general
print("IDs vacíos (después de limpiar):", n_vacios)
print("IDs duplicados (después de limpiar):", int(mask_dups.sum()))
print("IDs duplicados en ID_SQL (final):", int(mask_dups_sql.sum()))

filas_vacias_id = df.loc[mask_vacios, "Fila_Excel"].tolist()
filas_dups_id   = df.loc[mask_dups, "Fila_Excel"].tolist()
filas_dups_sql  = df.loc[mask_dups_sql, "Fila_Excel"].tolist()

print("Filas Excel con IDs vacíos:", filas_vacias_id)
print("Filas Excel con IDs duplicados (ID_LIMPIO):", filas_dups_id)
print("Filas Excel con IDs duplicados (ID_SQL):", filas_dups_sql)

# 9) DataFrames para exportar problemas
cols_export = ["Fila_Excel", "ID", "ID_RAW", "ID_LIMPIO", "ID_SQL"]

df_vacios = df.loc[mask_vacios, cols_export].copy()
df_dups   = df.loc[mask_dups, cols_export].copy()
df_dups_sql = df.loc[mask_dups_sql, cols_export].copy()

if not df_vacios.empty:
    df_vacios.to_excel("ID_con_problemas_vacios.xlsx", index=False)

if not df_dups.empty:
    df_dups.to_excel("ID_con_problemas_duplicados.xlsx", index=False)

# Si llegan a existir duplicados en el ID final, se deja evidencia aparte
if not df_dups_sql.empty:
    df_dups_sql.to_excel("ID_con_problemas_duplicados_ID_SQL.xlsx", index=False)

# 10) Excel con TODOS los registros (original + limpio + para migrar)
df[cols_export].to_excel("ID_original_limpio_y_SQL.xlsx", index=False)

print("\nArchivos creados:")
if not df_vacios.empty:
    print(" - ID_con_problemas_vacios.xlsx")
if not df_dups.empty:
    print(" - ID_con_problemas_duplicados.xlsx")
if not df_dups_sql.empty:
    print(" - ID_con_problemas_duplicados_ID_SQL.xlsx")
print(" - ID_original_limpio_y_SQL.xlsx")

# 11) Ejemplo de las primeras filas en consola
print("\n=== Ejemplo de ID original, limpio y para migrar ===")
print(df.loc[:, cols_export].head(20).to_string(index=False))

# 12) Ejemplos de vacíos
print("\n=== Ejemplos de IDs vacíos (después de limpiar) ===")
if not df_vacios.empty:
    print(df_vacios.head(20).to_string(index=False))
else:
    print("No hay IDs vacíos.")

# 13) Ejemplos de duplicados (ID_LIMPIO)
print("\n=== Ejemplos de IDs duplicados (después de limpiar) ===")
if not df_dups.empty:
    print(df_dups.head(20).to_string(index=False))
else:
    print("No hay IDs duplicados.")

# 14) Ejemplos de duplicados (ID_SQL final)
print("\n=== Ejemplos de IDs duplicados en ID_SQL (final) ===")
if not df_dups_sql.empty:
    print(df_dups_sql.head(20).to_string(index=False))
else:
    print("No hay IDs duplicados en ID_SQL.")


c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


IDs vacíos (después de limpiar): 22
IDs duplicados (después de limpiar): 0
IDs duplicados en ID_SQL (final): 0
Filas Excel con IDs vacíos: [13239, 20715, 20718, 21082, 23234, 24032, 24033, 24042, 24043, 24044, 24045, 24046, 24047, 24048, 24049, 24050, 24051, 24052, 24053, 24054, 24055, 24056]
Filas Excel con IDs duplicados (ID_LIMPIO): []
Filas Excel con IDs duplicados (ID_SQL): []

Archivos creados:
 - ID_con_problemas_vacios.xlsx
 - ID_original_limpio_y_SQL.xlsx

=== Ejemplo de ID original, limpio y para migrar ===
 Fila_Excel      ID  ID_RAW ID_LIMPIO  ID_SQL
          2 ID8099B ID8099B   ID8099B ID8099B
          3 ID8099C ID8099C   ID8099C ID8099C
          4 ID8099D ID8099D   ID8099D ID8099D
          5 ID8099E ID8099E   ID8099E ID8099E
          6 ID9080A ID9080A   ID9080A ID9080A
          7 ID8099F ID8099F   ID8099F ID8099F
          8 ID8099G ID8099G   ID8099G ID8099G
          9 ID10439 ID10439   ID10439 ID10439
         10  ID6287  ID6287    ID6287  ID6287
         11 ID809

## FMI

In [4]:
df["FMI_LIMPIO"] = (
    df["FMI"]
      .astype("string")
      .str.replace("\u00A0", " ", regex=False)
      .str.strip()
)

mask_vacios_fmi = df["FMI_LIMPIO"].isna() | (df["FMI_LIMPIO"] == "")

# Formato esperado: 3 dígitos, guion, 5 dígitos  -> 000-00000
pat_fmi = re.compile(r"^\d{3}-\d{5}$")

mask_formato_malo_fmi = ~mask_vacios_fmi & ~df["FMI_LIMPIO"].str.match(pat_fmi)
mask_dups_fmi = ~mask_vacios_fmi & df["FMI_LIMPIO"].duplicated(keep=False)

filas_fmi_vacios = (df.index[mask_vacios_fmi] + 2).tolist()
filas_fmi_formato_malo = (df.index[mask_formato_malo_fmi] + 2).tolist()
filas_fmi_dups = (df.index[mask_dups_fmi] + 2).tolist()

print("Filas Excel FMI vacíos:", filas_fmi_vacios)
print("Filas Excel FMI formato incorrecto:", filas_fmi_formato_malo)
print("Filas Excel FMI duplicados:", filas_fmi_dups)

Filas Excel FMI vacíos: [895, 2905, 3967, 3969, 4298, 6077, 7385, 20942, 20963, 20964, 21130, 21132, 21133, 21134, 21135, 21136, 21141, 21142, 21143, 21145, 21146, 21147, 21148, 21149, 21150, 21151, 21152, 21153, 21154, 21155]
Filas Excel FMI formato incorrecto: [9, 15, 18, 20, 27, 29, 33, 34, 36, 37, 38, 40, 42, 43, 45, 48, 49, 50, 53, 54, 56, 60, 64, 65, 69, 70, 71, 73, 79, 80, 81, 82, 101, 103, 104, 105, 106, 107, 108, 124, 128, 130, 134, 135, 143, 144, 145, 154, 155, 156, 157, 159, 160, 164, 165, 167, 168, 172, 175, 176, 177, 178, 181, 182, 183, 188, 190, 194, 198, 203, 208, 212, 213, 215, 216, 217, 225, 226, 227, 231, 234, 235, 236, 238, 240, 243, 245, 246, 248, 252, 256, 259, 261, 264, 265, 266, 268, 269, 271, 273, 274, 276, 282, 284, 285, 287, 290, 293, 295, 299, 302, 303, 307, 308, 310, 311, 312, 314, 321, 324, 327, 330, 331, 335, 336, 337, 338, 341, 342, 344, 346, 348, 350, 351, 353, 354, 357, 359, 360, 361, 368, 369, 371, 373, 374, 375, 376, 379, 383, 386, 388, 389, 393, 394,

In [5]:
import pandas as pd
import numpy as np
import re
import unicodedata

# Se asume que df ya está cargado con la matriz completa
# y que ya existe o no la columna Fila_Excel.

# 1) Asegurar columna auxiliar con la fila real de Excel
#    (encabezado de datos empieza en la fila 2)
df["Fila_Excel"] = df.index + 2

# 2) Columnas base para FMI (crudo y limpio)
df["FMI_RAW"] = df["FMI"].astype("string")

df["FMI_LIMPIO"] = (
    df["FMI_RAW"]
      .astype("string")
      .str.replace("\u00A0", " ", regex=False)   # NBSP → espacio normal
      .str.replace(r"\s+", " ", regex=True)     # espacios múltiples → uno
      .str.strip()
)

# 3) Columnas de estado / diagnóstico
mask_vacios_fmi = df["FMI_LIMPIO"].isna() | (df["FMI_LIMPIO"] == "")

# En FMI NO vamos a validar formato (no tiene patrón fijo),
# sólo limpiamos, marcamos vacíos y duplicados.

mask_no_vacios_fmi = ~mask_vacios_fmi
mask_dups_fmi = mask_no_vacios_fmi & df["FMI_LIMPIO"].duplicated(keep=False)

# 4) Valor final para migrar → FMI_SQL
df["FMI_SQL"] = df["FMI_LIMPIO"]
df.loc[mask_vacios_fmi, "FMI_SQL"] = "SIN REGISTRO"

# 5) Listas de filas problemáticas (para que las tengas claras)
filas_fmi_vacios = df.loc[mask_vacios_fmi, "Fila_Excel"].tolist()
filas_fmi_dups   = df.loc[mask_dups_fmi,   "Fila_Excel"].tolist()

print("FMI vacíos (después de limpiar):", int(mask_vacios_fmi.sum()))
print("FMI duplicados (después de limpiar):", int(mask_dups_fmi.sum()))
print("Filas Excel FMI vacíos:", filas_fmi_vacios)
print("Filas Excel FMI duplicados:", filas_fmi_dups)

# 6) DataFrames para exportar a Excel
cols_export = ["Fila_Excel", "FMI", "FMI_RAW", "FMI_LIMPIO", "FMI_SQL"]

df_vacios_fmi = df.loc[mask_vacios_fmi, cols_export].copy()
df_dups_fmi   = df.loc[mask_dups_fmi,   cols_export].copy()

# Excel solo con vacíos
if not df_vacios_fmi.empty:
    df_vacios_fmi.to_excel("FMI_con_problemas_vacios.xlsx", index=False)

# Excel solo con duplicados
if not df_dups_fmi.empty:
    df_dups_fmi.to_excel("FMI_con_problemas_duplicados.xlsx", index=False)

# Excel con TODOS los registros (crudo + limpio + valor final para migrar)
df[cols_export].to_excel("FMI_original_limpio_y_SQL.xlsx", index=False)

print("\nArchivos creados:")
if not df_vacios_fmi.empty:
    print(" - FMI_con_problemas_vacios.xlsx")
if not df_dups_fmi.empty:
    print(" - FMI_con_problemas_duplicados.xlsx")
print(" - FMI_original_limpio_y_SQL.xlsx")

# 7) Muestra en consola para revisión rápida
print("\n=== Ejemplo FMI original, limpio y para migrar ===")
print(
    df.loc[:, cols_export]
      .head(20)
      .to_string(index=False)
)

print("\n=== Ejemplos de FMI vacíos (después de limpiar) ===")
if not df_vacios_fmi.empty:
    print(df_vacios_fmi.head(20).to_string(index=False))
else:
    print("No hay FMI vacíos.")

print("\n=== Ejemplos de FMI duplicados (después de limpiar) ===")
if not df_dups_fmi.empty:
    print(df_dups_fmi.head(20).to_string(index=False))
else:
    print("No hay FMI duplicados.")


FMI vacíos (después de limpiar): 30
FMI duplicados (después de limpiar): 24
Filas Excel FMI vacíos: [895, 2905, 3967, 3969, 4298, 6077, 7385, 20942, 20963, 20964, 21130, 21132, 21133, 21134, 21135, 21136, 21141, 21142, 21143, 21145, 21146, 21147, 21148, 21149, 21150, 21151, 21152, 21153, 21154, 21155]
Filas Excel FMI duplicados: [1041, 7497, 7695, 14804, 16491, 19200, 19781, 19834, 19931, 20966, 21028, 21061, 21131, 21137, 21138, 21139, 21140, 21157, 21158, 21167, 21168, 21169, 21170, 23615]

Archivos creados:
 - FMI_con_problemas_vacios.xlsx
 - FMI_con_problemas_duplicados.xlsx
 - FMI_original_limpio_y_SQL.xlsx

=== Ejemplo FMI original, limpio y para migrar ===
 Fila_Excel        FMI    FMI_RAW FMI_LIMPIO    FMI_SQL
          2  062-37357  062-37357  062-37357  062-37357
          3  062-37359  062-37359  062-37359  062-37359
          4  062-37361  062-37361  062-37361  062-37361
          5  062-37354  062-37354  062-37354  062-37354
          6  142-23909  142-23909  142-23909  14

## No.Expediente

In [6]:
import pandas as pd
import re
import numpy as np

# -------------------------------------------------------------------
# 1. Asegurar columna de fila aproximada de Excel (encabezado en fila 2)
# -------------------------------------------------------------------
if 'Fila_Excel_aprox' not in df.columns:
    df['Fila_Excel_aprox'] = df.index + 2

# -------------------------------------------------------------------
# 2. Renombrar la columna original y conservar valor crudo
# -------------------------------------------------------------------
if 'No. Expediente' in df.columns and 'NumeroExpediente' not in df.columns:
    df = df.rename(columns={"No. Expediente": "NumeroExpediente"})

df['NumeroExpediente_RAW'] = df['NumeroExpediente']  # crudo tal cual viene

# -------------------------------------------------------------------
# 3. Limpiar espacios / NBSP y generar columna LIMPIA (una sola vez)
# -------------------------------------------------------------------
def limpiar_basico_expediente(valor):
    if pd.isna(valor):
        return None
    v = str(valor).replace("\u00A0", " ").strip()
    if v == "":
        return None
    return v

df['NumeroExpediente_LIMPIO'] = df['NumeroExpediente_RAW'].apply(limpiar_basico_expediente)

# -------------------------------------------------------------------
# 4. Normalizar para migrar (SQL): convertir "sin dato" → None
#    IMPORTANT: SQL sale de la columna LIMPIA, no del RAW
# -------------------------------------------------------------------
VALORES_SIN_DATO = {
    "NA", "N/A",
    "NO APLICA", "NO APLICA.",
    "NO REGISTRA", "NO REGISTRA.",
    "NO REGISTRO",
    "SIN REGISTRO"
}

def normalizar_numero_expediente(valor_limpio):
    if pd.isna(valor_limpio):
        return None
    v = str(valor_limpio).replace("\u00A0", " ").strip()
    if v == "":
        return None
    if v.upper() in VALORES_SIN_DATO:
        return None
    return v  # sin imponer formato ni unicidad

df['NumeroExpediente_SQL'] = df['NumeroExpediente_LIMPIO'].apply(normalizar_numero_expediente)

# -------------------------------------------------------------------
# 5. Longitud y diagnósticos (evitar LEN=4 en nulos)
# -------------------------------------------------------------------
s_sql = df['NumeroExpediente_SQL'].astype('string')     # nulos quedan <NA>
df['NumeroExpediente_LEN'] = s_sql.str.len()            # <NA> en nulos

mask_con_dato = df['NumeroExpediente_SQL'].notna()
mask_largos_100 = mask_con_dato & (df['NumeroExpediente_LEN'] > 100)

total = len(df)
sin_dato = df['NumeroExpediente_SQL'].isna().sum()
con_dato = mask_con_dato.sum()

print("=== RESUMEN NumeroExpediente ===")
print(f"Total registros: {total}")
print(f"Sin dato (NumeroExpediente_SQL = NULL): {sin_dato}")
print(f"Con dato (NumeroExpediente_SQL no nulo): {con_dato}")

print("\n=== Top 20 valores distintos (NumeroExpediente_SQL) ===")
print(df['NumeroExpediente_SQL'].value_counts(dropna=True).head(20))

if mask_largos_100.any():
    print("\n=== Registros con NumeroExpediente_SQL > 100 caracteres (REVISAR POR MIGRACIÓN) ===")
    print(
        df.loc[mask_largos_100,
               ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
                'NumeroExpediente_RAW', 'NumeroExpediente_LIMPIO', 'NumeroExpediente_SQL', 'NumeroExpediente_LEN']
        ].to_string(index=False)
    )
else:
    print("\nNingún NumeroExpediente_SQL supera los 100 caracteres (ok para varchar(100)).")

print("\n=== Ejemplo de primeras filas (RAW / LIMPIO / SQL / LEN) ===")
print(
    df[['Fila_Excel_aprox',
        'NumeroExpediente_RAW',
        'NumeroExpediente_LIMPIO',
        'NumeroExpediente_SQL',
        'NumeroExpediente_LEN'
      ]].head(10).to_string(index=False)
)

# -------------------------------------------------------------------
# 6. Exportar a Excel (opcional) para revisión visual
# -------------------------------------------------------------------
cols_export = [
    'Fila_Excel_aprox',
    'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    'NumeroExpediente_RAW',
    'NumeroExpediente_LIMPIO',
    'NumeroExpediente_SQL',
    'NumeroExpediente_LEN'
]
cols_export = [c for c in cols_export if c in df.columns]

df_export = df[cols_export].copy()
nombre_excel = "NumeroExpediente_raw_limpio_sql.xlsx"
df_export.to_excel(nombre_excel, index=False)

print(f"\nArchivo creado: {nombre_excel}")


=== RESUMEN NumeroExpediente ===
Total registros: 24055
Sin dato (NumeroExpediente_SQL = NULL): 15683
Con dato (NumeroExpediente_SQL no nulo): 8372

=== Top 20 valores distintos (NumeroExpediente_SQL) ===
NumeroExpediente_SQL
202440003401515432E    3
202440003401515433E    3
202440003401515471E    3
202440003401520684E    3
202440003401515790E    3
202440003401515982E    3
202440003401519194E    3
202440003401515430E    3
202440003401515363E    3
202440003401519082E    3
202440003401518989E    3
202440003401520680E    3
202440003401521208E    3
202440003401521434E    3
202440003401521435E    3
202440003401521437E    3
202440003401521438E    3
202440003401521439E    3
202440003401517209E    2
202440003401517363E    2
Name: count, dtype: int64

Ningún NumeroExpediente_SQL supera los 100 caracteres (ok para varchar(100)).

=== Ejemplo de primeras filas (RAW / LIMPIO / SQL / LEN) ===
 Fila_Excel_aprox NumeroExpediente_RAW NumeroExpediente_LIMPIO NumeroExpediente_SQL  NumeroExpediente_LEN
 

In [7]:
# Filas con NumeroExpediente_SQL más largo que el límite del campo SQL (100)
mask_largos = df["NumeroExpediente_SQL"].notna() & (df["NumeroExpediente_LEN"] > 100)

cols_largos = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "NumeroExpediente_RAW",
    "NumeroExpediente_LIMPIO",
    "NumeroExpediente_SQL",
    "NumeroExpediente_LEN",
]

df_largos = df.loc[mask_largos, cols_largos].copy()

print("Total NumeroExpediente > 100 caracteres:", len(df_largos))

df_largos.to_excel(
    "NumeroExpediente_largos_para_corregir.xlsx",
    index=False
)
print("Archivo creado: NumeroExpediente_largos_para_corregir.xlsx")


Total NumeroExpediente > 100 caracteres: 0
Archivo creado: NumeroExpediente_largos_para_corregir.xlsx


## FUENTE

In [8]:
import pandas as pd
import unicodedata

# ==========================================
# FUENTE -> IdFuenteProceso_SQL (SIN ERRORES)
# Mantiene los diagnósticos y los excels como antes.
# Solo agrega validaciones para que no reviente.
# ==========================================

# 0) Validaciones mínimas
if "FUENTE" not in df.columns:
    # Muestra columnas para que ubiques el nombre real si está distinto
    raise ValueError(
        "No existe la columna 'FUENTE' en df. "
        "Revisa df.columns (puede llamarse distinto, p.ej. 'FUENTE DEL PROCESO')."
    )

# Aseguramos columna de fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2  # header=1 => datos empiezan en fila 2

# 1) Función de normalización (igual que antes)
def normalizar_texto_fuente(s):
    if pd.isna(s):
        return None
    s = str(s).strip()
    if s == "":
        return None

    s_up = s.upper()
    if s_up in {"NA", "N/A", "NO APLICA", "SIN DATO"}:
        return None

    s_norm = unicodedata.normalize("NFD", s_up)
    s_norm = "".join(c for c in s_norm if unicodedata.category(c) != "Mn")

    for ch in ("_", "-", "–", "—"):
        s_norm = s_norm.replace(ch, " ")

    s_norm = " ".join(s_norm.split())
    return s_norm


# 2) Catálogo (igual que antes)
CATALOGO_FUENTE_ID = {
    1:  "ARGOS",
    2:  "BANCO AGRARIO DE COLOMBIA",
    3:  "CELSIA - DONACIÓN",
    4:  "CISA",
    5:  "COMPROMISOS ORGANIZACIONES SOCIALES",
    6:  "COMPROMISOS PMU - DAE ROSAS",
    7:  "CONTRALORÍA",
    8:  "CORREO COMPRAS",
    9:  "DAT",
    10: "DIAN",
    11: "DIRECCIÓN GENERAL",
    12: "DONACIÓN PARTICULAR",
    13: "ECOPETROL",
    14: "FEDEGÁN",
    15: "FISCALÍA",
    16: "FONDO ADAPTACIÓN",
    17: "LA MOJANA",
    18: "MADR",
    19: "MIN. DEFENSA",
    20: "OC - AGROVAR",
    21: "OC - ANUC CAUCA",
    22: "OC - ANUC CÓRDOBA",
    23: "OC - ANUC FENSUAGRO",
    24: "OC - ANUC NACIONAL",
    25: "OC - BELLACRUZ DIÁLOGO SOCIAL",
    26: "OC - CACEP ANAFRO",
    27: "OC - CACEP ANZORC",
    28: "OC - CACEP CONGRESO DE LOS PUEBLOS",
    29: "OC - CACEP MARCHA PATRIÓTICA",
    30: "OC - CATATUMBO",
    31: "OC - CIMA",
    32: "OC - CIMA CAUCA",
    33: "OC - CONPES NARIÑO",
    34: "OC - CUMBRE AGRARIA",
    35: "OC - FENSUAGRO",
    36: "OC - PARQUES NACIONALES",
    37: "OC - PUPSOC",
    38: "OC - PUPSOC CAUCA",
    39: "OC - QUIMBO",
    40: "OC - REINCORPORACIÓN",
    41: "OC - SOS COMPROMISOS ORGANIZACIONES",
    42: "ORFEO",
    43: "ORGANIZACIÓN CAMPESINA",
    44: "ROSAS",
    45: "SENTENCIA",
    46: "SIT",
    47: "SOMOS TIERRA",
    48: "UNGRD",
    49: "WHATSAPP",
    50: "WHATSAPP FEDEGÁN",
}

MAP_FUENTE_NORM_TO_ID = {}
MAP_FUENTE_NORM_TO_NOMBRE = {}
for id_fuente, nombre in CATALOGO_FUENTE_ID.items():
    clave_norm = normalizar_texto_fuente(nombre)
    MAP_FUENTE_NORM_TO_ID[clave_norm] = id_fuente
    MAP_FUENTE_NORM_TO_NOMBRE[clave_norm] = nombre


# 3) Crear columnas (igual que antes)
df["FUENTE_RAW"] = df["FUENTE"].astype("string")
df["FUENTE_TXT_LIMPIO"] = df["FUENTE_RAW"].apply(normalizar_texto_fuente)
df["FUENTE_NOMBRE_SQL"] = df["FUENTE_TXT_LIMPIO"].map(MAP_FUENTE_NORM_TO_NOMBRE)
df["IdFuenteProceso_SQL"] = df["FUENTE_TXT_LIMPIO"].map(MAP_FUENTE_NORM_TO_ID).astype("Int64")

# (Opcional) Reordenar columnas junto a FUENTE (pero sin romper si ya existen)
cols = list(df.columns)
if "FUENTE" in cols:
    pos = cols.index("FUENTE") + 1
    insertar = ["FUENTE_RAW", "FUENTE_TXT_LIMPIO", "FUENTE_NOMBRE_SQL", "IdFuenteProceso_SQL"]
    for c in insertar:
        if c in cols:
            cols.remove(c)
    cols[pos:pos] = insertar
    df = df[cols]

# 4) Diagnósticos por consola (igual que antes)
print("=== Valores CRUDOS de FUENTE (top 30) ===")
print(df["FUENTE_RAW"].value_counts(dropna=False).head(30))

print("\n=== Valores NORMALIZADOS de FUENTE_TXT_LIMPIO (top 30) ===")
print(df["FUENTE_TXT_LIMPIO"].value_counts(dropna=False).head(30))

print("\n=== Conteo IdFuenteProceso_SQL (incluyendo nulos) ===")
print(df["IdFuenteProceso_SQL"].value_counts(dropna=False))

mask_sin_mapeo = df["FUENTE_TXT_LIMPIO"].notna() & df["IdFuenteProceso_SQL"].isna()
fuentes_sin_id = (
    df.loc[mask_sin_mapeo, ["FUENTE_RAW", "FUENTE_TXT_LIMPIO"]]
      .drop_duplicates()
      .sort_values("FUENTE_TXT_LIMPIO")
)

print("\n=== FUENTE con texto limpio pero SIN IdFuenteProceso_SQL (para revisar a mano) ===")
print(fuentes_sin_id.to_string(index=False))
print("Total sin IdFuenteProceso_SQL:", len(fuentes_sin_id))

print("\n=== Ejemplo de mapeo FUENTE (primeras 10 filas) ===")
print(
    df[[
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "FUENTE_RAW",
        "FUENTE_TXT_LIMPIO",
        "FUENTE_NOMBRE_SQL",
        "IdFuenteProceso_SQL",
    ]].head(10).to_string(index=False)
)

# 5) Excels (igual que antes)
cols_export_full = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "FUENTE_RAW",
    "FUENTE_TXT_LIMPIO",
    "FUENTE_NOMBRE_SQL",
    "IdFuenteProceso_SQL",
]

df[cols_export_full].to_excel("FUENTE_mapeo_completo.xlsx", index=False)
df[mask_sin_mapeo][cols_export_full].to_excel("FUENTE_sin_mapeo.xlsx", index=False)

print("\nArchivos generados:")
print(" - FUENTE_mapeo_completo.xlsx")
print(" - FUENTE_sin_mapeo.xlsx")


=== Valores CRUDOS de FUENTE (top 30) ===
FUENTE_RAW
WHATSAPP                               11478
SIT                                     6570
ORFEO                                   1561
FEDEGAN                                 1377
CORREO_COMPRAS                          1100
OC-REINCORPORACION                       525
OC-ANUC NACIONAL                         289
WHATSAPP-FEDEGAN                         178
UNGRD                                    120
OC-QUIMBO                                114
OC-AGROVAR                                71
OC-ANUC CAUCA                             64
DAT                                       57
OC-CATATUMBO                              53
<NA>                                      43
LA MOJANA                                 43
OC-CONPES-NARIÑO                          38
ARGOS                                     34
MIN. DEFENSA                              31
CISA                                      28
OC-PUPSOC-CAUCA                           27
OC

*avance excel*

In [9]:
import pandas as pd

# =========================================================
# DF LISTO HASTA AHORA (RegistroProceso) - COLUMNAS APROBADAS
# Incluye SOLO lo que ya tienes bien:
#   - IdPostulacion  <- ID_SQL (si ya lo generaste)
#   - FMI            <- FMI_SQL
#   - NumeroExpediente <- NumeroExpediente_SQL
#   - IdFuenteProceso  <- IdFuenteProceso_SQL
# (IdTipoProceso_SQL e IdEtapaProcesal_SQL todavía NO)
# =========================================================

# 1) Trazabilidad
if "Fila_Excel" not in df.columns:
    df["Fila_Excel"] = df.index + 2

# 2) Asegurar IdPostulacion_SQL desde ID_SQL (sin reventar)
#    (si ya existe IdPostulacion_SQL, lo respeta)
if "IdPostulacion_SQL" not in df.columns:
    if "ID_SQL" in df.columns:
        df["IdPostulacion_SQL"] = df["ID_SQL"].astype("string")
    else:
        raise ValueError("Falta 'IdPostulacion_SQL' y no existe 'ID_SQL'. Ejecuta primero el bloque de ID que crea ID_SQL.")

# 3) Columnas requeridas (las que dices que ya están bien)
req = ["IdPostulacion_SQL", "FMI_SQL", "NumeroExpediente_SQL", "IdFuenteProceso_SQL"]
faltan = [c for c in req if c not in df.columns]
if faltan:
    raise ValueError(f"Faltan columnas *_SQL en df (aún no ejecutadas en esta sesión): {faltan}")

# 4) Construir df_migracion_hasta_ahora
df_migracion_hasta_ahora = pd.DataFrame({
    "Fila_Excel": df["Fila_Excel"],
    "IdPostulacion": df["IdPostulacion_SQL"].astype("string"),
    "FMI": df["FMI_SQL"].astype("string"),
    "NumeroExpediente": df["NumeroExpediente_SQL"].astype("string"),
    "IdFuenteProceso": df["IdFuenteProceso_SQL"].astype("Int64"),
}, index=df.index)

# 5) Validaciones mínimas según DDL (NOT NULL: IdPostulacion, FMI)
mask_idpost = df_migracion_hasta_ahora["IdPostulacion"].isna() | (df_migracion_hasta_ahora["IdPostulacion"].str.strip() == "")
mask_fmi    = df_migracion_hasta_ahora["FMI"].isna() | (df_migracion_hasta_ahora["FMI"].str.strip() == "")

if mask_idpost.any():
    df_migracion_hasta_ahora.loc[mask_idpost, ["Fila_Excel", "IdPostulacion"]].to_excel("ERROR_IdPostulacion_NULL.xlsx", index=False)
    raise ValueError("Hay IdPostulacion vacíos. Ver: ERROR_IdPostulacion_NULL.xlsx")

if mask_fmi.any():
    df_migracion_hasta_ahora.loc[mask_fmi, ["Fila_Excel", "FMI"]].to_excel("ERROR_FMI_NULL.xlsx", index=False)
    raise ValueError("Hay FMI vacíos. Ver: ERROR_FMI_NULL.xlsx")

# 6) Longitudes (DDL)
len_checks = {"IdPostulacion": 30, "FMI": 100, "NumeroExpediente": 100}
for col, maxlen in len_checks.items():
    s = df_migracion_hasta_ahora[col].astype("string")
    mask_len = s.notna() & (s.str.len() > maxlen)
    if mask_len.any():
        df_migracion_hasta_ahora.loc[mask_len, ["Fila_Excel", col]].to_excel(f"ERROR_{col}_MAS_DE_{maxlen}.xlsx", index=False)
        raise ValueError(f"Hay valores en {col} con longitud > {maxlen}. Ver: ERROR_{col}_MAS_DE_{maxlen}.xlsx")

# 7) Vista rápida + export
print("=== Columnas en df_migracion_hasta_ahora ===")
print(list(df_migracion_hasta_ahora.columns))

print("\n=== Primeras 15 filas ===")
print(df_migracion_hasta_ahora.head(15).to_string(index=False))

print("\n=== Últimas 15 filas ===")
print(df_migracion_hasta_ahora.tail(15).to_string(index=False))

df_migracion_hasta_ahora.drop(columns=["Fila_Excel"]).to_excel("RegistroProceso_hasta_ahora.xlsx", index=False)
df_migracion_hasta_ahora.to_excel("RegistroProceso_hasta_ahora_con_trazabilidad.xlsx", index=False)

print("\nArchivos creados:")
print(" - RegistroProceso_hasta_ahora.xlsx")
print(" - RegistroProceso_hasta_ahora_con_trazabilidad.xlsx")


=== Columnas en df_migracion_hasta_ahora ===
['Fila_Excel', 'IdPostulacion', 'FMI', 'NumeroExpediente', 'IdFuenteProceso']

=== Primeras 15 filas ===
 Fila_Excel IdPostulacion        FMI NumeroExpediente  IdFuenteProceso
          2       ID8099B  062-37357             <NA>               49
          3       ID8099C  062-37359             <NA>               49
          4       ID8099D  062-37361             <NA>               49
          5       ID8099E  062-37354             <NA>               49
          6       ID9080A  142-23909             <NA>               49
          7       ID8099F  062-37360             <NA>               49
          8       ID8099G  062-37356             <NA>               49
          9       ID10439 060-123317             <NA>               49
         10        ID6287  148-28711             <NA>               14
         11       ID8099A  062-37355             <NA>               49
         12        ID4656  062-10701             <NA>               4

## Tipo de proceso agrario

In [10]:
import pandas as pd
import unicodedata

# ============================================================
# TIPO DE PROCESO AGRARIO -> IdTipoProceso_SQL
# CORRECCIÓN: NO recarga df desde Excel. Usa el df ya existente.
# No cambia nada más.
# ============================================================

# Número de fila de Excel (encabezado en fila 2)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

def normalizar_texto(s):
    if pd.isna(s):
        return None
    s = str(s).strip()
    if s == "":
        return None

    s_up = s.upper()
    if s_up in {"NA", "N/A", "NO APLICA", "SIN DATO"}:
        return None

    s_norm = unicodedata.normalize("NFD", s_up)
    s_norm = "".join(c for c in s_norm if unicodedata.category(c) != "Mn")

    for ch in ("_", "-", "–", "—"):
        s_norm = s_norm.replace(ch, " ")

    s_norm = " ".join(s_norm.split())
    return s_norm

CATALOGO_TIPO_PROCESO = [
    (1, "CLARIFICACIÓN"),
    (2, "DESLINDE"),
    (3, "EXTINCIÓN"),
    (4, "NUEVO_SU_288"),
    (5, "BIENES FISCALES PATRIMONIALES"),
    (6, "RECUPERACIÓN"),
    (7, "TITULACIÓN DE BALDÍO A PERSONA NATURAL"),
    (8, "RECONOCIMIENTO DE SENTENCIA SU-288"),
    (9, "TITULACIÓN DE LA POSESIÓN"),
]

MAP_TIPO_PROC_NORM_A_ID = {}
MAP_TIPO_PROC_NORM_A_NOMBRE = {}

for id_tipo, nombre in CATALOGO_TIPO_PROCESO:
    clave_norm = normalizar_texto(nombre)
    MAP_TIPO_PROC_NORM_A_ID[clave_norm] = id_tipo
    MAP_TIPO_PROC_NORM_A_NOMBRE[clave_norm] = nombre

col_raw = "TIPO DE PROCESO AGRARIO"

# Copia cruda por si luego quieres mantenerla con otro nombre
df["TipoProceso_RAW"] = df[col_raw]

# Texto limpio (normalizado)
df["TIPO_PROCESO_TXT_LIMPIO"] = df[col_raw].apply(normalizar_texto)

# Nombre catálogo y ID para SQL
df["TipoProceso_NOMBRE_SQL"] = df["TIPO_PROCESO_TXT_LIMPIO"].map(MAP_TIPO_PROC_NORM_A_NOMBRE)
df["IdTipoProceso_SQL"] = df["TIPO_PROCESO_TXT_LIMPIO"].map(MAP_TIPO_PROC_NORM_A_ID).astype("Int64")

# Reubicar columnas nuevas junto a la columna original
cols = list(df.columns)
idx_raw = cols.index(col_raw)

for c in ["TipoProceso_RAW", "TIPO_PROCESO_TXT_LIMPIO", "TipoProceso_NOMBRE_SQL", "IdTipoProceso_SQL"]:
    if c in cols:
        cols.remove(c)

insert_pos = idx_raw + 1
cols[insert_pos:insert_pos] = [
    "TipoProceso_RAW",
    "TIPO_PROCESO_TXT_LIMPIO",
    "TipoProceso_NOMBRE_SQL",
    "IdTipoProceso_SQL",
]

df = df[cols]

print("=== Distribución de TIPO_PROCESO_TXT_LIMPIO ===")
print(df["TIPO_PROCESO_TXT_LIMPIO"].value_counts(dropna=False))

print("\n=== Resumen de mapeo a IdTipoProceso_SQL ===")
resumen_mapeo = (
    df.groupby(["TipoProceso_RAW", "TIPO_PROCESO_TXT_LIMPIO", "TipoProceso_NOMBRE_SQL", "IdTipoProceso_SQL"], dropna=False)
      .size()
      .reset_index(name="conteo")
      .sort_values(["IdTipoProceso_SQL", "TipoProceso_NOMBRE_SQL", "TIPO_PROCESO_TXT_LIMPIO"])
)
print(resumen_mapeo.to_string(index=False))

mask_sin_id = df["TIPO_PROCESO_TXT_LIMPIO"].notna() & df["IdTipoProceso_SQL"].isna()

tipo_sin_mapeo = (
    df.loc[mask_sin_id, ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
                         "TipoProceso_RAW", "TIPO_PROCESO_TXT_LIMPIO"]]
      .drop_duplicates()
      .sort_values(["TIPO_PROCESO_TXT_LIMPIO", "TipoProceso_RAW"])
)

print("\n=== TIPO DE PROCESO AGRARIO con texto pero SIN IdTipoProceso_SQL (para revisar) ===")
print(tipo_sin_mapeo.to_string(index=False))
print("Total sin IdTipoProceso_SQL:", len(tipo_sin_mapeo))

cols_export_full = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "TipoProceso_RAW",
    "TIPO_PROCESO_TXT_LIMPIO",
    "TipoProceso_NOMBRE_SQL",
    "IdTipoProceso_SQL",
]

df[cols_export_full].to_excel("TipoProceso_mapeo_completo.xlsx", index=False)
tipo_sin_mapeo.to_excel("TipoProceso_sin_mapeo.xlsx", index=False)

print("\nArchivos creados:")
print(" - TipoProceso_mapeo_completo.xlsx")
print(" - TipoProceso_sin_mapeo.xlsx")
print("\nTotal filas sin IdTipoProceso_SQL (en df):", df["IdTipoProceso_SQL"].isna().sum())


=== Distribución de TIPO_PROCESO_TXT_LIMPIO ===
TIPO_PROCESO_TXT_LIMPIO
None                                      23229
CLARIFICACION                               365
DESLINDE                                    104
BIENES FISCALES PATRIMONIALES                92
EXTINCION                                    69
RECUPERACION                                 69
TITULACION DE BALDIO A PERSONA NATURAL       68
NUEVO SU 288                                 30
TITULACION DE LA POSESION                    23
RECONOCIMIENTO DE SENTENCIA SU 288            6
Name: count, dtype: int64

=== Resumen de mapeo a IdTipoProceso_SQL ===
                       TipoProceso_RAW                TIPO_PROCESO_TXT_LIMPIO                 TipoProceso_NOMBRE_SQL  IdTipoProceso_SQL  conteo
                         CLARIFICACION                          CLARIFICACION                          CLARIFICACIÓN                  1     365
                              DESLINDE                               DESLINDE           

## ETAPA PROCESAL

In [11]:
import pandas as pd
import unicodedata

# ========================
# 1. Función de normalizar
# ========================
def normalizar_texto(s):
    if pd.isna(s):
        return None
    s = str(s).strip().upper()
    if s in ("", "NA", "N/A", "NO APLICA", "SIN DATO"):
        return None

    # quitar tildes
    s = unicodedata.normalize("NFD", s)
    s = "".join(c for c in s if unicodedata.category(c) != "Mn")

    # unificar separadores y espacios
    for ch in ("_", "-"):
        s = s.replace(ch, " ")
    s = " ".join(s.split())

    return s

# ===============================
# 2. Catálogo ETAPA PROCESAL SQL
# ===============================
CATALOGO_ETAPA = [
    "APERTURA TRAMITE ADMINISTRATIVO",  # 1
    "ARCHIVO FINAL",                    # 2
    "CIERRE ETAPA PROBATORIA",          # 3
    "EJECUTORIADO",                     # 4
    "ETAPA FINAL",                      # 5
    "ETAPA INICIAL",                    # 6
    "ETAPA PRELIMINAR",                 # 7
    "ETAPA PREVIA",                     # 8
    "ETAPA PROBATORIA",                 # 9
    "NUEVO_SU_288",                     #10
    "TRASLADADO",                       #11
    "ETAPA DE APERTURA",                #12
    "ETAPA DE CIERRE",                  #13
    "EN MISIONAL SIN GESTION",          #14
    "SIN INICIAR",                      #15
    "REGISTRO",                         #16
    "SUSPENDIDO",                       #17
    "DECIDIDO",                         #18
]

MAP_ETAPA_NOMBRE = {
    normalizar_texto(nombre): nombre
    for nombre in CATALOGO_ETAPA
}
MAP_ETAPA_ID = {
    normalizar_texto(nombre): idx
    for idx, nombre in enumerate(CATALOGO_ETAPA, start=1)
}

# ===============================
# 3. Sinónimos que trae el Excel
# ===============================
SINONIMOS = {
    "CIERRE PROBATORIA": "CIERRE ETAPA PROBATORIA",
    "APERTURA PROBATORIA": "ETAPA PROBATORIA",
    "ETAPA INICIO": "ETAPA INICIAL",
    "SININICIAR": "SIN INICIAR",
}

for variante, oficial in SINONIMOS.items():
    MAP_ETAPA_ID[normalizar_texto(variante)] = MAP_ETAPA_ID[normalizar_texto(oficial)]
    MAP_ETAPA_NOMBRE[normalizar_texto(variante)] = MAP_ETAPA_NOMBRE[normalizar_texto(oficial)]

# ==========================
# 4. Columnas en el DataFrame
# ==========================
# Aseguramos columna de fila de Excel (si no la tienes ya creada)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2  # porque header=1

# RAW directamente de la matriz
df["EtapaProcesal_RAW"] = df["ETAPA PROCESAL"]

# Texto normalizado
df["EtapaProcesal_TXT_LIMPIO"] = df["EtapaProcesal_RAW"].apply(normalizar_texto)

# Nombre catálogo y FK
df["EtapaProcesal_NOMBRE_SQL"] = df["EtapaProcesal_TXT_LIMPIO"].map(MAP_ETAPA_NOMBRE)
df["IdEtapaProcesal_SQL"] = df["EtapaProcesal_TXT_LIMPIO"].map(MAP_ETAPA_ID).astype("Int64")

# ==============================
# 5. Resumen por combinación
# ==============================
resumen = (
    df.groupby(
        ["EtapaProcesal_RAW",
         "EtapaProcesal_TXT_LIMPIO",
         "EtapaProcesal_NOMBRE_SQL",
         "IdEtapaProcesal_SQL"],
        dropna=False
    )
    .size()
    .reset_index(name="conteo")
    .sort_values("conteo", ascending=False)
)

print("=== Distribución de EtapaProcesal_TXT_LIMPIO ===")
print(df["EtapaProcesal_TXT_LIMPIO"].value_counts(dropna=False))

print("\n=== Resumen de mapeo a IdEtapaProcesal_SQL ===")
print(resumen.to_string(index=False))

# ==========================
# 6. Filas sin Id (para revisar)
# ==========================
mask_sin_id = (
    df["EtapaProcesal_TXT_LIMPIO"].notna()
    & df["IdEtapaProcesal_SQL"].isna()
)

etapa_sin_id = df.loc[
    mask_sin_id,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "EtapaProcesal_RAW",
        "EtapaProcesal_TXT_LIMPIO",
    ],
]

print("\n=== ETAPA PROCESAL con texto pero SIN IdEtapaProcesal_SQL (para revisar) ===")
print(etapa_sin_id.to_string(index=False))
print("Total sin IdEtapaProcesal_SQL:", len(etapa_sin_id))

# ==========================
# 7. Excels de salida
# ==========================
# 7.1 Todas las filas (para inspección visual)
cols_mapeo_filas = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "EtapaProcesal_RAW",
    "EtapaProcesal_TXT_LIMPIO",
    "EtapaProcesal_NOMBRE_SQL",
    "IdEtapaProcesal_SQL",
]
df[cols_mapeo_filas].to_excel(
    "EtapaProcesal_mapeo_completo_filas.xlsx", index=False
)

# 7.2 Solo las filas sin Id (si hubiera)
etapa_sin_id.to_excel(
    "EtapaProcesal_sin_IdEtapaProcesal_SQL.xlsx", index=False
)

# 7.3 Resumen por combinación (como el que ya viste)
resumen.to_excel(
    "EtapaProcesal_resumen_valores.xlsx", index=False
)

print("\nArchivos creados:")
print(" - EtapaProcesal_mapeo_completo_filas.xlsx")
print(" - EtapaProcesal_sin_IdEtapaProcesal_SQL.xlsx")
print(" - EtapaProcesal_resumen_valores.xlsx")


=== Distribución de EtapaProcesal_TXT_LIMPIO ===
EtapaProcesal_TXT_LIMPIO
None                               23223
ETAPA PRELIMINAR                     212
APERTURA TRAMITE ADMINISTRATIVO      112
ETAPA DE CIERRE                       85
EN MISIONAL SIN GESTION               62
ETAPA FINAL                           55
SININICIAR                            49
TRASLADADO                            47
ARCHIVO FINAL                         44
APERTURA PROBATORIA                   42
ETAPA PREVIA                          24
REGISTRO                              24
NUEVO SU 288                          21
ETAPA DE APERTURA                     19
EJECUTORIADO                          19
CIERRE PROBATORIA                      8
ETAPA INICIO                           7
SUSPENDIDO                             1
DECIDIDO                               1
Name: count, dtype: int64

=== Resumen de mapeo a IdEtapaProcesal_SQL ===
              EtapaProcesal_RAW        EtapaProcesal_TXT_LIMPIO        Et

## DEPENDENCIA

In [12]:
# ==========================
# DEPENDENCIA (texto libre)
# ==========================

# Si no existe aún, creamos la columna con la fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2   # porque leíste con header=1

# 1. Asegurar nombre de columna consistente con SQL
if "Dependencia" not in df.columns:
    if "DEPENDENCIA" in df.columns:
        df = df.rename(columns={"DEPENDENCIA": "Dependencia"})
    else:
        raise ValueError("No existe la columna 'DEPENDENCIA' (ni 'Dependencia') en df. Revisa df.columns.")

# 2-5) SOLO crear/llenar columnas si NO existen (para que NO recargue/NO sobrescriba)
if "Dependencia_RAW" not in df.columns:
    df["Dependencia_RAW"] = df["Dependencia"]

def limpiar_dependencia(valor):
    if pd.isna(valor):
        return None

    s = str(valor)

    # NBSP → espacio normal
    s = s.replace("\u00A0", " ")

    # quitar espacios al inicio/fin
    s = s.strip()

    # valores que tratamos como “sin dato”
    if s == "":
        return None

    s_upper = s.upper()
    if s_upper in {
        "NA", "N/A", "NO APLICA", "SIN DATO",
        "NO REGISTRA", "NO REGISTRA."
    }:
        return None

    # comprimir espacios internos múltiples a uno solo
    s = " ".join(s.split())

    return s

if "Dependencia_TXT_LIMPIO" not in df.columns:
    df["Dependencia_TXT_LIMPIO"] = df["Dependencia_RAW"].apply(limpiar_dependencia)

# 5. Valor que se migrará a SQL (respetando longitud de la columna)
MAX_LEN_DEPENDENCIA_SQL = 500   # según la regla que venías usando

if "Dependencia_SQL" not in df.columns:
    df["Dependencia_SQL"] = df["Dependencia_TXT_LIMPIO"]

if "Dependencia_SQL_longitud" not in df.columns:
    df["Dependencia_SQL_longitud"] = df["Dependencia_SQL"].astype("string").str.len()

mask_larga = df["Dependencia_SQL"].notna() & (df["Dependencia_SQL_longitud"] > MAX_LEN_DEPENDENCIA_SQL)

print("Total filas con Dependencia vacía (para migrar como NULL):", df["Dependencia_SQL"].isna().sum())
print("Filas con Dependencia_SQL que exceden", MAX_LEN_DEPENDENCIA_SQL, "caracteres:", int(mask_larga.sum()))

# Truncar SOLO para la columna que se va a migrar
df.loc[mask_larga, "Dependencia_SQL"] = df.loc[mask_larga, "Dependencia_SQL"].str.slice(0, MAX_LEN_DEPENDENCIA_SQL)

# Recalcular longitud solo si existe la columna (para que refleje el truncamiento)
df["Dependencia_SQL_longitud"] = df["Dependencia_SQL"].astype("string").str.len()

# 6. Resumen rápido de valores (ya limpios)
print("\n=== Top 30 valores de Dependencia_TXT_LIMPIO ===")
print(df["Dependencia_TXT_LIMPIO"].value_counts(dropna=False).head(30))

# 7. DataFrame con las filas problemáticas (texto demasiado largo)
dependencia_larga = df.loc[
    mask_larga,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "Dependencia_RAW",
        "Dependencia_TXT_LIMPIO",
        "Dependencia_SQL",
        "Dependencia_SQL_longitud",
    ],
].copy()

print("\n=== Ejemplos de Dependencia que exceden longitud máxima ===")
print(dependencia_larga.head(20).to_string(index=False))

# 8. Excel con TODAS las filas (para inspección visual)
cols_mapeo_filas = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "Dependencia_RAW",
    "Dependencia_TXT_LIMPIO",
    "Dependencia_SQL",
    "Dependencia_SQL_longitud",
]

df[cols_mapeo_filas].to_excel(
    "Dependencia_mapeo_completo_filas.xlsx",
    index=False,
)

# 9. Excel solo con las filas que exceden la longitud (si hay)
dependencia_larga.to_excel(
    "Dependencia_dependencia_larga.xlsx",
    index=False,
)

print("\nArchivos creados:")
print(" - Dependencia_mapeo_completo_filas.xlsx")
print(" - Dependencia_dependencia_larga.xlsx")


Total filas con Dependencia vacía (para migrar como NULL): 23223
Filas con Dependencia_SQL que exceden 500 caracteres: 0

=== Top 30 valores de Dependencia_TXT_LIMPIO ===
Dependencia_TXT_LIMPIO
None                                                            23223
SPAGJ                                                             420
UGT                                                               119
DIRECCION DE ACCESO A TIERRAS -PROGRAMAS ESPECIALES                97
SSJ                                                                92
SUBDIRECCION DE ACCESO A TIERRAS POR ZONAS FOCALIZADAS             61
SUBDIRECCION DE SEGURIDAD JURIDICA                                 23
SUBDIRECCION DE ACCESO A TIERRAS POR DEMANDA Y DESCONGESTION        9
SPAGJ - UGT                                                         5
UGT CORDOBA                                                         2
UGT CAUCA                                                           2
UGT MAGDALENA                       

## ID ORFEO

In [13]:
import pandas as pd

# ==========================
# ID ORFEO  →  RadicadoOrfeo
# (SIN RECARGA / SIN SOBRESCRIBIR)
# ==========================

# 0. Asegurar columna de fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2   # porque leíste con header=1

# Validación mínima (evitar KeyError)
if "ID ORFEO" not in df.columns:
    raise ValueError("No existe la columna 'ID ORFEO' en df. Revisa df.columns por si el nombre viene distinto.")

# 1. Guardar valor crudo (SOLO si no existe)
if "RadicadoOrfeo_RAW" not in df.columns:
    df["RadicadoOrfeo_RAW"] = df["ID ORFEO"]

# 2. Función de limpieza (se deja igual)
def limpiar_radicado_orfeo(valor):
    if pd.isna(valor):
        return None

    s = str(valor)

    # NBSP → espacio normal
    s = s.replace("\u00A0", " ")

    # quitar espacios al inicio/fin
    s = s.strip()

    # textos que significan "sin dato"
    if s == "":
        return None

    s_upper = s.upper()
    if s_upper in {
        "NA", "N/A", "NO APLICA", "SIN DATO",
        "NO REGISTRA", "NO REGISTRA."
    }:
        return None

    # comprimir espacios internos
    s = " ".join(s.split())

    return s

# 3. Texto limpio (SOLO si no existe)
if "RadicadoOrfeo_TXT_LIMPIO" not in df.columns:
    df["RadicadoOrfeo_TXT_LIMPIO"] = df["RadicadoOrfeo_RAW"].apply(limpiar_radicado_orfeo)

# 4. Valor final para migrar a SQL (varchar(50)) (SOLO si no existe)
MAX_LEN_ORFEO_SQL = 50
if "RadicadoOrfeo_SQL" not in df.columns:
    df["RadicadoOrfeo_SQL"] = df["RadicadoOrfeo_TXT_LIMPIO"]

# Longitud (SOLO si no existe)
if "RadicadoOrfeo_SQL_longitud" not in df.columns:
    df["RadicadoOrfeo_SQL_longitud"] = df["RadicadoOrfeo_SQL"].astype("string").str.len()

mask_largos = df["RadicadoOrfeo_SQL"].notna() & (df["RadicadoOrfeo_SQL_longitud"] > MAX_LEN_ORFEO_SQL)

# 5. Duplicados (solo informativo, no se tocan)
mask_no_nulo = df["RadicadoOrfeo_SQL"].notna()
mask_dups = mask_no_nulo & df["RadicadoOrfeo_SQL"].duplicated(keep=False)

# 6. Resumen por pantalla (igual)
print("Total filas RadicadoOrfeo_SQL = NULL:",
      df["RadicadoOrfeo_SQL"].isna().sum())
print("Filas con RadicadoOrfeo_SQL > 50 caracteres:", mask_largos.sum())
print("Filas con RadicadoOrfeo_SQL duplicados:", mask_dups.sum())

print("\nDistribución de longitudes RadicadoOrfeo_SQL (top 10):")
print(df["RadicadoOrfeo_SQL_longitud"].value_counts().head(10))

print("\n=== Ejemplo de mapeo (primeras 10 filas) ===")
print(
    df.loc[:, [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "RadicadoOrfeo_RAW",
        "RadicadoOrfeo_TXT_LIMPIO",
        "RadicadoOrfeo_SQL",
        "RadicadoOrfeo_SQL_longitud",
    ]].head(10).to_string(index=False)
)

print("\n=== Ejemplos de RadicadoOrfeo largos (> 50) ===")
radicado_largo = df.loc[
    mask_largos,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "RadicadoOrfeo_RAW",
        "RadicadoOrfeo_TXT_LIMPIO",
        "RadicadoOrfeo_SQL",
        "RadicadoOrfeo_SQL_longitud",
    ],
].copy()
print(radicado_largo.head(20).to_string(index=False))

print("\n=== Ejemplos de RadicadoOrfeo duplicados ===")
radicado_dups = df.loc[
    mask_dups,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "RadicadoOrfeo_SQL",
    ],
].sort_values("RadicadoOrfeo_SQL")
print(radicado_dups.head(20).to_string(index=False))

# 7. (Opcional) truncar SOLO los largos en la columna que migra
#    (SIN RECARGAR todo: solo modifica si hay largos)
if mask_largos.any():
    df.loc[mask_largos, "RadicadoOrfeo_SQL"] = df.loc[mask_largos, "RadicadoOrfeo_SQL"].astype("string").str[:MAX_LEN_ORFEO_SQL]
    df["RadicadoOrfeo_SQL_longitud"] = df["RadicadoOrfeo_SQL"].astype("string").str.len()

# 8. Excels por FILA para revisión (igual)

# 8.1. Mapeo completo (todas las filas)
cols_mapeo = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "RadicadoOrfeo_RAW",
    "RadicadoOrfeo_TXT_LIMPIO",
    "RadicadoOrfeo_SQL",
    "RadicadoOrfeo_SQL_longitud",
]
df[cols_mapeo].to_excel(
    "RadicadoOrfeo_mapeo_completo_filas.xlsx",
    index=False,
)

# 8.2. Solo valores largos
radicado_largo.to_excel(
    "RadicadoOrfeo_largos.xlsx",
    index=False,
)

# 8.3. Solo duplicados (informativo)
radicado_dups.to_excel(
    "RadicadoOrfeo_duplicados.xlsx",
    index=False,
)

print("\nArchivos creados:")
print(" - RadicadoOrfeo_mapeo_completo_filas.xlsx")
print(" - RadicadoOrfeo_largos.xlsx")
print(" - RadicadoOrfeo_duplicados.xlsx")


Total filas RadicadoOrfeo_SQL = NULL: 23411
Filas con RadicadoOrfeo_SQL > 50 caracteres: 0
Filas con RadicadoOrfeo_SQL duplicados: 163

Distribución de longitudes RadicadoOrfeo_SQL (top 10):
RadicadoOrfeo_SQL_longitud
19    641
18      1
39      1
8       1
Name: count, dtype: Int64

=== Ejemplo de mapeo (primeras 10 filas) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO     MUNICIPIO RadicadoOrfeo_RAW RadicadoOrfeo_TXT_LIMPIO RadicadoOrfeo_SQL  RadicadoOrfeo_SQL_longitud
                2  062-37357      BOLIVAR      ZAMBRANO               NaN                     None              None                        <NA>
                3  062-37359      BOLIVAR      ZAMBRANO               NaN                     None              None                        <NA>
                4  062-37361      BOLIVAR      ZAMBRANO               NaN                     None              None                        <NA>
                5  062-37354      BOLIVAR      ZAMBRANO               NaN                 

In [14]:
import pandas as pd

# =========================================================
# DF LISTO HASTA AHORA (RegistroProceso) - COLUMNAS APROBADAS
# Incluye SOLO lo que ya tienes listo:
#   - IdPostulacion       <- IdPostulacion_SQL (o ID_SQL)
#   - FMI                 <- FMI_SQL
#   - NumeroExpediente    <- NumeroExpediente_SQL
#   - IdFuenteProceso     <- IdFuenteProceso_SQL
#   - IdTipoProceso       <- IdTipoProceso_SQL
#   - IdEtapaProcesal     <- IdEtapaProcesal_SQL
#   - Dependencia         <- Dependencia_SQL
#   - RadicadoOrfeo       <- RadicadoOrfeo_SQL
# =========================================================

# 1) Trazabilidad (usa Fila_Excel si existe; si no, crea)
if "Fila_Excel" not in df.columns:
    df["Fila_Excel"] = df.index + 2

# 2) Asegurar IdPostulacion_SQL (sin reventar)
if "IdPostulacion_SQL" not in df.columns:
    if "ID_SQL" in df.columns:
        df["IdPostulacion_SQL"] = df["ID_SQL"].astype("string")
    else:
        raise ValueError("Falta 'IdPostulacion_SQL' y no existe 'ID_SQL'. Ejecuta primero el bloque de ID que crea ID_SQL.")

# 3) Columnas requeridas (las que dijiste que ya están listas)
req = [
    "IdPostulacion_SQL",
    "FMI_SQL",
    "NumeroExpediente_SQL",
    "IdFuenteProceso_SQL",
    "IdTipoProceso_SQL",
    "IdEtapaProcesal_SQL",
    "Dependencia_SQL",
    "RadicadoOrfeo_SQL",
]
faltan = [c for c in req if c not in df.columns]
if faltan:
    raise ValueError(f"Faltan columnas *_SQL en df (aún no ejecutadas en esta sesión): {faltan}")

# 4) Construir df_migracion_hasta_ahora
df_migracion_hasta_ahora = pd.DataFrame({
    "Fila_Excel": df["Fila_Excel"],
    "IdPostulacion": df["IdPostulacion_SQL"].astype("string"),
    "FMI": df["FMI_SQL"].astype("string"),
    "NumeroExpediente": df["NumeroExpediente_SQL"].astype("string"),
    "IdFuenteProceso": df["IdFuenteProceso_SQL"].astype("Int64"),
    "IdTipoProceso": df["IdTipoProceso_SQL"].astype("Int64"),
    "IdEtapaProcesal": df["IdEtapaProcesal_SQL"].astype("Int64"),
    "Dependencia": df["Dependencia_SQL"].astype("string"),
    "RadicadoOrfeo": df["RadicadoOrfeo_SQL"].astype("string"),
}, index=df.index)

# 5) Validaciones mínimas según DDL (NOT NULL: IdPostulacion, FMI)
mask_idpost = df_migracion_hasta_ahora["IdPostulacion"].isna() | (df_migracion_hasta_ahora["IdPostulacion"].str.strip() == "")
mask_fmi    = df_migracion_hasta_ahora["FMI"].isna() | (df_migracion_hasta_ahora["FMI"].str.strip() == "")

if mask_idpost.any():
    df_migracion_hasta_ahora.loc[mask_idpost, ["Fila_Excel", "IdPostulacion"]].to_excel("ERROR_IdPostulacion_NULL.xlsx", index=False)
    raise ValueError("Hay IdPostulacion vacíos. Ver: ERROR_IdPostulacion_NULL.xlsx")

if mask_fmi.any():
    df_migracion_hasta_ahora.loc[mask_fmi, ["Fila_Excel", "FMI"]].to_excel("ERROR_FMI_NULL.xlsx", index=False)
    raise ValueError("Hay FMI vacíos. Ver: ERROR_FMI_NULL.xlsx")

# 6) Longitudes (según DDL conocido)
# Nota: ajusta Dependencia si tu DDL final es varchar(10) u otro.
len_checks = {
    "IdPostulacion": 30,
    "FMI": 100,
    "NumeroExpediente": 100,
    "RadicadoOrfeo": 50,
    "Dependencia": 500,  # cambia aquí si tu SQL es varchar(10) u otro
}
for col, maxlen in len_checks.items():
    s = df_migracion_hasta_ahora[col].astype("string")
    mask_len = s.notna() & (s.str.len() > maxlen)
    if mask_len.any():
        df_migracion_hasta_ahora.loc[mask_len, ["Fila_Excel", col]].to_excel(f"ERROR_{col}_MAS_DE_{maxlen}.xlsx", index=False)
        raise ValueError(f"Hay valores en {col} con longitud > {maxlen}. Ver: ERROR_{col}_MAS_DE_{maxlen}.xlsx")

# 7) Vista rápida + export
print("=== Columnas en df_migracion_hasta_ahora ===")
print(list(df_migracion_hasta_ahora.columns))

print("\n=== Primeras 15 filas ===")
print(df_migracion_hasta_ahora.head(15).to_string(index=False))

print("\n=== Últimas 15 filas ===")
print(df_migracion_hasta_ahora.tail(15).to_string(index=False))

df_migracion_hasta_ahora.drop(columns=["Fila_Excel"]).to_excel("RegistroProceso_hasta_ahora.xlsx", index=False)
df_migracion_hasta_ahora.to_excel("RegistroProceso_hasta_ahora_con_trazabilidad.xlsx", index=False)

print("\nArchivos creados:")
print(" - RegistroProceso_hasta_ahora.xlsx")
print(" - RegistroProceso_hasta_ahora_con_trazabilidad.xlsx")


=== Columnas en df_migracion_hasta_ahora ===
['Fila_Excel', 'IdPostulacion', 'FMI', 'NumeroExpediente', 'IdFuenteProceso', 'IdTipoProceso', 'IdEtapaProcesal', 'Dependencia', 'RadicadoOrfeo']

=== Primeras 15 filas ===
 Fila_Excel IdPostulacion        FMI NumeroExpediente  IdFuenteProceso  IdTipoProceso  IdEtapaProcesal                                         Dependencia       RadicadoOrfeo
          2       ID8099B  062-37357             <NA>               49           <NA>             <NA>                                                <NA>                <NA>
          3       ID8099C  062-37359             <NA>               49           <NA>             <NA>                                                <NA>                <NA>
          4       ID8099D  062-37361             <NA>               49           <NA>             <NA>                                                <NA>                <NA>
          5       ID8099E  062-37354             <NA>               49           <

Para RegistroProceso (tabla destino: ViabilidadJuridica.Postulacion.RegistroProceso), el Excel final debe tener estas columnas destino (mapeo Excel → SQL):

IdPostulacion → RegistroProceso.IdPostulacion (varchar(30), NOT NULL)

FMI → RegistroProceso.FMI (varchar(100), NOT NULL)

NumeroExpediente → RegistroProceso.NumeroExpediente (varchar(100), NULL)

IdFuenteProceso → RegistroProceso.IdFuenteProceso (int, NOT NULL en SQL; si vas a dejar vacíos en Excel, toca definir cómo los manejarás antes de insertar)

IdTipoProceso → RegistroProceso.IdTipoProceso (int, NOT NULL)

IdEtapaProcesal → RegistroProceso.IdEtapaProcesal (int, NOT NULL)

Dependencia → RegistroProceso.Dependencia (varchar(10), NULL)

RadicadoOrfeo → RegistroProceso.RadicadoOrfeo (varchar(50), NULL)

Notas rápidas:

No incluyas IdRegistroProceso en el Excel: lo genera SQL con la secuencia.

Fila_Excel (si la agregas) es solo para trazabilidad, no va a SQL.

## Localización

In [15]:
# DEPARTAMENTO / MUNICIPIO  → versiones limpias

df["DEPARTAMENTO_LIMPIO"] = df["DEPARTAMENTO"].apply(normalizar_texto)
df["MUNICIPIO_LIMPIO"]    = df["MUNICIPIO"].apply(normalizar_texto)

# Ver un resumen de cómo quedaron (opcional, solo para revisar)
loc_revision = (
    df
      .groupby(
          ["DEPARTAMENTO", "MUNICIPIO",
           "DEPARTAMENTO_LIMPIO", "MUNICIPIO_LIMPIO"],
          dropna=False
      )
      .size()
      .reset_index(name="conteo")
)

print("=== Resumen DEPARTAMENTO / MUNICIPIO ===")
print(loc_revision.to_string(index=False))


=== Resumen DEPARTAMENTO / MUNICIPIO ===
                                            DEPARTAMENTO                   MUNICIPIO                                      DEPARTAMENTO_LIMPIO            MUNICIPIO_LIMPIO  conteo
                                                AMAZONAS                     LETICIA                                                 AMAZONAS                     LETICIA       5
                                               ANTIOQUIA                   ABEJORRAL                                                ANTIOQUIA                   ABEJORRAL       9
                                               ANTIOQUIA                    ABRIAQUI                                                ANTIOQUIA                    ABRIAQUI       4
                                               ANTIOQUIA                       AMAGA                                                ANTIOQUIA                       AMAGA       2
                                               ANTIOQUIA             

## Conectarme a la base de datos

In [16]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
import pyodbc


In [17]:
server = r"172.23.92.97\MSSQLSERVER2022"
database = "ViabilidadJuridica"
user = "andres.garciav"
password = "4ndr3s.g4rc14v_3610@"

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={user};"
    f"PWD={password};"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


## Localización

In [18]:
import pandas as pd
import unicodedata

# ============================================================
# LOCALIZACION -> IdLocalizacion_SQL (UNIFICADO, SIN RECARGAR df)
# - NO vuelve a leer el Excel
# - Aplica correcciones puntuales:
#   1) CAUCA / "SOTARA PAISPAMBA"  ->  "SOTARÁ - PAISPAMBA"
#   2) NaN/NaN (DEPTO+MUNI vacíos) -> "SIN REGISTRO" / "SIN REGISTRO"
#      (asume que YA crearás en SQL el registro con:
#       IdLocalizacion=1123, CodigoDepartamento=100, NombreDepartamento='SIN REGISTRO',
#       CodigoMunicipio=100773, NombreMunicipio='SIN REGISTRO', Tipo según tu tabla)
# - Vuelve a leer el catálogo desde SQL para mapear con lo ya corregido
# ============================================================

# -------------------------
# 0) Validaciones mínimas
# -------------------------
for c in ["DEPARTAMENTO", "MUNICIPIO"]:
    if c not in df.columns:
        raise ValueError(f"Falta la columna '{c}' en df. Revisa el nombre real en df.columns")

if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2  # header=1 => datos empiezan en fila 2

# -------------------------
# 1) Normalizador
# -------------------------
def normalizar_texto(s):
    if pd.isna(s):
        return None
    s = str(s).replace("\u00A0", " ").strip().upper()
    if s in ("", "NA", "N/A", "NO APLICA", "SIN DATO"):
        return None
    s = unicodedata.normalize("NFD", s)
    s = "".join(c for c in s if unicodedata.category(c) != "Mn")
    s = " ".join(s.split())
    return s

# -------------------------
# 2) Correcciones puntuales (NO recarga df)
# -------------------------

# 2.1 META / FUENTEDEORO  ->  FUENTE DE ORO
mask_meta_fuente = (df["DEPARTAMENTO"] == "META") & (df["MUNICIPIO"] == "FUENTEDEORO")
df.loc[mask_meta_fuente, "MUNICIPIO"] = "FUENTE DE ORO"

# 2.2 MAGDALENA / ARIGUIANI  ->  ARIGUANÍ
mask_mag_arig = (df["DEPARTAMENTO"] == "MAGDALENA") & (df["MUNICIPIO"] == "ARIGUIANI")
df.loc[mask_mag_arig, "MUNICIPIO"] = "ARIGUANÍ"

# 2.3 CAUCA / SOTARA PAISPAMBA -> SOTARÁ - PAISPAMBA
mask_cauca_sotara = (
    (df["DEPARTAMENTO"] == "CAUCA") &
    (df["MUNICIPIO"].astype("string").fillna("") == "SOTARA PAISPAMBA")
)
df.loc[mask_cauca_sotara, "MUNICIPIO"] = "SOTARÁ - PAISPAMBA"

# 2.4 NaN/NaN -> SIN REGISTRO / SIN REGISTRO (para que capture el nuevo registro SQL)
mask_nan_nan = df["DEPARTAMENTO"].isna() & df["MUNICIPIO"].isna()
df.loc[mask_nan_nan, "DEPARTAMENTO"] = "SIN REGISTRO"
df.loc[mask_nan_nan, "MUNICIPIO"] = "SIN REGISTRO"

# -------------------------
# 3) RAW / LIMPIO / LLAVE
# -------------------------
df["DEPARTAMENTO_RAW"] = df["DEPARTAMENTO"]
df["MUNICIPIO_RAW"] = df["MUNICIPIO"]

df["DEPARTAMENTO_LIMPIO"] = df["DEPARTAMENTO_RAW"].apply(normalizar_texto)
df["MUNICIPIO_LIMPIO"] = df["MUNICIPIO_RAW"].apply(normalizar_texto)

df["LLAVE_LOC_TXT_LIMPIO"] = df["DEPARTAMENTO_LIMPIO"].astype("string") + " || " + df["MUNICIPIO_LIMPIO"].astype("string")

# -------------------------
# 4) Leer catálogo Localizacion desde SQL (requiere engine)
# -------------------------
consulta_localizacion = """
SELECT
    IdLocalizacion,
    CodigoDepartamento,
    NombreDepartamento,
    CodigoMunicipio,
    NombreMunicipio,
    Tipo
FROM ViabilidadJuridica.Postulacion.Localizacion;
"""
cat_loc = pd.read_sql(consulta_localizacion, con=engine)

cat_loc["DEP_LOC_LIMPIO"] = cat_loc["NombreDepartamento"].apply(normalizar_texto)
cat_loc["MUN_LOC_LIMPIO"] = cat_loc["NombreMunicipio"].apply(normalizar_texto)
cat_loc["LLAVE_LOC_CAT"] = cat_loc["DEP_LOC_LIMPIO"].astype("string") + " || " + cat_loc["MUN_LOC_LIMPIO"].astype("string")

# Duplicados de llave en catálogo (si existen, toca resolver en BD)
dup_loc = (
    cat_loc.groupby(["LLAVE_LOC_CAT"], dropna=False)["IdLocalizacion"]
           .nunique()
           .reset_index(name="num_IdLocalizacion")
)
dup_prob = dup_loc[dup_loc["num_IdLocalizacion"] > 1]
print("=== Posibles duplicados en catálogo Localizacion (DEPTO+MUNI) ===")
print(dup_prob.to_string(index=False))

# -------------------------
# 5) Mapear a IdLocalizacion
# -------------------------
map_llave_to_id = dict(zip(cat_loc["LLAVE_LOC_CAT"], cat_loc["IdLocalizacion"]))
map_llave_to_nombre = dict(
    zip(
        cat_loc["LLAVE_LOC_CAT"],
        cat_loc["NombreMunicipio"].astype("string") + " (" + cat_loc["NombreDepartamento"].astype("string") + ")"
    )
)

df["IdLocalizacion_SQL"] = df["LLAVE_LOC_TXT_LIMPIO"].map(map_llave_to_id).astype("Int64")
df["Localizacion_NOMBRE_SQL"] = df["LLAVE_LOC_TXT_LIMPIO"].map(map_llave_to_nombre)

# -------------------------
# 6) Resumen y pendientes
# -------------------------
resumen_loc = (
    df.groupby(
        [
            "DEPARTAMENTO_RAW",
            "MUNICIPIO_RAW",
            "DEPARTAMENTO_LIMPIO",
            "MUNICIPIO_LIMPIO",
            "Localizacion_NOMBRE_SQL",
            "IdLocalizacion_SQL",
        ],
        dropna=False,
    )
    .size()
    .reset_index(name="conteo")
    .sort_values(["IdLocalizacion_SQL", "DEPARTAMENTO_LIMPIO", "MUNICIPIO_LIMPIO"], na_position="last")
)

print("\n=== Resumen de mapeo Localización (agrupado) ===")
print(resumen_loc.to_string(index=False))

sin_loc = resumen_loc[
    resumen_loc["DEPARTAMENTO_LIMPIO"].notna()
    & resumen_loc["MUNICIPIO_LIMPIO"].notna()
    & resumen_loc["IdLocalizacion_SQL"].isna()
]

print("\n=== Combinaciones DEPARTAMENTO/MUNICIPIO sin IdLocalizacion_SQL ===")
print(sin_loc.to_string(index=False))
print("Total sin IdLocalizacion_SQL (sum conteo):", int(sin_loc["conteo"].sum()))

# -------------------------
# 7) Excels de apoyo (mismo estilo)
# -------------------------
resumen_loc.to_excel("Localizacion_mapeo_completo.xlsx", index=False)

sin_loc[[
    "DEPARTAMENTO_RAW",
    "MUNICIPIO_RAW",
    "DEPARTAMENTO_LIMPIO",
    "MUNICIPIO_LIMPIO",
    "conteo",
]].to_excel("pendientes_localizacion.xlsx", index=False)

cols_detalle = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO_RAW",
    "DEPARTAMENTO_LIMPIO",
    "MUNICIPIO_RAW",
    "MUNICIPIO_LIMPIO",
    "Localizacion_NOMBRE_SQL",
    "IdLocalizacion_SQL",
]
cols_detalle = [c for c in cols_detalle if c in df.columns]
df[cols_detalle].to_excel("Localizacion_raw_limpio_sql.xlsx", index=False)

print("\nArchivos creados:")
print(" - Localizacion_mapeo_completo.xlsx")
print(" - pendientes_localizacion.xlsx")
print(" - Localizacion_raw_limpio_sql.xlsx")


=== Posibles duplicados en catálogo Localizacion (DEPTO+MUNI) ===
Empty DataFrame
Columns: [LLAVE_LOC_CAT, num_IdLocalizacion]
Index: []

=== Resumen de mapeo Localización (agrupado) ===
                                        DEPARTAMENTO_RAW               MUNICIPIO_RAW                                      DEPARTAMENTO_LIMPIO            MUNICIPIO_LIMPIO                                                Localizacion_NOMBRE_SQL  IdLocalizacion_SQL  conteo
                                               ANTIOQUIA                    MEDELLIN                                                ANTIOQUIA                    MEDELLIN                                                   MEDELLÍN (ANTIOQUIA)                   1      20
                                               ANTIOQUIA                   ABEJORRAL                                                ANTIOQUIA                   ABEJORRAL                                                  ABEJORRAL (ANTIOQUIA)                   2       9
      

## AREA_REGISTRAL_ORIGINAL

In [19]:
import re
import pandas as pd

# ==========================================================
# AREA REGISTRAL -> AreaRegistral_SQL (NO RECARGA df)
# Corrección: si viene "SIN REGISTRO" (o similares) -> NULL (pd.NA)
# Mantiene diagnósticos + ejemplo en consola + Excel (3 hojas)
# ==========================================================

# 0) Asegurar fila Excel aproximada (sin sobrescribir si ya existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2  # header=1 => datos empiezan en fila 2

# 1) Identificar columna de área registral (tolerante a tildes)
cols_area = [
    c for c in df.columns
    if ("AREA REGISTRAL" in c.upper()) or ("ÁREA REGISTRAL" in c.upper())
]
if not cols_area:
    raise ValueError("No se encontró ninguna columna que contenga 'AREA REGISTRAL' o 'ÁREA REGISTRAL' en el nombre.")
col_area = cols_area[0]

# 2) Copia cruda (siempre apunta a la columna detectada)
df["AreaRegistral_RAW"] = df[col_area]

LIMITE_DECIMAL_18_4 = 99999999999999.9999  # NUMERIC(18,4)

# Valores que deben ir a NULL
VALORES_SIN_DATO = {
    "SIN REGISTRO", "SIN DATO", "NO APLICA", "NA", "N/A", "NULL", "NONE", "-"
}

def limpiar_area_registral(valor):
    """
    Retorna (txt_limpio, valor_sql):
      - txt_limpio: string estandarizado (punto decimal) o None
      - valor_sql: float redondeado a 4 decimales o pd.NA si no es válido / fuera de rango / sin dato
    """
    if pd.isna(valor):
        return (None, pd.NA)

    s_raw = str(valor).replace("\u00A0", " ").strip()
    if s_raw == "":
        return (None, pd.NA)

    # Normalizar para detectar "SIN REGISTRO" aunque venga con espacios/guiones
    s_raw_up = " ".join(s_raw.upper().split())
    if s_raw_up in VALORES_SIN_DATO:
        return (None, pd.NA)

    # Quitar espacios internos para parseo numérico
    s = s_raw.replace(" ", "")

    # Extraer parte numérica inicial
    m = re.match(r"^-?[\d\.,]+", s)
    if not m:
        # Hay texto pero no es número -> inválido (se reporta)
        return (s_raw, pd.NA)

    s_num = m.group(0)

    # Normalizar separadores
    if "," in s_num and "." in s_num:
        s_norm = s_num.replace(".", "").replace(",", ".")
    elif "," in s_num:
        s_norm = s_num.replace(",", ".")
    else:
        s_norm = s_num

    try:
        v = float(s_norm)
    except ValueError:
        return (s_norm, pd.NA)

    v_red = round(v, 4)

    if abs(v_red) > LIMITE_DECIMAL_18_4:
        return (s_norm, pd.NA)

    return (s_norm, v_red)

# 3) Aplicar limpieza (pisando solo estas columnas “de trabajo”)
df[["AreaRegistral_TXT_LIMPIO", "AreaRegistral_SQL"]] = df["AreaRegistral_RAW"].apply(
    lambda v: pd.Series(limpiar_area_registral(v))
)
df["AreaRegistral_SQL"] = pd.to_numeric(df["AreaRegistral_SQL"], errors="coerce")

# 4) Diagnósticos (incluye conteo de “sin dato” textual)
raw_str = df["AreaRegistral_RAW"].astype("string")
mask_con_texto = raw_str.notna() & (raw_str.str.strip() != "")

raw_up = raw_str.fillna("").str.replace("\u00A0", " ", regex=False).str.strip().str.upper()
raw_up = raw_up.str.replace(r"\s+", " ", regex=True)
mask_sin_dato_texto = mask_con_texto & raw_up.isin(VALORES_SIN_DATO)

mask_validos = df["AreaRegistral_SQL"].notna()
mask_invalidos = (mask_con_texto & ~mask_sin_dato_texto) & df["AreaRegistral_SQL"].isna()
mask_negativos = mask_validos & (df["AreaRegistral_SQL"] < 0)

print("=== Resumen AREA REGISTRAL ===")
print("Columna detectada:", col_area)
print("Total registros:", len(df))
print("Con dato en original (no vacío):", int(mask_con_texto.sum()))
print("Sin dato textual (SIN REGISTRO/NA/... -> NULL):", int(mask_sin_dato_texto.sum()))
print("Validos (AreaRegistral_SQL no nulo):", int(mask_validos.sum()))
print("Invalidos (texto pero sin numero valido):", int(mask_invalidos.sum()))
print("Negativos (validos pero < 0):", int(mask_negativos.sum()))

cols_demo = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "AreaRegistral_RAW", "AreaRegistral_TXT_LIMPIO", "AreaRegistral_SQL"
]
cols_demo = [c for c in cols_demo if c in df.columns]

print("\n=== Ejemplo AREA REGISTRAL (primeras 20 con dato) ===")
print(df.loc[mask_con_texto, cols_demo].head(20).to_string(index=False))

# Mensaje de “ya quedó limpio” vs “falta algo”
if int(mask_invalidos.sum()) == 0 and int(mask_negativos.sum()) == 0:
    print("\nOK: AreaRegistral queda limpia para migrar (los 'SIN REGISTRO/NA/...' quedaron como NULL).")
else:
    print("\nPENDIENTE: aún hay registros por revisar (ver Area_invalidos y/o Area_negativos).")

# 5) DataFrames de revisión
area_invalidos = df.loc[mask_invalidos, cols_demo].copy()
area_negativos = df.loc[mask_negativos, cols_demo].copy()

# 6) Excel de revisión
with pd.ExcelWriter("AreaRegistral_revision.xlsx", engine="openpyxl") as writer:
    df.loc[:, cols_demo].to_excel(writer, sheet_name="Area_raw_limpio_sql", index=False)
    area_invalidos.to_excel(writer, sheet_name="Area_invalidos", index=False)
    area_negativos.to_excel(writer, sheet_name="Area_negativos", index=False)

print("\nArchivos creados:")
print(" - AreaRegistral_revision.xlsx (3 hojas: mapeo completo, invalidos, negativos)")


=== Resumen AREA REGISTRAL ===
Columna detectada: ÁREA REGISTRAL
 (0,0000 = Has,M2)
Total registros: 24055
Con dato en original (no vacío): 19939
Sin dato textual (SIN REGISTRO/NA/... -> NULL): 13
Validos (AreaRegistral_SQL no nulo): 19926
Invalidos (texto pero sin numero valido): 0
Negativos (validos pero < 0): 0

=== Ejemplo AREA REGISTRAL (primeras 20 con dato) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO            MUNICIPIO AreaRegistral_RAW AreaRegistral_TXT_LIMPIO  AreaRegistral_SQL
                2  062-37357      BOLIVAR             ZAMBRANO            3.0034                   3.0034             3.0034
                3  062-37359      BOLIVAR             ZAMBRANO            8.1976                   8.1976             8.1976
                4  062-37361      BOLIVAR             ZAMBRANO             50.33                    50.33            50.3300
                5  062-37354      BOLIVAR             ZAMBRANO           400.504                  400.504           400.5040
    

## CÍRCULO REGISTRAL

In [20]:
import pandas as pd

# ==========================================================
# CÍRCULO REGISTRAL -> CirculoRegistral_SQL (NO RECARGA df)
# - NO hace df = pd.read_excel(...)
# - Crea/actualiza:
#   CirculoRegistral_RAW, CirculoRegistral_TXT_LIMPIO, CirculoRegistral_SQL, CirculoRegistral_LEN
# - Diagnósticos + Excel de revisión (3 hojas)
# ==========================================================

# 0) Asegurar fila Excel aproximada (sin sobrescribir si ya existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2  # header=1 => datos empiezan en fila 2

# 1) Localizar la columna original de CÍRCULO REGISTRAL en el Excel
cols_circ = [
    c for c in df.columns
    if ("CÍRCULO REGISTRAL" in c.upper()) or ("CIRCULO REGISTRAL" in c.upper())
]
if not cols_circ:
    raise ValueError("No se encontró la columna de CÍRCULO REGISTRAL en df.")
col_circulo = cols_circ[0]

# Posición de la columna original para insertar las nuevas al lado
idx = df.columns.get_loc(col_circulo)

# 2) Validar que exista normalizar_texto (la que vienes usando)
if "normalizar_texto" not in globals():
    raise ValueError("No existe la función normalizar_texto() en esta sesión. Ejecuta primero el bloque donde la defines.")

# 3) Crear columnas estándar: RAW / TXT_LIMPIO / SQL (sin borrar original)
#    OJO: insert solo si no existen (para NO reventar por columnas duplicadas)
if "CirculoRegistral_RAW" not in df.columns:
    df.insert(idx + 1, "CirculoRegistral_RAW", df[col_circulo])
else:
    df["CirculoRegistral_RAW"] = df[col_circulo]

if "CirculoRegistral_TXT_LIMPIO" not in df.columns:
    df.insert(idx + 2, "CirculoRegistral_TXT_LIMPIO", pd.NA)

if "CirculoRegistral_SQL" not in df.columns:
    df.insert(idx + 3, "CirculoRegistral_SQL", pd.NA)

# 4) Limpieza de texto (usa normalizar_texto)
def limpiar_circulo(valor):
    if pd.isna(valor):
        return None
    s = str(valor).replace("\u00A0", " ").strip()
    if s == "":
        return None
    return normalizar_texto(s)

df["CirculoRegistral_TXT_LIMPIO"] = df["CirculoRegistral_RAW"].apply(limpiar_circulo)

# 5) Valor definitivo para migrar a SQL (varchar(100) NULL)
df["CirculoRegistral_SQL"] = df["CirculoRegistral_TXT_LIMPIO"]

# 6) Validaciones: nulos, inválidos, longitud > 100
df["CirculoRegistral_LEN"] = df["CirculoRegistral_SQL"].astype("string").str.len()

mask_con_dato = df["CirculoRegistral_RAW"].notna() & (df["CirculoRegistral_RAW"].astype("string").str.strip() != "")
mask_validos = df["CirculoRegistral_SQL"].notna()
mask_invalidos = mask_con_dato & df["CirculoRegistral_SQL"].isna()

mask_fuera_long = df["CirculoRegistral_SQL"].notna() & (df["CirculoRegistral_LEN"] > 100)

print("=== RESUMEN CirculoRegistral ===")
print("Columna detectada:", col_circulo)
print(f"Total registros: {len(df)}")
print(f"Con dato en original (no vacío): {int(mask_con_dato.sum())}")
print(f"Válidos (CirculoRegistral_SQL no nulo): {int(mask_validos.sum())}")
print(f"Inválidos (texto pero sin dato válido): {int(mask_invalidos.sum())}")
print(f"Fuera de longitud (>100 caracteres): {int(mask_fuera_long.sum())}")

print("\n=== Top 30 valores distintos (CirculoRegistral_SQL) ===")
print(df["CirculoRegistral_SQL"].value_counts(dropna=False).head(30))

print("\n=== Ejemplo de CirculoRegistral (primeras 20 con dato) ===")
cols_demo = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "CirculoRegistral_RAW", "CirculoRegistral_TXT_LIMPIO", "CirculoRegistral_SQL"
]
cols_demo = [c for c in cols_demo if c in df.columns]
print(df.loc[mask_con_dato, cols_demo].head(20).to_string(index=False))

# 7) DataFrames de apoyo: inválidos y fuera de longitud
df_invalidos = df.loc[mask_invalidos, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "CirculoRegistral_RAW", "CirculoRegistral_TXT_LIMPIO"
]].copy()

df_fuera_long = df.loc[mask_fuera_long, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "CirculoRegistral_RAW", "CirculoRegistral_TXT_LIMPIO",
    "CirculoRegistral_SQL", "CirculoRegistral_LEN"
]].copy()

# 8) Excel de revisión (3 hojas)
with pd.ExcelWriter("CirculoRegistral_raw_limpio_sql.xlsx", engine="openpyxl") as writer:
    df[[
        "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
        col_circulo,
        "CirculoRegistral_RAW",
        "CirculoRegistral_TXT_LIMPIO",
        "CirculoRegistral_SQL",
        "CirculoRegistral_LEN",
    ]].to_excel(writer, index=False, sheet_name="Circulo_raw_limpio_sql")

    df_invalidos.to_excel(writer, index=False, sheet_name="Circulo_invalidos")
    df_fuera_long.to_excel(writer, index=False, sheet_name="Circulo_fuera_long")

print("\nArchivos creados:")
print(" - CirculoRegistral_raw_limpio_sql.xlsx (3 hojas: mapeo completo, invalidos, fuera_long)")


=== RESUMEN CirculoRegistral ===
Columna detectada: CÍRCULO REGISTRAL
Total registros: 24055
Con dato en original (no vacío): 5461
Válidos (CirculoRegistral_SQL no nulo): 5461
Inválidos (texto pero sin dato válido): 0
Fuera de longitud (>100 caracteres): 0

=== Top 30 valores distintos (CirculoRegistral_SQL) ===
CirculoRegistral_SQL
None                    18594
VALLEDUPAR                292
EL CARMEN DE BOLIVAR      262
SABANALARGA               250
PLATO                     220
CHIMICHAGUA               212
MONTERIA                  197
CARTAGENA                 159
MONTELIBANO               153
SAN JUAN DEL CESAR        145
RIOHACHA                  144
BARRANCABERMEJA           140
SINCELEJO                 132
SAHAGUN                   106
CHAPARRAL                  94
MAGANGUE                   91
CIENAGA                    87
AGUACHICA                  87
COROZAL                    83
LORICA                     79
POPAYAN                    78
VELEZ                      76
SIMIT

# ÁREA CALCULADA


In [21]:
import pandas as pd
import numpy as np
import re

# ==========================================================
# AREA CALCULADA -> AreaCalculada_SQL (NO RECARGA df)
# - NO hace df = pd.read_excel(...)
# - Crea/actualiza: AreaCalculada_RAW, AreaCalculada_TXT_LIMPIO, AreaCalculada_SQL
# - Convierte "SIN REGISTRO/NA/..." -> NULL
# - Diagnósticos + Excel de revisión (3 hojas)
# ==========================================================

# 0) Asegurar fila Excel aproximada (sin sobrescribir si ya existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2  # header=1 => datos empiezan en fila 2

# 1) Localizar columna de ÁREA CALCULADA
cols_area_calc = [
    c for c in df.columns
    if ("ÁREA CALCULADA" in c.upper()) or ("AREA CALCULADA" in c.upper())
]
if not cols_area_calc:
    raise ValueError("No se encontró la columna de ÁREA CALCULADA en df.")

col_area_calc = cols_area_calc[0]
idx = df.columns.get_loc(col_area_calc)

# 2) Crear columnas estándar RAW / TXT_LIMPIO / SQL junto a la original
if "AreaCalculada_RAW" not in df.columns:
    df.insert(idx + 1, "AreaCalculada_RAW", df[col_area_calc])
else:
    df["AreaCalculada_RAW"] = df[col_area_calc]

if "AreaCalculada_TXT_LIMPIO" not in df.columns:
    df.insert(idx + 2, "AreaCalculada_TXT_LIMPIO", pd.NA)

if "AreaCalculada_SQL" not in df.columns:
    df.insert(idx + 3, "AreaCalculada_SQL", pd.NA)

# 3) Parseo: texto → (texto_normalizado, número con 4 decimales)
LIMITE_DECIMAL_18_4 = 99999999999999.9999  # NUMERIC(18,4)
VALORES_SIN_DATO = {
    "NA", "N/A", "NO APLICA", "SIN DATO",
    "NO REGISTRA", "NO REGISTRA.", "SIN REGISTRO"
}

def parsear_area_calculada(valor):
    if pd.isna(valor):
        return (None, pd.NA)

    s_raw = str(valor).replace("\u00A0", " ").strip()
    if s_raw == "":
        return (None, pd.NA)

    # "sin dato" -> NULL
    if s_raw.upper() in VALORES_SIN_DATO:
        return (None, pd.NA)

    # quitar espacios internos
    s = s_raw.replace(" ", "")

    # quedarnos solo con signo, dígitos, puntos y comas
    m = re.match(r"^-?[\d\.,]+", s)
    if not m:
        return (s_raw, pd.NA)

    s_num = m.group(0)

    # normalizar separadores
    if "." in s_num and "," in s_num:
        if s_num.rfind(",") > s_num.rfind("."):
            s_std = s_num.replace(".", "").replace(",", ".")  # 1.234,56 -> 1234.56
        else:
            s_std = s_num.replace(",", "")                    # 1,234.56 -> 1234.56
    elif "," in s_num and "." not in s_num:
        s_std = s_num.replace(",", ".")                       # 123,45 -> 123.45
    else:
        s_std = s_num                                         # 123.45 o 123

    try:
        num = float(s_std)
    except ValueError:
        return (s_std, pd.NA)

    num = round(num, 4)

    # fuera de rango -> NULL (para no migrar basura)
    if abs(num) > LIMITE_DECIMAL_18_4:
        return (s_std, pd.NA)

    return (s_std, num)

df[["AreaCalculada_TXT_LIMPIO", "AreaCalculada_SQL"]] = df["AreaCalculada_RAW"].apply(
    lambda v: pd.Series(parsear_area_calculada(v))
)

df["AreaCalculada_SQL"] = pd.to_numeric(df["AreaCalculada_SQL"], errors="coerce")

# 4) Diagnósticos
mask_con_texto = df["AreaCalculada_RAW"].notna() & (df["AreaCalculada_RAW"].astype("string").str.strip() != "")
mask_sin_dato_txt = mask_con_texto & df["AreaCalculada_RAW"].astype("string").str.strip().str.upper().isin(VALORES_SIN_DATO)

mask_validos = df["AreaCalculada_SQL"].notna()
mask_invalidos = mask_con_texto & (~mask_sin_dato_txt) & df["AreaCalculada_SQL"].isna()

mask_negativos = mask_validos & (df["AreaCalculada_SQL"] < 0)
mask_fuera_rango = df["AreaCalculada_SQL"].notna() & (df["AreaCalculada_SQL"].abs() > LIMITE_DECIMAL_18_4)

print("=== RESUMEN Área Calculada ===")
print("Columna detectada:", col_area_calc)
print(f"Total registros: {len(df)}")
print(f"Con dato en original (no vacío): {int(mask_con_texto.sum())}")
print(f"Sin dato textual (SIN REGISTRO/NA/... -> NULL): {int(mask_sin_dato_txt.sum())}")
print(f"Válidos (AreaCalculada_SQL no nulo): {int(mask_validos.sum())}")
print(f"Inválidos (texto pero sin número válido): {int(mask_invalidos.sum())}")
print(f"Valores negativos (válidos pero <0): {int(mask_negativos.sum())}")
print(f"Fuera de rango DECIMAL(18,4): {int(mask_fuera_rango.sum())}")

print("\n=== Ejemplo de ÁREA CALCULADA (primeras 20 con dato) ===")
cols_demo = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "AreaCalculada_RAW", "AreaCalculada_TXT_LIMPIO", "AreaCalculada_SQL"
]
cols_demo = [c for c in cols_demo if c in df.columns]
print(df.loc[mask_con_texto, cols_demo].head(20).to_string(index=False))

# 5) DataFrames para revisión
df_invalidos = df.loc[mask_invalidos, cols_demo].copy()
df_prob_valor = df.loc[mask_negativos | mask_fuera_rango, cols_demo].copy()

# 6) Exportar a Excel
with pd.ExcelWriter("AreaCalculada_revision.xlsx", engine="openpyxl") as writer:
    df[[
        "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
        col_area_calc,
        "AreaCalculada_RAW",
        "AreaCalculada_TXT_LIMPIO",
        "AreaCalculada_SQL",
    ]].to_excel(writer, index=False, sheet_name="AreaCalculada_mapeo_completo")

    df_invalidos.to_excel(writer, index=False, sheet_name="AreaCalculada_invalidos")
    df_prob_valor.to_excel(writer, index=False, sheet_name="AreaCalculada_negativos_fuera_rango")

print("\nArchivos creados:")
print(" - AreaCalculada_revision.xlsx (3 hojas: mapeo completo, invalidos, negativos_fuera_rango)")


=== RESUMEN Área Calculada ===
Columna detectada: ÁREA CALCULADA
Total registros: 24055
Con dato en original (no vacío): 24040
Sin dato textual (SIN REGISTRO/NA/... -> NULL): 0
Válidos (AreaCalculada_SQL no nulo): 24040
Inválidos (texto pero sin número válido): 0
Valores negativos (válidos pero <0): 0
Fuera de rango DECIMAL(18,4): 0

=== Ejemplo de ÁREA CALCULADA (primeras 20 con dato) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO            MUNICIPIO AreaCalculada_RAW AreaCalculada_TXT_LIMPIO  AreaCalculada_SQL
                2  062-37357      BOLIVAR             ZAMBRANO          2.906059                 2.906059             2.9061
                3  062-37359      BOLIVAR             ZAMBRANO          8.597642                 8.597642             8.5976
                4  062-37361      BOLIVAR             ZAMBRANO         49.576705            49.5767053911            49.5767
                5  062-37354      BOLIVAR             ZAMBRANO        878.681979               878.681979  

c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")



Archivos creados:
 - AreaCalculada_revision.xlsx (3 hojas: mapeo completo, invalidos, negativos_fuera_rango)


*excel hasta aca*

In [22]:
import pandas as pd

# =========================================================
# EXPORTAR EXCEL DE AVANCE (SIN RECARGAR df)
# - NO hace df = pd.read_excel(...)
# - Crea un Excel con las columnas SQL que YA EXISTAN
# - NO revienta por nulos: solo genera reportes de pendientes
# =========================================================

# 1) Trazabilidad base
if "Fila_Excel" not in df.columns:
    df["Fila_Excel"] = df.index + 2

# 2) Asegurar IdPostulacion_SQL desde ID_SQL si aplica (sin recargar nada)
if "IdPostulacion_SQL" not in df.columns and "ID_SQL" in df.columns:
    df["IdPostulacion_SQL"] = df["ID_SQL"].astype("string")

# 3) Mapeo: columna en df -> columna destino en Excel de avance
# (solo se agrega si existe en df)
mapeo = {
    "IdPostulacion_SQL": "IdPostulacion",
    "FMI_SQL": "FMI",
    "NumeroExpediente_SQL": "NumeroExpediente",
    "IdFuenteProceso_SQL": "IdFuenteProceso",
    "IdTipoProceso_SQL": "IdTipoProceso",
    "IdEtapaProcesal_SQL": "IdEtapaProcesal",
    "Dependencia_SQL": "Dependencia",
    "RadicadoOrfeo_SQL": "RadicadoOrfeo",
    "IdLocalizacion_SQL": "IdLocalizacion",
    "AreaRegistral_SQL": "AreaRegistral",
    "CirculoRegistral_SQL": "CirculoRegistral",
    "AreaCalculada_SQL": "AreaCalculada",
}

# 4) Construir df_avance con lo que haya
df_avance = pd.DataFrame(index=df.index)
df_avance["Fila_Excel"] = df["Fila_Excel"]

cols_agregadas = []
cols_faltan = []

for col_origen, col_destino in mapeo.items():
    if col_origen in df.columns:
        # mantener Int64 donde corresponda (si ya viene Int64, perfecto)
        df_avance[col_destino] = df[col_origen]
        cols_agregadas.append(col_origen)
    else:
        cols_faltan.append(col_origen)

print("=== Columnas SQL AGREGADAS al avance ===")
print(cols_agregadas)

print("\n=== Columnas SQL AÚN NO EXISTEN en df (pendientes) ===")
print(cols_faltan)

# 5) Validaciones: obligatorias vs pendientes (SIN bloquear export)
# Ajusta estas si cambias la regla.
obligatorias = ["IdPostulacion", "FMI"]  # RegistroProceso NOT NULL
pendientes_fk = ["IdFuenteProceso", "IdTipoProceso", "IdEtapaProcesal"]  # si aún te faltan mapeos o hay nulos

# 5.1 Obligatorias: se reportan, pero no se frena (tú decides si frenar)
for col in obligatorias:
    if col in df_avance.columns:
        mask = df_avance[col].isna() | (df_avance[col].astype("string").str.strip() == "")
        if mask.any():
            df_avance.loc[mask, ["Fila_Excel", col]].to_excel(f"PENDIENTE_{col}_NULL.xlsx", index=False)
            print(f"ALERTA: {col} tiene nulos/vacíos. Archivo: PENDIENTE_{col}_NULL.xlsx")

# 5.2 FKs pendientes: se reportan si están y tienen nulos
for col in pendientes_fk:
    if col in df_avance.columns:
        mask = df_avance[col].isna()
        if mask.any():
            out_cols = [c for c in ["Fila_Excel", "FMI", col] if c in df_avance.columns]
            df_avance.loc[mask, out_cols].to_excel(f"PENDIENTE_{col}_NULL.xlsx", index=False)
            print(f"Pendiente: {col} tiene nulos. Archivo: PENDIENTE_{col}_NULL.xlsx")

# 6) Vista rápida
print("\n=== DF AVANCE (primeras 15 filas) ===")
print(df_avance.head(15).to_string(index=False))

print("\n=== DF AVANCE (últimas 15 filas) ===")
print(df_avance.tail(15).to_string(index=False))

# 7) Exportar Excel de avance
# - con trazabilidad
df_avance.to_excel("AVANCE_migracion_hasta_ahora_con_trazabilidad.xlsx", index=False)
# - sin trazabilidad (si lo necesitas para cargar)
df_avance.drop(columns=["Fila_Excel"]).to_excel("AVANCE_migracion_hasta_ahora.xlsx", index=False)

print("\nArchivos creados:")
print(" - AVANCE_migracion_hasta_ahora.xlsx")
print(" - AVANCE_migracion_hasta_ahora_con_trazabilidad.xlsx")
print(" - (opcionales) PENDIENTE_*_NULL.xlsx si hubo pendientes")


=== Columnas SQL AGREGADAS al avance ===
['IdPostulacion_SQL', 'FMI_SQL', 'NumeroExpediente_SQL', 'IdFuenteProceso_SQL', 'IdTipoProceso_SQL', 'IdEtapaProcesal_SQL', 'Dependencia_SQL', 'RadicadoOrfeo_SQL', 'IdLocalizacion_SQL', 'AreaRegistral_SQL', 'CirculoRegistral_SQL', 'AreaCalculada_SQL']

=== Columnas SQL AÚN NO EXISTEN en df (pendientes) ===
[]
Pendiente: IdFuenteProceso tiene nulos. Archivo: PENDIENTE_IdFuenteProceso_NULL.xlsx
Pendiente: IdTipoProceso tiene nulos. Archivo: PENDIENTE_IdTipoProceso_NULL.xlsx
Pendiente: IdEtapaProcesal tiene nulos. Archivo: PENDIENTE_IdEtapaProcesal_NULL.xlsx

=== DF AVANCE (primeras 15 filas) ===
 Fila_Excel IdPostulacion        FMI NumeroExpediente  IdFuenteProceso  IdTipoProceso  IdEtapaProcesal                                         Dependencia       RadicadoOrfeo  IdLocalizacion  AreaRegistral CirculoRegistral  AreaCalculada
          2       ID8099B  062-37357             None               49           <NA>             <NA>                  

## "PERSONA TITULAR
(Natural / Jurídica)"


In [23]:
import pandas as pd

# ======================================================
# PERSONA TITULAR → TipoPersonaTitular_SQL (Natural / Jurídica / NULL)
# (NO RECARGA df)
# ======================================================

# 1) Detectar columna de PERSONA TITULAR
candidatas = [c for c in df.columns if "PERSONA" in c.upper() and "TITULAR" in c.upper()]

print("Columnas candidatas para PERSONA TITULAR:", candidatas)
if not candidatas:
    raise ValueError("No se encontró columna para PERSONA TITULAR en el DataFrame.")

col_persona = candidatas[0]
print("Usando la columna:", col_persona)

# 2) Asegurar columna con número de fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2  # encabezado en fila 2

# 3) Crear/actualizar columnas RAW y LIMPIO
df["PersonaTitular_RAW"] = df[col_persona]

df["PERSONA_TITULAR_TXT_LIMPIO"] = df[col_persona].apply(
    lambda x: normalizar_texto(x) if pd.notna(x) else None
)

# 4) Función de mapeo final a texto para SQL
def mapear_tipo_persona(valor_limpio):
    if valor_limpio is None or valor_limpio == "":
        return None

    naturales = {
        "NATURAL",
        "PERSONA NATURAL",
        "P NATURAL",
        "P.NATURAL",
        "P NAT",
    }
    juridicas = {
        "JURIDICA",
        "JURÍDICA",
        "PERSONA JURIDICA",
        "PERSONA JURÍDICA",
        "P JURIDICA",
        "P.JURIDICA",
        "P JUR",
    }
    sin_dato = {
        "NA",
        "N/A",
        "NO APLICA",
        "NO REGISTRA",
        "SIN INFORMACION",
        "SIN INFORMACIÓN",
        "NINGUNO",
        "NINGUNA",
        "0",
    }

    if valor_limpio in naturales:
        return "Natural"
    if valor_limpio in juridicas:
        return "Jurídica"
    if valor_limpio in sin_dato:
        return None

    # Cualquier otro valor lo dejamos en NULL para revisión
    return None

# 5) Columna final para migrar a SQL (EstudioTerreno.TipoPersonaTitular)
df["TipoPersonaTitular_SQL"] = df["PERSONA_TITULAR_TXT_LIMPIO"].apply(mapear_tipo_persona)

# 6) Resúmenes en consola
print("\n=== Distribución PERSONA_TITULAR_TXT_LIMPIO ===")
print(df["PERSONA_TITULAR_TXT_LIMPIO"].value_counts(dropna=False))

res_mapeo = (
    df
      .groupby(
          ["PersonaTitular_RAW", "PERSONA_TITULAR_TXT_LIMPIO", "TipoPersonaTitular_SQL"],
          dropna=False
      )
      .size()
      .reset_index(name="conteo")
)

print("\n=== Resumen de mapeo a TipoPersonaTitular_SQL ===")
print(res_mapeo.to_string(index=False))

# 7) Casos sin clasificar (texto limpio pero sin tipo asignado)
mask_sin_clasificar = (
    df["PERSONA_TITULAR_TXT_LIMPIO"].notna()
    & df["TipoPersonaTitular_SQL"].isna()
)

sin_clasificar_distintos = (
    df.loc[mask_sin_clasificar, ["PersonaTitular_RAW", "PERSONA_TITULAR_TXT_LIMPIO"]]
      .drop_duplicates()
      .reset_index(drop=True)
)

print("\n=== Valores PERSONA TITULAR sin clasificar (distintos) ===")
print(sin_clasificar_distintos.to_string(index=False))
print(
    "\nTotal filas sin TipoPersonaTitular_SQL (con texto limpio):",
    int(mask_sin_clasificar.sum())
)

# 8) Detalle de filas sin clasificar (con ubicación)
sin_clasificar_detalle = df.loc[
    mask_sin_clasificar,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "PersonaTitular_RAW",
        "PERSONA_TITULAR_TXT_LIMPIO",
    ],
].copy().sort_values("Fila_Excel_aprox")

print("\n=== Ejemplo de filas sin clasificar (primeras 20) ===")
print(sin_clasificar_detalle.head(20).to_string(index=False))

# 9) Reordenar columnas: dejar RAW, LIMPIO y SQL al lado de la original (SIN romper df)
#    FIX: no usar df["TipoPersonaTitular_SQL"] después de pop sin guardarla antes.
tipo_sql = df["TipoPersonaTitular_SQL"].copy()

for c in ["PersonaTitular_RAW", "PERSONA_TITULAR_TXT_LIMPIO", "TipoPersonaTitular_SQL"]:
    if c in df.columns:
        df.pop(c)

pos = df.columns.get_loc(col_persona) + 1
df.insert(pos,     "PersonaTitular_RAW", df[col_persona])
df.insert(pos + 1, "PERSONA_TITULAR_TXT_LIMPIO",
          df[col_persona].apply(lambda x: normalizar_texto(x) if pd.notna(x) else None))
df.insert(pos + 2, "TipoPersonaTitular_SQL", tipo_sql)

# 10) Excels de salida
# 10.1) Mapeo completo (agrupado)
res_mapeo.to_excel("PersonaTitular_mapeo_completo.xlsx", index=False)

# 10.2) Todas las filas con valores crudos / limpios / SQL
export_completo = df[[
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_persona,                 # original tal cual
    "PersonaTitular_RAW",
    "PERSONA_TITULAR_TXT_LIMPIO",
    "TipoPersonaTitular_SQL",
]].copy()

export_completo.to_excel("PersonaTitular_raw_limpio_sql.xlsx", index=False)

# 10.3) Solo filas sin clasificar (para revisión manual)
sin_clasificar_detalle.to_excel("PersonaTitular_sin_clasificar.xlsx", index=False)

print("\nArchivos creados:")
print(" - PersonaTitular_mapeo_completo.xlsx")
print(" - PersonaTitular_raw_limpio_sql.xlsx")
print(" - PersonaTitular_sin_clasificar.xlsx")


Columnas candidatas para PERSONA TITULAR: ['PERSONA TITULAR\n(Natural / Jurídica)']
Usando la columna: PERSONA TITULAR
(Natural / Jurídica)

=== Distribución PERSONA_TITULAR_TXT_LIMPIO ===
PERSONA_TITULAR_TXT_LIMPIO
None        16121
NATURAL      6225
JURIDICA     1709
Name: count, dtype: int64

=== Resumen de mapeo a TipoPersonaTitular_SQL ===
PersonaTitular_RAW PERSONA_TITULAR_TXT_LIMPIO TipoPersonaTitular_SQL  conteo
          JURÍDICA                   JURIDICA               Jurídica      11
          Jurídica                   JURIDICA               Jurídica    1697
           NATURAL                    NATURAL                Natural     223
           Natural                    NATURAL                Natural    5993
          jurídica                   JURIDICA               Jurídica       1
           natural                    NATURAL                Natural       9
               NaN                        NaN                    NaN   16121

=== Valores PERSONA TITULAR sin clas

## NOMBRE / APELLIDOS

*NATURAL*

In [24]:
import re
import os
import pandas as pd

# ============================
# NATURALES (VARCHAR 450): separar Nombre/Apellido + revisión
# (NO RECARGA df, y DEJA columnas en df principal)
# ============================

MAX_NOMBRE = 450
MAX_APELLIDO = 450
MAX_TOTAL = MAX_NOMBRE + MAX_APELLIDO  # 900

# 1) Detectar columna PROPIETARIO
cols_prop_candidatas = [c for c in df.columns if "PROPIETARIO" in c.upper()]
print("Columnas candidatas para PROPIETARIO:", cols_prop_candidatas)
if not cols_prop_candidatas:
    raise ValueError("No se encontró columna que contenga 'PROPIETARIO' en el nombre.")

col_prop = cols_prop_candidatas[0]
print("Columna usada:", col_prop)

# 2) Asegurar Fila_Excel_aprox
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 3) Validar que exista TipoPersonaTitular_SQL (porque dependes de eso)
if "TipoPersonaTitular_SQL" not in df.columns:
    raise ValueError("Falta la columna 'TipoPersonaTitular_SQL'. Ejecuta primero el bloque de Persona Titular.")

# 4) Normalización (usa tu normalizar_texto ya existente)
SIN_DATO_NOMBRE = {
    "NA","N/A","NO APLICA","SIN DATO",
    "SIN INFORMACION","SIN INFORMACIÓN",
    "NO REGISTRA","NINGUNO","NINGUNA","0",
}

def normalizar_nombre_propietario(x):
    if pd.isna(x):
        return None
    s = str(x).replace("\u00A0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    if s == "":
        return None

    # usa tu función global normalizar_texto
    s_norm = normalizar_texto(s)

    if s_norm in SIN_DATO_NOMBRE:
        return None
    return s_norm

def contar_palabras(texto):
    if not isinstance(texto, str):
        return 0
    return len([p for p in texto.strip().split(" ") if p])

def partir_nombre_apellido_natural(nombre_limpio):
    if nombre_limpio is None:
        return (None, None)

    palabras = [p for p in str(nombre_limpio).split(" ") if p]
    n = len(palabras)

    if n == 0:
        return (None, None)
    elif n == 1:
        return (palabras[0], None)
    elif n == 2:
        return (" ".join(palabras[:2]), None)
    elif n == 3:
        return (" ".join(palabras[:2]), palabras[2])
    elif n == 4:
        return (" ".join(palabras[:2]), " ".join(palabras[2:]))
    else:
        mitad = (n + 1) // 2  # ceil(n/2)
        return (" ".join(palabras[:mitad]), " ".join(palabras[mitad:]))

# 5) Crear columnas en df (si no existen) SIN pisar otras lógicas
for c in [
    "NombrePropietario_RAW",
    "NombrePropietario_TXT_LIMPIO",
    "NumPalabras_Nombre",
    "Propietario_LEN_TOTAL",
    "NombrePropietario_SQL",
    "ApellidoPropietario_SQL",
    "NombrePropietario_LEN_SQL",
    "ApellidoPropietario_LEN_SQL",
]:
    if c not in df.columns:
        df[c] = pd.NA

# 6) Filtrar SOLO NATURALES (pero escribiendo en df)
mask_nat = df["TipoPersonaTitular_SQL"].eq("Natural")

# RAW + LIMPIO + métricas (solo naturales)
df.loc[mask_nat, "NombrePropietario_RAW"] = df.loc[mask_nat, col_prop]
df.loc[mask_nat, "NombrePropietario_TXT_LIMPIO"] = df.loc[mask_nat, "NombrePropietario_RAW"].apply(normalizar_nombre_propietario)
df.loc[mask_nat, "NumPalabras_Nombre"] = df.loc[mask_nat, "NombrePropietario_TXT_LIMPIO"].apply(contar_palabras)
df.loc[mask_nat, "Propietario_LEN_TOTAL"] = df.loc[mask_nat, "NombrePropietario_TXT_LIMPIO"].astype("string").str.len()

# Partición (solo naturales) -> escribir en df
partes = df.loc[mask_nat, "NombrePropietario_TXT_LIMPIO"].apply(lambda s: pd.Series(partir_nombre_apellido_natural(s)))
df.loc[mask_nat, "NombrePropietario_SQL"] = partes[0].values
df.loc[mask_nat, "ApellidoPropietario_SQL"] = partes[1].values

# Longitudes por campo (VARCHAR(450))
df.loc[mask_nat, "NombrePropietario_LEN_SQL"] = df.loc[mask_nat, "NombrePropietario_SQL"].astype("string").str.len()
df.loc[mask_nat, "ApellidoPropietario_LEN_SQL"] = df.loc[mask_nat, "ApellidoPropietario_SQL"].astype("string").str.len()

# 7) Flags (solo para reporte; no altera datos)
df_nat = df.loc[mask_nat].copy()

df_nat["FLAG_TOTAL_GT900"] = df_nat["Propietario_LEN_TOTAL"].fillna(0).astype(int) > MAX_TOTAL
df_nat["FLAG_NOMBRE_GT450"] = df_nat["NombrePropietario_LEN_SQL"].fillna(0).astype(int) > MAX_NOMBRE
df_nat["FLAG_APELLIDO_GT450"] = df_nat["ApellidoPropietario_LEN_SQL"].fillna(0).astype(int) > MAX_APELLIDO
df_nat["FLAG_ALGUNO_GT450"] = df_nat["FLAG_NOMBRE_GT450"] | df_nat["FLAG_APELLIDO_GT450"]

# 8) Resumen en consola
print("\n=== NATURALES: resumen (VARCHAR(450)) ===")
print("Total Naturales:", int(mask_nat.sum()))
print("Con propietario (TXT_LIMPIO no nulo):", int(df_nat["NombrePropietario_TXT_LIMPIO"].notna().sum()))
print(f"TOTAL > {MAX_TOTAL} (diagnóstico):", int(df_nat["FLAG_TOTAL_GT900"].sum()))
print(f"Alguno > {MAX_NOMBRE} (Nombre o Apellido):", int(df_nat["FLAG_ALGUNO_GT450"].sum()))
print(f" - Nombre > {MAX_NOMBRE}:", int(df_nat["FLAG_NOMBRE_GT450"].sum()))
print(f" - Apellido > {MAX_APELLIDO}:", int(df_nat["FLAG_APELLIDO_GT450"].sum()))

# 9) EJEMPLOS (para referencia)
cols_demo = [
    "Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO",
    col_prop,
    "NombrePropietario_TXT_LIMPIO","NumPalabras_Nombre",
    "NombrePropietario_SQL","ApellidoPropietario_SQL",
    "NombrePropietario_LEN_SQL","ApellidoPropietario_LEN_SQL",
    "Propietario_LEN_TOTAL"
]

print("\n=== Ejemplo NATURALES (primeras 15 con dato) ===")
print(
    df_nat.loc[df_nat["NombrePropietario_TXT_LIMPIO"].notna(), cols_demo]
          .head(15)
          .to_string(index=False)
)

print("\n=== Ejemplo NATURALES que exceden (top 10 por longitud total) ===")
print(
    df_nat.loc[df_nat["FLAG_TOTAL_GT900"] | df_nat["FLAG_ALGUNO_GT450"], cols_demo + ["FLAG_TOTAL_GT900","FLAG_ALGUNO_GT450"]]
          .sort_values(["Propietario_LEN_TOTAL","Fila_Excel_aprox"], ascending=[False, True])
          .head(10)
          .to_string(index=False)
)

# 10) Excel de revisión (SOLO NATURALES)
cols_out = cols_demo + ["FLAG_TOTAL_GT900","FLAG_NOMBRE_GT450","FLAG_APELLIDO_GT450","FLAG_ALGUNO_GT450"]
nat_all = df_nat.loc[:, cols_out].copy()

nat_gt900 = nat_all.loc[nat_all["FLAG_TOTAL_GT900"]].copy().sort_values(
    ["Propietario_LEN_TOTAL", "Fila_Excel_aprox"], ascending=[False, True]
)

nat_excede450 = nat_all.loc[nat_all["FLAG_ALGUNO_GT450"]].copy().sort_values(
    ["Propietario_LEN_TOTAL", "Fila_Excel_aprox"], ascending=[False, True]
)

out_xlsx = "Propietarios_Naturales_revision_450.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    nat_all.to_excel(writer, index=False, sheet_name="Nat_all")
    nat_gt900.to_excel(writer, index=False, sheet_name="Nat_gt900")
    nat_excede450.to_excel(writer, index=False, sheet_name="Nat_excede450")

print("\nArchivo creado:")
print(" -", out_xlsx)
print("Ruta (carpeta actual):", os.getcwd())
print("\nOK: NombrePropietario_SQL y ApellidoPropietario_SQL quedaron guardados en df (solo para Naturales).")


Columnas candidatas para PROPIETARIO: ['NOMBRE DEL (OS) PROPIETARIO (S)']
Columna usada: NOMBRE DEL (OS) PROPIETARIO (S)


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4210443705.py:114: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_nat["FLAG_TOTAL_GT900"] = df_nat["Propietario_LEN_TOTAL"].fillna(0).astype(int) > MAX_TOTAL
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4210443705.py:115: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_nat["FLAG_NOMBRE_GT450"] = df_nat["NombrePropietario_LEN_SQL"].fillna(0).astype(int) > MAX_NOMBRE
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4210443705.py:116: FutureWarning: Downcasting object dtyp


=== NATURALES: resumen (VARCHAR(450)) ===
Total Naturales: 6225
Con propietario (TXT_LIMPIO no nulo): 6180
TOTAL > 900 (diagnóstico): 0
Alguno > 450 (Nombre o Apellido): 0
 - Nombre > 450: 0
 - Apellido > 450: 0

=== Ejemplo NATURALES (primeras 15 con dato) ===
 Fila_Excel_aprox       FMI DEPARTAMENTO           MUNICIPIO                                                                                                                                                                                                                                    NOMBRE DEL (OS) PROPIETARIO (S)                                                                                                                                                                                                                              NombrePropietario_TXT_LIMPIO NumPalabras_Nombre                                                                                                         NombrePropietario_SQL                       

*Juridico*

In [25]:
import re
import os
import pandas as pd

# ============================
# JURÍDICAS (VARCHAR 450): todo a Nombre, Apellido NULL + revisar longitudes
# (NO RECARGA df, y DEJA columnas en df principal)
# ============================

MAX_NOMBRE = 450
MAX_APELLIDO = 450
MAX_TOTAL = MAX_NOMBRE + MAX_APELLIDO  # 900 (diagnóstico)

# 1) Detectar columna PROPIETARIO
cols_prop_candidatas = [c for c in df.columns if "PROPIETARIO" in c.upper()]
print("Columnas candidatas para PROPIETARIO:", cols_prop_candidatas)
if not cols_prop_candidatas:
    raise ValueError("No se encontró columna que contenga 'PROPIETARIO' en el nombre.")

col_prop = cols_prop_candidatas[0]
print("Columna usada:", col_prop)

# 2) Asegurar Fila_Excel_aprox
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 3) Validar que exista TipoPersonaTitular_SQL
if "TipoPersonaTitular_SQL" not in df.columns:
    raise ValueError("Falta la columna 'TipoPersonaTitular_SQL'. Ejecuta primero el bloque de Persona Titular.")

# 4) Normalización (usa tu normalizar_texto ya existente)
SIN_DATO_NOMBRE = {
    "NA","N/A","NO APLICA","SIN DATO",
    "SIN INFORMACION","SIN INFORMACIÓN",
    "NO REGISTRA","NINGUNO","NINGUNA","0",
}

def normalizar_nombre_propietario(x):
    if pd.isna(x):
        return None
    s = str(x).replace("\u00A0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    if s == "":
        return None

    # usa tu función global normalizar_texto
    s_norm = normalizar_texto(s)

    if s_norm in SIN_DATO_NOMBRE:
        return None
    return s_norm

# 5) Crear columnas en df si no existen (para no reventar)
for c in [
    "NombrePropietario_RAW",
    "NombrePropietario_TXT_LIMPIO",
    "Propietario_LEN_TOTAL",
    "NombrePropietario_SQL",
    "ApellidoPropietario_SQL",
    "NombrePropietario_LEN_SQL",
    "ApellidoPropietario_LEN_SQL",
]:
    if c not in df.columns:
        df[c] = pd.NA

# Flags (si no existen)
for c in ["FLAG_NOMBRE_GT450", "FLAG_TOTAL_GT900"]:
    if c not in df.columns:
        df[c] = False

# 6) Filtrar SOLO JURÍDICAS
mask_jur = df["TipoPersonaTitular_SQL"].eq("Jurídica")

# 7) RAW + LIMPIO (directo en df)
df.loc[mask_jur, "NombrePropietario_RAW"] = df.loc[mask_jur, col_prop]
df.loc[mask_jur, "NombrePropietario_TXT_LIMPIO"] = df.loc[mask_jur, "NombrePropietario_RAW"].apply(normalizar_nombre_propietario)

# 8) Reglas Jurídica:
# - Todo va a NombrePropietario_SQL
# - ApellidoPropietario_SQL = NULL
df.loc[mask_jur, "NombrePropietario_SQL"] = df.loc[mask_jur, "NombrePropietario_TXT_LIMPIO"]
df.loc[mask_jur, "ApellidoPropietario_SQL"] = pd.NA

# 9) Longitudes
df.loc[mask_jur, "NombrePropietario_LEN_SQL"] = df.loc[mask_jur, "NombrePropietario_SQL"].astype("string").str.len()
df.loc[mask_jur, "ApellidoPropietario_LEN_SQL"] = pd.NA  # siempre NULL
df.loc[mask_jur, "Propietario_LEN_TOTAL"] = df.loc[mask_jur, "NombrePropietario_TXT_LIMPIO"].astype("string").str.len()

# 10) Flags
df.loc[mask_jur, "FLAG_NOMBRE_GT450"] = (
    df.loc[mask_jur, "NombrePropietario_LEN_SQL"].fillna(0).astype(int) > MAX_NOMBRE
)
df.loc[mask_jur, "FLAG_TOTAL_GT900"] = (
    df.loc[mask_jur, "Propietario_LEN_TOTAL"].fillna(0).astype(int) > MAX_TOTAL
)

# 11) Vista para reportes/Excel (solo Jurídicas, sin afectar df principal)
df_jur = df.loc[mask_jur].copy()

print("\n=== JURÍDICAS: resumen (VARCHAR(450)) ===")
print("Total Jurídicas:", int(mask_jur.sum()))
print("Con propietario (TXT_LIMPIO no nulo):", int(df_jur["NombrePropietario_TXT_LIMPIO"].notna().sum()))
print(f"Nombre > {MAX_NOMBRE}:", int(df_jur["FLAG_NOMBRE_GT450"].sum()))
print(f"TOTAL > {MAX_TOTAL} (diagnóstico):", int(df_jur["FLAG_TOTAL_GT900"].sum()))

cols_demo = [
    "Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO",
    col_prop,
    "NombrePropietario_TXT_LIMPIO",
    "NombrePropietario_SQL","ApellidoPropietario_SQL",
    "NombrePropietario_LEN_SQL","Propietario_LEN_TOTAL",
    "FLAG_NOMBRE_GT450","FLAG_TOTAL_GT900"
]
cols_demo = [c for c in cols_demo if c in df_jur.columns]

print("\n=== Ejemplo JURÍDICAS (primeras 15 con dato) ===")
print(
    df_jur.loc[df_jur["NombrePropietario_TXT_LIMPIO"].notna(), cols_demo]
          .head(15)
          .to_string(index=False)
)

print("\n=== Ejemplo JURÍDICAS que exceden (top 10 por longitud) ===")
print(
    df_jur.loc[df_jur["FLAG_NOMBRE_GT450"], cols_demo]
          .sort_values(["NombrePropietario_LEN_SQL","Fila_Excel_aprox"], ascending=[False, True])
          .head(10)
          .to_string(index=False)
)

# 12) Excel de revisión (SOLO JURÍDICAS)
jur_all = df_jur.loc[:, cols_demo].copy()
jur_excede450 = jur_all.loc[jur_all["FLAG_NOMBRE_GT450"]].copy().sort_values(
    ["NombrePropietario_LEN_SQL","Fila_Excel_aprox"], ascending=[False, True]
)

out_xlsx = "Propietarios_Juridicas_revision_450.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    jur_all.to_excel(writer, index=False, sheet_name="Jur_all")
    jur_excede450.to_excel(writer, index=False, sheet_name="Jur_gt450")

print("\nArchivo creado:")
print(" -", out_xlsx)
print("Ruta (carpeta actual):", os.getcwd())
print("\nOK: NombrePropietario_SQL y ApellidoPropietario_SQL quedaron guardados en df (solo para Jurídicas).")


Columnas candidatas para PROPIETARIO: ['NOMBRE DEL (OS) PROPIETARIO (S)', 'NombrePropietario_RAW', 'NombrePropietario_TXT_LIMPIO', 'Propietario_LEN_TOTAL', 'NombrePropietario_SQL', 'ApellidoPropietario_SQL', 'NombrePropietario_LEN_SQL', 'ApellidoPropietario_LEN_SQL']
Columna usada: NOMBRE DEL (OS) PROPIETARIO (S)

=== JURÍDICAS: resumen (VARCHAR(450)) ===
Total Jurídicas: 1709
Con propietario (TXT_LIMPIO no nulo): 1707
Nombre > 450: 2
TOTAL > 900 (diagnóstico): 0

=== Ejemplo JURÍDICAS (primeras 15 con dato) ===
 Fila_Excel_aprox       FMI DEPARTAMENTO           MUNICIPIO                            NOMBRE DEL (OS) PROPIETARIO (S)                              NombrePropietario_TXT_LIMPIO                                     NombrePropietario_SQL ApellidoPropietario_SQL NombrePropietario_LEN_SQL Propietario_LEN_TOTAL  FLAG_NOMBRE_GT450  FLAG_TOTAL_GT900
                2 062-37357      BOLIVAR            ZAMBRANO                                       PALMARES DEL RIO SAS                  

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1364926542.py:91: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[mask_jur, "NombrePropietario_LEN_SQL"].fillna(0).astype(int) > MAX_NOMBRE
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1364926542.py:94: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[mask_jur, "Propietario_LEN_TOTAL"].fillna(0).astype(int) > MAX_TOTAL



Archivo creado:
 - Propietarios_Juridicas_revision_450.xlsx
Ruta (carpeta actual): c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica

OK: NombrePropietario_SQL y ApellidoPropietario_SQL quedaron guardados en df (solo para Jurídicas).


*avance del excel*

In [26]:
import pandas as pd

# =========================================================
# EXPORTAR AVANCE A EXCEL (SIN RECARGAR df)
# - Genera un Excel con lo que llevas listo hasta ahora
# - NO revienta por nulos (solo los reporta en una hoja)
# =========================================================

# 0) Trazabilidad
if "Fila_Excel" not in df.columns:
    df["Fila_Excel"] = df.index + 2

if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Helpers: tomar columna si existe, si no crearla en NA
def col_or_na(colname, dtype=None):
    if colname in df.columns:
        s = df[colname]
    else:
        s = pd.Series([pd.NA] * len(df), index=df.index)
    if dtype is not None:
        try:
            s = s.astype(dtype)
        except Exception:
            pass
    return s

# =========================================================
# 2) REGISTROPROCESO (Postulacion.RegistroProceso)
# Destino SQL:
#   IdPostulacion (varchar 30) NOT NULL
#   FMI (varchar 100) NOT NULL
#   NumeroExpediente (varchar 100) NULL
#   IdFuenteProceso (int) NOT NULL
#   IdTipoProceso (int) NOT NULL (si ya lo tienes)
#   IdEtapaProcesal (int) NOT NULL (si ya lo tienes)
#   Dependencia (varchar 10) NULL  [si tu SQL es varchar(10)]
#   RadicadoOrfeo (varchar 50) NULL
# =========================================================

# IdPostulacion: si ya tienes IdPostulacion_SQL úsala; si no, intenta con ID_SQL; si no, deja NA
if "IdPostulacion_SQL" in df.columns:
    idpost = df["IdPostulacion_SQL"].astype("string")
elif "ID_SQL" in df.columns:
    idpost = df["ID_SQL"].astype("string")
else:
    idpost = pd.Series([pd.NA] * len(df), index=df.index, dtype="string")

df_registroproceso = pd.DataFrame({
    "Fila_Excel": df["Fila_Excel"],
    "IdPostulacion": idpost,
    "FMI": col_or_na("FMI_SQL", "string"),
    "NumeroExpediente": col_or_na("NumeroExpediente_SQL", "string"),
    "IdFuenteProceso": col_or_na("IdFuenteProceso_SQL", "Int64"),
    "IdTipoProceso": col_or_na("IdTipoProceso_SQL", "Int64"),
    "IdEtapaProcesal": col_or_na("IdEtapaProcesal_SQL", "Int64"),
    "Dependencia": col_or_na("Dependencia_SQL", "string"),
    "RadicadoOrfeo": col_or_na("RadicadoOrfeo_SQL", "string"),
}, index=df.index)

# =========================================================
# 3) ESTUDIOTERRENO (Postulacion.EstudioTerreno)
# Destino SQL (según tu DDL guardado):
#   IdRegistroProceso (BIGINT) -> se llenará luego al insertar RegistroProceso
#   IdLocalizacion (INT) NOT NULL
#   AreaRegistral (NUMERIC(18,4)) NOT NULL
#   CirculoRegistral (VARCHAR(100)) NULL
#   AreaCalculada (NUMERIC(18,4)) NOT NULL
#   TipoPersonaTitular (VARCHAR(50)) NULL
#   NombrePropietario (VARCHAR(100)) NULL [tu SQL dice 100, ojo si usas 450 en limpieza]
#   ApellidoPropietario (VARCHAR(100)) NULL
#   Identificacion (BIGINT) NULL
#   NaturalezaJuridica (VARCHAR(100)) NULL
#   AcreditacionPropiedad (VARCHAR(100)) NULL
# =========================================================

df_estudioterreno = pd.DataFrame({
    "Fila_Excel": df["Fila_Excel"],
    "FMI": col_or_na("FMI_SQL", "string"),
    "IdLocalizacion": col_or_na("IdLocalizacion_SQL", "Int64"),
    "AreaRegistral": col_or_na("AreaRegistral_SQL"),
    "CirculoRegistral": col_or_na("CirculoRegistral_SQL", "string"),
    "AreaCalculada": col_or_na("AreaCalculada_SQL"),
    "TipoPersonaTitular": col_or_na("TipoPersonaTitular_SQL", "string"),
    "NombrePropietario": col_or_na("NombrePropietario_SQL", "string"),
    "ApellidoPropietario": col_or_na("ApellidoPropietario_SQL", "string"),
    # Si todavía no has hecho estas columnas, quedarán en NA:
    "Identificacion": col_or_na("Identificacion_SQL"),
    "NaturalezaJuridica": col_or_na("NaturalezaJuridica_SQL", "string"),
    "AcreditacionPropiedad": col_or_na("AcreditacionPropiedad_SQL", "string"),
}, index=df.index)

# =========================================================
# 4) DIAGNÓSTICOS (no detiene, solo reporta)
# =========================================================
diag = []

# RegistroProceso - requeridos
diag.append(("RegistroProceso", "IdPostulacion", int(df_registroproceso["IdPostulacion"].isna().sum())))
diag.append(("RegistroProceso", "FMI", int(df_registroproceso["FMI"].isna().sum())))
diag.append(("RegistroProceso", "IdFuenteProceso", int(df_registroproceso["IdFuenteProceso"].isna().sum())))
diag.append(("RegistroProceso", "IdTipoProceso", int(df_registroproceso["IdTipoProceso"].isna().sum())))
diag.append(("RegistroProceso", "IdEtapaProcesal", int(df_registroproceso["IdEtapaProcesal"].isna().sum())))

# EstudioTerreno - requeridos
diag.append(("EstudioTerreno", "IdLocalizacion", int(df_estudioterreno["IdLocalizacion"].isna().sum())))
diag.append(("EstudioTerreno", "AreaRegistral", int(df_estudioterreno["AreaRegistral"].isna().sum())))
diag.append(("EstudioTerreno", "AreaCalculada", int(df_estudioterreno["AreaCalculada"].isna().sum())))

df_diag = pd.DataFrame(diag, columns=["Tabla", "Campo", "Nulos"])

# Detalle de nulos (muestras)
det_reg = df_registroproceso[
    df_registroproceso[["IdPostulacion", "FMI", "IdFuenteProceso"]].isna().any(axis=1)
][["Fila_Excel", "IdPostulacion", "FMI", "IdFuenteProceso", "IdTipoProceso", "IdEtapaProcesal"]].copy()

det_est = df_estudioterreno[
    df_estudioterreno[["IdLocalizacion", "AreaRegistral", "AreaCalculada"]].isna().any(axis=1)
][["Fila_Excel", "FMI", "IdLocalizacion", "AreaRegistral", "AreaCalculada"]].copy()

# =========================================================
# 5) EXPORTAR
# =========================================================
out_file = "AVANCE_MIGRACION_HASTA_AHORA.xlsx"
with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
    df_registroproceso.to_excel(writer, index=False, sheet_name="RegistroProceso_listo")
    df_estudioterreno.to_excel(writer, index=False, sheet_name="EstudioTerreno_listo")
    df_diag.to_excel(writer, index=False, sheet_name="Diagnostico_nulos")
    det_reg.to_excel(writer, index=False, sheet_name="Detalle_nulos_Registro")
    det_est.to_excel(writer, index=False, sheet_name="Detalle_nulos_Estudio")

print("OK. Archivo creado:", out_file)
print("Hojas:", ["RegistroProceso_listo","EstudioTerreno_listo","Diagnostico_nulos","Detalle_nulos_Registro","Detalle_nulos_Estudio"])


OK. Archivo creado: AVANCE_MIGRACION_HASTA_AHORA.xlsx
Hojas: ['RegistroProceso_listo', 'EstudioTerreno_listo', 'Diagnostico_nulos', 'Detalle_nulos_Registro', 'Detalle_nulos_Estudio']


## "NUMERO DE IDENTIFICACIÓN (CC / NIT)"


In [27]:
import re
import numpy as np
import pandas as pd

# ============================================================
# NUMERO IDENTIFICACION (CC/NIT) -> Identificacion_SQL_TXT_MIGRAR + IdTipoDocumento_SQL_MIGRAR
# (UNIFICADO, PROGRESIVO, SIN RECARGAR df)
#
# - Detecta la columna de Excel por "NUMERO" + "IDENTIFICACION" (tolerante a saltos)
# - Crea/actualiza SOLO columnas de trabajo (no hace df=...)
# - Parser base:
#     Identificacion_RAW_TEXT
#     Identificacion_SQL_TXT
#     TipoDoc_Evidencia, TiposDoc_Detectados, IdTipoDocumento_SQL
#     Estado_ID_TXT, Identificacion_LEN_TXT
# - Fix OK (si en realidad hay 2+ IDs pegados): separa y pone ';'
# - Fix MULTIPLE: reconstruye IDs bien separados por ';' y pasa a ok si cabe
# - Construye columnas finales para migrar:
#     Identificacion_SQL_TXT_MIGRAR  (texto final; ok => ids; sin_digitos/vacio => NULL)
#     IdTipoDocumento_SQL_MIGRAR     (FK; si no hay evidencia queda NULL)
#     Estado_MIGRAR                  (ok/sin_digitos/vacio/excede_max/pendiente)
# - Genera Excel de revisión: NumeroIdentificacion_listo_migrar.xlsx
# ============================================================

# -----------------------------
# CONFIG
# -----------------------------
MAX_VARCHAR = 450
NULL_SI_MIXTO = True  # si detecta CC y NIT juntos => IdTipoDocumento_SQL = NULL
SALIDA_XLSX = "NumeroIdentificacion_listo_migrar.xlsx"

MAP_TIPO_ID = {
    "CC": 1, "TI": 2, "CE": 3, "RC": 4, "NIT": 5, "PP": 6, "PEP": 7, "DIE": 8
}

# -----------------------------
# 0) Detectar columna original en Excel (sin hardcode)
# -----------------------------
def _norm_colname(c: str) -> str:
    return str(c).upper().replace("\n", " ").replace("\r", " ").replace("\t", " ")

col_id_cands = []
for c in df.columns:
    cu = _norm_colname(c)
    if ("IDENTIFIC" in cu) and ("NUMERO" in cu or "NÚMERO" in cu):
        col_id_cands.append(c)

if not col_id_cands:
    raise ValueError("No encontré la columna de identificación. Debe contener 'NUMERO' y 'IDENTIFIC' en el nombre.")

COL_ID_EXCEL = col_id_cands[0]
print("Columna detectada de identificación:", repr(COL_ID_EXCEL))

# trazabilidad
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# -----------------------------
# 1) Normalización RAW_TEXT (para ver qué se limpió)
# -----------------------------
DASHES = {"–": "-", "—": "-", "−": "-", "‒": "-", "―": "-"}

def normalizar_raw_text(v):
    if pd.isna(v):
        return None
    s = str(v).replace("\u00A0", " ").strip()
    if s == "":
        return None
    for k, rep in DASHES.items():
        s = s.replace(k, rep)
    s = re.sub(r"\s+", " ", s)
    return s

df["Identificacion_RAW_TEXT"] = df[COL_ID_EXCEL].apply(normalizar_raw_text)

# -----------------------------
# 2) Parser base: tipos + ids
# -----------------------------
TIPO_REGEX = re.compile(
    r"\b(NIT|C\.C|CC|T\.I|TI|C\.E|CE|R\.C|RC|P\.P|PP|PEP|DIE|PASAPORTE)\b",
    re.IGNORECASE
)

def normalizar_tipo(t):
    t = t.upper().replace(".", "").replace(" ", "")
    if t == "CC": return "CC"
    if t == "NIT": return "NIT"
    if t == "TI": return "TI"
    if t == "CE": return "CE"
    if t == "RC": return "RC"
    if t in ("PP", "PASAPORTE"): return "PP"
    if t == "PEP": return "PEP"
    if t == "DIE": return "DIE"
    return t

def limpiar_identificaciones_y_tipo(raw_text, null_si_mixto=True):
    """
    Salidas:
      Identificacion_SQL_TXT: "900542709-3;5200583"
      TipoDoc_Evidencia: primer tipo detectado
      TiposDoc_Detectados: "CC;NIT"
      IdTipoDocumento_SQL: FK segun MAP_TIPO_ID (o NULL si mixto y null_si_mixto=True)
      Estado_ID_TXT: vacio/sin_digitos/ok/multiple/excede_max
      Identificacion_LEN_TXT: int
    """
    if raw_text is None:
        return (None, None, None, None, "vacio", 0)

    s = raw_text

    # Tipos detectados
    tipos = [normalizar_tipo(m.group(1)) for m in TIPO_REGEX.finditer(s)]
    seen = set()
    tipos_unicos = []
    for t in tipos:
        if t not in seen:
            tipos_unicos.append(t)
            seen.add(t)

    tipo_evidencia = tipos_unicos[0] if tipos_unicos else None
    tipos_detectados = ";".join(tipos_unicos) if tipos_unicos else None

    if null_si_mixto and len(tipos_unicos) > 1:
        id_tipo = None
    else:
        id_tipo = MAP_TIPO_ID.get(tipo_evidencia) if tipo_evidencia else None

    # Extraer números + DV (base + opcional -DV)
    patron = re.compile(r"(\d[\d\.\s]*\d)(?:\s*-\s*(\d))?")
    matches = list(patron.finditer(s))
    if not matches:
        return (None, tipo_evidencia, tipos_detectados, id_tipo, "sin_digitos", 0)

    ids = []
    for m in matches:
        base = re.sub(r"\D", "", m.group(1))
        dv = m.group(2)
        if not base:
            continue
        ids.append(f"{base}-{dv}" if dv is not None else base)

    if not ids:
        return (None, tipo_evidencia, tipos_detectados, id_tipo, "sin_digitos", 0)

    # dedupe
    seen_ids = set()
    ids_unicos = []
    for x in ids:
        if x not in seen_ids:
            ids_unicos.append(x)
            seen_ids.add(x)

    out = ";".join(ids_unicos) if ids_unicos else None
    out_len = len(out) if out else 0

    if out is None:
        return (None, tipo_evidencia, tipos_detectados, id_tipo, "sin_digitos", 0)

    if out_len > MAX_VARCHAR:
        estado = "excede_max"
    elif len(ids_unicos) == 1:
        estado = "ok"
    else:
        estado = "multiple"

    return (out, tipo_evidencia, tipos_detectados, id_tipo, estado, out_len)

df[[
    "Identificacion_SQL_TXT",
    "TipoDoc_Evidencia",
    "TiposDoc_Detectados",
    "IdTipoDocumento_SQL",
    "Estado_ID_TXT",
    "Identificacion_LEN_TXT"
]] = df["Identificacion_RAW_TEXT"].apply(
    lambda v: pd.Series(limpiar_identificaciones_y_tipo(v, null_si_mixto=NULL_SI_MIXTO))
)

df["IdTipoDocumento_SQL"] = df["IdTipoDocumento_SQL"].astype("Int64")
df["Identificacion_LEN_TXT"] = pd.to_numeric(df["Identificacion_LEN_TXT"], errors="coerce").fillna(0).astype(int)

# -----------------------------
# 3) OK-FIX: si el "ok" realmente tenía 2+ IDs (pegados) => separarlos y unir con ';'
# -----------------------------
def _normalizar_separadores(texto: str) -> str:
    if texto is None:
        return ""
    s = str(texto)
    s = s.replace("\u00A0", " ").replace("\t", " ")
    s = s.replace("\r\n", "\n").replace("\r", "\n").replace("\n", " ")
    s = re.sub(r"[,:]", " ", s)
    s = re.sub(r"\s+y\s+", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"\s+E\s+", " ", s, flags=re.IGNORECASE)
    s = s.replace("–", "-").replace("—", "-")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extraer_ids_okfix(texto: str) -> list[str]:
    s = _normalizar_separadores(texto)
    if not s:
        return []
    patron = r"\b\d{5,15}(?:-\d{1,2})?\b"
    ids = re.findall(patron, s)
    vistos = set()
    out = []
    for x in ids:
        if x not in vistos:
            vistos.add(x)
            out.append(x)
    return out

mask_ok = df["Estado_ID_TXT"].astype("string").str.lower().eq("ok")
df["_Identificacion_SQL_TXT_ANTES_OKFIX"] = df["Identificacion_SQL_TXT"]

def okfix_row(raw_text):
    ids = extraer_ids_okfix(raw_text)
    if len(ids) >= 2:
        nuevo = ";".join(ids)
        return nuevo, ("ok" if len(nuevo) <= MAX_VARCHAR else "excede_max"), len(ids), True
    if len(ids) == 1:
        nuevo = ids[0]
        return nuevo, ("ok" if len(nuevo) <= MAX_VARCHAR else "excede_max"), 1, False
    return None, "sin_digitos", 0, False

res_ok = df.loc[mask_ok, "Identificacion_RAW_TEXT"].apply(lambda t: pd.Series(okfix_row(t)))
res_ok.columns = ["Identificacion_SQL_TXT_OKCORR", "ESTADO_OKCORR", "N_IDS_OKCORR", "FLAG_OK_TENIA_MULTIPLES"]

# crear columnas si no existen
for c in res_ok.columns:
    if c not in df.columns:
        df[c] = pd.NA

df.loc[mask_ok, res_ok.columns] = res_ok.values

# sobrescribir identificación SOLO si la corrección detectó algo (1 o más ids)
df.loc[mask_ok & df["Identificacion_SQL_TXT_OKCORR"].notna(), "Identificacion_SQL_TXT"] = df.loc[
    mask_ok & df["Identificacion_SQL_TXT_OKCORR"].notna(), "Identificacion_SQL_TXT_OKCORR"
].values

# actualizar longitudes
df.loc[mask_ok, "Identificacion_LEN_TXT"] = (
    df.loc[mask_ok, "Identificacion_SQL_TXT"].astype("string").str.len().fillna(0).astype(int)
)

# -----------------------------
# 4) MULTIPLE-FIX: reconstruir separación real con ';' y pasar a OK si cabe
# -----------------------------
DOC_PATTERNS = {
    "CC":  r"\bC\.?\s*C\.?\b|\bCC\b",
    "NIT": r"\bN\.?\s*I\.?\s*T\.?\b|\bNIT\b",
    "TI":  r"\bT\.?\s*I\.?\b|\bTI\b",
    "CE":  r"\bC\.?\s*E\.?\b|\bCE\b",
    "RC":  r"\bR\.?\s*C\.?\b|\bRC\b",
    "PP":  r"\bP\.?\s*P\.?\b|\bPP\b|\bPASAPORTE\b",
    "PEP": r"\bPEP\b",
    "DIE": r"\bDIE\b",
}

def detectar_tipo_doc_primario(texto: str):
    if texto is None:
        return None
    s = str(texto)
    if not s.strip():
        return None
    s_up = s.upper()
    best = None
    best_pos = None
    for doc, pat in DOC_PATTERNS.items():
        m = re.search(pat, s_up, flags=re.IGNORECASE)
        if m:
            pos = m.start()
            if best_pos is None or pos < best_pos:
                best_pos = pos
                best = doc
    return best

ID_REGEX = re.compile(r"(?<!\d)(\d{5,15}\s*-\s*\d{1,2}|\d{5,15})(?!\d)")

def normalizar_texto_para_extraccion(s: str) -> str:
    s = s.replace("\u00A0", " ")
    s = s.replace("–", "-").replace("—", "-")
    s = re.sub(r"[\r\n]+", " ; ", s)
    s = re.sub(r"(?<=\d)\.(?=\d)", "", s)  # quita miles con punto
    s = s.replace(",", " ; ").replace(";", " ; ").replace(":", " ; ").replace("/", " ; ")
    s = re.sub(r"\brespectivamente\b", " ; ", s, flags=re.IGNORECASE)
    s = re.sub(r"\s+\by\b\s+", " ; ", s, flags=re.IGNORECASE)
    s = re.sub(r"\s+\be\b\s+", " ; ", s, flags=re.IGNORECASE)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extraer_ids_en_orden(texto: str):
    if texto is None:
        return []
    s = str(texto).strip()
    if not s:
        return []
    s = normalizar_texto_para_extraccion(s)
    matches = ID_REGEX.findall(s)
    if not matches:
        return []
    out, seen = [], set()
    for token in matches:
        t = token.strip()
        t = re.sub(r"\s*-\s*", "-", t).replace(" ", "")
        if t and t not in seen:
            seen.add(t)
            out.append(t)
    return out

mask_multiple = df["Estado_ID_TXT"].astype("string").str.lower().eq("multiple")
df_mul = df.loc[mask_multiple].copy()

df_mul["_TXT_FUENTE"] = df_mul["Identificacion_RAW_TEXT"]
df_mul.loc[df_mul["_TXT_FUENTE"].isna() | (df_mul["_TXT_FUENTE"].astype(str).str.strip() == ""), "_TXT_FUENTE"] = df_mul[COL_ID_EXCEL]

df_mul["TipoDoc_Primario_MULT"] = df_mul["_TXT_FUENTE"].apply(detectar_tipo_doc_primario)
df_mul["IdTipoDocumento_SQL_MULT"] = df_mul["TipoDoc_Primario_MULT"].map(MAP_TIPO_ID).astype("Int64")

df_mul["IDS_LIST_MULT"] = df_mul["_TXT_FUENTE"].apply(extraer_ids_en_orden)
df_mul["N_IDS_MULT"] = df_mul["IDS_LIST_MULT"].apply(len)

df_mul["Identificacion_SQL_TXT_MULTICORR"] = df_mul["IDS_LIST_MULT"].apply(lambda lst: ";".join(lst) if lst else None)
df_mul["Identificacion_LEN_MULT"] = df_mul["Identificacion_SQL_TXT_MULTICORR"].astype("string").str.len().fillna(0).astype(int)

df_mul["FLAG_MULT_EXCEDE_MAX"] = df_mul["Identificacion_LEN_MULT"] > MAX_VARCHAR
df_mul["ESTADO_MULT_CORR"] = np.select(
    [
        (df_mul["N_IDS_MULT"] >= 1) & (~df_mul["FLAG_MULT_EXCEDE_MAX"]),
        (df_mul["N_IDS_MULT"] >= 1) & (df_mul["FLAG_MULT_EXCEDE_MAX"]),
        (df_mul["N_IDS_MULT"] == 0),
    ],
    ["ok", "excede_max", "sin_digitos"],
    default="pendiente",
)

# volcar a df principal (sin recargar)
for c in ["Identificacion_SQL_TXT_MULTICORR","N_IDS_MULT","Identificacion_LEN_MULT","FLAG_MULT_EXCEDE_MAX",
          "TipoDoc_Primario_MULT","IdTipoDocumento_SQL_MULT","ESTADO_MULT_CORR"]:
    if c not in df.columns:
        df[c] = pd.NA

df.loc[mask_multiple, "Identificacion_SQL_TXT_MULTICORR"] = df_mul["Identificacion_SQL_TXT_MULTICORR"].values
df.loc[mask_multiple, "N_IDS_MULT"] = df_mul["N_IDS_MULT"].values
df.loc[mask_multiple, "Identificacion_LEN_MULT"] = df_mul["Identificacion_LEN_MULT"].values
df.loc[mask_multiple, "FLAG_MULT_EXCEDE_MAX"] = df_mul["FLAG_MULT_EXCEDE_MAX"].values
df.loc[mask_multiple, "TipoDoc_Primario_MULT"] = df_mul["TipoDoc_Primario_MULT"].values
df.loc[mask_multiple, "IdTipoDocumento_SQL_MULT"] = df_mul["IdTipoDocumento_SQL_MULT"].values
df.loc[mask_multiple, "ESTADO_MULT_CORR"] = df_mul["ESTADO_MULT_CORR"].values

# -----------------------------
# 5) Construir columnas finales MIGRAR
# -----------------------------
df["Estado_MIGRAR"] = df["Estado_ID_TXT"].astype("string").str.lower()

# MULTIPLE corregidos
df.loc[mask_multiple & df["ESTADO_MULT_CORR"].astype("string").str.lower().eq("ok"), "Estado_MIGRAR"] = "ok"
df.loc[mask_multiple & df["ESTADO_MULT_CORR"].astype("string").str.lower().eq("excede_max"), "Estado_MIGRAR"] = "excede_max"
df.loc[mask_multiple & df["ESTADO_MULT_CORR"].astype("string").str.lower().eq("sin_digitos"), "Estado_MIGRAR"] = "sin_digitos"

# Identificación final a migrar
df["Identificacion_SQL_TXT_MIGRAR"] = pd.NA

mask_ok_final = df["Estado_MIGRAR"].eq("ok")
mask_sin = df["Estado_MIGRAR"].eq("sin_digitos")
mask_vacio = df["Estado_MIGRAR"].eq("vacio")

# OK (no multiple): usar Identificacion_SQL_TXT ya corregida (incluye OKCORR)
mask_ok_no_mult = mask_ok_final & (~mask_multiple)
df.loc[mask_ok_no_mult, "Identificacion_SQL_TXT_MIGRAR"] = df.loc[mask_ok_no_mult, "Identificacion_SQL_TXT"].values

# MULTIPLE corregidos a OK: usar MULTICORR
mask_mult_ok = mask_multiple & df["ESTADO_MULT_CORR"].astype("string").str.lower().eq("ok")
df.loc[mask_mult_ok, "Identificacion_SQL_TXT_MIGRAR"] = df.loc[mask_mult_ok, "Identificacion_SQL_TXT_MULTICORR"].values

# SIN_DIGITOS y VACIO => NULL (dejar NA)

# IdTipoDocumento final a migrar (solo si hay evidencia; si no, queda NULL)
df["IdTipoDocumento_SQL_MIGRAR"] = df["IdTipoDocumento_SQL"].astype("Int64")

# para MULTIPLE corregidos: si NA, tomar el primario detectado
mask_need = mask_mult_ok & df["IdTipoDocumento_SQL_MIGRAR"].isna()
df.loc[mask_need, "IdTipoDocumento_SQL_MIGRAR"] = df.loc[mask_need, "IdTipoDocumento_SQL_MULT"].astype("Int64").values

# -----------------------------
# 6) Resumen y Excel de revisión
# -----------------------------
print("\n=== RESUMEN IDENTIFICACIÓN ===")
print("Columna Excel usada:", repr(COL_ID_EXCEL))
print("Estados (Estado_MIGRAR):")
print(df["Estado_MIGRAR"].value_counts(dropna=False))

mask_listo = df["Estado_MIGRAR"].isin(["ok", "sin_digitos", "vacio"])
print("\nLISTO PARA MIGRAR (ok + sin_digitos + vacio):", int(mask_listo.sum()))
print(" - ok:", int((df["Estado_MIGRAR"] == "ok").sum()))
print(" - sin_digitos:", int((df["Estado_MIGRAR"] == "sin_digitos").sum()))
print(" - vacio:", int((df["Estado_MIGRAR"] == "vacio").sum()))
print(" - excede_max (NO listo):", int((df["Estado_MIGRAR"] == "excede_max").sum()))
print(" - pendientes (NO listo):", int((~df["Estado_MIGRAR"].isin(["ok","sin_digitos","vacio","excede_max"])).sum()))

cols_out = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    COL_ID_EXCEL, "Identificacion_RAW_TEXT",
    "TipoDoc_Evidencia", "TiposDoc_Detectados",
    "Estado_ID_TXT",
    "Identificacion_SQL_TXT",
    "Estado_MIGRAR",
    "Identificacion_SQL_TXT_MIGRAR",
    "IdTipoDocumento_SQL_MIGRAR",
    "Identificacion_LEN_TXT",
    "FLAG_OK_TENIA_MULTIPLES",
    "ESTADO_OKCORR",
    "N_IDS_OKCORR",
    "Identificacion_SQL_TXT_MULTICORR",
    "N_IDS_MULT",
    "ESTADO_MULT_CORR",
    "FLAG_MULT_EXCEDE_MAX",
]
cols_out = [c for c in cols_out if c in df.columns]

df_listo = df.loc[mask_listo, cols_out].copy()
df_ok = df.loc[df["Estado_MIGRAR"].eq("ok"), cols_out].copy()
df_sin = df.loc[df["Estado_MIGRAR"].eq("sin_digitos"), cols_out].copy()
df_vac = df.loc[df["Estado_MIGRAR"].eq("vacio"), cols_out].copy()
df_exc = df.loc[df["Estado_MIGRAR"].eq("excede_max"), cols_out].copy()
df_pend = df.loc[~df["Estado_MIGRAR"].isin(["ok","sin_digitos","vacio","excede_max"]), cols_out].copy()
df_mult = df.loc[mask_multiple, cols_out].copy()

resumen_estado = (
    df["Estado_MIGRAR"].value_counts(dropna=False)
      .rename_axis("Estado_MIGRAR")
      .reset_index(name="conteo")
)

resumen_idtipo = (
    df.loc[mask_listo, "IdTipoDocumento_SQL_MIGRAR"]
      .value_counts(dropna=False)
      .rename_axis("IdTipoDocumento_SQL_MIGRAR")
      .reset_index(name="conteo")
)

with pd.ExcelWriter(SALIDA_XLSX, engine="openpyxl") as writer:
    df_listo.to_excel(writer, index=False, sheet_name="listo_total")
    df_ok.to_excel(writer, index=False, sheet_name="ok")
    df_sin.to_excel(writer, index=False, sheet_name="sin_digitos")
    df_vac.to_excel(writer, index=False, sheet_name="vacio")
    df_mult.to_excel(writer, index=False, sheet_name="multiple_original")
    df_exc.to_excel(writer, index=False, sheet_name="excede_max")
    df_pend.to_excel(writer, index=False, sheet_name="pendientes")
    resumen_estado.to_excel(writer, index=False, sheet_name="resumen_estado")
    resumen_idtipo.to_excel(writer, index=False, sheet_name="resumen_idtipo_listo")

print(f"\nExcel creado: {SALIDA_XLSX}")
print("Hojas: listo_total, ok, sin_digitos, vacio, multiple_original, excede_max, pendientes, resumen_estado, resumen_idtipo_listo")


Columna detectada de identificación: 'NUMERO DE IDENTIFICACIÓN \n(CC / NIT)'

=== RESUMEN IDENTIFICACIÓN ===
Columna Excel usada: 'NUMERO DE IDENTIFICACIÓN \n(CC / NIT)'
Estados (Estado_MIGRAR):
Estado_MIGRAR
vacio          16281
ok              7454
sin_digitos      320
Name: count, dtype: Int64

LISTO PARA MIGRAR (ok + sin_digitos + vacio): 24055
 - ok: 7454
 - sin_digitos: 320
 - vacio: 16281
 - excede_max (NO listo): 0
 - pendientes (NO listo): 0

Excel creado: NumeroIdentificacion_listo_migrar.xlsx
Hojas: listo_total, ok, sin_digitos, vacio, multiple_original, excede_max, pendientes, resumen_estado, resumen_idtipo_listo


## ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN	


In [28]:
import pandas as pd
import numpy as np
import re
import unicodedata

# ============================
# NATURALEZA JURÍDICA (SQL)  - NO RECARGA df
# - NO hace df = pd.read_excel(...)
# - Crea/actualiza columnas de trabajo:
#   NaturalezaJuridica_RAW, NaturalezaJuridica_SQL, NaturalezaJuridica_SQL_FINAL, NaturalezaJuridica_len
# - Exporta Excel de revisión
# ============================

# 0) Asegurar Fila_Excel_aprox (sin sobrescribir si ya existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Localizar la columna de Excel (NATURALEZA...)
candidatas_natjur = [c for c in df.columns if "NATURALEZA" in c.upper()]
print("Columnas candidatas para NATURALEZA JURÍDICA:", candidatas_natjur)

if not candidatas_natjur:
    raise ValueError("No se encontró columna que contenga 'NATURALEZA' en el encabezado.")

col_natjur = candidatas_natjur[0]
print("Usando columna de Excel:", col_natjur)

# 2) Guardar RAW (evidencia) sin dañar historial
#    - si ya existe, solo la refrescamos con la columna actual detectada
df["NaturalezaJuridica_RAW"] = df[col_natjur]

# 3) Limpieza base (sin cambiar significado)
def limpiar_natjur(v):
    if pd.isna(v):
        return np.nan
    t = str(v).replace("\u00A0", " ").strip()   # NBSP -> espacio
    t = re.sub(r"\s+", " ", t)                  # compactar espacios
    return np.nan if t == "" else t

df["NaturalezaJuridica_SQL"] = df["NaturalezaJuridica_RAW"].apply(limpiar_natjur)

# 4) Normalizador solo para comparar (quita tildes, etc.)
def norm_key(s: str) -> str:
    s = str(s)
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))  # sin tildes
    s = s.upper()
    s = re.sub(r"[^A-Z0-9]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# 5) Canonicalización final para migrar
MAP_NATJUR = {
    "PRIVADO": "PRIVADO",
    "BALDIO": "BALDÍO",                  # salida con tilde
    "SIN REGISTRO": "SIN REGISTRO",
    "MIXTO": "MIXTO",
    "FORMULA TRANSACCIONAL": "FORMULA TRANSACCIONAL",
    "PENDIENTE": "PENDIENTE",
    "PUBLICO": "PUBLICO",
}

def canon_natjur(v):
    # NaN/"" -> SIN REGISTRO
    if pd.isna(v):
        return "SIN REGISTRO"

    t = str(v).replace("\u00A0", " ").strip()
    t = re.sub(r"\s+", " ", t)
    if t == "":
        return "SIN REGISTRO"

    k = norm_key(t)

    # Variantes de "SIN REGISTRO"
    if k in {"SINREGISTRO", "SIN REGISTRO", "SIN REGISTROS", "SINREGISTROS"}:
        return "SIN REGISTRO"

    # Mapa principal (ojo: la llave es sin tildes)
    if k in MAP_NATJUR:
        return MAP_NATJUR[k]

    # Valor raro: lo dejamos en MAYÚSCULAS tal cual (con tildes si venían)
    return t.upper()

df["NaturalezaJuridica_SQL_FINAL"] = df["NaturalezaJuridica_SQL"].apply(canon_natjur)

# 6) Longitudes (tope 100 en BD)
df["NaturalezaJuridica_len"] = df["NaturalezaJuridica_SQL_FINAL"].fillna("").astype(str).str.len()

print("\n=== Resumen de longitudes NaturalezaJuridica_SQL_FINAL ===")
print(df["NaturalezaJuridica_len"].describe())

pendientes_longitud = df.loc[df["NaturalezaJuridica_len"] > 100, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    col_natjur, "NaturalezaJuridica_RAW", "NaturalezaJuridica_SQL", "NaturalezaJuridica_SQL_FINAL", "NaturalezaJuridica_len"
]].copy()

if pendientes_longitud.empty:
    print("\nOK: todos los valores caben en varchar(100).")
else:
    print("\n=== PENDIENTES (exceden 100) ===")
    print(pendientes_longitud.head(30).to_string(index=False))
    print("Total exceden 100:", len(pendientes_longitud))

# 7) Resumen de categorías finales
resumen_final = (
    df.groupby("NaturalezaJuridica_SQL_FINAL", dropna=False)
      .size()
      .reset_index(name="conteo")
      .sort_values("conteo", ascending=False)
)

print("\n=== Principales valores (SQL_FINAL) ===")
print(resumen_final.head(50).to_string(index=False))

# 8) Excel de revisión
ruta_excel = "NaturalezaJuridica_revision.xlsx"
with pd.ExcelWriter(ruta_excel, engine="openpyxl") as writer:
    df.loc[:, [
        "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
        col_natjur, "NaturalezaJuridica_RAW", "NaturalezaJuridica_SQL", "NaturalezaJuridica_SQL_FINAL", "NaturalezaJuridica_len"
    ]].to_excel(writer, index=False, sheet_name="all_mapeo")

    resumen_final.to_excel(writer, index=False, sheet_name="resumen_final")
    pendientes_longitud.to_excel(writer, index=False, sheet_name="excede_100")

print(f"\nExcel creado: {ruta_excel}")
print("Hojas: all_mapeo, resumen_final, excede_100")

# === Para migración a BD ===
# Usar df['NaturalezaJuridica_SQL_FINAL'] -> EstudioTerreno.NaturalezaJuridica


Columnas candidatas para NATURALEZA JURÍDICA: ['ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN\xa0']
Usando columna de Excel: ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN 

=== Resumen de longitudes NaturalezaJuridica_SQL_FINAL ===
count    24055.000000
mean         9.645479
std          2.590530
min          5.000000
25%          7.000000
50%         12.000000
75%         12.000000
max         21.000000
Name: NaturalezaJuridica_len, dtype: float64

OK: todos los valores caben en varchar(100).

=== Principales valores (SQL_FINAL) ===
NaturalezaJuridica_SQL_FINAL  conteo
                SIN REGISTRO   13094
                     PRIVADO    9176
                      BALDÍO    1699
                       MIXTO      82
       FORMULA TRANSACCIONAL       2
                   PENDIENTE       1
                     PUBLICO       1

Excel creado: NaturalezaJuridica_revision.xlsx
Hojas: all_mapeo, resumen_final, excede_100


## Título originario

In [29]:
import numpy as np
import pandas as pd
import re

# ============================
# TÍTULO ORIGINARIO / ACREDITACIÓN DE LA PROPIEDAD
# -> EstudioTerreno.AcreditacionPropiedad (varchar(100) NULL)
# Categorías objetivo:
#   - FÓRMULA TRANSACCIONAL
#   - TÍTULO ORIGINARIO
#   - SIN REGISTRO
#
# NOTA: NO RECARGA df (NO hace df = pd.read_excel(...))
# ============================

# 0) Fila Excel (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Localizar columna del Excel (más específica) - sin recargar df
cols_candidatas = [
    c for c in df.columns
    if ("ACREDIT" in c.upper())
    or ("PROPIEDAD" in c.upper())
    or ("TITULO" in c.upper())
    or ("TÍTULO" in c.upper())
    or ("ORIGINARIO" in c.upper())
]
print("Columnas candidatas para TÍTULO ORIGINARIO / ACREDITACIÓN:", cols_candidatas)

if not cols_candidatas:
    raise ValueError("No se encontró columna relacionada con 'ACREDIT', 'PROPIEDAD', 'TITULO/TÍTULO' u 'ORIGINARIO'.")

col_tit = cols_candidatas[0]
print("Usando columna:", col_tit)

# 2) RAW (si existe, se actualiza; no se duplica)
df["AcreditacionPropiedad_RAW"] = df[col_tit]

# 3) Normalización base (para análisis/mapeo)
#    Usa tu normalizar_texto (mayúsculas, sin tildes, etc.).
def _norm(v):
    if pd.isna(v):
        return None
    s = str(v).replace("\u00A0", " ").strip()
    s = re.sub(r"\s+", " ", s)
    if s == "":
        return None
    return normalizar_texto(s)

df["AcreditacionPropiedad_TXT_LIMPIO"] = df["AcreditacionPropiedad_RAW"].apply(_norm)

# 4) Catálogo (lo que queremos guardar en SQL)
CATALOGO = [
    "TÍTULO ORIGINARIO",
    "FÓRMULA TRANSACCIONAL",
    "SIN REGISTRO",
]

# Mapeo por clave normalizada
MAP = {normalizar_texto(x): x for x in CATALOGO}

# 5) Sinónimos / variantes frecuentes (se agregan al MAP normalizado)
SINONIMOS = {
    # TÍTULO ORIGINARIO
    "TITULO ORIGINARIO": "TÍTULO ORIGINARIO",
    "TITULO  ORIGINARIO": "TÍTULO ORIGINARIO",
    "TTITULO ORIGINARIO": "TÍTULO ORIGINARIO",
    "TITULO-ORIGINARIO": "TÍTULO ORIGINARIO",

    # FÓRMULA TRANSACCIONAL
    "FORMULA TRANSACCIONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRANSACCIOONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRANSACIONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRASNSACCIONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRANSACCIONAL.": "FÓRMULA TRANSACCIONAL",

    # SIN REGISTRO
    "SIN REGISTRO": "SIN REGISTRO",
    "NO REGISTRA": "SIN REGISTRO",
    "NO REGISTRO": "SIN REGISTRO",
    "SIN INFORMACION": "SIN REGISTRO",
    "SIN INFORMACIÓN": "SIN REGISTRO",
    "N/A": "SIN REGISTRO",
    "NA": "SIN REGISTRO",
    "N.R": "SIN REGISTRO",
    "NR": "SIN REGISTRO",
    "0": "SIN REGISTRO",
}
for k, v in SINONIMOS.items():
    MAP[normalizar_texto(k)] = v

# 6) Mapeo final + estado
def mapear_final(txt_norm):
    """
    Devuelve (AcreditacionPropiedad_SQL_FINAL, Estado_Acreditacion)
    Estado:
      - ok
      - sin_registro_auto
      - pendiente
    """
    if txt_norm is None or pd.isna(txt_norm):
        return ("SIN REGISTRO", "sin_registro_auto")

    if txt_norm == "0":
        return ("SIN REGISTRO", "sin_registro_auto")

    canon = MAP.get(txt_norm)
    if canon is not None:
        if canon == "SIN REGISTRO" and txt_norm != normalizar_texto("SIN REGISTRO"):
            return (canon, "sin_registro_auto")
        return (canon, "ok")

    # fallback (si no quieres auto-clasificar, comenta este bloque)
    if "FORMULA" in txt_norm or "TRANSACC" in txt_norm:
        return ("FÓRMULA TRANSACCIONAL", "ok")
    if "TITULO" in txt_norm or "ORIGINARIO" in txt_norm:
        return ("TÍTULO ORIGINARIO", "ok")

    # Pendiente real -> NULL para SQL
    return (np.nan, "pendiente")

df[["AcreditacionPropiedad_SQL_FINAL", "Estado_Acreditacion"]] = (
    df["AcreditacionPropiedad_TXT_LIMPIO"]
      .apply(lambda x: pd.Series(mapear_final(x)))
)

# 7) Longitud (control varchar(100))
df["AcreditacionPropiedad_LEN_SQL"] = df["AcreditacionPropiedad_SQL_FINAL"].fillna("").astype(str).str.len()

print("\n=== Resumen longitudes AcreditacionPropiedad_SQL_FINAL ===")
print(df["AcreditacionPropiedad_LEN_SQL"].describe())

excede_100 = df.loc[df["AcreditacionPropiedad_LEN_SQL"] > 100, [
    "Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO",col_tit,
    "AcreditacionPropiedad_RAW","AcreditacionPropiedad_TXT_LIMPIO",
    "AcreditacionPropiedad_SQL_FINAL","AcreditacionPropiedad_LEN_SQL","Estado_Acreditacion"
]].copy()

if excede_100.empty:
    print("\nOK: Todo cabe en varchar(100).")
else:
    print("\nPENDIENTE: Hay valores que exceden 100 (revisar). Total:", len(excede_100))

# 8) Resúmenes
print("\n=== Conteo por Estado_Acreditacion ===")
print(df["Estado_Acreditacion"].value_counts(dropna=False))

print("\n=== Distribución AcreditacionPropiedad_SQL_FINAL ===")
print(df["AcreditacionPropiedad_SQL_FINAL"].value_counts(dropna=False))

pendientes = df.loc[df["Estado_Acreditacion"].eq("pendiente"), [
    "Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO",col_tit,
    "AcreditacionPropiedad_RAW","AcreditacionPropiedad_TXT_LIMPIO"
]].copy()

if not pendientes.empty:
    print("\n=== Textos distintos PENDIENTES (top 50) ===")
    print(pendientes["AcreditacionPropiedad_TXT_LIMPIO"].value_counts().head(50).to_string())
else:
    print("\nNo hay pendientes de clasificar.")

# 9) Excel de informe
archivo = "AcreditacionPropiedad_revision.xlsx"

all_mapeo = df.loc[:, [
    "Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO",
    col_tit,
    "AcreditacionPropiedad_RAW",
    "AcreditacionPropiedad_TXT_LIMPIO",
    "AcreditacionPropiedad_SQL_FINAL",
    "AcreditacionPropiedad_LEN_SQL",
    "Estado_Acreditacion",
]].copy()

resumen_final = (
    df.groupby(["AcreditacionPropiedad_SQL_FINAL","Estado_Acreditacion"], dropna=False)
      .size().reset_index(name="conteo")
      .sort_values("conteo", ascending=False)
)

with pd.ExcelWriter(archivo, engine="openpyxl") as writer:
    all_mapeo.to_excel(writer, index=False, sheet_name="all_mapeo")
    resumen_final.to_excel(writer, index=False, sheet_name="resumen_final")
    pendientes.to_excel(writer, index=False, sheet_name="pendientes_filas")
    if not excede_100.empty:
        excede_100.to_excel(writer, index=False, sheet_name="excede_100")

print("\nExcel creado:", archivo)
print("Hojas: all_mapeo, resumen_final, pendientes_filas" + (", excede_100" if not excede_100.empty else ""))

# === Para migración a BD ===
# Usar df["AcreditacionPropiedad_SQL_FINAL"] -> EstudioTerreno.AcreditacionPropiedad


Columnas candidatas para TÍTULO ORIGINARIO / ACREDITACIÓN: ['Título originario\xa0']
Usando columna: Título originario 

=== Resumen longitudes AcreditacionPropiedad_SQL_FINAL ===
count    24055.000000
mean        14.336811
std          3.360687
min         12.000000
25%         12.000000
50%         12.000000
75%         17.000000
max         21.000000
Name: AcreditacionPropiedad_LEN_SQL, dtype: float64

OK: Todo cabe en varchar(100).

=== Conteo por Estado_Acreditacion ===
Estado_Acreditacion
sin_registro_auto    15025
ok                    9030
Name: count, dtype: int64

=== Distribución AcreditacionPropiedad_SQL_FINAL ===
AcreditacionPropiedad_SQL_FINAL
SIN REGISTRO             15523
TÍTULO ORIGINARIO         5144
FÓRMULA TRANSACCIONAL     3388
Name: count, dtype: int64

No hay pendientes de clasificar.

Excel creado: AcreditacionPropiedad_revision.xlsx
Hojas: all_mapeo, resumen_final, pendientes_filas


*avance excel*

In [30]:
import pandas as pd

# ============================================================
# AVANCE A EXCEL (SIN RECARGAR df)
# - No hace df = pd.read_excel(...)
# - Exporta lo que ya está listo hasta ahora
# ============================================================

# 0) Asegurar fila Excel (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Columnas "contexto" (solo si existen)
cols_ctx = [
    "Fila_Excel_aprox",
    "IdPostulacion_SQL",  # si la tienes
    "ID_SQL",             # por si todavía la usas
    "FMI", "FMI_SQL",
    "DEPARTAMENTO", "MUNICIPIO",
]
cols_ctx = [c for c in cols_ctx if c in df.columns]

# 2) Tabla destino: ViabilidadJuridica.Postulacion.RegistroProceso
cols_registro = [
    "IdPostulacion_SQL",  # -> RegistroProceso.IdPostulacion (varchar(30))
    "FMI_SQL",            # -> RegistroProceso.FMI (varchar(100))
    "NumeroExpediente_SQL",
    "IdFuenteProceso",    # o "IdFuenteProceso_SQL"
    "IdTipoProceso",      # o "IdTipoProceso_SQL"
    "IdEtapaProcesal",    # o "IdEtapaProcesal_SQL"
    "Dependencia_SQL",
    "RadicadoOrfeo_SQL",
]
# tomar solo las que existan
cols_registro_exist = [c for c in cols_registro if c in df.columns]

df_registro = df[cols_ctx + cols_registro_exist].copy()

# 3) Tabla destino: ViabilidadJuridica.Postulacion.EstudioTerreno
cols_estudio = [
    "IdRegistroProceso_SQL",     # si ya la creaste
    "IdLocalizacion_SQL",        # -> EstudioTerreno.IdLocalizacion (int)
    "AreaRegistral_SQL",         # -> EstudioTerreno.AreaRegistral (numeric(18,4))
    "CirculoRegistral_SQL",      # -> EstudioTerreno.CirculoRegistral (varchar(100))
    "AreaCalculada_SQL",         # -> EstudioTerreno.AreaCalculada (numeric(18,4))
    "TipoPersonaTitular_SQL",    # -> EstudioTerreno.TipoPersonaTitular (varchar(50))
    "NombrePropietario_SQL",     # -> EstudioTerreno.NombrePropietario (varchar(100))  (si luego lo vas a recortar, aquí igual lo exporta)
    "ApellidoPropietario_SQL",   # -> EstudioTerreno.ApellidoPropietario (varchar(100))
    "Identificacion_SQL_TXT_MIGRAR",  # -> EstudioTerreno.Identificacion (bigint) (si luego lo vas a convertir; aquí exporta el txt)
    "IdTipoDocumento_SQL_MIGRAR",     # -> EstudioTerreno.IdTipoDocumento (si aplica en tu modelo)
    "NaturalezaJuridica_SQL_FINAL",   # -> EstudioTerreno.NaturalezaJuridica (varchar(100))
    "AcreditacionPropiedad_SQL_FINAL" # -> EstudioTerreno.AcreditacionPropiedad (varchar(100))
]
cols_estudio_exist = [c for c in cols_estudio if c in df.columns]

df_estudio = df[cols_ctx + cols_estudio_exist].copy()

# 4) Hoja "diccionario" (opcional, útil para revisar)
dicc = []
for c in cols_registro_exist:
    dicc.append(["RegistroProceso", c])
for c in cols_estudio_exist:
    dicc.append(["EstudioTerreno", c])

df_dicc = pd.DataFrame(dicc, columns=["TablaDestino", "Columna_df"])

# 5) Exportar
out_xlsx = "AVANCE_MIGRACION_HASTA_HOY.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    df_registro.to_excel(writer, index=False, sheet_name="RegistroProceso_avance")
    df_estudio.to_excel(writer, index=False, sheet_name="EstudioTerreno_avance")
    df_dicc.to_excel(writer, index=False, sheet_name="Diccionario")

print("Excel creado:", out_xlsx)
print("Hojas: RegistroProceso_avance, EstudioTerreno_avance, Diccionario")
print("Filas RegistroProceso_avance:", len(df_registro))
print("Filas EstudioTerreno_avance:", len(df_estudio))


Excel creado: AVANCE_MIGRACION_HASTA_HOY.xlsx
Hojas: RegistroProceso_avance, EstudioTerreno_avance, Diccionario
Filas RegistroProceso_avance: 24055
Filas EstudioTerreno_avance: 24055


## Medida procesal

*Hipoteca*

In [31]:
import pandas as pd
import numpy as np
import re

# ============================================================
# MEDIDA PROCESAL - HIPOTECA (SIN RECARGAR df)
# Tabla destino: ViabilidadJuridica.Postulacion.MedidaProcesal
# ============================================================

# 0) Fila Excel aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

cols = list(df.columns)
pos = {c: i for i, c in enumerate(cols)}

# 1) Detectar columnas (HIPOTECA: primera pareja Si/No + primera Anotación)
cols_si = [c for c in cols if ("SI" in str(c).upper() and "NO" in str(c).upper())]
cols_anot = [c for c in cols if "ANOTAC" in str(c).upper()]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

col_hip_si = cols_si_ordenadas[0]
col_hip_anot = cols_anot_ordenadas[0]

print("\nSe asume que HIPOTECA usa estas columnas:")
print("  HIPOTECA Si/No   :", col_hip_si)
print("  HIPOTECA Anotac. :", col_hip_anot)

# 2) RAW para trabajar (sin insertar columnas en posiciones; solo crear/actualizar)
df["HIPOTECA_SI_RAW"] = df[col_hip_si]
df["HIPOTECA_ANOT_RAW"] = df[col_hip_anot]

print("\n=== Valores originales de HIPOTECA (Si/No) ===")
print(df["HIPOTECA_SI_RAW"].value_counts(dropna=False))

# 3) Normalización y mapeo (SI / NO / PENDIENTE)
def normalizar_si_no(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    if t == "":
        return None
    t = re.sub(r"\s+", " ", t)
    t_up = t.upper()
    t_up = t_up.replace("Í", "I")  # SÍ -> SI
    t_up = t_up.replace(".", "").replace(",", "")
    return t_up.strip()

MAP_HIPOTECA = {
    "SI": "SI",
    "S": "SI",
    "YES": "SI",
    "NO": "NO",
    "N": "NO",
    "PENDIENTE": "PENDIENTE",
    "PEND": "PENDIENTE",
    "SIN_REG": "SIN_REG",
    "SINREG": "SIN_REG",
    # "SIN": "SIN_REG",  # deja comentado si no lo estás usando en tu Excel
}

def mapear_hipoteca(valor_norm):
    """
    Devuelve:
      - HIPOTECA_VALOR_SQL: "SI"/"NO"/"PENDIENTE" o None
      - HIPOTECA_ESTADO: "ok" / "vacio" / "pendiente"
    """
    if valor_norm is None:
        return (None, "vacio")

    primera = valor_norm.split(" ")[0]

    if valor_norm in MAP_HIPOTECA:
        return (MAP_HIPOTECA[valor_norm], "ok")
    if primera in MAP_HIPOTECA:
        return (MAP_HIPOTECA[primera], "ok")

    return (None, "pendiente")

df["HIPOTECA_SI_NORM"] = df["HIPOTECA_SI_RAW"].apply(normalizar_si_no)
df[["HIPOTECA_VALOR_SQL", "HIPOTECA_ESTADO"]] = df["HIPOTECA_SI_NORM"].apply(
    lambda x: pd.Series(mapear_hipoteca(x))
)

# 4) Limpiar anotación (max 4000)
def limpiar_anotacion(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    if t == "":
        return None
    t = re.sub(r"\s+", " ", t)
    return t

df["HIPOTECA_ANOT_SQL"] = df["HIPOTECA_ANOT_RAW"].apply(limpiar_anotacion)
df["HIPOTECA_ANOT_LEN"] = df["HIPOTECA_ANOT_SQL"].fillna("").str.len()
df["FLAG_ANOT_GT4000"] = df["HIPOTECA_ANOT_LEN"] > 4000

# Truncar si se pasa de 4000
df.loc[df["FLAG_ANOT_GT4000"], "HIPOTECA_ANOT_SQL"] = (
    df.loc[df["FLAG_ANOT_GT4000"], "HIPOTECA_ANOT_SQL"].str.slice(0, 4000)
)

# 4.1) Si HIPOTECA vacía pero hay anotación -> Valor="SIN_REG"
mask_solo_anot = df["HIPOTECA_VALOR_SQL"].isna() & df["HIPOTECA_ANOT_SQL"].notna()
mask_sinreg = df["HIPOTECA_ESTADO"].eq("vacio") & mask_solo_anot

df.loc[mask_sinreg, "HIPOTECA_VALOR_SQL"] = "SIN_REG"
df.loc[mask_sinreg, "HIPOTECA_ESTADO"] = "sin_reg"

# 5) Preparar columnas destino MedidaProcesal (ok + sin_reg)
mask_insertar = df["HIPOTECA_ESTADO"].isin(["ok", "sin_reg"])

# Crear/actualizar columnas destino (sin resetear df ni recargar)
for c in ["MP_Objeto", "MP_Valor", "MP_Anotacion", "MP_TipoClasificacion"]:
    if c not in df.columns:
        df[c] = None

# IMPORTANTE: solo llenar donde aplica hipoteca para no dañar otras MedidasProcesales si ya existen
df.loc[mask_insertar, "MP_Objeto"] = "HIPOTECA"
df.loc[mask_insertar, "MP_Valor"] = df.loc[mask_insertar, "HIPOTECA_VALOR_SQL"]
df.loc[mask_insertar, "MP_Anotacion"] = df.loc[mask_insertar, "HIPOTECA_ANOT_SQL"]
# MP_TipoClasificacion queda None

# 6) Resúmenes
print("\n=== Conteo por HIPOTECA_ESTADO ===")
print(df["HIPOTECA_ESTADO"].value_counts(dropna=False))

print("\n=== Distribución HIPOTECA_VALOR_SQL (insertables: ok + sin_reg) ===")
print(df.loc[mask_insertar, "HIPOTECA_VALOR_SQL"].value_counts(dropna=False))

pendientes = df.loc[df["HIPOTECA_ESTADO"].eq("pendiente"), [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    col_hip_si, col_hip_anot,
    "HIPOTECA_SI_RAW", "HIPOTECA_SI_NORM", "HIPOTECA_ANOT_SQL"
]].copy()

print("\n=== Pendientes (no clasificados) ===")
if pendientes.empty:
    print("No hay pendientes. Todo quedó en SI/NO/PENDIENTE/SIN_REG o vacio.")
else:
    print(pendientes.head(50).to_string(index=False))
    print("\nTextos distintos pendientes:")
    print(pendientes["HIPOTECA_SI_NORM"].value_counts(dropna=False).to_string())

# 7) Dataset listo para insertar en MedidaProcesal (ok + sin_reg)
mp_listo = df.loc[mask_insertar, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "MP_Objeto", "MP_Valor", "MP_Anotacion", "MP_TipoClasificacion"
]].copy()

print("\n=== Ejemplos LISTOS (primeros 30) ===")
print(mp_listo.head(30).to_string(index=False))

# 8) Exportar Excel de revisión
archivo = "MedidaProcesal_Hipoteca_revision.xlsx"

df_all = df[[
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    col_hip_si, col_hip_anot,
    "HIPOTECA_SI_RAW", "HIPOTECA_SI_NORM",
    "HIPOTECA_VALOR_SQL", "HIPOTECA_ESTADO",
    "HIPOTECA_ANOT_SQL", "HIPOTECA_ANOT_LEN", "FLAG_ANOT_GT4000",
    "MP_Objeto", "MP_Valor", "MP_Anotacion", "MP_TipoClasificacion"
]].copy()

resumen_estado = (
    df["HIPOTECA_ESTADO"].value_counts(dropna=False)
      .rename_axis("HIPOTECA_ESTADO")
      .reset_index(name="conteo")
)

resumen_valor = (
    df["HIPOTECA_VALOR_SQL"].value_counts(dropna=False)
      .rename_axis("HIPOTECA_VALOR_SQL")
      .reset_index(name="conteo")
)

with pd.ExcelWriter(archivo, engine="openpyxl") as writer:
    df_all.to_excel(writer, index=False, sheet_name="all_mapeo")
    resumen_estado.to_excel(writer, index=False, sheet_name="resumen_estado")
    resumen_valor.to_excel(writer, index=False, sheet_name="resumen_valor")
    mp_listo.to_excel(writer, index=False, sheet_name="mp_listo_insertar")
    pendientes.to_excel(writer, index=False, sheet_name="pendientes")

print(f"\nExcel creado: {archivo}")
print("Hojas: all_mapeo, resumen_estado, resumen_valor, mp_listo_insertar, pendientes")


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6', 'ENTREGÓ CARPETA DE SOPORTES\n(SI / NO)']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']

Se asume que HIPOTECA usa estas columnas:
  HIPOTECA Si/No   : Si / No
  HIPOTECA Anotac. : Anotación

=== Valores originales de HIPOTECA (Si/No) ===
HIPOTECA_SI_RAW
NaN          14909
NO            7296
SI            1810
no              25
si               8
Si               3
Si               2
No               1
PENDIENTE        1
Name: count, dtype: int64

=== Conteo por HIPOTECA_ESTADO ===
HIPOTECA_ESTADO
vacio      14595
ok          9146
sin_reg      314
Name: count, dtype: int64

=== Distribución HIPOTECA_VALOR_SQL (insertables: ok + sin_reg) ===
HIPOTECA_VALOR_SQL
NO           7322
SI           1823
SIN_REG       314
PENDIENTE       1
Name: count, dtype: int64

=== Pendientes (no clasificados)

## SERVIDUMBRES	


In [32]:
import pandas as pd
import numpy as np
import re

# ======================================
# MEDIDA PROCESAL - SERVIDUMBRES (AJUSTADO A REGLA "SIN_REG")
# (CORREGIDO: NO recarga/reasigna df)
# ======================================

# 0) Fila aproximada de Excel (por si aún no existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

cols = list(df.columns)

# 1) Localizar columnas de SERVIDUMBRES (Si/No + Anotación)
cols_si = [c for c in cols if "SI" in str(c).upper() and "NO" in str(c).upper()]
cols_anot = [c for c in cols if "ANOTAC" in str(c).upper()]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

# Ordenar por posición (en el Excel las columnas van en orden)
pos = {c: i for i, c in enumerate(cols)}
cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# SERVIDUMBRES = segunda pareja Si/No + Anotación
if "Si / No.1" in df.columns:
    col_serv_si = "Si / No.1"
else:
    if len(cols_si_ordenadas) < 2:
        raise ValueError("No se encontró una segunda columna 'Si / No' para SERVIDUMBRES.")
    col_serv_si = cols_si_ordenadas[1]

if "Anotación.1" in df.columns:
    col_serv_anot = "Anotación.1"
else:
    if len(cols_anot_ordenadas) < 2:
        raise ValueError("No se encontró una segunda columna 'Anotación' para SERVIDUMBRES.")
    col_serv_anot = cols_anot_ordenadas[1]

print("\nSERVIDUMBRES Si/No   :", col_serv_si)
print("SERVIDUMBRES Anotac. :", col_serv_anot)

# 2) Copias RAW de trabajo
df["SERVIDUMBRES_SI_RAW"]   = df[col_serv_si]
df["SERVIDUMBRES_ANOT_RAW"] = df[col_serv_anot]

# 3) Helpers: detectar vacío real (incluye strings con espacios)
def es_vacio(v) -> bool:
    if pd.isna(v):
        return True
    if isinstance(v, str) and v.strip() == "":
        return True
    return False

# 4) Normalizar valor SI / NO / PENDIENTE (y tratar espacios como None)
def normalizar_servidumbre_valor(v):
    if es_vacio(v):
        return None

    txt = str(v).strip().upper()

    # Quitar tildes comunes
    txt = (txt
           .replace("Á", "A")
           .replace("É", "E")
           .replace("Í", "I")
           .replace("Ó", "O")
           .replace("Ú", "U"))

    # Quitar signos simples
    txt = txt.replace(".", "").replace(",", "").replace(";", "")
    txt = " ".join(txt.split()).strip()

    # Normalizar variantes
    if txt in {"SI", "S"}:
        return "SI"
    if txt in {"NO", "N"}:
        return "NO"
    if "PEND" in txt:
        return "PENDIENTE"

    # Cualquier cosa rara queda None (se revisa)
    return None

df["SERVIDUMBRES_VALOR_TXT_LIMPIO"] = df["SERVIDUMBRES_SI_RAW"].apply(normalizar_servidumbre_valor)
df["SERVIDUMBRES_VALOR_SQL"] = df["SERVIDUMBRES_VALOR_TXT_LIMPIO"]

print("\n=== Conteo SERVIDUMBRES_VALOR_TXT_LIMPIO ===")
print(df["SERVIDUMBRES_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

# Valores raros reales: hay algo NO vacío en la celda, pero no se pudo mapear
mask_valor_raro = (
    df["SERVIDUMBRES_SI_RAW"].apply(lambda x: not es_vacio(x))
    & df["SERVIDUMBRES_VALOR_TXT_LIMPIO"].isna()
)

servid_valores_raros = df.loc[
    mask_valor_raro,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", "SERVIDUMBRES_SI_RAW"],
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\nTotal filas SERVIDUMBRES con valor Si/No raro (sin mapeo REAL):", len(servid_valores_raros))
if not servid_valores_raros.empty:
    print("=== Ejemplos de valores SERVIDUMBRES_SI_RAW sin mapeo ===")
    print(servid_valores_raros.head(30).to_string(index=False))

# 5) Limpiar anotación SERVIDUMBRES y revisar longitud (varchar(4000))
def limpiar_anotacion_servidumbre(v):
    if es_vacio(v):
        return None
    txt = str(v)
    txt = txt.replace("\r", " ").replace("\n", " ").replace("\t", " ")
    txt = " ".join(txt.split()).strip()
    return txt if txt != "" else None

df["SERVIDUMBRES_ANOT_SQL"] = df["SERVIDUMBRES_ANOT_RAW"].apply(limpiar_anotacion_servidumbre)
df["SERVIDUMBRES_ANOT_LEN"] = df["SERVIDUMBRES_ANOT_SQL"].fillna("").str.len()

print("\n=== Resumen longitudes SERVIDUMBRES_ANOT_SQL ===")
print(df["SERVIDUMBRES_ANOT_LEN"].describe())

mask_anot_larga = df["SERVIDUMBRES_ANOT_LEN"] > 4000
servid_anot_largas = df.loc[
    mask_anot_larga,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "SERVIDUMBRES_VALOR_SQL",
        "SERVIDUMBRES_ANOT_LEN",
        "SERVIDUMBRES_ANOT_SQL",
    ],
].copy()

print("\nTotal filas con SERVIDUMBRES_ANOT_SQL > 4000 caracteres:", len(servid_anot_largas))
if not servid_anot_largas.empty:
    print("=== Ejemplos de anotaciones demasiado largas ===")
    print(servid_anot_largas.head(20).to_string(index=False))

# Si se pasa de 4000, truncar para que sea migrable
df["SERVIDUMBRES_FLAG_ANOT_GT4000"] = df["SERVIDUMBRES_ANOT_LEN"] > 4000
df.loc[df["SERVIDUMBRES_FLAG_ANOT_GT4000"], "SERVIDUMBRES_ANOT_SQL"] = (
    df.loc[df["SERVIDUMBRES_FLAG_ANOT_GT4000"], "SERVIDUMBRES_ANOT_SQL"].str.slice(0, 4000)
)
df.loc[df["SERVIDUMBRES_FLAG_ANOT_GT4000"], "SERVIDUMBRES_ANOT_LEN"] = 4000

# 6) Regla de negocio: si hay anotación y no hay valor -> Valor = SIN_REG
df["SERVIDUMBRES_VALOR_SQL_FINAL"] = df["SERVIDUMBRES_VALOR_SQL"].copy()

mask_anot_sin_valor = df["SERVIDUMBRES_ANOT_SQL"].notna() & df["SERVIDUMBRES_VALOR_SQL_FINAL"].isna()
df.loc[mask_anot_sin_valor, "SERVIDUMBRES_VALOR_SQL_FINAL"] = "SIN_REG"

print("\n=== Conteo SERVIDUMBRES_VALOR_SQL_FINAL ===")
print(df["SERVIDUMBRES_VALOR_SQL_FINAL"].value_counts(dropna=False))

# 7) Estado para control (ok / sin_reg / vacio / pendiente)
df["SERVIDUMBRES_ESTADO"] = "vacio"
df.loc[df["SERVIDUMBRES_VALOR_TXT_LIMPIO"].notna(), "SERVIDUMBRES_ESTADO"] = "ok"
df.loc[mask_anot_sin_valor, "SERVIDUMBRES_ESTADO"] = "sin_reg"
df.loc[mask_valor_raro, "SERVIDUMBRES_ESTADO"] = "pendiente"

print("\n=== Conteo por SERVIDUMBRES_ESTADO ===")
print(df["SERVIDUMBRES_ESTADO"].value_counts(dropna=False))

# 8) (CAMBIO CLAVE) NO reordenar ni hacer df = df[cols_all]
#     Solo dejamos las columnas creadas en df tal cual (sin recargar/reasignar df)

# 9) Export ALL
cols_export = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_serv_si,
    "SERVIDUMBRES_SI_RAW",
    "SERVIDUMBRES_VALOR_TXT_LIMPIO",
    "SERVIDUMBRES_VALOR_SQL",
    "SERVIDUMBRES_VALOR_SQL_FINAL",
    "SERVIDUMBRES_ESTADO",
    col_serv_anot,
    "SERVIDUMBRES_ANOT_RAW",
    "SERVIDUMBRES_ANOT_SQL",
    "SERVIDUMBRES_ANOT_LEN",
    "SERVIDUMBRES_FLAG_ANOT_GT4000",
]
cols_export = [c for c in cols_export if c in df.columns]
df_servid_all = df[cols_export].copy()

archivo_completo = "MedidaProcesal_Servidumbres_revision.xlsx"
df_servid_all.to_excel(archivo_completo, index=False)

# 10) Excels adicionales
archivo_valores_raros = None
archivo_anot_largas = None

if not servid_valores_raros.empty:
    archivo_valores_raros = "MedidaProcesal_Servidumbres_valores_raros_reales.xlsx"
    servid_valores_raros.to_excel(archivo_valores_raros, index=False)

if not servid_anot_largas.empty:
    archivo_anot_largas = "MedidaProcesal_Servidumbres_anotaciones_largas.xlsx"
    servid_anot_largas.to_excel(archivo_anot_largas, index=False)

# 11) Dataset “listo para insertar” (solo donde hay Valor final)
mask_listo_insertar = df["SERVIDUMBRES_VALOR_SQL_FINAL"].notna()
df_listo_insertar = df.loc[
    mask_listo_insertar,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL"],
].copy()

df_listo_insertar.rename(columns={
    "SERVIDUMBRES_VALOR_SQL_FINAL": "Valor",
    "SERVIDUMBRES_ANOT_SQL": "Anotacion",
}, inplace=True)

df_listo_insertar["Objeto"] = "SERVIDUMBRES"
# df_listo_insertar["IdEstudioTerreno"] = ...  # se completa luego con el cruce

archivo_listo = "MedidaProcesal_Servidumbres_listo_insertar.xlsx"
df_listo_insertar.to_excel(archivo_listo, index=False)

print("\nArchivos creados:")
print(f" - {archivo_completo}")
print(f" - {archivo_listo} (solo filas con Valor final para insertar)")
if archivo_valores_raros:
    print(f" - {archivo_valores_raros} (valores Si/No raros reales)")
if archivo_anot_largas:
    print(f" - {archivo_anot_largas} (anotaciones > 4000 caracteres)")


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6', 'ENTREGÓ CARPETA DE SOPORTES\n(SI / NO)', 'HIPOTECA_SI_NORM']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6', 'MP_Anotacion']

SERVIDUMBRES Si/No   : Si / No.1
SERVIDUMBRES Anotac. : Anotación.1

=== Conteo SERVIDUMBRES_VALOR_TXT_LIMPIO ===
SERVIDUMBRES_VALOR_TXT_LIMPIO
None         15317
NO            8011
SI             726
PENDIENTE        1
Name: count, dtype: int64

Total filas SERVIDUMBRES con valor Si/No raro (sin mapeo REAL): 0

=== Resumen longitudes SERVIDUMBRES_ANOT_SQL ===
count    24055.000000
mean         0.160091
std          3.339388
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        297.000000
Name: SERVIDUMBRES_ANOT_LEN, dtype: float64

Total filas con SERVIDUMBRES_ANOT_SQL > 4000 caracteres: 0

=== Conteo SERVIDUMBRES_VALOR_S

## MEDIDAS CAUTELARES	


In [33]:
# ======================================
# MEDIDA PROCESAL - MEDIDAS CAUTELARES (AJUSTADO A REGLA "SIN_REG")
# - Normaliza SI/NO/PENDIENTE
# - Trata strings con espacios como vacíos
# - Limpia anotación (trunca a 4000 si se pasa)
# - Regla de negocio:
#     * Si hay anotación y no hay valor -> Valor = "SIN_REG"  (para insertar fila)
#     * Si no hay valor y no hay anotación -> NO insertar fila
# - Exporta:
#     * revision (all)
#     * listo_insertar (solo filas insertables)
#     * valores raros reales (si existen)
# ======================================


import pandas as pd
import numpy as np
import re

# ======================================
# MEDIDA PROCESAL - MEDIDAS CAUTELARES (AJUSTADO A REGLA "SIN_REG")
# (CORREGIDO: NO recarga/reasigna df; el resto queda igual)
# ======================================

# 0) Fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

cols = list(df.columns)

# 1) Localizar columnas (tercer par Si/No + Anotación = MEDIDAS CAUTELARES)
cols_si = [c for c in cols if "SI" in str(c).upper() and "NO" in str(c).upper()]
cols_anot = [c for c in cols if "ANOTAC" in str(c).upper()]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

pos = {c: i for i, c in enumerate(cols)}
cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# MEDIDAS CAUTELARES = tercera pareja (índice 2)
if "Si / No.2" in df.columns:
    col_med_si = "Si / No.2"
else:
    if len(cols_si_ordenadas) < 3:
        raise ValueError("No se encontró una tercera columna 'Si / No' para MEDIDAS CAUTELARES.")
    col_med_si = cols_si_ordenadas[2]

if "Anotación.2" in df.columns:
    col_med_anot = "Anotación.2"
else:
    if len(cols_anot_ordenadas) < 3:
        raise ValueError("No se encontró una tercera columna 'Anotación' para MEDIDAS CAUTELARES.")
    col_med_anot = cols_anot_ordenadas[2]

print("\nMEDIDAS CAUTELARES Si/No   :", col_med_si)
print("MEDIDAS CAUTELARES Anotac. :", col_med_anot)

# 2) RAW de trabajo
df["MEDIDAS_CAUT_SI_RAW"] = df[col_med_si]
df["MEDIDAS_CAUT_ANOT_RAW"] = df[col_med_anot]

# 3) Helpers: vacío real
def es_vacio(v) -> bool:
    if pd.isna(v):
        return True
    if isinstance(v, str) and v.strip() == "":
        return True
    return False

# 4) Normalizar valor SI/NO/PENDIENTE
def normalizar_medidas_caut_valor(v):
    if es_vacio(v):
        return None

    txt = str(v).strip().upper()

    # quitar tildes comunes
    txt = (txt
           .replace("Á", "A")
           .replace("É", "E")
           .replace("Í", "I")
           .replace("Ó", "O")
           .replace("Ú", "U"))

    # limpiar signos simples
    txt = txt.replace(".", "").replace(",", "").replace(";", "")
    txt = " ".join(txt.split()).strip()

    if txt in {"SI", "S"}:
        return "SI"
    if txt in {"NO", "N"}:
        return "NO"
    if "PEND" in txt:
        return "PENDIENTE"

    return None  # raro

df["MEDIDAS_CAUT_VALOR_TXT_LIMPIO"] = df["MEDIDAS_CAUT_SI_RAW"].apply(normalizar_medidas_caut_valor)
df["MEDIDAS_CAUT_VALOR_SQL"] = df["MEDIDAS_CAUT_VALOR_TXT_LIMPIO"]

print("\n=== Conteo MEDIDAS_CAUT_VALOR_TXT_LIMPIO ===")
print(df["MEDIDAS_CAUT_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

# Valores raros reales (no vacío pero no mapea)
mask_valor_raro = (
    df["MEDIDAS_CAUT_SI_RAW"].apply(lambda x: not es_vacio(x))
    & df["MEDIDAS_CAUT_VALOR_TXT_LIMPIO"].isna()
)

medidas_caut_valores_raros = df.loc[
    mask_valor_raro,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", "MEDIDAS_CAUT_SI_RAW"],
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\nTotal filas MEDIDAS CAUTELARES con valor raro REAL:", len(medidas_caut_valores_raros))
if not medidas_caut_valores_raros.empty:
    print(medidas_caut_valores_raros.head(30).to_string(index=False))

# 5) Limpiar anotación (varchar(4000))
def limpiar_anotacion_medidas_caut(v):
    if es_vacio(v):
        return None
    txt = str(v)
    txt = txt.replace("\r", " ").replace("\n", " ").replace("\t", " ")
    txt = " ".join(txt.split()).strip()
    return txt if txt != "" else None

df["MEDIDAS_CAUT_ANOT_SQL"] = df["MEDIDAS_CAUT_ANOT_RAW"].apply(limpiar_anotacion_medidas_caut)
df["MEDIDAS_CAUT_ANOT_LEN"] = df["MEDIDAS_CAUT_ANOT_SQL"].fillna("").str.len()

print("\n=== Resumen longitudes MEDIDAS_CAUT_ANOT_SQL ===")
print(df["MEDIDAS_CAUT_ANOT_LEN"].describe())

df["MEDIDAS_CAUT_FLAG_ANOT_GT4000"] = df["MEDIDAS_CAUT_ANOT_LEN"] > 4000
df.loc[df["MEDIDAS_CAUT_FLAG_ANOT_GT4000"], "MEDIDAS_CAUT_ANOT_SQL"] = (
    df.loc[df["MEDIDAS_CAUT_FLAG_ANOT_GT4000"], "MEDIDAS_CAUT_ANOT_SQL"].str.slice(0, 4000)
)
df.loc[df["MEDIDAS_CAUT_FLAG_ANOT_GT4000"], "MEDIDAS_CAUT_ANOT_LEN"] = 4000

# 6) Regla de negocio: si hay anotación y no hay valor -> Valor = SIN_REG
df["MEDIDAS_CAUT_VALOR_SQL_FINAL"] = df["MEDIDAS_CAUT_VALOR_SQL"].copy()
mask_anot_sin_valor = df["MEDIDAS_CAUT_ANOT_SQL"].notna() & df["MEDIDAS_CAUT_VALOR_SQL_FINAL"].isna()
df.loc[mask_anot_sin_valor, "MEDIDAS_CAUT_VALOR_SQL_FINAL"] = "SIN_REG"

print("\n=== Conteo MEDIDAS_CAUT_VALOR_SQL_FINAL ===")
print(df["MEDIDAS_CAUT_VALOR_SQL_FINAL"].value_counts(dropna=False))

# 7) Estado de control (ok / sin_reg / vacio / pendiente)
df["MEDIDAS_CAUT_ESTADO"] = "vacio"
df.loc[df["MEDIDAS_CAUT_VALOR_TXT_LIMPIO"].notna(), "MEDIDAS_CAUT_ESTADO"] = "ok"
df.loc[mask_anot_sin_valor, "MEDIDAS_CAUT_ESTADO"] = "sin_reg"
df.loc[mask_valor_raro, "MEDIDAS_CAUT_ESTADO"] = "pendiente"

print("\n=== Conteo por MEDIDAS_CAUT_ESTADO ===")
print(df["MEDIDAS_CAUT_ESTADO"].value_counts(dropna=False))

# 8) (CAMBIO CLAVE) NO reordenar ni hacer df = df[cols_all]
#     Solo dejamos las columnas creadas en df tal cual (sin recargar/reasignar df)

# 9) Excel revisión (all)
cols_export = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    col_med_si,
    "MEDIDAS_CAUT_SI_RAW",
    "MEDIDAS_CAUT_VALOR_TXT_LIMPIO",
    "MEDIDAS_CAUT_VALOR_SQL",
    "MEDIDAS_CAUT_VALOR_SQL_FINAL",
    "MEDIDAS_CAUT_ESTADO",
    col_med_anot,
    "MEDIDAS_CAUT_ANOT_RAW",
    "MEDIDAS_CAUT_ANOT_SQL",
    "MEDIDAS_CAUT_ANOT_LEN",
    "MEDIDAS_CAUT_FLAG_ANOT_GT4000",
]
cols_export = [c for c in cols_export if c in df.columns]
df_medidas_all = df[cols_export].copy()

archivo_revision = "MedidaProcesal_MedidasCautelares_revision.xlsx"
df_medidas_all.to_excel(archivo_revision, index=False)

# 10) Dataset listo para insertar (solo filas insertables)
mask_listo_insertar = df["MEDIDAS_CAUT_VALOR_SQL_FINAL"].notna()
df_listo_insertar = df.loc[
    mask_listo_insertar,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL"],
].copy()

df_listo_insertar.rename(columns={
    "MEDIDAS_CAUT_VALOR_SQL_FINAL": "Valor",
    "MEDIDAS_CAUT_ANOT_SQL": "Anotacion",
}, inplace=True)

df_listo_insertar["Objeto"] = "MEDIDAS CAUTELARES"
# df_listo_insertar["IdEstudioTerreno"] = ...  # se completa luego con el cruce

archivo_listo = "MedidaProcesal_MedidasCautelares_listo_insertar.xlsx"
df_listo_insertar.to_excel(archivo_listo, index=False)

# 11) Excels adicionales
archivo_raros = None
if not medidas_caut_valores_raros.empty:
    archivo_raros = "MedidaProcesal_MedidasCautelares_valores_raros_reales.xlsx"
    medidas_caut_valores_raros.to_excel(archivo_raros, index=False)

print("\nArchivos creados:")
print(f" - {archivo_revision}")
print(f" - {archivo_listo} (solo filas con Valor final para insertar)")
if archivo_raros:
    print(f" - {archivo_raros} (valores Si/No raros reales)")


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6', 'ENTREGÓ CARPETA DE SOPORTES\n(SI / NO)', 'HIPOTECA_SI_NORM']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6', 'MP_Anotacion']

MEDIDAS CAUTELARES Si/No   : Si / No.2
MEDIDAS CAUTELARES Anotac. : Anotación.2

=== Conteo MEDIDAS_CAUT_VALOR_TXT_LIMPIO ===
MEDIDAS_CAUT_VALOR_TXT_LIMPIO
None         12688
NO           10125
SI            1241
PENDIENTE        1
Name: count, dtype: int64

Total filas MEDIDAS CAUTELARES con valor raro REAL: 0

=== Resumen longitudes MEDIDAS_CAUT_ANOT_SQL ===
count    24055.000000
mean         0.170692
std          3.314977
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        281.000000
Name: MEDIDAS_CAUT_ANOT_LEN, dtype: float64

=== Conteo MEDIDAS_CAUT_VALOR_SQL_FINAL ===
MEDIDAS_CAUT_VALOR_SQL_FINAL
None         12686

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2430834975.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["MEDIDAS_CAUT_SI_RAW"] = df[col_med_si]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2430834975.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["MEDIDAS_CAUT_ANOT_RAW"] = df[col_med_anot]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2430834975.py:103: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh


Archivos creados:
 - MedidaProcesal_MedidasCautelares_revision.xlsx
 - MedidaProcesal_MedidasCautelares_listo_insertar.xlsx (solo filas con Valor final para insertar)


## RUPTA

In [34]:
import pandas as pd
import numpy as np
import re

# ======================================
# MEDIDA PROCESAL - RUPTA (ROBUSTO)
# (CORREGIDO: NO recarga/reasigna df; resto igual)
# - NO hace: df = df.loc[..., ...].copy()
# - NO hace: df = pd.concat(...)
# - En su lugar:
#     * Elimina/limpia columnas RUPTA_* IN-PLACE (drop)
#     * Calcula en DF auxiliar (work)
#     * Asigna columnas al df IN-PLACE (df[col] = ...)
# ======================================

# 0) Fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas RUPTA_* si existen (SIN recargar df)
cols_rupta_prev = [c for c in df.columns if isinstance(c, str) and c.startswith("RUPTA_")]
if cols_rupta_prev:
    df.drop(columns=cols_rupta_prev, inplace=True, errors="ignore")

# (Opcional) desduplicar columnas idénticas NO se aplica aquí porque implica rearmar df

cols = list(df.columns)
pos = {c: i for i, c in enumerate(cols)}

# 1) Detectar SOLO columnas base del Excel: "Si / No", "Si / No.1", ... y "Anotación", "Anotación.1", ...
pat_si   = re.compile(r"^Si / No(\.\d+)?$")
pat_anot = re.compile(r"^Anotación(\.\d+)?$")

cols_si = [c for c in cols if isinstance(c, str) and pat_si.match(c)]
cols_anot = [c for c in cols if isinstance(c, str) and pat_anot.match(c)]

print("Columnas 'Si / No' (Excel):", cols_si)
print("Columnas 'Anotación' (Excel):", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas base 'Si / No' o 'Anotación' del Excel.")

cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# 2) RUPTA = 4to par (HIPOTECA=0, SERVIDUMBRES=1, MED.CAUT=2, RUPTA=3)
RUPTA_IDX = 3

col_rupta_si = f"Si / No.{RUPTA_IDX}" if f"Si / No.{RUPTA_IDX}" in df.columns else cols_si_ordenadas[RUPTA_IDX]
col_rupta_anot = f"Anotación.{RUPTA_IDX}" if f"Anotación.{RUPTA_IDX}" in df.columns else cols_anot_ordenadas[RUPTA_IDX]

col_rupta_clasif = "Colectivo / Individual"
if col_rupta_clasif not in df.columns:
    raise ValueError("No se encontró la columna 'Colectivo / Individual' para RUPTA.")

print("\nRUPTA Si/No   :", col_rupta_si)
print("RUPTA Anotac. :", col_rupta_anot)
print("RUPTA Clasif. :", col_rupta_clasif)

# 3) Helpers
def es_vacio(v) -> bool:
    if pd.isna(v):
        return True
    if isinstance(v, str) and v.strip() == "":
        return True
    return False

def normalizar_si_no_pend(v):
    if es_vacio(v):
        return None
    t = str(v).strip().upper()
    t = (t.replace("Á","A").replace("É","E").replace("Í","I").replace("Ó","O").replace("Ú","U"))
    t = t.replace(".", "").replace(",", "").replace(";", "")
    t = " ".join(t.split()).strip()

    if t in {"SI", "S"}:
        return "SI"
    if t in {"NO", "N"}:
        return "NO"
    if "PEND" in t:
        return "PENDIENTE"
    return None

def limpiar_anotacion(v):
    if es_vacio(v):
        return None
    t = str(v).replace("\r", " ").replace("\n", " ").replace("\t", " ")
    t = " ".join(t.split()).strip()
    return t if t else None

def limpiar_clasif(v):
    if es_vacio(v):
        return None
    t = str(v).strip().upper()
    t = (t.replace("Á","A").replace("É","E").replace("Í","I").replace("Ó","O").replace("Ú","U"))
    t = " ".join(t.split()).strip()
    return t if t else None

def mapear_clasif(t):
    if t is None:
        return None
    if t.startswith("COLECTIV"):
        return "COLECTIVO"
    if t.startswith("INDIVID"):
        return "INDIVIDUAL"
    if "PEND" in t:
        return "PENDIENTE"
    return None

# 4) DF auxiliar (work) para no fragmentar df
work = pd.DataFrame(index=df.index)

# RAW
work["RUPTA_SI_RAW"] = df[col_rupta_si]
work["RUPTA_ANOT_RAW"] = df[col_rupta_anot]
work["RUPTA_CLASIF_RAW"] = df[col_rupta_clasif]

# Valor
work["RUPTA_VALOR_TXT_LIMPIO"] = work["RUPTA_SI_RAW"].apply(normalizar_si_no_pend)
work["RUPTA_VALOR_SQL"] = work["RUPTA_VALOR_TXT_LIMPIO"]

print("\n=== Conteo RUPTA_VALOR_TXT_LIMPIO ===")
print(work["RUPTA_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

mask_valor_raro_real = (
    work["RUPTA_SI_RAW"].apply(lambda x: not es_vacio(x))
    & work["RUPTA_VALOR_TXT_LIMPIO"].isna()
)
print("\nTotal RUPTA con valor raro REAL (sin mapeo):", int(mask_valor_raro_real.sum()))

# Anotación (max 4000)
work["RUPTA_ANOT_SQL"] = work["RUPTA_ANOT_RAW"].apply(limpiar_anotacion)
work["RUPTA_ANOT_LEN"] = work["RUPTA_ANOT_SQL"].fillna("").str.len()
work["RUPTA_ANOT_GT4000"] = work["RUPTA_ANOT_LEN"] > 4000
work.loc[work["RUPTA_ANOT_GT4000"], "RUPTA_ANOT_SQL"] = work.loc[work["RUPTA_ANOT_GT4000"], "RUPTA_ANOT_SQL"].str.slice(0, 4000)
work["RUPTA_ANOT_LEN"] = work["RUPTA_ANOT_SQL"].fillna("").str.len()

print("\n=== Resumen longitudes RUPTA_ANOT_SQL ===")
print(work["RUPTA_ANOT_LEN"].describe())
print("Total anotaciones > 4000:", int(work["RUPTA_ANOT_GT4000"].sum()))

# Clasificación
work["RUPTA_CLASIF_TXT_LIMPIO"] = work["RUPTA_CLASIF_RAW"].apply(limpiar_clasif)
work["RUPTA_TIPOCLASIF_SQL"] = work["RUPTA_CLASIF_TXT_LIMPIO"].apply(mapear_clasif)

print("\n=== Conteo RUPTA_TIPOCLASIF_SQL ===")
print(work["RUPTA_TIPOCLASIF_SQL"].value_counts(dropna=False))

mask_clasif_raro_real = work["RUPTA_CLASIF_TXT_LIMPIO"].notna() & work["RUPTA_TIPOCLASIF_SQL"].isna()
print("\nTotal RUPTA con clasif rara REAL:", int(mask_clasif_raro_real.sum()))

# Valor final
work["RUPTA_VALOR_SQL_FINAL"] = work["RUPTA_VALOR_SQL"].copy()
mask_info_sin_valor = (
    work["RUPTA_VALOR_SQL_FINAL"].isna()
    & (work["RUPTA_ANOT_SQL"].notna() | work["RUPTA_TIPOCLASIF_SQL"].notna())
)
work.loc[mask_info_sin_valor, "RUPTA_VALOR_SQL_FINAL"] = "SIN_REG"

# Estado
work["RUPTA_ESTADO"] = "vacio"
work.loc[work["RUPTA_VALOR_SQL"].notna(), "RUPTA_ESTADO"] = "ok"
work.loc[mask_info_sin_valor, "RUPTA_ESTADO"] = "sin_reg"
work.loc[mask_valor_raro_real, "RUPTA_ESTADO"] = "pendiente"

print("\n=== Conteo por RUPTA_ESTADO ===")
print(work["RUPTA_ESTADO"].value_counts(dropna=False))

print("\n=== Conteo RUPTA_VALOR_SQL_FINAL ===")
print(work["RUPTA_VALOR_SQL_FINAL"].value_counts(dropna=False))

# 5) (CAMBIO CLAVE) Asignar columnas al df SIN df = pd.concat(...)
for c in work.columns:
    df[c] = work[c].values

# 6) Exportes
base_cols = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

# all_mapeo
cols_all = base_cols + [
    col_rupta_si,
    "RUPTA_SI_RAW",
    "RUPTA_VALOR_TXT_LIMPIO",
    "RUPTA_VALOR_SQL",
    "RUPTA_VALOR_SQL_FINAL",
    col_rupta_anot,
    "RUPTA_ANOT_RAW",
    "RUPTA_ANOT_SQL",
    "RUPTA_ANOT_LEN",
    "RUPTA_ANOT_GT4000",
    col_rupta_clasif,
    "RUPTA_CLASIF_RAW",
    "RUPTA_CLASIF_TXT_LIMPIO",
    "RUPTA_TIPOCLASIF_SQL",
    "RUPTA_ESTADO",
]
cols_all = [c for c in cols_all if c in df.columns]
df_all = df[cols_all].copy()

# raros
rupta_valores_raros = df.loc[
    mask_valor_raro_real,
    base_cols + ["RUPTA_SI_RAW"]
].copy()

rupta_clasif_raros = df.loc[
    mask_clasif_raro_real,
    base_cols + ["RUPTA_CLASIF_RAW", "RUPTA_CLASIF_TXT_LIMPIO"]
].copy()

# listo_insertar (solo donde hay Valor final)
col_valor_final = df["RUPTA_VALOR_SQL_FINAL"]
if isinstance(col_valor_final, pd.DataFrame):  # por si quedaron duplicadas por algo externo
    col_valor_final = col_valor_final.iloc[:, 0]

mask_listo = col_valor_final.notna()

cols_listo = base_cols + ["RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL"]
cols_listo = [c for c in cols_listo if c in df.columns]

df_listo = df.loc[mask_listo, cols_listo].copy()

df_listo.rename(columns={
    "RUPTA_VALOR_SQL_FINAL": "Valor",
    "RUPTA_ANOT_SQL": "Anotacion",
    "RUPTA_TIPOCLASIF_SQL": "TipoClasificacion",
}, inplace=True)

df_listo["Objeto"] = "RUPTA"
# df_listo["IdEstudioTerreno"] = ...  # se completa luego con el cruce

# Resúmenes
resumen_estado = (
    df["RUPTA_ESTADO"].value_counts(dropna=False)
      .rename_axis("RUPTA_ESTADO")
      .reset_index(name="conteo")
)
resumen_valor_final = (
    col_valor_final.value_counts(dropna=False)
      .rename_axis("RUPTA_VALOR_SQL_FINAL")
      .reset_index(name="conteo")
)
resumen_clasif = (
    df["RUPTA_TIPOCLASIF_SQL"].value_counts(dropna=False)
      .rename_axis("RUPTA_TIPOCLASIF_SQL")
      .reset_index(name="conteo")
)

archivo_revision = "MedidaProcesal_RUPTA_revision.xlsx"
archivo_listo = "MedidaProcesal_RUPTA_listo_insertar.xlsx"

with pd.ExcelWriter(archivo_revision, engine="openpyxl") as writer:
    df_all.to_excel(writer, index=False, sheet_name="all_mapeo")
    resumen_estado.to_excel(writer, index=False, sheet_name="resumen_estado")
    resumen_valor_final.to_excel(writer, index=False, sheet_name="resumen_valor_final")
    resumen_clasif.to_excel(writer, index=False, sheet_name="resumen_clasif")
    rupta_valores_raros.to_excel(writer, index=False, sheet_name="valor_raro_real")
    rupta_clasif_raros.to_excel(writer, index=False, sheet_name="clasif_rara_real")
    df_listo.to_excel(writer, index=False, sheet_name="listo_insertar")

df_listo.to_excel(archivo_listo, index=False)

print("\nArchivos creados:")
print(" -", archivo_revision)
print(" -", archivo_listo)


Columnas 'Si / No' (Excel): ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6']
Columnas 'Anotación' (Excel): ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']

RUPTA Si/No   : Si / No.3
RUPTA Anotac. : Anotación.3
RUPTA Clasif. : Colectivo / Individual

=== Conteo RUPTA_VALOR_TXT_LIMPIO ===
RUPTA_VALOR_TXT_LIMPIO
None         12710
NO           10903
SI             441
PENDIENTE        1
Name: count, dtype: int64

Total RUPTA con valor raro REAL (sin mapeo): 0

=== Resumen longitudes RUPTA_ANOT_SQL ===
count    24055.000000
mean         0.085762
std          3.020936
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        281.000000
Name: RUPTA_ANOT_LEN, dtype: float64
Total anotaciones > 4000: 0

=== Conteo RUPTA_TIPOCLASIF_SQL ===
RUPTA_TIPOCLASIF_SQL
None          23867
COLECTIVO        98
INDIVIDUAL       90
Name: count, dtype: int64

Total RUPTA c

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\790784771.py:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = work[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\790784771.py:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = work[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\790784771.py:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a


Archivos creados:
 - MedidaProcesal_RUPTA_revision.xlsx
 - MedidaProcesal_RUPTA_listo_insertar.xlsx


## RTDAF

In [35]:
import re
import numpy as np
import pandas as pd

# ============================================================
# MEDIDA PROCESAL - RTDAF LEY 1448 (AJUSTADO)
# (CORREGIDO: NO recarga/reasigna df; resto igual)
# - NO hace: df = df.copy()
# - NO hace: df = pd.concat([df, nuevo], axis=1)
# - En su lugar:
#     * Borra RTDAF_* previas IN-PLACE (drop) para re-ejecución segura
#     * Calcula en DF auxiliar (work/nuevo)
#     * Asigna columnas al df IN-PLACE: df[c] = nuevo[c].values
# ============================================================

# 0) Fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas RTDAF_* si existen (SIN recargar df)
cols_rtdaf_prev = [c for c in df.columns if isinstance(c, str) and c.startswith("RTDAF_")]
if cols_rtdaf_prev:
    df.drop(columns=cols_rtdaf_prev, inplace=True, errors="ignore")

cols = list(df.columns)
pos = {c: i for i, c in enumerate(cols)}

# 1) Detectar SOLO columnas originales del Excel
#    (Ej: "Si / No", "Si / No.1"... y "Anotación", "Anotación.1"...)
re_si = re.compile(r"^Si\s*/\s*No(\.\d+)?$", re.IGNORECASE)
re_an = re.compile(r"^Anotaci[oó]n(\.\d+)?$", re.IGNORECASE)  # soporta Anotación / Anotacion

cols_si = [c for c in cols if re_si.match(str(c).strip())]
cols_anot = [c for c in cols if re_an.match(str(c).strip())]

print("Columnas 'Si / No' (Excel):", cols_si)
print("Columnas 'Anotación' (Excel):", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas originales 'Si / No' o 'Anotación' del Excel en el DataFrame.")

# Orden real en el Excel
cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# RTDAF = 5to par (index 4): Si/No.4 y Anotación.4
if "Si / No.4" in df.columns:
    rtdaf_si_col = "Si / No.4"
else:
    if len(cols_si_ordenadas) < 5:
        raise ValueError("No se encontró una quinta columna 'Si / No' para RTDAF (par #5).")
    rtdaf_si_col = cols_si_ordenadas[4]

if "Anotación.4" in df.columns:
    rtdaf_anot_col = "Anotación.4"
elif "Anotacion.4" in df.columns:
    rtdaf_anot_col = "Anotacion.4"
else:
    if len(cols_anot_ordenadas) < 5:
        raise ValueError("No se encontró una quinta columna 'Anotación' para RTDAF (par #5).")
    rtdaf_anot_col = cols_anot_ordenadas[4]

print("\nRTDAF Si/No   :", rtdaf_si_col)
print("RTDAF Anotac. :", rtdaf_anot_col)

# 2) Helpers
def es_vacio(v) -> bool:
    if pd.isna(v):
        return True
    if isinstance(v, str) and v.strip() == "":
        return True
    return False

def quitar_tildes_upper(txt: str) -> str:
    txt = txt.strip().upper()
    return (txt.replace("Á", "A")
               .replace("É", "E")
               .replace("Í", "I")
               .replace("Ó", "O")
               .replace("Ú", "U"))

def normalizar_si_no_pend(v):
    if es_vacio(v):
        return None
    t = quitar_tildes_upper(str(v))
    # limpiar puntuación simple
    t = t.replace(".", "").replace(",", "").replace(";", "")
    t = " ".join(t.split())

    if t in {"SI", "S"}:
        return "SI"
    if t in {"NO", "N"}:
        return "NO"
    if "PEND" in t:
        return "PENDIENTE"
    return None  # raro real

def limpiar_anotacion(v):
    if es_vacio(v):
        return None
    txt = str(v).replace("\r", " ").replace("\n", " ").replace("\t", " ")
    txt = " ".join(txt.split()).strip()
    return txt if txt != "" else None

# 3) Construir columnas nuevas (en bloque) en DF auxiliar
rtdaf_valor_raw = df[rtdaf_si_col]
rtdaf_anot_raw  = df[rtdaf_anot_col]

rtdaf_valor_txt = rtdaf_valor_raw.apply(normalizar_si_no_pend)
rtdaf_valor_sql = rtdaf_valor_txt  # varchar(10)

rtdaf_anot_sql = rtdaf_anot_raw.apply(limpiar_anotacion)
rtdaf_anot_len = rtdaf_anot_sql.fillna("").str.len()

# Truncar anotación si excede 4000
RTDAF_ANOT_MAX = 4000
rtdaf_anot_gt4000 = rtdaf_anot_len > RTDAF_ANOT_MAX
rtdaf_anot_sql = rtdaf_anot_sql.where(~rtdaf_anot_gt4000, rtdaf_anot_sql.str.slice(0, RTDAF_ANOT_MAX))

# 4) Regla: si hay anotación y no hay valor -> SIN_REG
# (PENDIENTE se reserva para cuando el Excel realmente dice PENDIENTE)
rtdaf_valor_sql_final = rtdaf_valor_sql.copy()
mask_solo_anot = rtdaf_anot_sql.notna() & rtdaf_valor_sql_final.isna()
rtdaf_valor_sql_final.loc[mask_solo_anot] = "SIN_REG"

# 5) Estado (útil para auditoría)
rtdaf_estado = pd.Series("vacio", index=df.index, dtype="object")
rtdaf_estado.loc[rtdaf_valor_sql.notna()] = "ok"
rtdaf_estado.loc[mask_solo_anot] = "sin_reg"

# raros reales (hay contenido no vacío pero no mapeó)
mask_valor_raro_real = rtdaf_valor_raw.apply(lambda x: not es_vacio(x)) & rtdaf_valor_sql.isna()
rtdaf_estado.loc[mask_valor_raro_real] = "pendiente"

# 6) Adjuntar columnas al df SIN pd.concat y SIN recargar df
nuevo = pd.DataFrame({
    "RTDAF_VALOR_RAW": rtdaf_valor_raw,
    "RTDAF_VALOR_TXT_LIMPIO": rtdaf_valor_txt,
    "RTDAF_VALOR_SQL": rtdaf_valor_sql,
    "RTDAF_VALOR_SQL_FINAL": rtdaf_valor_sql_final,
    "RTDAF_ESTADO": rtdaf_estado,
    "RTDAF_ANOT_RAW": rtdaf_anot_raw,
    "RTDAF_ANOT_SQL": rtdaf_anot_sql,
    "RTDAF_ANOT_LEN": rtdaf_anot_len,
    "RTDAF_ANOT_GT4000": rtdaf_anot_gt4000,
}, index=df.index)

for c in nuevo.columns:
    df[c] = nuevo[c].values

# 7) Reportes por consola
print("\n=== Conteo RTDAF_VALOR_TXT_LIMPIO ===")
print(df["RTDAF_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

print("\n=== Conteo RTDAF_VALOR_SQL_FINAL ===")
print(df["RTDAF_VALOR_SQL_FINAL"].value_counts(dropna=False))

print("\n=== Conteo por RTDAF_ESTADO ===")
print(df["RTDAF_ESTADO"].value_counts(dropna=False))

print("\n=== Resumen longitudes RTDAF_ANOT_SQL ===")
print(df["RTDAF_ANOT_LEN"].describe())

# 8) Valores raros reales (para revisar)
rtdaf_valores_raros = df.loc[
    mask_valor_raro_real,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", rtdaf_si_col, "RTDAF_VALOR_RAW"]
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\nTotal RTDAF con valor raro REAL (sin mapeo):", len(rtdaf_valores_raros))
if not rtdaf_valores_raros.empty:
    print(rtdaf_valores_raros.head(30).to_string(index=False))

# 9) Dataset listo para insertar (solo donde hay Valor final)
base_cols = ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"]
base_cols = [c for c in base_cols if c in df.columns]

mask_listo = df["RTDAF_VALOR_SQL_FINAL"].notna()

df_listo = df.loc[
    mask_listo,
    base_cols + ["RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL"]
].copy()

df_listo.rename(columns={
    "RTDAF_VALOR_SQL_FINAL": "Valor",
    "RTDAF_ANOT_SQL": "Anotacion",
}, inplace=True)

df_listo["Objeto"] = "RTDAF LEY 1448"
df_listo["TipoClasificacion"] = None
# df_listo["IdEstudioTerreno"] = ...  # se completa después con el cruce

# 10) Exportar Excels
archivo_revision = "MedidaProcesal_RTDAF_revision.xlsx"
archivo_listo = "MedidaProcesal_RTDAF_listo_insertar.xlsx"
archivo_raros = None

# Vista completa de auditoría (orden de columnas)
cols_export = (
    base_cols
    + [rtdaf_si_col, rtdaf_anot_col]
    + [
        "RTDAF_VALOR_RAW",
        "RTDAF_VALOR_TXT_LIMPIO",
        "RTDAF_VALOR_SQL",
        "RTDAF_VALOR_SQL_FINAL",
        "RTDAF_ESTADO",
        "RTDAF_ANOT_RAW",
        "RTDAF_ANOT_SQL",
        "RTDAF_ANOT_LEN",
        "RTDAF_ANOT_GT4000",
    ]
)
cols_export = [c for c in cols_export if c in df.columns]

df_all = df[cols_export].copy()

resumen_estado = (
    df["RTDAF_ESTADO"].value_counts(dropna=False)
      .rename_axis("RTDAF_ESTADO")
      .reset_index(name="conteo")
)

resumen_valor_final = (
    df["RTDAF_VALOR_SQL_FINAL"].value_counts(dropna=False)
      .rename_axis("RTDAF_VALOR_SQL_FINAL")
      .reset_index(name="conteo")
)

with pd.ExcelWriter(archivo_revision, engine="openpyxl") as writer:
    df_all.to_excel(writer, index=False, sheet_name="all_mapeo")
    resumen_estado.to_excel(writer, index=False, sheet_name="resumen_estado")
    resumen_valor_final.to_excel(writer, index=False, sheet_name="resumen_valor_final")
    rtdaf_valores_raros.to_excel(writer, index=False, sheet_name="valor_raro_real")
    df_listo.to_excel(writer, index=False, sheet_name="listo_insertar")

df_listo.to_excel(archivo_listo, index=False)

if not rtdaf_valores_raros.empty:
    archivo_raros = "MedidaProcesal_RTDAF_valores_raros_reales.xlsx"
    rtdaf_valores_raros.to_excel(archivo_raros, index=False)

print("\nArchivos creados:")
print(f" - {archivo_revision} (hojas: all_mapeo, resumen_*, valor_raro_real, listo_insertar)")
print(f" - {archivo_listo} (solo filas listas para insertar)")
if archivo_raros:
    print(f" - {archivo_raros} (valores Si/No raros reales)")


Columnas 'Si / No' (Excel): ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6']
Columnas 'Anotación' (Excel): ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']

RTDAF Si/No   : Si / No.4
RTDAF Anotac. : Anotación.4

=== Conteo RTDAF_VALOR_TXT_LIMPIO ===
RTDAF_VALOR_TXT_LIMPIO
None         15307
NO            8652
SI              95
PENDIENTE        1
Name: count, dtype: int64

=== Conteo RTDAF_VALOR_SQL_FINAL ===
RTDAF_VALOR_SQL_FINAL
None         15296
NO            8652
SI              95
SIN_REG         11
PENDIENTE        1
Name: count, dtype: int64

=== Conteo por RTDAF_ESTADO ===
RTDAF_ESTADO
vacio      15296
ok          8748
sin_reg       11
Name: count, dtype: int64

=== Resumen longitudes RTDAF_ANOT_SQL ===
count    24055.000000
mean         0.013344
std          0.339638
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         23.000000
Name: 

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1579113328.py:149: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = nuevo[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1579113328.py:149: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = nuevo[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1579113328.py:149: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join


Archivos creados:
 - MedidaProcesal_RTDAF_revision.xlsx (hojas: all_mapeo, resumen_*, valor_raro_real, listo_insertar)
 - MedidaProcesal_RTDAF_listo_insertar.xlsx (solo filas listas para insertar)


## ofertas otra entidad

In [36]:
import re
import numpy as np
import pandas as pd

# ============================================================
# MEDIDA PROCESAL - OFERTA OTRAS ENTIDADES (AJUSTADO)
# (CORREGIDO: NO recarga/reasigna df; resto igual)
# - NO hace: df = df.copy()
# - NO hace: df = pd.concat([df, nuevo], axis=1)
# - En su lugar:
#     * Borra OFERTA_* previas IN-PLACE (drop) para re-ejecución segura
#     * Calcula en DF auxiliar (nuevo/work)
#     * Asigna columnas al df IN-PLACE: df[c] = nuevo[c].values
# ============================================================

# 0) Fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas OFERTA_* si existen (SIN recargar df)
cols_oferta_prev = [c for c in df.columns if isinstance(c, str) and c.startswith("OFERTA_")]
if cols_oferta_prev:
    df.drop(columns=cols_oferta_prev, inplace=True, errors="ignore")

cols = list(df.columns)
pos = {c: i for i, c in enumerate(cols)}

# 1) Columnas originales del Excel
re_si = re.compile(r"^Si\s*/\s*No(\.\d+)?$", re.IGNORECASE)
re_an = re.compile(r"^Anotaci[oó]n(\.\d+)?$", re.IGNORECASE)

cols_si = [c for c in cols if re_si.match(str(c).strip())]
cols_anot = [c for c in cols if re_an.match(str(c).strip())]

print("Columnas 'Si / No' (Excel):", cols_si)
print("Columnas 'Anotación' (Excel):", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas originales 'Si / No' o 'Anotación' del Excel en el DataFrame.")

cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# OFERTA OTRAS ENTIDADES = 6to par (index 5): Si / No.5 y Anotación.5
if "Si / No.5" in df.columns:
    oferta_si_col = "Si / No.5"
else:
    if len(cols_si_ordenadas) < 6:
        raise ValueError("No se encontró la 6ta columna 'Si / No' para OFERTA (par #6).")
    oferta_si_col = cols_si_ordenadas[5]

if "Anotación.5" in df.columns:
    oferta_anot_col = "Anotación.5"
elif "Anotacion.5" in df.columns:
    oferta_anot_col = "Anotacion.5"
else:
    if len(cols_anot_ordenadas) < 6:
        raise ValueError("No se encontró la 6ta columna 'Anotación' para OFERTA (par #6).")
    oferta_anot_col = cols_anot_ordenadas[5]

print("\nOFERTA Si/No   :", oferta_si_col)
print("OFERTA Anotac. :", oferta_anot_col)

# 2) Helpers
def es_vacio(v) -> bool:
    if pd.isna(v):
        return True
    if isinstance(v, str) and v.strip() == "":
        return True
    return False

def quitar_tildes_upper(txt: str) -> str:
    txt = txt.strip().upper()
    return (txt.replace("Á", "A")
               .replace("É", "E")
               .replace("Í", "I")
               .replace("Ó", "O")
               .replace("Ú", "U"))

def normalizar_si_no_pend(v):
    if es_vacio(v):
        return None
    t = quitar_tildes_upper(str(v))
    t = t.replace(".", "").replace(",", "").replace(";", "")
    t = " ".join(t.split())

    if t in {"SI", "S"}:
        return "SI"
    if t in {"NO", "N"}:
        return "NO"
    if "PEND" in t:
        return "PENDIENTE"
    return None

def limpiar_anotacion(v):
    if es_vacio(v):
        return None
    txt = str(v).replace("\r", " ").replace("\n", " ").replace("\t", " ")
    txt = " ".join(txt.split()).strip()
    return txt if txt != "" else None

# 3) Transformación (en bloque)
oferta_valor_raw = df[oferta_si_col]
oferta_anot_raw  = df[oferta_anot_col]

oferta_valor_txt = oferta_valor_raw.apply(normalizar_si_no_pend)
oferta_valor_sql = oferta_valor_txt

oferta_anot_sql = oferta_anot_raw.apply(limpiar_anotacion)
oferta_anot_len = oferta_anot_sql.fillna("").str.len()

# Truncar anotación si excede 4000
OFERTA_ANOT_MAX = 4000
oferta_anot_gt4000 = oferta_anot_len > OFERTA_ANOT_MAX
oferta_anot_sql = oferta_anot_sql.where(~oferta_anot_gt4000, oferta_anot_sql.str.slice(0, OFERTA_ANOT_MAX))

# 4) Regla: si hay anotación y no hay valor -> SIN_REG
oferta_valor_sql_final = oferta_valor_sql.copy()
mask_solo_anot = oferta_anot_sql.notna() & oferta_valor_sql_final.isna()
oferta_valor_sql_final.loc[mask_solo_anot] = "SIN_REG"

# 5) Estado
oferta_estado = pd.Series("vacio", index=df.index, dtype="object")
oferta_estado.loc[oferta_valor_sql.notna()] = "ok"
oferta_estado.loc[mask_solo_anot] = "sin_reg"

mask_valor_raro_real = oferta_valor_raw.apply(lambda x: not es_vacio(x)) & oferta_valor_sql.isna()
oferta_estado.loc[mask_valor_raro_real] = "pendiente"

# 6) Adjuntar columnas al df (SIN pd.concat y SIN recargar df)
nuevo = pd.DataFrame({
    "OFERTA_VALOR_RAW": oferta_valor_raw,
    "OFERTA_VALOR_TXT_LIMPIO": oferta_valor_txt,
    "OFERTA_VALOR_SQL": oferta_valor_sql,
    "OFERTA_VALOR_SQL_FINAL": oferta_valor_sql_final,
    "OFERTA_ESTADO": oferta_estado,
    "OFERTA_ANOT_RAW": oferta_anot_raw,
    "OFERTA_ANOT_SQL": oferta_anot_sql,
    "OFERTA_ANOT_LEN": oferta_anot_len,
    "OFERTA_ANOT_GT4000": oferta_anot_gt4000,
}, index=df.index)

for c in nuevo.columns:
    df[c] = nuevo[c].values

# 7) Reportes
print("\n=== Conteo OFERTA_VALOR_TXT_LIMPIO ===")
print(df["OFERTA_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

print("\n=== Conteo OFERTA_VALOR_SQL_FINAL ===")
print(df["OFERTA_VALOR_SQL_FINAL"].value_counts(dropna=False))

print("\n=== Conteo por OFERTA_ESTADO ===")
print(df["OFERTA_ESTADO"].value_counts(dropna=False))

print("\n=== Resumen longitudes OFERTA_ANOT_SQL ===")
print(df["OFERTA_ANOT_LEN"].describe())

# 8) Raros reales
oferta_valores_raros = df.loc[
    mask_valor_raro_real,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", oferta_si_col, "OFERTA_VALOR_RAW"]
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\nTotal OFERTA con valor raro REAL (sin mapeo):", len(oferta_valores_raros))
if not oferta_valores_raros.empty:
    print(oferta_valores_raros.head(30).to_string(index=False))

# 9) Listo para insertar (solo donde hay Valor final)
base_cols = ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"]
base_cols = [c for c in base_cols if c in df.columns]

mask_listo = df["OFERTA_VALOR_SQL_FINAL"].notna()

df_listo = df.loc[
    mask_listo,
    base_cols + ["OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL"]
].copy()

df_listo.rename(columns={
    "OFERTA_VALOR_SQL_FINAL": "Valor",
    "OFERTA_ANOT_SQL": "Anotacion",
}, inplace=True)

df_listo["Objeto"] = "OFERTA OTRAS ENTIDADES"
df_listo["TipoClasificacion"] = None
# df_listo["IdEstudioTerreno"] = ...  # se completa luego con el cruce

# 10) Exportar
archivo_revision = "MedidaProcesal_OFERTA_revision.xlsx"
archivo_listo = "MedidaProcesal_OFERTA_listo_insertar.xlsx"
archivo_raros = None

cols_export = (
    base_cols
    + [oferta_si_col, oferta_anot_col]
    + [
        "OFERTA_VALOR_RAW",
        "OFERTA_VALOR_TXT_LIMPIO",
        "OFERTA_VALOR_SQL",
        "OFERTA_VALOR_SQL_FINAL",
        "OFERTA_ESTADO",
        "OFERTA_ANOT_RAW",
        "OFERTA_ANOT_SQL",
        "OFERTA_ANOT_LEN",
        "OFERTA_ANOT_GT4000",
    ]
)
cols_export = [c for c in cols_export if c in df.columns]

df_all = df[cols_export].copy()

resumen_estado = (
    df["OFERTA_ESTADO"].value_counts(dropna=False)
      .rename_axis("OFERTA_ESTADO")
      .reset_index(name="conteo")
)

resumen_valor_final = (
    df["OFERTA_VALOR_SQL_FINAL"].value_counts(dropna=False)
      .rename_axis("OFERTA_VALOR_SQL_FINAL")
      .reset_index(name="conteo")
)

with pd.ExcelWriter(archivo_revision, engine="openpyxl") as writer:
    df_all.to_excel(writer, index=False, sheet_name="all_mapeo")
    resumen_estado.to_excel(writer, index=False, sheet_name="resumen_estado")
    resumen_valor_final.to_excel(writer, index=False, sheet_name="resumen_valor_final")
    oferta_valores_raros.to_excel(writer, index=False, sheet_name="valor_raro_real")
    df_listo.to_excel(writer, index=False, sheet_name="listo_insertar")

df_listo.to_excel(archivo_listo, index=False)

if not oferta_valores_raros.empty:
    archivo_raros = "MedidaProcesal_OFERTA_valores_raros_reales.xlsx"
    oferta_valores_raros.to_excel(archivo_raros, index=False)

print("\nArchivos creados:")
print(f" - {archivo_revision} (hojas: all_mapeo, resumen_*, valor_raro_real, listo_insertar)")
print(f" - {archivo_listo} (solo filas listas para insertar)")
if archivo_raros:
    print(f" - {archivo_raros} (valores Si/No raros reales)")


Columnas 'Si / No' (Excel): ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6']
Columnas 'Anotación' (Excel): ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']

OFERTA Si/No   : Si / No.5
OFERTA Anotac. : Anotación.5

=== Conteo OFERTA_VALOR_TXT_LIMPIO ===
OFERTA_VALOR_TXT_LIMPIO
None         12746
NO           11255
SI              53
PENDIENTE        1
Name: count, dtype: int64

=== Conteo OFERTA_VALOR_SQL_FINAL ===
OFERTA_VALOR_SQL_FINAL
None         12737
NO           11255
SI              53
SIN_REG          9
PENDIENTE        1
Name: count, dtype: int64

=== Conteo por OFERTA_ESTADO ===
OFERTA_ESTADO
vacio      12737
ok         11309
sin_reg        9
Name: count, dtype: int64

=== Resumen longitudes OFERTA_ANOT_SQL ===
count    24055.000000
mean         0.010434
std          0.998885
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        154.0000

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1106489865.py:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = nuevo[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1106489865.py:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = nuevo[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1106489865.py:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join


Archivos creados:
 - MedidaProcesal_OFERTA_revision.xlsx (hojas: all_mapeo, resumen_*, valor_raro_real, listo_insertar)
 - MedidaProcesal_OFERTA_listo_insertar.xlsx (solo filas listas para insertar)


## Proceso de clasificaion

In [37]:
import re
import numpy as np
import pandas as pd

# ============================================================
# MEDIDA PROCESAL - PROCESOS DE CLARIFICACIÓN (AJUSTADO)
# (CORREGIDO: NO recarga/reasigna df; resto igual)
# - NO hace: df = df.copy()
# - NO hace: df = pd.concat([df, nuevo], axis=1)
# - En su lugar:
#     * Borra PROC_CLARIF_* previas IN-PLACE (drop) para re-ejecución segura
#     * Calcula en DF auxiliar (nuevo)
#     * Asigna columnas al df IN-PLACE: df[c] = nuevo[c].values
# ============================================================

# 0) Fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas PROC_CLARIF_* si existen (SIN recargar df)
cols_proc_prev = [c for c in df.columns if isinstance(c, str) and c.startswith("PROC_CLARIF_")]
if cols_proc_prev:
    df.drop(columns=cols_proc_prev, inplace=True, errors="ignore")

cols = list(df.columns)
pos = {c: i for i, c in enumerate(cols)}

# 1) Columnas originales del Excel
re_si = re.compile(r"^Si\s*/\s*No(\.\d+)?$", re.IGNORECASE)
re_an = re.compile(r"^Anotaci[oó]n(\.\d+)?$", re.IGNORECASE)

cols_si = [c for c in cols if re_si.match(str(c).strip())]
cols_anot = [c for c in cols if re_an.match(str(c).strip())]

print("Columnas 'Si / No' (Excel):", cols_si)
print("Columnas 'Anotación' (Excel):", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas originales 'Si / No' o 'Anotación' del Excel en el DataFrame.")

cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# PROCESOS DE CLARIFICACIÓN = 7mo par (index 6): Si / No.6 y Anotación.6
if "Si / No.6" in df.columns:
    proc_si_col = "Si / No.6"
else:
    if len(cols_si_ordenadas) < 7:
        raise ValueError("No se encontró la 7ma columna 'Si / No' para PROCESOS DE CLARIFICACIÓN (par #7).")
    proc_si_col = cols_si_ordenadas[6]

# Nota: a veces viene "Anotacion.6" sin tilde
if "Anotación.6" in df.columns:
    proc_anot_col = "Anotación.6"
elif "Anotacion.6" in df.columns:
    proc_anot_col = "Anotacion.6"
else:
    if len(cols_anot_ordenadas) < 7:
        raise ValueError("No se encontró la 7ma columna 'Anotación' para PROCESOS DE CLARIFICACIÓN (par #7).")
    proc_anot_col = cols_anot_ordenadas[6]

print("\nPROC. CLARIF Si/No   :", proc_si_col)
print("PROC. CLARIF Anotac. :", proc_anot_col)

# 2) Helpers
def es_vacio(v) -> bool:
    if pd.isna(v):
        return True
    if isinstance(v, str) and v.strip() == "":
        return True
    return False

def quitar_tildes_upper(txt: str) -> str:
    txt = txt.strip().upper()
    return (txt.replace("Á", "A")
               .replace("É", "E")
               .replace("Í", "I")
               .replace("Ó", "O")
               .replace("Ú", "U"))

def normalizar_si_no_pend(v):
    if es_vacio(v):
        return None
    t = quitar_tildes_upper(str(v))
    t = t.replace(".", "").replace(",", "").replace(";", "")
    t = " ".join(t.split())

    if t in {"SI", "S"}:
        return "SI"
    if t in {"NO", "N"}:
        return "NO"
    if "PEND" in t:
        return "PENDIENTE"
    return None

def limpiar_anotacion(v):
    if es_vacio(v):
        return None
    txt = str(v).replace("\r", " ").replace("\n", " ").replace("\t", " ")
    txt = " ".join(txt.split()).strip()
    return txt if txt != "" else None

# 3) Transformación (en bloque)
proc_valor_raw = df[proc_si_col]
proc_anot_raw  = df[proc_anot_col]

proc_valor_txt = proc_valor_raw.apply(normalizar_si_no_pend)
proc_valor_sql = proc_valor_txt

proc_anot_sql = proc_anot_raw.apply(limpiar_anotacion)
proc_anot_len = proc_anot_sql.fillna("").str.len()

# Truncar anotación si excede 4000
PROC_ANOT_MAX = 4000
proc_anot_gt4000 = proc_anot_len > PROC_ANOT_MAX
proc_anot_sql = proc_anot_sql.where(~proc_anot_gt4000, proc_anot_sql.str.slice(0, PROC_ANOT_MAX))

# 4) Regla: si hay anotación y no hay valor -> SIN_REG
proc_valor_sql_final = proc_valor_sql.copy()
mask_solo_anot = proc_anot_sql.notna() & proc_valor_sql_final.isna()
proc_valor_sql_final.loc[mask_solo_anot] = "SIN_REG"

# 5) Estado + raros reales
proc_estado = pd.Series("vacio", index=df.index, dtype="object")
proc_estado.loc[proc_valor_sql.notna()] = "ok"
proc_estado.loc[mask_solo_anot] = "sin_reg"

mask_valor_raro_real = proc_valor_raw.apply(lambda x: not es_vacio(x)) & proc_valor_sql.isna()
proc_estado.loc[mask_valor_raro_real] = "pendiente"

# 6) Adjuntar columnas al df (SIN pd.concat y SIN recargar df)
nuevo = pd.DataFrame({
    "PROC_CLARIF_VALOR_RAW": proc_valor_raw,
    "PROC_CLARIF_VALOR_TXT_LIMPIO": proc_valor_txt,
    "PROC_CLARIF_VALOR_SQL": proc_valor_sql,
    "PROC_CLARIF_VALOR_SQL_FINAL": proc_valor_sql_final,
    "PROC_CLARIF_ESTADO": proc_estado,
    "PROC_CLARIF_ANOT_RAW": proc_anot_raw,
    "PROC_CLARIF_ANOT_SQL": proc_anot_sql,
    "PROC_CLARIF_ANOT_LEN": proc_anot_len,
    "PROC_CLARIF_ANOT_GT4000": proc_anot_gt4000,
}, index=df.index)

for c in nuevo.columns:
    df[c] = nuevo[c].values

# 7) Reportes
print("\n=== Conteo PROC_CLARIF_VALOR_TXT_LIMPIO ===")
print(df["PROC_CLARIF_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

print("\n=== Conteo PROC_CLARIF_VALOR_SQL_FINAL ===")
print(df["PROC_CLARIF_VALOR_SQL_FINAL"].value_counts(dropna=False))

print("\n=== Conteo por PROC_CLARIF_ESTADO ===")
print(df["PROC_CLARIF_ESTADO"].value_counts(dropna=False))

print("\n=== Resumen longitudes PROC_CLARIF_ANOT_SQL ===")
print(df["PROC_CLARIF_ANOT_LEN"].describe())

# 8) Raros reales (NO vacío y sin mapeo)
proc_valores_raros = df.loc[
    mask_valor_raro_real,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", proc_si_col, "PROC_CLARIF_VALOR_RAW"]
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\nTotal PROC_CLARIF con valor raro REAL (sin mapeo):", len(proc_valores_raros))
if not proc_valores_raros.empty:
    print(proc_valores_raros.head(30).to_string(index=False))

# 9) Listo para insertar (solo donde hay Valor final)
base_cols = ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"]
base_cols = [c for c in base_cols if c in df.columns]

mask_listo = df["PROC_CLARIF_VALOR_SQL_FINAL"].notna()

df_listo = df.loc[
    mask_listo,
    base_cols + ["PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL"]
].copy()

df_listo.rename(columns={
    "PROC_CLARIF_VALOR_SQL_FINAL": "Valor",
    "PROC_CLARIF_ANOT_SQL": "Anotacion",
}, inplace=True)

df_listo["Objeto"] = "PROCESOS DE CLARIFICACIÓN"
df_listo["TipoClasificacion"] = None
# df_listo["IdEstudioTerreno"] = ...  # se completa luego con el cruce

# 10) Exportar
archivo_revision = "MedidaProcesal_PROC_CLARIF_revision.xlsx"
archivo_listo = "MedidaProcesal_PROC_CLARIF_listo_insertar.xlsx"
archivo_raros = None

cols_export = (
    base_cols
    + [proc_si_col, proc_anot_col]
    + [
        "PROC_CLARIF_VALOR_RAW",
        "PROC_CLARIF_VALOR_TXT_LIMPIO",
        "PROC_CLARIF_VALOR_SQL",
        "PROC_CLARIF_VALOR_SQL_FINAL",
        "PROC_CLARIF_ESTADO",
        "PROC_CLARIF_ANOT_RAW",
        "PROC_CLARIF_ANOT_SQL",
        "PROC_CLARIF_ANOT_LEN",
        "PROC_CLARIF_ANOT_GT4000",
    ]
)
cols_export = [c for c in cols_export if c in df.columns]

df_all = df[cols_export].copy()

resumen_estado = (
    df["PROC_CLARIF_ESTADO"].value_counts(dropna=False)
      .rename_axis("PROC_CLARIF_ESTADO")
      .reset_index(name="conteo")
)

resumen_valor_final = (
    df["PROC_CLARIF_VALOR_SQL_FINAL"].value_counts(dropna=False)
      .rename_axis("PROC_CLARIF_VALOR_SQL_FINAL")
      .reset_index(name="conteo")
)

with pd.ExcelWriter(archivo_revision, engine="openpyxl") as writer:
    df_all.to_excel(writer, index=False, sheet_name="all_mapeo")
    resumen_estado.to_excel(writer, index=False, sheet_name="resumen_estado")
    resumen_valor_final.to_excel(writer, index=False, sheet_name="resumen_valor_final")
    proc_valores_raros.to_excel(writer, index=False, sheet_name="valor_raro_real")
    df_listo.to_excel(writer, index=False, sheet_name="listo_insertar")

df_listo.to_excel(archivo_listo, index=False)

if not proc_valores_raros.empty:
    archivo_raros = "MedidaProcesal_PROC_CLARIF_valores_raros_reales.xlsx"
    proc_valores_raros.to_excel(archivo_raros, index=False)

print("\nArchivos creados:")
print(f" - {archivo_revision} (hojas: all_mapeo, resumen_*, valor_raro_real, listo_insertar)")
print(f" - {archivo_listo} (solo filas listas para insertar)")
if archivo_raros:
    print(f" - {archivo_raros} (valores Si/No raros reales)")


Columnas 'Si / No' (Excel): ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6']
Columnas 'Anotación' (Excel): ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']

PROC. CLARIF Si/No   : Si / No.6
PROC. CLARIF Anotac. : Anotación.6

=== Conteo PROC_CLARIF_VALOR_TXT_LIMPIO ===
PROC_CLARIF_VALOR_TXT_LIMPIO
None         15479
NO            8542
SI              33
PENDIENTE        1
Name: count, dtype: int64

=== Conteo PROC_CLARIF_VALOR_SQL_FINAL ===
PROC_CLARIF_VALOR_SQL_FINAL
None         15467
NO            8542
SI              33
SIN_REG         12
PENDIENTE        1
Name: count, dtype: int64

=== Conteo por PROC_CLARIF_ESTADO ===
PROC_CLARIF_ESTADO
vacio      15467
ok          8576
sin_reg       12
Name: count, dtype: int64

=== Resumen longitudes PROC_CLARIF_ANOT_SQL ===
count    24055.000000
mean         0.004864
std          0.201065
min          0.000000
25%          0.000000
50%          0.0

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1187409248.py:145: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = nuevo[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1187409248.py:145: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = nuevo[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1187409248.py:145: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join


Archivos creados:
 - MedidaProcesal_PROC_CLARIF_revision.xlsx (hojas: all_mapeo, resumen_*, valor_raro_real, listo_insertar)
 - MedidaProcesal_PROC_CLARIF_listo_insertar.xlsx (solo filas listas para insertar)


*avance excel*

In [38]:
import pandas as pd

# ============================================================
# AVANCE A EXCEL (SIN RECARGAR df) + MEDIDA PROCESAL CONSOLIDADA
# - No hace df = ...
# - Construye dataframes auxiliares y exporta
# ============================================================

# 0) Asegurar fila Excel (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Columnas "contexto" (solo si existen)
cols_ctx = [
    "Fila_Excel_aprox",
    "IdPostulacion_SQL",
    "ID_SQL",
    "FMI", "FMI_SQL",
    "DEPARTAMENTO", "MUNICIPIO",
]
cols_ctx = [c for c in cols_ctx if c in df.columns]

# ------------------------------------------------------------
# 2) RegistroProceso (solo las que existan)
# ------------------------------------------------------------
cols_registro = [
    "IdPostulacion_SQL",
    "FMI_SQL",
    "NumeroExpediente_SQL",
    "IdFuenteProceso",
    "IdFuenteProceso_SQL",
    "IdTipoProceso",
    "IdTipoProceso_SQL",
    "IdEtapaProcesal",
    "IdEtapaProcesal_SQL",
    "Dependencia_SQL",
    "RadicadoOrfeo_SQL",
]
cols_registro_exist = [c for c in cols_registro if c in df.columns]
df_registro = df[cols_ctx + cols_registro_exist].copy()

# ------------------------------------------------------------
# 3) EstudioTerreno (solo las que existan)
# ------------------------------------------------------------
cols_estudio = [
    "IdRegistroProceso_SQL",
    "IdLocalizacion_SQL",
    "AreaRegistral_SQL",
    "CirculoRegistral_SQL",
    "AreaCalculada_SQL",
    "TipoPersonaTitular_SQL",
    "NombrePropietario_SQL",
    "ApellidoPropietario_SQL",
    "Identificacion_SQL_TXT_MIGRAR",
    "IdTipoDocumento_SQL_MIGRAR",
    "NaturalezaJuridica_SQL_FINAL",
    "AcreditacionPropiedad_SQL_FINAL",
]
cols_estudio_exist = [c for c in cols_estudio if c in df.columns]
df_estudio = df[cols_ctx + cols_estudio_exist].copy()

# ------------------------------------------------------------
# 4) MedidaProcesal consolidada (listo_insertar)
#    (NO modifica df; solo lee columnas ya calculadas)
# ------------------------------------------------------------
base_cols_mp = ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"]
base_cols_mp = [c for c in base_cols_mp if c in df.columns]

mp_parts = []

def add_mp(objeto: str, col_valor_final: str, col_anot: str, col_tipo: str = None, col_estado: str = None, estados_ok=None):
    """
    Agrega un bloque de MedidaProcesal en formato largo:
      base_cols + [Valor, Anotacion, TipoClasificacion, Objeto]
    Filtra:
      - Si existe col_estado y estados_ok, usa eso.
      - Si no, usa Valor notna().
    """
    if col_valor_final not in df.columns:
        return
    if col_anot not in df.columns:
        return

    valor = df[col_valor_final]
    anot = df[col_anot]

    if (col_estado is not None) and (col_estado in df.columns) and estados_ok:
        mask = df[col_estado].isin(estados_ok)
    else:
        mask = valor.notna()

    if not mask.any():
        return

    out = df.loc[mask, base_cols_mp].copy()
    out["Valor"] = valor.loc[mask].values
    out["Anotacion"] = anot.loc[mask].values

    if col_tipo and (col_tipo in df.columns):
        out["TipoClasificacion"] = df.loc[mask, col_tipo].values
    else:
        out["TipoClasificacion"] = None

    out["Objeto"] = objeto
    mp_parts.append(out)

# HIPOTECA: usa estado ok/sin_reg (tu código ya pone SIN_REG en HIPOTECA_VALOR_SQL cuando aplica)
add_mp(
    objeto="HIPOTECA",
    col_valor_final="HIPOTECA_VALOR_SQL",
    col_anot="HIPOTECA_ANOT_SQL",
    col_tipo=None,
    col_estado="HIPOTECA_ESTADO",
    estados_ok=["ok", "sin_reg"]
)

# SERVIDUMBRES
add_mp(
    objeto="SERVIDUMBRES",
    col_valor_final="SERVIDUMBRES_VALOR_SQL_FINAL",
    col_anot="SERVIDUMBRES_ANOT_SQL",
    col_tipo=None,
    col_estado="SERVIDUMBRES_ESTADO",
    estados_ok=["ok", "sin_reg"]
)

# MEDIDAS CAUTELARES
add_mp(
    objeto="MEDIDAS CAUTELARES",
    col_valor_final="MEDIDAS_CAUT_VALOR_SQL_FINAL",
    col_anot="MEDIDAS_CAUT_ANOT_SQL",
    col_tipo=None,
    col_estado="MEDIDAS_CAUT_ESTADO",
    estados_ok=["ok", "sin_reg"]
)

# RUPTA (incluye TipoClasificacion)
add_mp(
    objeto="RUPTA",
    col_valor_final="RUPTA_VALOR_SQL_FINAL",
    col_anot="RUPTA_ANOT_SQL",
    col_tipo="RUPTA_TIPOCLASIF_SQL",
    col_estado="RUPTA_ESTADO",
    estados_ok=["ok", "sin_reg"]
)

# RTDAF LEY 1448
add_mp(
    objeto="RTDAF LEY 1448",
    col_valor_final="RTDAF_VALOR_SQL_FINAL",
    col_anot="RTDAF_ANOT_SQL",
    col_tipo=None,
    col_estado="RTDAF_ESTADO",
    estados_ok=["ok", "sin_reg"]
)

# OFERTA OTRAS ENTIDADES
add_mp(
    objeto="OFERTA OTRAS ENTIDADES",
    col_valor_final="OFERTA_VALOR_SQL_FINAL",
    col_anot="OFERTA_ANOT_SQL",
    col_tipo=None,
    col_estado="OFERTA_ESTADO",
    estados_ok=["ok", "sin_reg"]
)

# PROCESOS DE CLARIFICACIÓN
add_mp(
    objeto="PROCESOS DE CLARIFICACIÓN",
    col_valor_final="PROC_CLARIF_VALOR_SQL_FINAL",
    col_anot="PROC_CLARIF_ANOT_SQL",
    col_tipo=None,
    col_estado="PROC_CLARIF_ESTADO",
    estados_ok=["ok", "sin_reg"]
)

if mp_parts:
    df_medidaprocesal = pd.concat(mp_parts, ignore_index=True)
else:
    df_medidaprocesal = pd.DataFrame(columns=base_cols_mp + ["Valor", "Anotacion", "TipoClasificacion", "Objeto"])

# (Opcional) ordenar para revisión
sort_cols = [c for c in ["DEPARTAMENTO", "MUNICIPIO", "FMI", "Objeto"] if c in df_medidaprocesal.columns]
if sort_cols:
    df_medidaprocesal = df_medidaprocesal.sort_values(sort_cols).reset_index(drop=True)

# ------------------------------------------------------------
# 5) Diccionario (actualizado)
# ------------------------------------------------------------
dicc = []
for c in cols_registro_exist:
    dicc.append(["RegistroProceso", c])
for c in cols_estudio_exist:
    dicc.append(["EstudioTerreno", c])

# MedidaProcesal: columnas físicas del export
dicc.append(["MedidaProcesal_listo_insertar", "Valor"])
dicc.append(["MedidaProcesal_listo_insertar", "Anotacion"])
dicc.append(["MedidaProcesal_listo_insertar", "TipoClasificacion"])
dicc.append(["MedidaProcesal_listo_insertar", "Objeto"])

df_dicc = pd.DataFrame(dicc, columns=["TablaDestino/Hoja", "Columna"])

# ------------------------------------------------------------
# 6) Exportar Excel consolidado
# ------------------------------------------------------------
out_xlsx = "AVANCE_MIGRACION_HASTA_HOY.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    df_registro.to_excel(writer, index=False, sheet_name="RegistroProceso_avance")
    df_estudio.to_excel(writer, index=False, sheet_name="EstudioTerreno_avance")
    df_medidaprocesal.to_excel(writer, index=False, sheet_name="MedidaProcesal_listo_insertar")
    df_dicc.to_excel(writer, index=False, sheet_name="Diccionario")

print("Excel creado:", out_xlsx)
print("Hojas: RegistroProceso_avance, EstudioTerreno_avance, MedidaProcesal_listo_insertar, Diccionario")
print("Filas RegistroProceso_avance:", len(df_registro))
print("Filas EstudioTerreno_avance:", len(df_estudio))
print("Filas MedidaProcesal_listo_insertar:", len(df_medidaprocesal))


Excel creado: AVANCE_MIGRACION_HASTA_HOY.xlsx
Hojas: RegistroProceso_avance, EstudioTerreno_avance, MedidaProcesal_listo_insertar, Diccionario
Filas RegistroProceso_avance: 24055
Filas EstudioTerreno_avance: 24055
Filas MedidaProcesal_listo_insertar: 69662


In [39]:
import pandas as pd
import numpy as np

# ============================================================
# EXCEL UNIFICADO "HASTA HOY" (SIN RECARGAR df)
# - No hace df = ... (ni copy, ni concat sobre df)
# - Usa solo lecturas de df y crea dataframes auxiliares
# - Genera un Excel con:
#   1) UNIFICADO_FILAS (una fila por registro: RegistroProceso + EstudioTerreno + campos listos)
#   2) MedidaProcesal_listo_insertar (todas las filas insertables ya consolidadas)
#   3) Diccionario (tabla->columna)
#   4) Resumen_MP (conteos por objeto y por valor)
# ============================================================

# 0) Asegurar Fila Excel (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Columnas contexto (solo si existen)
cols_ctx = [
    "Fila_Excel_aprox",
    "IdPostulacion_SQL",  # si existe
    "ID_SQL",             # si aún existe
    "FMI", "FMI_SQL",
    "DEPARTAMENTO", "MUNICIPIO",
]
cols_ctx = [c for c in cols_ctx if c in df.columns]

# 2) RegistroProceso (solo si existen)
cols_registro = [
    "IdPostulacion_SQL",   # -> RegistroProceso.IdPostulacion (varchar(30))
    "FMI_SQL",             # -> RegistroProceso.FMI (varchar(100))
    "NumeroExpediente_SQL",
    "IdFuenteProceso",     # o IdFuenteProceso_SQL
    "IdTipoProceso",       # o IdTipoProceso_SQL
    "IdEtapaProcesal",     # o IdEtapaProcesal_SQL
    "Dependencia_SQL",
    "RadicadoOrfeo_SQL",
]
cols_registro_exist = [c for c in cols_registro if c in df.columns]

# 3) EstudioTerreno (solo si existen)
cols_estudio = [
    "IdRegistroProceso_SQL",          # si existe
    "IdLocalizacion_SQL",
    "AreaRegistral_SQL",
    "CirculoRegistral_SQL",
    "AreaCalculada_SQL",
    "TipoPersonaTitular_SQL",
    "NombrePropietario_SQL",
    "ApellidoPropietario_SQL",
    "Identificacion_SQL_TXT_MIGRAR",
    "IdTipoDocumento_SQL_MIGRAR",
    "NaturalezaJuridica_SQL_FINAL",
    "AcreditacionPropiedad_SQL_FINAL",
]
cols_estudio_exist = [c for c in cols_estudio if c in df.columns]

# 4) UNIFICADO (una fila por registro)
#    (NO reasignar df; solo construir df_unificado)
cols_unificado = []
for c in cols_ctx + cols_registro_exist + cols_estudio_exist:
    if c not in cols_unificado:
        cols_unificado.append(c)

df_unificado = df[cols_unificado].copy()

# 5) Construir MedidaProcesal_listo_insertar UNIFICADO (todas las filas insertables)
base_cols_mp = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

mp_frames = []

def add_mp_from_columns(prefix_objeto: str, col_valor_final: str, col_anot: str, col_tipoclasif: str | None, objeto: str):
    """
    Agrega al consolidado MP:
      - Usa df[col_valor_final].notna() como máscara
      - Mapea a columnas estándar: Objeto, Valor, Anotacion, TipoClasificacion
    """
    if col_valor_final not in df.columns:
        return
    if col_anot not in df.columns:
        return

    valor = df[col_valor_final]
    # Si por alguna razón hay duplicidad de columnas (DataFrame), toma la primera
    if isinstance(valor, pd.DataFrame):
        valor = valor.iloc[:, 0]

    mask = valor.notna()
    if not mask.any():
        return

    cols_take = base_cols_mp + [col_valor_final, col_anot]
    if col_tipoclasif and col_tipoclasif in df.columns:
        cols_take.append(col_tipoclasif)

    tmp = df.loc[mask, cols_take].copy()

    rename_map = {
        col_valor_final: "Valor",
        col_anot: "Anotacion",
    }
    tmp.rename(columns=rename_map, inplace=True)

    if col_tipoclasif and col_tipoclasif in tmp.columns:
        tmp.rename(columns={col_tipoclasif: "TipoClasificacion"}, inplace=True)
    else:
        tmp["TipoClasificacion"] = None

    tmp["Objeto"] = objeto

    # Orden estándar
    cols_std = base_cols_mp + ["Objeto", "Valor", "Anotacion", "TipoClasificacion"]
    cols_std = [c for c in cols_std if c in tmp.columns]
    tmp = tmp[cols_std].copy()

    mp_frames.append(tmp)

# HIPOTECA: en tu script quedó como HIPOTECA_VALOR_SQL (ya incluye SIN_REG cuando aplica)
# (si también tienes HIPOTECA_VALOR_SQL_FINAL, úsalo; aquí priorizamos FINAL si existe)
if "HIPOTECA_VALOR_SQL_FINAL" in df.columns:
    add_mp_from_columns("HIPOTECA", "HIPOTECA_VALOR_SQL_FINAL", "HIPOTECA_ANOT_SQL", None, "HIPOTECA")
else:
    add_mp_from_columns("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL", None, "HIPOTECA")

# SERVIDUMBRES
add_mp_from_columns("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL", None, "SERVIDUMBRES")

# MEDIDAS CAUTELARES
add_mp_from_columns("MEDIDAS_CAUT", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL", None, "MEDIDAS CAUTELARES")

# RUPTA (incluye TipoClasificacion)
add_mp_from_columns("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL", "RUPTA")

# RTDAF LEY 1448
add_mp_from_columns("RTDAF", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL", None, "RTDAF LEY 1448")

# OFERTA OTRAS ENTIDADES
add_mp_from_columns("OFERTA", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL", None, "OFERTA OTRAS ENTIDADES")

# PROCESOS DE CLARIFICACIÓN
add_mp_from_columns("PROC_CLARIF", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL", None, "PROCESOS DE CLARIFICACIÓN")

if mp_frames:
    df_mp_unificado = pd.concat(mp_frames, ignore_index=True)
else:
    df_mp_unificado = pd.DataFrame(columns=base_cols_mp + ["Objeto", "Valor", "Anotacion", "TipoClasificacion"])

# 6) Diccionario (tabla->columna) para el unificado
dicc = []
for c in cols_registro_exist:
    dicc.append(["RegistroProceso", c])
for c in cols_estudio_exist:
    dicc.append(["EstudioTerreno", c])

# MedidaProcesal no es 1:1 (sale de columnas específicas)
mp_cols_used = [
    # hipoteca
    "HIPOTECA_VALOR_SQL_FINAL", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL",
    # servidumbres
    "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL",
    # cautelares
    "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL",
    # rupta
    "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL",
    # rtdaf
    "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL",
    # oferta
    "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL",
    # proc clarif
    "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL",
]
mp_cols_used = [c for c in mp_cols_used if c in df.columns]

for c in mp_cols_used:
    dicc.append(["MedidaProcesal_origen_en_df", c])

df_dicc = pd.DataFrame(dicc, columns=["TablaDestino", "Columna_df"])

# 7) Resúmenes MP
resumen_mp_objeto = (
    df_mp_unificado["Objeto"].value_counts(dropna=False)
      .rename_axis("Objeto")
      .reset_index(name="conteo_filas")
)

resumen_mp_valor = (
    df_mp_unificado["Valor"].value_counts(dropna=False)
      .rename_axis("Valor")
      .reset_index(name="conteo_filas")
)

# Duplicados por llave lógica (si FMI existe)
df_dup = pd.DataFrame()
if "FMI" in df_mp_unificado.columns:
    dup_mask = df_mp_unificado.duplicated(subset=["FMI", "Objeto"], keep=False)
    df_dup = df_mp_unificado.loc[dup_mask].copy()

# 8) Exportar Excel UNIFICADO
out_xlsx = "AVANCE_MIGRACION_UNIFICADO_HASTA_HOY.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    df_unificado.to_excel(writer, index=False, sheet_name="UNIFICADO_FILAS")
    df_mp_unificado.to_excel(writer, index=False, sheet_name="MedidaProcesal_listo_insertar")
    df_dicc.to_excel(writer, index=False, sheet_name="Diccionario")
    resumen_mp_objeto.to_excel(writer, index=False, sheet_name="Resumen_MP_objeto")
    resumen_mp_valor.to_excel(writer, index=False, sheet_name="Resumen_MP_valor")
    if not df_dup.empty:
        df_dup.to_excel(writer, index=False, sheet_name="MP_duplicados_FMI_Obj")

print("Excel creado:", out_xlsx)
print("Hojas: UNIFICADO_FILAS, MedidaProcesal_listo_insertar, Diccionario, Resumen_MP_objeto, Resumen_MP_valor (+ MP_duplicados_FMI_Obj si aplica)")
print("Filas UNIFICADO_FILAS:", len(df_unificado))
print("Filas MedidaProcesal_listo_insertar:", len(df_mp_unificado))


Excel creado: AVANCE_MIGRACION_UNIFICADO_HASTA_HOY.xlsx
Hojas: UNIFICADO_FILAS, MedidaProcesal_listo_insertar, Diccionario, Resumen_MP_objeto, Resumen_MP_valor (+ MP_duplicados_FMI_Obj si aplica)
Filas UNIFICADO_FILAS: 24055
Filas MedidaProcesal_listo_insertar: 69662


## Fecha de informe previo

In [40]:
import pandas as pd
import numpy as np
import re

# ======================================
# FECHA INFORME PREVIO REPORTADA
# (limpieza + clasificación + excels)
# (CORREGIDO: NO recarga/reasigna df; resto igual)
# - NO hace: df = ...
# - NO hace: df = pd.concat(...)
# - En su lugar:
#     * Borra columnas FechaInformePrevio_* previas (drop) para re-ejecución segura
#     * Calcula en DF auxiliar (newcols)
#     * Asigna al df IN-PLACE: df[c] = newcols[c].values
# ======================================

# 0) Fila aproximada de Excel (por si acaso)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas FechaInformePrevio_* si existen (SIN recargar df)
cols_prev = [
    c for c in df.columns
    if isinstance(c, str) and c.startswith("FechaInformePrevio_")
]
if cols_prev:
    df.drop(columns=cols_prev, inplace=True, errors="ignore")

# 1) Identificar columna
cols_fecha_prev = [
    c for c in df.columns
    if 'FECHA' in str(c).upper() and ('PREVIO' in str(c).upper() or 'INFORME' in str(c).upper())
]

print("Columnas candidatas para FECHA INFORME PREVIO:", cols_fecha_prev)

if 'FECHA INFORME PREVIO REPORTADA' in df.columns:
    col_fecha_prev = 'FECHA INFORME PREVIO REPORTADA'
else:
    if not cols_fecha_prev:
        raise ValueError("No se encontró columna para FECHA INFORME PREVIO; revisa encabezados.")
    col_fecha_prev = cols_fecha_prev[0]

print("Usando columna de fecha:", col_fecha_prev)

raw = df[col_fecha_prev]

# Helpers
def norm_text(x):
    if pd.isna(x):
        return None
    t = str(x).strip().upper()
    t = re.sub(r"\s+", " ", t)
    t = t.replace("Á","A").replace("É","E").replace("Í","I").replace("Ó","O").replace("Ú","U")
    t = re.sub(r"[.,;:]+$", "", t)  # quita puntuación final
    return t if t != "" else None

TOKENS_A_VACIO = {
    "NO", "SI",
    "NO APLICA", "NOAPLICA",
    "NO REGISTRA", "NO REGISTRADO", "NO REGISTRADA", "NO REGISTRO",
    "N/A", "NA"
}

raw_txt_norm = raw.apply(norm_text)

mask_vacio = raw.isna() | raw_txt_norm.isna()
mask_token = raw_txt_norm.isin(TOKENS_A_VACIO)

# 2) Conversión DIRECTA (si ya es fecha, queda OK)
dt_direct = pd.to_datetime(raw, errors="coerce", dayfirst=True)

# 3) Excel serial (si aplica)
num = pd.to_numeric(raw, errors="coerce")
mask_excel_serial = num.notna() & (num >= 20000) & (num <= 60000)
dt_excel = pd.to_datetime(num[mask_excel_serial], unit="D", origin="1899-12-30", errors="coerce")

# 4) Extracción de fecha al INICIO del texto: "dd/mm/yyyy." o "dd/mm/yyyy "
#    (esto soluciona el caso 9015)
pat_ini = r"^\s*(\d{1,2}/\d{1,2}/\d{4})"
ini_fecha_str = raw.astype("string").str.extract(pat_ini, expand=False)
dt_ini = pd.to_datetime(ini_fecha_str, errors="coerce", dayfirst=True)

# 5) Arreglos de formato solo donde aún no hay fecha y no es token/vacío/serial
mask_need_fix = dt_direct.isna() & dt_ini.isna() & ~mask_excel_serial & ~mask_token & ~mask_vacio

s_fix = raw.astype("string")

# 2103/2024 -> 21/03/2024
s_fix = s_fix.str.replace(r"^(\d{2})(\d{2})/(\d{4})$", r"\1/\2/\3", regex=True)

# 25/042024 -> 25/04/2024
s_fix = s_fix.str.replace(r"^(\d{1,2})/(\d{2})(\d{4})$", r"\1/\2/\3", regex=True)

dt_fix = pd.to_datetime(s_fix.where(mask_need_fix), errors="coerce", dayfirst=True)

# 6) dt_final (prioridades)
dt_final = dt_direct.copy()
dt_final.loc[mask_excel_serial] = dt_excel
m_ini = dt_direct.isna() & dt_ini.notna()
dt_final.loc[m_ini] = dt_ini.loc[m_ini]
dt_final.loc[mask_need_fix] = dt_fix.loc[mask_need_fix]

# 7) Estado
estado = pd.Series("ok", index=df.index, dtype="string")
estado.loc[mask_vacio] = "vacio"
estado.loc[mask_token] = "token_no_fecha"
estado.loc[mask_excel_serial & dt_final.notna()] = "ok_excel_serial"
estado.loc[m_ini] = "ok_extraida_inicio"
estado.loc[dt_final.isna() & ~mask_vacio & ~mask_token] = "no_convertida"

# 8) (Opcional) override explícito SOLO para esa fila si quieres dejarlo fijo sí o sí
overrides = [
    {"Fila_Excel_aprox": 3508, "Fecha": "08/04/2024", "Motivo": "Texto largo: 'Estudio realizado ...'"},
    {"Fila_Excel_aprox": 9015, "Fecha": "16/04/2024", "Motivo": "Texto largo inicia con 16/04/2024"},
]
for o in overrides:
    m = df["Fila_Excel_aprox"].eq(o["Fila_Excel_aprox"])
    if m.any():
        dt_final.loc[m] = pd.to_datetime(o["Fecha"], dayfirst=True, errors="coerce")
        estado.loc[m] = "ok_manual"

# 9) Guardar columnas en df (SIN pd.concat y SIN recargar df)
newcols = pd.DataFrame({
    "FechaInformePrevio_RAW": raw,
    "FechaInformePrevio_RAW_NORM": raw_txt_norm,
    "FechaInformePrevio_SQL": dt_final,
    "FechaInformePrevio_ESTADO": estado,
}, index=df.index)

for c in newcols.columns:
    df[c] = newcols[c].values

# 10) Resumen
print("\n=== Resumen FECHA INFORME PREVIO (final) ===")
print("Total filas:", len(df))
print("Fechas válidas:", df["FechaInformePrevio_SQL"].notna().sum())
print("Fechas inválidas:", df["FechaInformePrevio_SQL"].isna().sum())

print("\nConteo por ESTADO:")
print(df["FechaInformePrevio_ESTADO"].value_counts(dropna=False))

print("\nRango de fechas válidas:")
print("  Mínima:", df["FechaInformePrevio_SQL"].min())
print("  Máxima:", df["FechaInformePrevio_SQL"].max())

# 11) Pendientes reales
pendientes = df.loc[
    df["FechaInformePrevio_ESTADO"].eq("no_convertida"),
    ["Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO","FechaInformePrevio_RAW","FechaInformePrevio_RAW_NORM"]
].copy()

print("\n=== Ejemplos PENDIENTES reales (máx. 30) ===")
print(pendientes.head(30).to_string(index=False))

# 12) Excels
cols_export = [
    "Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO",
    col_fecha_prev,
    "FechaInformePrevio_RAW",
    "FechaInformePrevio_RAW_NORM",
    "FechaInformePrevio_SQL",
    "FechaInformePrevio_ESTADO",
]
cols_export = [c for c in cols_export if c in df.columns]

df_revision = df[cols_export].copy()
df_revision.to_excel("FechaInformePrevio_revision.xlsx", index=False)

if not pendientes.empty:
    pendientes.to_excel("FechaInformePrevio_pendientes_para_corregir.xlsx", index=False)

tokens = df.loc[df["FechaInformePrevio_ESTADO"].eq("token_no_fecha"), cols_export].copy()
if not tokens.empty:
    tokens.to_excel("FechaInformePrevio_tokens_a_vacio.xlsx", index=False)

print("\nArchivos creados:")
print(" - FechaInformePrevio_revision.xlsx")
if not pendientes.empty:
    print(" - FechaInformePrevio_pendientes_para_corregir.xlsx")
if not tokens.empty:
    print(" - FechaInformePrevio_tokens_a_vacio.xlsx")


Columnas candidatas para FECHA INFORME PREVIO: ['FECHA INFORME PREVIO REPORTADA', 'FECHA DE INFORME', 'FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL', 'FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO']
Usando columna de fecha: FECHA INFORME PREVIO REPORTADA

=== Resumen FECHA INFORME PREVIO (final) ===
Total filas: 24055
Fechas válidas: 305
Fechas inválidas: 23750

Conteo por ESTADO:
FechaInformePrevio_ESTADO
vacio              23729
ok                   295
token_no_fecha        21
ok_excel_serial        8
ok_manual              2
Name: count, dtype: Int64

Rango de fechas válidas:
  Mínima: 1979-03-07 00:00:00
  Máxima: 2026-05-24 00:00:00

=== Ejemplos PENDIENTES reales (máx. 30) ===
Empty DataFrame
Columns: [Fila_Excel_aprox, FMI, DEPARTAMENTO, MUNICIPIO, FechaInformePrevio_RAW, FechaInformePrevio_RAW_NORM]
Index: []


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3551823389.py:82: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt_ini = pd.to_datetime(ini_fecha_str, errors="coerce", dayfirst=True)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3551823389.py:95: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_fix = pd.to_datetime(s_fix.where(mask_need_fix), errors="coerce", dayfirst=True)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3551823389.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
 


Archivos creados:
 - FechaInformePrevio_revision.xlsx
 - FechaInformePrevio_tokens_a_vacio.xlsx


## CONCEPTO ANTIGUO

In [41]:
import pandas as pd
import numpy as np
import re
import unicodedata

# ============================================================
# CONCEPTO PREVIO (SIN RECARGAR df; resto igual)
# - NO hace: df = df.copy()
# - NO hace: df = ...
# - En su lugar:
#     * Borra ConceptoPrevio_* previas IN-PLACE (drop) para re-ejecución segura
#     * Calcula en DF auxiliar (work/newcols)
#     * Asigna columnas al df IN-PLACE: df[c] = newcols[c].values
# ============================================================

# 0) Fila aproximada (si no existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas ConceptoPrevio_* si existen (SIN recargar df)
cols_prev = [c for c in df.columns if isinstance(c, str) and c.startswith("ConceptoPrevio_")]
if cols_prev:
    df.drop(columns=cols_prev, inplace=True, errors="ignore")

# =========================
# 1) Identificar columna de concepto previo (Excel)
# =========================
candidatas = [c for c in df.columns if 'CONCEPTO' in str(c).upper()]
print("Columnas candidatas para CONCEPTO:", candidatas)

if 'CONCEPTO ANTIGUO' in df.columns:
    col_conc = 'CONCEPTO ANTIGUO'
else:
    # fallback: si cambia el nombre, intentamos escoger la que tenga "ANTIGUO"
    cand_ant = [c for c in candidatas if 'ANTIGUO' in str(c).upper()]
    if cand_ant:
        col_conc = cand_ant[0]
    else:
        raise ValueError("No se encontró 'CONCEPTO ANTIGUO'. Revisa encabezados y ajusta col_conc.")

print("Usando columna de concepto previo:", col_conc)

# =========================
# 2) Limpieza y clasificación
# =========================
def _norm_text(x):
    if pd.isna(x):
        return None
    t = str(x)
    t = t.replace('\r\n', '\n').replace('\r', '\n').replace('\xa0', ' ')
    # colapsar espacios/tabs repetidos (sin tocar saltos de línea)
    t = re.sub(r'[ \t]+', ' ', t).strip()
    return t if t else None

def _key_token(t: str) -> str:
    # Quita tildes, pasa a mayúscula, normaliza espacios y recorta puntitos/guiones extremos
    t2 = unicodedata.normalize('NFKD', t)
    t2 = ''.join(ch for ch in t2 if not unicodedata.combining(ch))
    t2 = t2.upper().strip()
    t2 = re.sub(r'\s+', ' ', t2)
    t2 = re.sub(r'^[\.\-_/]+|[\.\-_/]+$', '', t2)
    return t2

# Tokens que SÍ se van a NULL (solo si el valor es EXACTAMENTE eso)
SIN_CONCEPTO = {
    'NO', 'NO REGISTRA', 'NO APLICA',
    'NINGUNO', 'SIN CONCEPTO',
    'NR', 'N R', 'N/R'
}

def limpiar_y_estado(v):
    t = _norm_text(v)
    if t is None:
        return None, 'vacio'

    k = _key_token(t)
    if k in SIN_CONCEPTO:
        return None, 'token_sin_concepto'

    return t, 'ok'

# =========================
# 3) Calcular en DF auxiliar (evita fragmentación)
# =========================
work = pd.DataFrame(index=df.index)

# Guardar original
work['ConceptoPrevio_TXT_ORIG'] = df[col_conc]

tmp = work['ConceptoPrevio_TXT_ORIG'].apply(limpiar_y_estado)
work['ConceptoPrevio_SQL'] = tmp.apply(lambda x: x[0])
work['ConceptoPrevio_ESTADO'] = tmp.apply(lambda x: x[1])
work['ConceptoPrevio_LEN'] = work['ConceptoPrevio_SQL'].fillna('').astype(str).str.len()

# =========================
# 4) Asignar columnas al df IN-PLACE (SIN df = ...)
# =========================
for c in work.columns:
    df[c] = work[c].values

# =========================
# 5) Resúmenes
# =========================
print("\n=== Resumen CONCEPTO PREVIO (limpieza) ===")
print("Total filas:", len(df))
print("Con concepto (no NULL):", df['ConceptoPrevio_SQL'].notna().sum())
print("Sin concepto (NULL):", df['ConceptoPrevio_SQL'].isna().sum())

print("\nConteo por ESTADO:")
print(df['ConceptoPrevio_ESTADO'].value_counts(dropna=False))

print("\n=== Resumen longitudes ConceptoPrevio_SQL ===")
print(df['ConceptoPrevio_LEN'].describe())

# Cortos para revisar (pero NO vacíos)
ej_cortos = df[(df['ConceptoPrevio_ESTADO'] == 'ok') & (df['ConceptoPrevio_LEN'] <= 20)][
    ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO', 'ConceptoPrevio_SQL', 'ConceptoPrevio_LEN']
].head(50)

print("\n=== Ejemplos conceptos cortos (<=20) para revisar (máx 50) ===")
print(ej_cortos.to_string(index=False))

# Largos (ejemplo >= 500)
ej_largos = df[(df['ConceptoPrevio_ESTADO'] == 'ok') & (df['ConceptoPrevio_LEN'] >= 500)][
    ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO', 'ConceptoPrevio_LEN']
].head(30)

print("\n=== Ejemplos conceptos largos (>=500) (máx 30) ===")
print(ej_largos.to_string(index=False))

# Tokens que se mandaron a NULL para validar
tokens_a_null = df[df['ConceptoPrevio_ESTADO'] == 'token_sin_concepto'][
    ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO', 'ConceptoPrevio_TXT_ORIG']
].head(200)

# =========================
# 6) Excels (como vienes pidiendo)
# =========================
cols_base = ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO']
cols_base = [c for c in cols_base if c in df.columns]

# 6.1 Revisión completa
rev_cols = cols_base + ['ConceptoPrevio_TXT_ORIG', 'ConceptoPrevio_SQL', 'ConceptoPrevio_ESTADO', 'ConceptoPrevio_LEN']
rev_cols = [c for c in rev_cols if c in df.columns]
rev = df[rev_cols].copy()

# 6.2 Pendientes: conceptos cortos "ok" (para revisión humana)
pend_cols = cols_base + ['ConceptoPrevio_TXT_ORIG', 'ConceptoPrevio_SQL', 'ConceptoPrevio_LEN']
pend_cols = [c for c in pend_cols if c in df.columns]
pendientes = df[(df['ConceptoPrevio_ESTADO'] == 'ok') & (df['ConceptoPrevio_LEN'] <= 20)][pend_cols].copy()

# 6.3 Tokens a vacío (NULL)
tok_cols = cols_base + ['ConceptoPrevio_TXT_ORIG', 'ConceptoPrevio_ESTADO']
tok_cols = [c for c in tok_cols if c in df.columns]
tokens = df[df['ConceptoPrevio_ESTADO'] == 'token_sin_concepto'][tok_cols].copy()

# 6.4 Archivo “migración” (si existe IdRegistroProceso en el df)
posibles_id = [
    c for c in df.columns
    if str(c).upper().replace('_','').replace(' ','') in ('IDREGISTROPROCESO','IDREGISTROPROCESOSQL','IDREGPROCESO')
]
col_idrp = posibles_id[0] if posibles_id else None

if col_idrp:
    mig = df[[col_idrp, 'ConceptoPrevio_SQL']].copy()
    mig.columns = ['IdRegistroProceso', 'Concepto']  # nombres listos para SQL
else:
    mig = pd.DataFrame()
    print("\n[OJO] No encontré columna IdRegistroProceso en df. El Excel de migración saldrá vacío. "
          "Si ya la tienes con otro nombre, ajusta 'posibles_id' o asigna col_idrp manual.")

# Escribir archivos
rev.to_excel("ConceptoPrevio_revision.xlsx", index=False)
pendientes.to_excel("ConceptoPrevio_pendientes_para_corregir.xlsx", index=False)
tokens.to_excel("ConceptoPrevio_tokens_a_vacio.xlsx", index=False)
mig.to_excel("ConceptoPrevio_migracion.xlsx", index=False)

print("\nArchivos creados:")
print(" - ConceptoPrevio_revision.xlsx")
print(" - ConceptoPrevio_pendientes_para_corregir.xlsx")
print(" - ConceptoPrevio_tokens_a_vacio.xlsx")
print(" - ConceptoPrevio_migracion.xlsx")


Columnas candidatas para CONCEPTO: ['CONCEPTO ANTIGUO', 'ANALISIS JURÍDICO / CONCEPTO FINAL\n(DEFINICIÓN DE VIABILIDAD JURÍDICA)']
Usando columna de concepto previo: CONCEPTO ANTIGUO


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3110312664.py:99: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = work[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3110312664.py:99: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = work[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3110312664.py:99: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a


=== Resumen CONCEPTO PREVIO (limpieza) ===
Total filas: 24055
Con concepto (no NULL): 5845
Sin concepto (NULL): 18210

Conteo por ESTADO:
ConceptoPrevio_ESTADO
vacio                 18202
ok                     5845
token_sin_concepto        8
Name: count, dtype: int64

=== Resumen longitudes ConceptoPrevio_SQL ===
count    24055.000000
mean       814.465724
std       2117.303567
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      32765.000000
Name: ConceptoPrevio_LEN, dtype: float64

=== Ejemplos conceptos cortos (<=20) para revisar (máx 50) ===
 Fila_Excel_aprox              FMI DEPARTAMENTO              MUNICIPIO   ConceptoPrevio_SQL  ConceptoPrevio_LEN
             1041 CÓDIGO CATASTRAL      VICHADA               CUMARIBO      No registra FMI                  15
             1072        ESCRITURA      CAQUETA SAN VICENTE DEL CAGUAN      No registra FMI                  15
             3026        076-23960       BOYACA                  

## "ANALISIS JURÍDICO / CONCEPTO FINAL (DEFINICIÓN DE VIABILIDAD JURÍDICA)"


In [42]:
import pandas as pd
import numpy as np
import re

# ===========================================
# CONCEPTO FINAL VIABILIDAD JURÍDICA (SIN RECARGAR df; resto igual)
# Excel -> AnalisisJuridico.ConceptoFinal.ConceptoActualDeViabilidadJuridica (varchar(MAX))
# ===========================================

# 0) Asegurar columna de fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas ConceptoFinal_* si existen (SIN recargar df)
cols_prev = [c for c in df.columns if isinstance(c, str) and (c.startswith("ConceptoFinal_") or c.startswith("ConceptoActualDeViabilidadJuridica_"))]
if cols_prev:
    df.drop(columns=cols_prev, inplace=True, errors="ignore")

# 1) Forzar la columna exacta del Excel (sin depender de candidatas[0])
COL_EXACTA = "ANALISIS JURÍDICO / CONCEPTO FINAL\n(DEFINICIÓN DE VIABILIDAD JURÍDICA)"

if COL_EXACTA in df.columns:
    col_conc_final = COL_EXACTA
else:
    candidatas = [
        c for c in df.columns
        if "CONCEPTO FINAL" in str(c).upper()
           or "ANALISIS JURIDICO" in str(c).upper()
           or "ANÁLISIS JURÍDICO" in str(c).upper()
    ]
    print("Columnas candidatas para CONCEPTO FINAL:", candidatas)
    if not candidatas:
        raise ValueError(
            "No se encontró una columna que parezca ser el CONCEPTO FINAL "
            "(revisa encabezados en el Excel)."
        )
    col_conc_final = candidatas[0]

print("Usando columna de concepto final:", col_conc_final)

# 2) Tokens que significan “sin información” (solo se vuelven NULL si el valor es EXACTAMENTE eso)
NO_INFO = {
    "NO",
    "NO.",
    "NO REGISTRA",
    "NO REGISTRA.",
    "NO APLICA",
    "NO APLICA.",
    "NO SE APLICA",
    "NO SE APLICA.",
    "NO REPORTA",
    "N/A",
    "NA",
    "N.A.",
    "SIN INFORMACION",
    "SIN INFORMACIÓN",
    "SIN REGISTRO",
    "SIN REGISTRO.",
    "SIN_REG",
    "NINGUNO",
    "NINGUNA",
    "0",
}

# Pre-normalizar el set una sola vez (para no recalcular dentro del apply)
def normalizar_token(s: str) -> str:
    """
    Normaliza SOLO para comparar contra NO_INFO.
    No altera el texto que se migra; el texto migrado se conserva tal cual.
    """
    t = str(s).strip()
    t = re.sub(r"\s+", " ", t)
    t = t.strip().strip(" .,:;")
    return t.upper()

NO_INFO_NORM = {normalizar_token(x) for x in NO_INFO}

def limpiar_concepto_final(v):
    """
    Retorna (valor_para_sql, estado)
    - No se “reduce” el texto: si no es token NO_INFO, se conserva EXACTO como viene.
    """
    if pd.isna(v):
        return (None, "vacio")

    t = str(v)
    t_strip = t.strip()
    if t_strip == "":
        return (None, "vacio")

    tok = normalizar_token(t_strip)
    if tok in NO_INFO_NORM:
        return (None, "token_no_info")

    return (t, "ok")

# 3) Calcular en DF auxiliar (evita fragmentación y NO recarga df)
work = pd.DataFrame(index=df.index)

# 3.1) Copia del texto original (NO se modifica; trazabilidad)
work["ConceptoFinal_TXT_ORIG"] = df[col_conc_final]

# 4) Generar columna lista para migrar + estado (sin perder información)
res = work["ConceptoFinal_TXT_ORIG"].apply(limpiar_concepto_final)
work["ConceptoActualDeViabilidadJuridica_SQL"] = res.apply(lambda x: x[0])
work["ConceptoFinal_ESTADO"] = res.apply(lambda x: x[1])

# 5) Versión “limpia” SOLO para análisis (espacios consolidados). No se usa para migrar.
work["ConceptoFinal_TXT_LIMPIO"] = (
    work["ConceptoActualDeViabilidadJuridica_SQL"]
      .astype("string")
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
)
work.loc[work["ConceptoActualDeViabilidadJuridica_SQL"].isna(), "ConceptoFinal_TXT_LIMPIO"] = None

# 6) Longitud (para revisar extremos)
work["ConceptoFinal_LEN"] = work["ConceptoFinal_TXT_LIMPIO"].fillna("").str.len()

# 7) Asignar al df IN-PLACE (SIN df = ...)
for c in work.columns:
    df[c] = work[c].values

print("\n=== Resumen longitudes ConceptoActualDeViabilidadJuridica_SQL (solo análisis) ===")
print(df["ConceptoFinal_LEN"].describe())

print("\n=== Conteo por ConceptoFinal_ESTADO ===")
print(df["ConceptoFinal_ESTADO"].value_counts(dropna=False))

# Tokens mapeados a NULL (top 30 normalizados)
tokens_null = (
    df.loc[df["ConceptoFinal_ESTADO"] == "token_no_info", "ConceptoFinal_TXT_ORIG"]
      .dropna()
      .apply(normalizar_token)
      .value_counts()
      .head(30)
)
print("\n=== Top 30 tokens (normalizados) mapeados a NULL ===")
print(tokens_null)

# 8) Ejemplos cortos (0 < len <= 20) - SOLO los que quedaron 'ok'
print("\n=== Ejemplos de conceptos muy cortos (0 < longitud <= 20) [estado=ok] ===")
conceptos_cortos = df[
    (df["ConceptoFinal_ESTADO"] == "ok") &
    (df["ConceptoFinal_LEN"] > 0) &
    (df["ConceptoFinal_LEN"] <= 20)
][[
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "ConceptoFinal_TXT_ORIG",
    "ConceptoActualDeViabilidadJuridica_SQL",
    "ConceptoFinal_LEN",
    "ConceptoFinal_ESTADO"
]]
print(conceptos_cortos.head(30).to_string(index=False))

# 9) Ejemplos largos (>= 500) - SOLO los que quedaron 'ok'
print("\n=== Ejemplos de conceptos largos (>= 500 caracteres) [estado=ok] ===")
conceptos_largos = df[
    (df["ConceptoFinal_ESTADO"] == "ok") &
    (df["ConceptoFinal_LEN"] >= 500)
][[
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "ConceptoFinal_TXT_ORIG",
    "ConceptoActualDeViabilidadJuridica_SQL",
    "ConceptoFinal_LEN",
    "ConceptoFinal_ESTADO"
]]
print(conceptos_largos.head(20).to_string(index=False))

# 10) Export de revisión
cols_base = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    col_conc_final,
    "ConceptoFinal_TXT_ORIG",
    "ConceptoFinal_TXT_LIMPIO",
    "ConceptoActualDeViabilidadJuridica_SQL",
    "ConceptoFinal_LEN",
    "ConceptoFinal_ESTADO",
]
cols_base = [c for c in cols_base if c in df.columns]

nombre_archivo = "ConceptoFinal_ViabilidadJuridica_revision.xlsx"

with pd.ExcelWriter(nombre_archivo, engine="openpyxl") as writer:
    df[cols_base].to_excel(writer, sheet_name="Todos", index=False)

    if not conceptos_cortos.empty:
        conceptos_cortos.to_excel(writer, sheet_name="Cortos_<=20", index=False)

    if not conceptos_largos.empty:
        conceptos_largos.to_excel(writer, sheet_name="Largos_>=500", index=False)

    # Hoja extra: tokens a NULL (para auditoría)
    df_tokens = df.loc[df["ConceptoFinal_ESTADO"] == "token_no_info", [
        "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", "ConceptoFinal_TXT_ORIG"
    ]].copy()
    if not df_tokens.empty:
        df_tokens["Token_NORM"] = df_tokens["ConceptoFinal_TXT_ORIG"].apply(normalizar_token)
        df_tokens.to_excel(writer, sheet_name="Tokens_a_NULL", index=False)

print(f"\nArchivo creado: {nombre_archivo}")

# =========================
# COLUMNA A MIGRAR:
# df["ConceptoActualDeViabilidadJuridica_SQL"]
# =========================


Usando columna de concepto final: ANALISIS JURÍDICO / CONCEPTO FINAL
(DEFINICIÓN DE VIABILIDAD JURÍDICA)


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3251873678.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = work[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3251873678.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c] = work[c].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3251873678.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin


=== Resumen longitudes ConceptoActualDeViabilidadJuridica_SQL (solo análisis) ===
count        24055.0
mean     3447.021077
std      5044.179609
min              0.0
25%              0.0
50%             75.0
75%           7030.0
max          32678.0
Name: ConceptoFinal_LEN, dtype: Float64

=== Conteo por ConceptoFinal_ESTADO ===
ConceptoFinal_ESTADO
ok               12339
vacio            11713
token_no_info        3
Name: count, dtype: int64

=== Top 30 tokens (normalizados) mapeados a NULL ===
ConceptoFinal_TXT_ORIG
NO APLICA    2
NO           1
Name: count, dtype: int64

=== Ejemplos de conceptos muy cortos (0 < longitud <= 20) [estado=ok] ===
 Fila_Excel_aprox         FMI       DEPARTAMENTO              MUNICIPIO ConceptoFinal_TXT_ORIG ConceptoActualDeViabilidadJuridica_SQL  ConceptoFinal_LEN ConceptoFinal_ESTADO
             2743    355-4203             TOLIMA               PLANADAS          FOLIO CERRADO                          FOLIO CERRADO                 13                  

## Fechainforme

In [43]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# ===============================
# FECHA DE INFORME (CONCEPTO FINAL)
# Destino: AnalisisJuridico.ConceptoFinal.FechaInforme (datetime NULL)
# ===============================

# 0) Asegurar columna de fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Re-ejecución segura SIN recargar df:
#      si ya existen columnas derivadas de este bloque, se borran IN-PLACE
cols_prev = [
    "FECHA_INFORME_RAW",
    "FECHA_INFORME_TXT_LIMPIO",
    "FechaInforme_SQL",
    "FechaInforme_ESTADO",
    "FechaInforme_METODO",
    "FechaInforme_FUERA_RANGO",
]
df.drop(columns=[c for c in cols_prev if c in df.columns], inplace=True, errors="ignore")

# 1) Elegir la columna correcta (preferir EXACTA "FECHA DE INFORME")
COL_EXACTA = "FECHA DE INFORME"

# Candidatas "FECHA"+"INFORME" excluyendo PREVIO y excluyendo columnas derivadas (_RAW/_SQL/_TXT)
cols_fecha_candidatas = [
    c for c in df.columns
    if ("FECHA" in str(c).upper())
    and ("INFORME" in str(c).upper())
    and ("PREVIO" not in str(c).upper())
    and (not str(c).upper().endswith("_RAW"))
    and (not str(c).upper().endswith("_SQL"))
    and (not str(c).upper().endswith("_TXT_LIMPIO"))
]

print("Columnas candidatas para FECHA DE INFORME:", cols_fecha_candidatas)

if COL_EXACTA in df.columns:
    col_fecha_informe = COL_EXACTA
elif cols_fecha_candidatas:
    col_fecha_informe = cols_fecha_candidatas[0]
else:
    raise ValueError("No se encontró una columna para FECHA DE INFORME (sin PREVIO).")

print("Usando columna para ConceptoFinal.FechaInforme:", col_fecha_informe)

# 2) RAW (no se altera) + TXT limpio SOLO para análisis (respetando nulos)
df["FECHA_INFORME_RAW"] = df[col_fecha_informe]

df["FECHA_INFORME_TXT_LIMPIO"] = (
    df["FECHA_INFORME_RAW"]
      .astype("string")              # no vuelve NaT -> 'NaT'
      .str.strip()
)

# 3) Tokens equivalentes a "sin información" (se vuelven NULL si el valor es EXACTAMENTE eso)
VALORES_VACIOS_INFORME = {
    "NO", "NO.", "NO REGISTRA", "NO REGISTRA.", "NO APLICA", "NO APLICA.",
    "NO EXISTE", "NO EXISTE.", "NO REPORTA", "N/A", "NA", "N.A.", "SIN REGISTRO", "SIN_REG", "0"
}

def normalizar_token(s: str) -> str:
    t = str(s).strip()
    t = re.sub(r"\s+", " ", t)
    t = t.strip().strip(" .,:;")
    return t.upper()

NO_INFO_NORM = {normalizar_token(x) for x in VALORES_VACIOS_INFORME}

# 4) Parse robusto sin perder info (devuelve fecha + estado + método)
limite_inf = pd.Timestamp("1900-01-01")
limite_sup = pd.Timestamp("2035-12-31")

def parse_fecha_informe(v):
    """
    Retorna (Timestamp/NaT, estado, metodo)
    estados: vacio | token_no_info | ok | parse_fail | num_fuera_rango
    """
    if pd.isna(v):
        return (pd.NaT, "vacio", "na")

    # si ya viene como fecha
    if isinstance(v, (pd.Timestamp, datetime)):
        dt = pd.to_datetime(v, errors="coerce")
        if pd.isna(dt):
            return (pd.NaT, "parse_fail", "timestamp_coerce")
        return (dt, "ok", "timestamp")

    # si viene numérico (posible serial de Excel)
    if isinstance(v, (int, float, np.integer, np.floating)):
        if not np.isfinite(v):
            return (pd.NaT, "vacio", "num_nan")
        # serial típico Excel ~ [1..60000]
        if 0 < float(v) < 60000:
            dt = pd.to_datetime(float(v), unit="D", origin="1899-12-30", errors="coerce")
            if pd.isna(dt):
                return (pd.NaT, "parse_fail", "excel_serial")
            return (dt, "ok", "excel_serial")
        else:
            return (pd.NaT, "num_fuera_rango", "num")

    t = str(v).strip()
    if t == "":
        return (pd.NaT, "vacio", "blank")

    tok = normalizar_token(t)
    if tok in NO_INFO_NORM:
        return (pd.NaT, "token_no_info", "token")

    # Normalizaciones suaves SOLO para parseo (no cambian RAW)
    t2 = re.sub(r"\s+", " ", t).strip()

    dt = pd.NaT
    metodo = "fallback"

    # YYYY-MM-DD
    if re.fullmatch(r"\d{4}-\d{2}-\d{2}", t2):
        dt = pd.to_datetime(t2, format="%Y-%m-%d", errors="coerce")
        metodo = "fmt_%Y-%m-%d"

    # YYYY/MM/DD
    elif re.fullmatch(r"\d{4}/\d{1,2}/\d{1,2}", t2):
        dt = pd.to_datetime(t2, format="%Y/%m/%d", errors="coerce")
        metodo = "fmt_%Y/%m/%d"

    # DD/MM/YYYY o MM/DD/YYYY (se prueba dd/mm y si falla, mm/dd)
    elif re.fullmatch(r"\d{1,2}/\d{1,2}/\d{4}", t2):
        dt = pd.to_datetime(t2, format="%d/%m/%Y", errors="coerce")
        metodo = "fmt_%d/%m/%Y"
        if pd.isna(dt):
            dt = pd.to_datetime(t2, format="%m/%d/%Y", errors="coerce")
            metodo = "fmt_%m/%d/%Y"

    # DD-MM-YYYY o MM-DD-YYYY
    elif re.fullmatch(r"\d{1,2}-\d{1,2}-\d{4}", t2):
        dt = pd.to_datetime(t2, format="%d-%m-%Y", errors="coerce")
        metodo = "fmt_%d-%m-%Y"
        if pd.isna(dt):
            dt = pd.to_datetime(t2, format="%m-%d-%Y", errors="coerce")
            metodo = "fmt_%m-%d-%Y"

    else:
        # último recurso
        dt = pd.to_datetime(t2, dayfirst=True, errors="coerce")
        metodo = "dateutil_dayfirst"

    if pd.isna(dt):
        return (pd.NaT, "parse_fail", metodo)

    return (dt, "ok", metodo)

# === CAMBIO CLAVE: NO usar df = df.copy() ni df = pd.concat(...)
parsed = df["FECHA_INFORME_RAW"].apply(parse_fecha_informe)
parsed_df = pd.DataFrame(
    parsed.tolist(),
    columns=["FechaInforme_SQL", "FechaInforme_ESTADO", "FechaInforme_METODO"],
    index=df.index
)

# Asignación IN-PLACE (mejor para performance / sin recargar df)
for c in parsed_df.columns:
    df[c] = parsed_df[c].values

# 5) Flag fuera de rango (NO borra la fecha; solo marca)
df["FechaInforme_FUERA_RANGO"] = (
    df["FechaInforme_SQL"].notna()
    & ((df["FechaInforme_SQL"] < limite_inf) | (df["FechaInforme_SQL"] > limite_sup))
)

# 6) Resumen correcto
total_filas = len(df)
n_validas = df["FechaInforme_SQL"].notna().sum()
n_nulas = df["FechaInforme_SQL"].isna().sum()

print("\n=== Resumen FECHA DE INFORME (ConceptoFinal) ===")
print("Total filas:", total_filas)
print("Fechas válidas (no NaT):", n_validas)
print("Valores nulos (NaT):", n_nulas)
print("\nConteo por estado:")
print(df["FechaInforme_ESTADO"].value_counts(dropna=False))

print("\nFuera de rango 1900–2035 (solo marcadas, no borradas):", int(df["FechaInforme_FUERA_RANGO"].sum()))

# 7) Tokens a NULL (top)
tokens_null = (
    df.loc[df["FechaInforme_ESTADO"] == "token_no_info", "FECHA_INFORME_TXT_LIMPIO"]
      .dropna()
      .apply(normalizar_token)
      .value_counts()
      .head(30)
)
print("\n=== Top tokens mapeados a NULL ===")
print(tokens_null)

# 8) Ejemplos OK (original vs normalizada)
print("\n=== Ejemplos FECHA DE INFORME original vs normalizada (ok) ===")
ej_ok = df[df["FechaInforme_SQL"].notna()][
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
     "FECHA_INFORME_RAW", "FechaInforme_SQL", "FechaInforme_METODO", "FechaInforme_FUERA_RANGO"]
].head(30)
print(ej_ok.to_string(index=False))

# 9) Lo que falta arreglar: parse_fail con texto real (no vacío)
mask_texto_real = df["FECHA_INFORME_TXT_LIMPIO"].notna() & (df["FECHA_INFORME_TXT_LIMPIO"].str.len() > 0)
mask_fail = (df["FechaInforme_ESTADO"] == "parse_fail") & mask_texto_real

print("\n=== Filas con texto que NO se pudo convertir (parse_fail) ===")
print("Total parse_fail con texto real:", int(mask_fail.sum()))

ej_fail = df.loc[mask_fail, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "FECHA_INFORME_RAW", "FECHA_INFORME_TXT_LIMPIO", "FechaInforme_METODO"
]].head(100)
print(ej_fail.to_string(index=False))

# 10) Export de revisión
nombre_archivo = "FechaInforme_ConceptoFinal_revision.xlsx"
cols_export = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    col_fecha_informe,
    "FECHA_INFORME_RAW", "FECHA_INFORME_TXT_LIMPIO",
    "FechaInforme_SQL", "FechaInforme_ESTADO", "FechaInforme_METODO", "FechaInforme_FUERA_RANGO",
]
cols_export = [c for c in cols_export if c in df.columns]

with pd.ExcelWriter(nombre_archivo) as writer:
    df[cols_export].to_excel(writer, sheet_name="Todos", index=False)
    if mask_fail.any():
        df.loc[mask_fail, cols_export].to_excel(writer, sheet_name="ParseFail", index=False)
    if (df["FechaInforme_ESTADO"] == "token_no_info").any():
        df.loc[df["FechaInforme_ESTADO"] == "token_no_info", cols_export].to_excel(writer, sheet_name="Tokens_a_NULL", index=False)
    if df["FechaInforme_FUERA_RANGO"].any():
        df.loc[df["FechaInforme_FUERA_RANGO"], cols_export].to_excel(writer, sheet_name="Fuera_Rango", index=False)

print(f"\nArchivo creado: {nombre_archivo}")

# =========================
# COLUMNA A MIGRAR:
# df["FechaInforme_SQL"]  -> ConceptoFinal.FechaInforme (datetime NULL)
# =========================


Columnas candidatas para FECHA DE INFORME: ['FECHA DE INFORME', 'FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL', 'FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO']
Usando columna para ConceptoFinal.FechaInforme: FECHA DE INFORME

=== Resumen FECHA DE INFORME (ConceptoFinal) ===
Total filas: 24055
Fechas válidas (no NaT): 9885
Valores nulos (NaT): 14170

Conteo por estado:
FechaInforme_ESTADO
vacio    14170
ok        9885
Name: count, dtype: int64

Fuera de rango 1900–2035 (solo marcadas, no borradas): 0

=== Top tokens mapeados a NULL ===
Series([], Name: count, dtype: Int64)

=== Ejemplos FECHA DE INFORME original vs normalizada (ok) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO           MUNICIPIO FECHA_INFORME_RAW FechaInforme_SQL FechaInforme_METODO  FechaInforme_FUERA_RANGO
                2  062-37357      BOLIVAR            ZAMBRANO        2024-10-17       2024-10-17           timestamp                     False
                3  062-37359      BOLIVAR            ZAMBRANO        

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1125889187.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["FECHA_INFORME_RAW"] = df[col_fecha_informe]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1125889187.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["FECHA_INFORME_TXT_LIMPIO"] = (
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1125889187.py:167: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has


Archivo creado: FechaInforme_ConceptoFinal_revision.xlsx


*aca voy*

## VIABLE / NO VIABLE / PENDIENTE

*Los nulos se pasaron como Sin definir*

In [44]:
import pandas as pd
import numpy as np
import re
import unicodedata

# ======================================================
# VIABILIDAD: VIABLE / NO VIABLE / PENDIENTE / PARCIAL
# Destino: AnalisisJuridico.ConceptoFinal.IdTipoViabilidad (tinyint)
# ======================================================

# 0) Asegurar columna de fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Re-ejecución segura SIN recargar df:
#      borrar columnas derivadas de este bloque si existen (IN-PLACE)
cols_prev = [
    "Viabilidad_RAW",
    "Viabilidad_TXT_LIMPIO",
    "IdTipoViabilidad_SQL",
    "NombreTipoViabilidad_SQL",
    "Viabilidad_ESTADO",
]
df.drop(columns=[c for c in cols_prev if c in df.columns], inplace=True, errors="ignore")

# -------------------------------------------
# 1) Ubicar la columna de Excel
# -------------------------------------------
preferida = "VIABLE / NO VIABLE / PENDIENTE"

if preferida in df.columns:
    col_viab = preferida
else:
    # Búsqueda robusta
    cols_viab = [c for c in df.columns if "VIABLE" in str(c).upper()]
    if not cols_viab:
        raise ValueError("No se encontró ninguna columna candidata para VIABILIDAD (que contenga 'VIABLE').")
    # Si existe una que también contenga 'PEND', preferirla
    cols_viab_pend = [c for c in cols_viab if "PEND" in str(c).upper()]
    col_viab = cols_viab_pend[0] if cols_viab_pend else cols_viab[0]

print("Usando columna:", col_viab)

# -------------------------------------------
# 2) Guardar RAW (sin tocar) + texto limpio solo para comparar
# -------------------------------------------
viab_raw = df[col_viab]  # NO modificar: es la fuente original

def _to_str_keep_na(v):
    if pd.isna(v):
        return pd.NA
    return str(v)

viab_txt = viab_raw.map(_to_str_keep_na).astype("string")

# Limpieza mínima SOLO para comparación (no cambia el RAW)
viab_txt_limpio = (
    viab_txt
      .str.replace("\u00A0", " ", regex=False)   # NBSP -> espacio normal
      .str.strip()
      .str.replace(r"\s+", " ", regex=True)     # colapsar espacios
)

def quitar_acentos(txt: str) -> str:
    return "".join(
        ch for ch in unicodedata.normalize("NFD", txt)
        if unicodedata.category(ch) != "Mn"
    )

def normalizar_cmp(s):
    if pd.isna(s):
        return pd.NA
    return quitar_acentos(str(s)).upper()

viab_cmp = viab_txt_limpio.map(normalizar_cmp).astype("string")

# -------------------------------------------
# 3) Mapeo a catálogo (prioridad: parcial -> no viable -> pendiente -> viable)
# -------------------------------------------
NO_INFO_EXACTO = {
    "NO APLICA", "NO SE APLICA", "N/A", "NA", "N.A.", "SIN INFORMACION", "SIN INFORMACIÓN",
    "SIN REGISTRO", "NO REGISTRA", "SIN DEFINIR", "NINGUNO", "NINGUNA", "0", ""
}

def mapear_viabilidad_cmp(t_up: str):
    """
    Devuelve (IdTipoViabilidad, NombreTipoViabilidad, Estado)
      Estado: vacio | ok | otro
    """
    if pd.isna(t_up):
        return 0, "Sin definir", "vacio"

    t = str(t_up).strip()
    if t in NO_INFO_EXACTO:
        return 0, "Sin definir", "vacio"

    # 3) Viabilidad parcial
    if ("VIABILIDAD PARCIAL" in t) or ("PARCIALMENTE" in t) or (("PARCIAL" in t) and ("VIAB" in t or "VIABLE" in t)):
        return 3, "Viabilidad parcial", "ok"

    # 4) No viable
    if ("NO VIABLE" in t) or re.search(r"\bNO\b.*\bVIABLE\b", t):
        return 4, "No viable", "ok"

    # 1) Pendiente
    if "PEND" in t:
        return 1, "Pendiente", "ok"

    # 2) Viable
    if "VIABLE" in t:
        return 2, "Viable", "ok"

    return 0, f"OTRO: {t}", "otro"

mapped = viab_cmp.apply(lambda x: pd.Series(mapear_viabilidad_cmp(x), index=[
    "IdTipoViabilidad_SQL", "NombreTipoViabilidad_SQL", "Viabilidad_ESTADO"
]))

# === CAMBIO CLAVE: NO df = df.copy() ni df = pd.concat(...)
# Asignación IN-PLACE
df["Viabilidad_RAW"] = viab_raw
df["Viabilidad_TXT_LIMPIO"] = viab_txt_limpio
df["IdTipoViabilidad_SQL"] = mapped["IdTipoViabilidad_SQL"].values
df["NombreTipoViabilidad_SQL"] = mapped["NombreTipoViabilidad_SQL"].values
df["Viabilidad_ESTADO"] = mapped["Viabilidad_ESTADO"].values

# Asegurar tipo numérico (tinyint)
df["IdTipoViabilidad_SQL"] = pd.to_numeric(df["IdTipoViabilidad_SQL"], errors="coerce").fillna(0).astype("int64")

# -------------------------------------------
# 4) Resúmenes
# -------------------------------------------
print("\n=== Resumen IdTipoViabilidad_SQL ===")
print(df["IdTipoViabilidad_SQL"].value_counts(dropna=False).sort_index())

print("\n=== Resumen NombreTipoViabilidad_SQL ===")
print(df["NombreTipoViabilidad_SQL"].value_counts(dropna=False))

# Casos fuera de catálogo (OTRO)
mask_otros = df["Viabilidad_ESTADO"].eq("otro")
otros = df.loc[mask_otros, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "Viabilidad_RAW", "Viabilidad_TXT_LIMPIO",
    "IdTipoViabilidad_SQL", "NombreTipoViabilidad_SQL", "Viabilidad_ESTADO"
]]

print("\n=== Casos fuera de catálogo (OTRO) ===")
if otros.empty:
    print("No hay textos fuera de catálogo.")
else:
    print(otros.head(200).to_string(index=False))

# -------------------------------------------
# 5) Archivo de revisión (incluye hoja de valores distintos)
# -------------------------------------------
cols_revision = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    col_viab, "Viabilidad_RAW", "Viabilidad_TXT_LIMPIO",
    "IdTipoViabilidad_SQL", "NombreTipoViabilidad_SQL", "Viabilidad_ESTADO"
]
cols_revision = [c for c in cols_revision if c in df.columns]

# Tabla de valores distintos (útil para auditoría)
distinct = (
    df[["Viabilidad_RAW", "Viabilidad_TXT_LIMPIO", "IdTipoViabilidad_SQL", "NombreTipoViabilidad_SQL", "Viabilidad_ESTADO"]]
      .drop_duplicates()
      .sort_values(by=["IdTipoViabilidad_SQL", "Viabilidad_ESTADO", "NombreTipoViabilidad_SQL"], kind="stable")
)

archivo = "Viabilidad_revision.xlsx"
with pd.ExcelWriter(archivo) as writer:
    df[cols_revision].to_excel(writer, sheet_name="Todos", index=False)
    distinct.to_excel(writer, sheet_name="Valores_distintos", index=False)
    if not otros.empty:
        otros.to_excel(writer, sheet_name="OTRO_para_revision", index=False)

print(f"\nArchivo creado: {archivo}")


Usando columna: VIABLE / NO VIABLE / PENDIENTE


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2669909780.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Viabilidad_RAW"] = viab_raw
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2669909780.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Viabilidad_TXT_LIMPIO"] = viab_txt_limpio
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2669909780.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po


=== Resumen IdTipoViabilidad_SQL ===
IdTipoViabilidad_SQL
0     7251
1     1249
2    10081
3       63
4     5411
Name: count, dtype: int64

=== Resumen NombreTipoViabilidad_SQL ===
NombreTipoViabilidad_SQL
Viable                10081
Sin definir            7251
No viable              5411
Pendiente              1249
Viabilidad parcial       63
Name: count, dtype: int64

=== Casos fuera de catálogo (OTRO) ===
No hay textos fuera de catálogo.

Archivo creado: Viabilidad_revision.xlsx


## TIPO DE INFORME

In [45]:
import pandas as pd
import numpy as np
import unicodedata

# ============================================================
# TIPO DE INFORME
# Destino: AnalisisJuridico.ConceptoFinal.IdTipoInforme (tinyint NULL)
# Catalogo:
#   1 = Nuevo
#   2 = Actualización
# ============================================================

# 0) Asegurar columna de fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Re-ejecución segura SIN recargar df:
#      borrar columnas derivadas de este bloque si existen (IN-PLACE)
cols_prev = [
    "TipoInforme_RAW",
    "TipoInforme_TXT_LIMPIO",
    "NombreTipoInforme_SQL",
    "IdTipoInforme_SQL",
    "TipoInforme_ESTADO",
]
df.drop(columns=[c for c in cols_prev if c in df.columns], inplace=True, errors="ignore")

# ============================================================
# 1) Ubicar columna de tipo de informe
# ============================================================
candidatas = [c for c in df.columns if ("TIPO" in str(c).upper()) and ("INFORME" in str(c).upper())]

print("Columnas candidatas para TIPO DE INFORME:", candidatas)

if not candidatas:
    raise ValueError("No se encontró columna para TIPO DE INFORME; revisa encabezados.")

col_tipo_inf = candidatas[0]
print("Usando columna:", col_tipo_inf)

tipo_raw = df[col_tipo_inf]  # NO tocar

# ============================================================
# 2) Texto limpio para clasificar (MAYÚSC, sin tildes, sin dobles espacios)
# ============================================================
def limpiar_texto_tipo_inf(v):
    if pd.isna(v):
        return None
    t = str(v).replace("\u00A0", " ").strip()  # NBSP
    if t == "":
        return None
    t = " ".join(t.split())
    t_up = t.upper()
    t_norm = unicodedata.normalize("NFD", t_up)
    t_sin_tilde = "".join(ch for ch in t_norm if unicodedata.category(ch) != "Mn")
    return t_sin_tilde

tipo_limpio = tipo_raw.apply(limpiar_texto_tipo_inf)

# ============================================================
# 3) Mapear a catálogo (Nuevo / Actualización) + SIN DATO
# ============================================================
SIN_DATO = {
    "NO APLICA",
    "NO APLICA.",
    "NO REGISTRA",
    "NO REGISTRA.",
    "SIN INFORME",
    "SIN INFORMACION",
    "NINGUNO",
    "NINGUNA",
    "N/A",
    "NA",
    "N.A.",
    "0",
    ""
}

def normalizar_tipo_informe(t):
    if t is None:
        return None

    # Sin dato -> NULL
    if t in SIN_DATO:
        return None

    # NUEVO (variantes)
    if t in {
        "NUEVO",
        "INICIAL",
        "PRIMERA VEZ",
        "INFORME NUEVO",
        "NVO"
    }:
        return "NUEVO"

    # ACTUALIZACION (variantes)
    if t in {
        "ACTUALIZACION",
        "ACTUALIZACION DEL INFORME",
        "ACTUALIZADO",
        "INFORME ACTUALIZADO",
        "ACTUALIZACION INFORME",
        "ACTUALIZACION."
    }:
        return "ACTUALIZACION"

    # Otro texto -> queda sin clasificar (para revisar), pero NO se pierde (queda en RAW)
    return None

nombre_sql = tipo_limpio.apply(normalizar_tipo_informe)

mapa_tipo_informe = {
    "NUEVO": 1,
    "ACTUALIZACION": 2,
}
id_sql = nombre_sql.map(mapa_tipo_informe)

# Estado (para auditoría)
estado = np.where(tipo_limpio.isna(), "vacio",
          np.where(nombre_sql.isna(), "pendiente", "ok"))

# === CAMBIO CLAVE: NO df = df.copy() ni df = pd.concat(...)
# Asignación IN-PLACE
df["TipoInforme_RAW"] = tipo_raw
df["TipoInforme_TXT_LIMPIO"] = tipo_limpio
df["NombreTipoInforme_SQL"] = nombre_sql
df["IdTipoInforme_SQL"] = id_sql
df["TipoInforme_ESTADO"] = estado

# ============================================================
# 4) Resúmenes + pendientes
# ============================================================
print("\n=== Resumen NombreTipoInforme_SQL (catálogo) ===")
print(df["NombreTipoInforme_SQL"].value_counts(dropna=False))

print("\n=== Conteo IdTipoInforme_SQL ===")
print(df["IdTipoInforme_SQL"].value_counts(dropna=False))

mask_no_clas = (
    df["TipoInforme_TXT_LIMPIO"].notna()
    & (df["TipoInforme_TXT_LIMPIO"].astype("string").str.len() > 0)
    & df["NombreTipoInforme_SQL"].isna()
)

no_clasificados = df.loc[
    mask_no_clas,
    ["Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO","TipoInforme_RAW","TipoInforme_TXT_LIMPIO"]
].copy()

print("\n=== Valores de TIPO DE INFORME no clasificados (para revisar) ===")
if not no_clasificados.empty:
    print(no_clasificados.head(200).to_string(index=False))
else:
    print("No hay valores pendientes de clasificar (todos NULL o mapeados a catálogo).")

pendientes_resumen = (
    df.loc[mask_no_clas, "TipoInforme_TXT_LIMPIO"]
      .value_counts(dropna=False)
      .reset_index()
)
pendientes_resumen.columns = ["TipoInforme_TXT_LIMPIO", "conteo"]

# ============================================================
# 5) Excel de revisión (varias hojas)
# ============================================================
resumen_mapeo = (
    df
      .groupby(
          ["TipoInforme_RAW","TipoInforme_TXT_LIMPIO","NombreTipoInforme_SQL","IdTipoInforme_SQL","TipoInforme_ESTADO"],
          dropna=False
      )
      .size()
      .reset_index(name="conteo")
      .sort_values(by=["TipoInforme_ESTADO","IdTipoInforme_SQL","conteo"], ascending=[True, True, False], kind="stable")
)

ruta_salida = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\TipoInforme_revision.xlsx"

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    resumen_mapeo.to_excel(writer, sheet_name="mapeo_tipo_informe", index=False)
    no_clasificados.to_excel(writer, sheet_name="pendientes_clasificacion", index=False)
    pendientes_resumen.to_excel(writer, sheet_name="pendientes_resumen", index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)


Columnas candidatas para TIPO DE INFORME: ['TIPO DE INFORME']
Usando columna: TIPO DE INFORME

=== Resumen NombreTipoInforme_SQL (catálogo) ===
NombreTipoInforme_SQL
None             17144
NUEVO             5111
ACTUALIZACION     1800
Name: count, dtype: int64

=== Conteo IdTipoInforme_SQL ===
IdTipoInforme_SQL
NaN    17144
1.0     5111
2.0     1800
Name: count, dtype: int64

=== Valores de TIPO DE INFORME no clasificados (para revisar) ===
No hay valores pendientes de clasificar (todos NULL o mapeados a catálogo).

Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\TipoInforme_revision.xlsx


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\509742708.py:125: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["TipoInforme_RAW"] = tipo_raw
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\509742708.py:126: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["TipoInforme_TXT_LIMPIO"] = tipo_limpio
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\509742708.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

## "CAUSAL NO VIABILIDAD"


In [46]:
import pandas as pd
import numpy as np
import unicodedata
import re

# ==========================================================
# CAUSAL NO VIABILIDAD  → ConceptoFinal.IdTipoNoViabilidad (tinyint NULL, DEFAULT 0 en BD)
# ==========================================================

# 0) Asegurar columna de fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Re-ejecución segura SIN recargar df: borrar columnas derivadas si existen (IN-PLACE)
cols_prev = [
    "CausalNoViab_RAW",
    "CausalNoViab_TXT_LIMPIO",
    "CausalNoViab_MAP_TXT",
    "IdTipoNoViabilidad_SQL",
    "NombreTipoNoViabilidad_SQL",
    # auditoría reglas
    "IdTipoNoViabilidad_ANTES",
    "IdTipoNoViabilidad_DESPUES",
    "CausalRegla_Aplicada",
]
df.drop(columns=[c for c in cols_prev if c in df.columns], inplace=True, errors="ignore")

# 1) Localizar la columna de Excel
candidatas = [
    c for c in df.columns
    if "CAUSAL" in str(c).upper() and "VIABILIDAD" in str(c).upper()
]

print("Columnas candidatas para CAUSAL NO VIABILIDAD:", candidatas)
if not candidatas:
    raise ValueError("No se encontró columna para CAUSAL NO VIABILIDAD.")

col_causal = candidatas[0]  # esperado: 'CAUSAL \n NO VIABILIDAD'
print("Usando columna:", col_causal)

# 2) Columna RAW (exactamente igual al Excel)
df["CausalNoViab_RAW"] = df[col_causal]

# 3) Texto limpio para análisis/mapeo (SIN perder RAW)
def limpiar_txt(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    if t == "":
        return None
    t_norm = unicodedata.normalize("NFD", t)
    t_norm = "".join(ch for ch in t_norm if unicodedata.category(ch) != "Mn")
    t_norm = " ".join(t_norm.split())
    return t_norm.upper()

df["CausalNoViab_TXT_LIMPIO"] = df["CausalNoViab_RAW"].apply(limpiar_txt)

print("\n=== Valores CRUDOS (RAW) top 30 ===")
print(df["CausalNoViab_RAW"].value_counts(dropna=False).head(30))

print("\n=== Valores NORMALIZADOS (TXT_LIMPIO) top 30 ===")
print(df["CausalNoViab_TXT_LIMPIO"].value_counts(dropna=False).head(30))

# 4) Catálogo / mapeo exacto (SQL Server)
map_causal_a_id = {
    "PREDIO PRIVADO CON FALSA TRADICION": 1,
    "PRESUNTO BALDIO CON FALSA TRADICION": 2,
    "NUMERO DE FMI INCONSISTENTE": 3,
    "FOLIO CERRADO": 4,
    "PERTENENCIA - ART. 48 L160/94": 5,
    "ES UN URBANO": 6,
    "PRESENTA UNA MEDIDA CAUTELAR": 7,
    "URT": 8,
    "ESTA EN PROCESO DE EXTINCION DE DOMINIO": 9,
    "PROPIETARIO TITULAR FALLECIDO": 10,
    "SOCIEDAD COMERCIAL LIQUIDADA Y CANCELADA": 11,
    "ANTECEDENTES QUE INHABILITAN PARA CONTRATAR": 12,
    "FIDEICOMISO NO COMERCIALIZABLE": 13,
    "CON PROCESO AGRARIO": 14,
    "REPORTADO EN LISTA RESTRICTIVA": 15,
    "OTROS": 16,
    "PERTENENCIA - SU 288/2022": 17,
    "BALDIO": 18,
    "DEMANDA INSCRITA": 19,
}

# 5) Tokens que deben convertirse a NULL (no texto real)
TOKENS_NULL = {
    "NO", "NO APLICA", "NO REGISTRA", "SIN REGISTRO", "N/A", "NA",
    "VACIO", "VACIOS", "VACIA", "VACIAS",
    "SIN DATO", "SIN INFORMACION", "SIN INFORMACIÓN",
}

def preparar_para_mapeo(txt_norm):
    if txt_norm is None:
        return None
    t = txt_norm.strip()
    if t == "" or t in TOKENS_NULL:
        return None

    for pref in ("ES UN ", "ES UNA ", "ES ", "PRESENTA "):
        if t.startswith(pref):
            t = t[len(pref):].strip()
            break

    t = " ".join(t.split()).strip()
    return t if t else None

df["CausalNoViab_MAP_TXT"] = df["CausalNoViab_TXT_LIMPIO"].apply(preparar_para_mapeo)

# 7) Asignación por reglas + exactos
def asignar_id(txt_map):
    if txt_map is None:
        return None

    if txt_map == "URBANO":
        return 6
    if txt_map == "UNA MEDIDA CAUTELAR":
        return 7

    if "PRESUNTO BALDIO" in txt_map and "FALSA TRADICION" in txt_map:
        return 2

    if txt_map == "FALSA TRADICION":
        return 2
    if txt_map == "PRESUNTO BALDIO":
        return 2
    if txt_map == "PREINSCRIPCION":
        return 16
    if txt_map == "INCONFORME SOCIAL":
        return 16
    if txt_map == "DEMANDA INSCRITA, MEDIDA RUPTA":
        return 16

    if txt_map in map_causal_a_id:
        return map_causal_a_id[txt_map]

    pref_variantes = [f"ES UN {txt_map}", f"ES UNA {txt_map}", f"PRESENTA {txt_map}"]
    for v in pref_variantes:
        if v in map_causal_a_id:
            return map_causal_a_id[v]

    return None

df["IdTipoNoViabilidad_SQL"] = df["CausalNoViab_MAP_TXT"].apply(asignar_id).astype("Int64")

# 9) Nombre catálogo
catalogo_tipo_no_viab = {
    1:  "PREDIO PRIVADO CON FALSA TRADICIÓN",
    2:  "PRESUNTO BALDÍO CON FALSA TRADICIÓN",
    3:  "NÚMERO DE FMI INCONSISTENTE",
    4:  "FOLIO CERRADO",
    5:  "PERTENENCIA - ART. 48 L160/94",
    6:  "ES UN URBANO",
    7:  "PRESENTA UNA MEDIDA CAUTELAR",
    8:  "URT",
    9:  "ESTÁ EN PROCESO DE EXTINCIÓN DE DOMINIO",
    10: "PROPIETARIO TITULAR FALLECIDO",
    11: "SOCIEDAD COMERCIAL LIQUIDADA Y CANCELADA",
    12: "ANTECEDENTES QUE INHABILITAN PARA CONTRATAR",
    13: "FIDEICOMISO NO COMERCIALIZABLE",
    14: "CON PROCESO AGRARIO",
    15: "REPORTADO EN LISTA RESTRICTIVA",
    16: "OTROS",
    17: "PERTENENCIA - SU 288/2022",
    18: "BALDÍO",
    19: "DEMANDA INSCRITA",
}
df["NombreTipoNoViabilidad_SQL"] = df["IdTipoNoViabilidad_SQL"].map(catalogo_tipo_no_viab)

print("\n=== Resumen IdTipoNoViabilidad_SQL (listo para migrar; <NA>=NULL) ===")
print(df["IdTipoNoViabilidad_SQL"].value_counts(dropna=False).sort_index())

# 10) Pendientes reales
mask_pend = (
    df["CausalNoViab_MAP_TXT"].notna()
    & (df["CausalNoViab_MAP_TXT"] != "")
    & df["IdTipoNoViabilidad_SQL"].isna()
)

pendientes = df.loc[
    mask_pend,
    ["Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO",
     "CausalNoViab_RAW","CausalNoViab_TXT_LIMPIO","CausalNoViab_MAP_TXT"]
].copy()

print("\nPendientes (texto pero queda NULL):", len(pendientes))
if not pendientes.empty:
    print("\n=== TOP 30 valores pendientes por CausalNoViab_MAP_TXT ===")
    print(pendientes["CausalNoViab_MAP_TXT"].value_counts().head(30))

    print("\n=== 30 ejemplos (1 por texto distinto) ===")
    ejemplos_30 = (
        pendientes.sort_values(["CausalNoViab_MAP_TXT","Fila_Excel_aprox"], na_position="last")
        .drop_duplicates(subset=["CausalNoViab_MAP_TXT"])
        .head(30)
    )
    print(ejemplos_30.to_string(index=False))

# 12) Valores distintos
valores_distintos = (
    df[["CausalNoViab_RAW","CausalNoViab_TXT_LIMPIO","CausalNoViab_MAP_TXT","IdTipoNoViabilidad_SQL"]]
    .drop_duplicates()
    .sort_values(["CausalNoViab_MAP_TXT","CausalNoViab_RAW"], na_position="last")
)

# 13) Bien clasificados
cols_base = [
    "Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO",
    "CausalNoViab_RAW","CausalNoViab_TXT_LIMPIO","CausalNoViab_MAP_TXT",
    "IdTipoNoViabilidad_SQL","NombreTipoNoViabilidad_SQL",
]
cols_base = [c for c in cols_base if c in df.columns]

mask_bien = df["IdTipoNoViabilidad_SQL"].notna()
df_bien = df.loc[mask_bien, cols_base].copy().sort_values(
    by=["IdTipoNoViabilidad_SQL","CausalNoViab_MAP_TXT"],
    ascending=[True, True],
)

# 14) Exportar Excel de revisión
ruta_salida = (
    r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT"
    r"\Documentos\Migracion base de datos viablidad juridica\CausalNoViab_revision.xlsx"
)

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    valores_distintos.to_excel(writer, sheet_name="valores_distintos", index=False)
    df_bien.to_excel(writer, sheet_name="bien_clasificados", index=False)
    pendientes.to_excel(writer, sheet_name="pendientes_NULL", index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)

print("\nColumna lista para migrar a ConceptoFinal.IdTipoNoViabilidad:", "IdTipoNoViabilidad_SQL")


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\395492068.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["CausalNoViab_RAW"] = df[col_causal]


Columnas candidatas para CAUSAL NO VIABILIDAD: ['CAUSAL \n NO VIABILIDAD']
Usando columna: CAUSAL 
 NO VIABILIDAD

=== Valores CRUDOS (RAW) top 30 ===
CausalNoViab_RAW
NaN                                            20944
PERTENENCIA - ART. 48 L160/94                    475
Propietario titular fallecido                    458
Presunto baldío con falsa tradición              402
Folio cerrado                                    234
Es un presunto baldío con falsa tradición        214
Predio privado con falsa tradición               166
Número de FMI inconsistente                      163
Presenta una medida cautelar                     155
Es un urbano                                     154
Otros                                            144
PERTENENCIA - SU 288/2022                        127
BALDIO                                           116
URT                                              107
DEMANDA INSCRITA                                  74
Con proceso agrario                  

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\395492068.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["CausalNoViab_TXT_LIMPIO"] = df["CausalNoViab_RAW"].apply(limpiar_txt)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\395492068.py:109: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["CausalNoViab_MAP_TXT"] = df["CausalNoViab_TXT_LIMPIO"].apply(preparar_para_mapeo)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\395492068.py:145: PerformanceWarning: DataFrame is highly fragmented. 


Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\CausalNoViab_revision.xlsx

Columna lista para migrar a ConceptoFinal.IdTipoNoViabilidad: IdTipoNoViabilidad_SQL


In [47]:
import pandas as pd
import numpy as np

ID_NO_VIABLE = 4
ID_OTROS = 16

req = [
    "IdTipoViabilidad_SQL",
    "IdTipoNoViabilidad_SQL",
    "CausalNoViab_RAW",
    "CausalNoViab_TXT_LIMPIO",
    "CausalNoViab_MAP_TXT",
]
faltan = [c for c in req if c not in df.columns]
if faltan:
    raise ValueError(f"Faltan columnas para aplicar reglas: {faltan}")

# Re-ejecución segura (solo columnas auditoría de este bloque)
cols_prev_reglas = ["IdTipoNoViabilidad_ANTES","IdTipoNoViabilidad_DESPUES","CausalRegla_Aplicada"]
df.drop(columns=[c for c in cols_prev_reglas if c in df.columns], inplace=True, errors="ignore")

# Asegurar dtype nullable
df["IdTipoNoViabilidad_SQL"] = df["IdTipoNoViabilidad_SQL"].astype("Int64")

es_no_viable = df["IdTipoViabilidad_SQL"].eq(ID_NO_VIABLE)

causal_texto_presente = (
    df["CausalNoViab_TXT_LIMPIO"].notna()
    & (df["CausalNoViab_TXT_LIMPIO"].astype("string").str.strip() != "")
)
causal_vacia = ~causal_texto_presente

# Auditoría antes/después
df["IdTipoNoViabilidad_ANTES"] = df["IdTipoNoViabilidad_SQL"]
df["CausalRegla_Aplicada"] = pd.NA

# Regla 1
mask_r1 = es_no_viable & causal_vacia & df["IdTipoNoViabilidad_SQL"].isna()
df.loc[mask_r1, "IdTipoNoViabilidad_SQL"] = ID_OTROS
df.loc[mask_r1, "CausalRegla_Aplicada"] = "R1_NO_VIABLE_SIN_CAUSAL_OTROS_16"

# Regla 2
mask_r2 = (~es_no_viable) & causal_texto_presente & df["IdTipoNoViabilidad_SQL"].notna()
df.loc[mask_r2, "IdTipoNoViabilidad_SQL"] = pd.NA
df.loc[mask_r2, "CausalRegla_Aplicada"] = "R2_NO_NO_VIABLE_CON_CAUSAL_LIMPIAR_NULL"

df["IdTipoNoViabilidad_SQL"] = df["IdTipoNoViabilidad_SQL"].astype("Int64")
df["IdTipoNoViabilidad_DESPUES"] = df["IdTipoNoViabilidad_SQL"]

print("\n=== APLICACION REGLAS CAUSAL vs VIABILIDAD ===")
print("Regla 1 (No viable sin causal -> OTROS 16):", int(mask_r1.sum()))
print("Regla 2 (No No-viable con causal -> NULL):", int(mask_r2.sum()))

afectados = df.loc[
    (mask_r1 | mask_r2),
    [
        "Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO",
        "Viabilidad_RAW","Viabilidad_TXT_LIMPIO",
        "IdTipoViabilidad_SQL","NombreTipoViabilidad_SQL",
        "CausalNoViab_RAW","CausalNoViab_TXT_LIMPIO","CausalNoViab_MAP_TXT",
        "IdTipoNoViabilidad_ANTES","IdTipoNoViabilidad_DESPUES",
        "CausalRegla_Aplicada",
    ]
].copy()

resumen_afectados = (
    afectados.groupby(["CausalRegla_Aplicada"], dropna=False)
    .size().reset_index(name="conteo")
)
print("\n=== Resumen afectados ===")
print(resumen_afectados.to_string(index=False))

tiene_causal_post = df["IdTipoNoViabilidad_SQL"].notna()
es_no_viable_post = df["IdTipoViabilidad_SQL"].eq(ID_NO_VIABLE)

tabla_post = pd.crosstab(
    tiene_causal_post.rename("TieneCausalNoViab_POST"),
    es_no_viable_post.rename("EsNoViable_POST"),
    dropna=False
)
print("\n=== CRUCE POST-REGLAS ===")
print(tabla_post)

ruta_reglas = (
    r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT"
    r"\Documentos\Migracion base de datos viablidad juridica\Correccion_Reglas_Viabilidad_Causal.xlsx"
)

with pd.ExcelWriter(ruta_reglas, engine="openpyxl") as writer:
    resumen_afectados.to_excel(writer, sheet_name="resumen", index=False)
    afectados.sort_values(["CausalRegla_Aplicada","FMI","Fila_Excel_aprox"]).to_excel(
        writer, sheet_name="afectados", index=False
    )
    tabla_post.reset_index().to_excel(writer, sheet_name="crosstab_post", index=False)

print("\nExcel de correcciones generado en:")
print(ruta_reglas)

# Validación final: NO VIABLE y sin causal (post)
mask_unico = df["IdTipoViabilidad_SQL"].eq(ID_NO_VIABLE) & df["IdTipoNoViabilidad_SQL"].isna()
unico = df.loc[mask_unico, [
    "Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO",
    "Viabilidad_RAW","Viabilidad_TXT_LIMPIO","IdTipoViabilidad_SQL",
    "CausalNoViab_RAW","CausalNoViab_TXT_LIMPIO","CausalNoViab_MAP_TXT","IdTipoNoViabilidad_SQL",
]].copy()

print("Casos NO VIABLE sin causal (post):", len(unico))
if not unico.empty:
    print(unico.to_string(index=False))



=== APLICACION REGLAS CAUSAL vs VIABILIDAD ===
Regla 1 (No viable sin causal -> OTROS 16): 2374
Regla 2 (No No-viable con causal -> NULL): 60

=== Resumen afectados ===
                   CausalRegla_Aplicada  conteo
       R1_NO_VIABLE_SIN_CAUSAL_OTROS_16    2374
R2_NO_NO_VIABLE_CON_CAUSAL_LIMPIAR_NULL      60

=== CRUCE POST-REGLAS ===
EsNoViable_POST         False  True 
TieneCausalNoViab_POST              
False                   18644      0
True                        0   5411


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1195259680.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["IdTipoNoViabilidad_ANTES"] = df["IdTipoNoViabilidad_SQL"]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1195259680.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["CausalRegla_Aplicada"] = pd.NA
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1195259680.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim


Excel de correcciones generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Correccion_Reglas_Viabilidad_Causal.xlsx
Casos NO VIABLE sin causal (post): 0


## INSUMOS PENDIENTES 

In [48]:
import pandas as pd
import numpy as np
import unicodedata
import re

# =====================================================
# INSUMOS PENDIENTES  → ConceptoFinal.InsumosPendientes (varchar(MAX) NULL)
# Metodología:
# - InsumosPendientes_RAW: copia exacta del Excel (NO se modifica)
# - InsumosPendientes_TXT_LIMPIO: solo para clasificar (mayúsculas, sin tildes, espacios normales)
# - InsumosPendientes_SQL: listo para migrar (texto real) o NULL (placeholders)
# - Reportes + Excel de revisión
#
# AJUSTE SOLICITADO:
# - NO recarga df (NO df = df.copy(), NO df = pd.concat)
# - Re-ejecución segura: drop IN-PLACE de columnas derivadas InsumosPendientes_*
# - Resto queda igual
# =====================================================

# 0) Asegurar columna de fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Re-ejecución segura SIN recargar df: borrar columnas derivadas si existen (IN-PLACE)
cols_prev = [c for c in df.columns if isinstance(c, str) and c.startswith("InsumosPendientes_")]
if cols_prev:
    df.drop(columns=cols_prev, inplace=True, errors="ignore")

# 1) Detectar columna ORIGINAL de Excel
candidatas_insumos = [
    c for c in df.columns
    if ("INSUMOS" in str(c).upper() and "PENDIENT" in str(c).upper())
]

print("Columnas candidatas para INSUMOS PENDIENTES (antes de filtrar):", candidatas_insumos)

# Excluir derivadas
excluir = {
    "INSUMOSPENDIENTES_RAW",
    "INSUMOSPENDIENTES_TXT_LIMPIO",
    "INSUMOSPENDIENTES_SQL",
    "INSUMOSPENDIENTES_LEN",
}
candidatas_insumos = [
    c for c in candidatas_insumos
    if str(c).strip().upper() not in excluir and not str(c).strip().upper().startswith("INSUMOSPENDIENTES_")
]

# Priorizar nombre exacto
prioritarias = [c for c in candidatas_insumos if str(c).strip().upper() == "INSUMOS PENDIENTES"]

if prioritarias:
    col_insumos = prioritarias[0]
else:
    if not candidatas_insumos:
        raise ValueError("No se encontró la columna ORIGINAL de INSUMOS PENDIENTES.")
    col_insumos = candidatas_insumos[0]

print("Usando columna de Excel:", col_insumos)

# 2) RAW
df["InsumosPendientes_RAW"] = df[col_insumos]

# 3) TXT_LIMPIO (solo para clasificar)
def _limpiar_txt(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    if t == "":
        return None
    t_norm = unicodedata.normalize("NFD", t)
    t_norm = "".join(ch for ch in t_norm if unicodedata.category(ch) != "Mn")
    t_norm = t_norm.replace("\u00A0", " ")          # NBSP
    t_norm = " ".join(t_norm.split())               # colapsar espacios
    return t_norm.upper()

df["InsumosPendientes_TXT_LIMPIO"] = df["InsumosPendientes_RAW"].apply(_limpiar_txt)

print("\n=== Valores NORMALIZADOS (TXT_LIMPIO) top 40 ===")
print(df["InsumosPendientes_TXT_LIMPIO"].value_counts(dropna=False).head(40))

# 4) Placeholders -> NULL (ampliado y robusto)
PLACEHOLDERS = {
    "NO", "NO.", "NO APLICA", "NO APLICA.", "NO REGISTRA", "SIN REGISTRO",
    "N/A", "NA", "N.A.", "NINGUNO", "NINGUNA", "SIN INFORMACION", "SIN INFORMACIÓN",
    "SIN DATO", "SIN DATOS", "NO HAY", "NO TIENE", "NO TIENE INSUMOS",
    "NO PRESENTA", "NO PRESENTA INSUMOS", "NO APORTA", "NO APORTA INSUMOS",
    "0",
}

def normalizar_insumos_sql(v_raw, v_txt):
    """
    - Si es placeholder -> NULL
    - Si es texto real -> se conserva el RAW recortado (sin modificar contenido real)
    """
    if pd.isna(v_raw):
        return None
    t = str(v_raw).strip()
    if t == "":
        return None

    t_up = v_txt if (v_txt is not None) else _limpiar_txt(t)
    if t_up is None:
        return None

    # exactos
    if t_up in PLACEHOLDERS:
        return None

    # patrones cortos típicos ("NO APLICA - ..." no debería volverse null)
    # si empieza con NO/NO APLICA pero trae más texto, se conserva como texto real
    # Ej: "NO APLICA POR FALTA DE..." -> se mantiene
    if t_up.startswith("NO APLICA") and t_up not in {"NO APLICA", "NO APLICA."}:
        return t
    if t_up.startswith("NO REGISTRA") and t_up != "NO REGISTRA":
        return t
    if t_up.startswith("SIN REGISTRO") and t_up != "SIN REGISTRO":
        return t

    return t  # texto real, se conserva

# (sin df = df.copy / sin df = pd.concat)
df["InsumosPendientes_SQL"] = df.apply(
    lambda r: normalizar_insumos_sql(r["InsumosPendientes_RAW"], r["InsumosPendientes_TXT_LIMPIO"]),
    axis=1
)

# 5) Longitudes (para revisar outliers)
df["InsumosPendientes_LEN"] = df["InsumosPendientes_SQL"].fillna("").astype(str).str.len()

print("\n=== Resumen INSUMOS PENDIENTES (SQL nulo vs no nulo) ===")
print(df["InsumosPendientes_SQL"].isna().value_counts(dropna=False))

print("\n=== Resumen estadístico longitudes (LEN) ===")
print(df["InsumosPendientes_LEN"].describe())

# 6) DataFrames de revisión
contexto = ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"]
contexto = [c for c in contexto if c in df.columns]

cols_detalle = contexto + [
    "InsumosPendientes_RAW",
    "InsumosPendientes_TXT_LIMPIO",
    "InsumosPendientes_SQL",
    "InsumosPendientes_LEN",
]

# a) Valores distintos (para auditoría)
valores_distintos = (
    df[["InsumosPendientes_RAW", "InsumosPendientes_TXT_LIMPIO", "InsumosPendientes_SQL"]]
    .drop_duplicates()
)

valores_distintos = valores_distintos.sort_values(
    by=["InsumosPendientes_TXT_LIMPIO", "InsumosPendientes_RAW"],
    key=lambda col: col.astype(str),
    na_position="last"
)

# b) Registros con texto real
df_con_texto = df[df["InsumosPendientes_SQL"].notna()][cols_detalle].copy()

# c) Placeholders (RAW no nulo pero SQL nulo)
mask_placeholders = df["InsumosPendientes_RAW"].notna() & df["InsumosPendientes_SQL"].isna()
df_placeholders = df[mask_placeholders][cols_detalle].copy()

# d) Outliers por longitud (top 200 más largos)
df_largos = (
    df[df["InsumosPendientes_SQL"].notna()][cols_detalle]
    .sort_values("InsumosPendientes_LEN", ascending=False)
    .head(200)
    .copy()
)

print("\nFilas con texto real (SQL no nulo):", len(df_con_texto))
print("Filas con placeholders (RAW no nulo pero SQL nulo):", len(df_placeholders))
print("Top largos (muestra):", len(df_largos))

# 7) Exportar Excel de revisión
ruta_salida = (
    r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT"
    r"\Documentos\Migracion base de datos viablidad juridica\InsumosPendientes_revision.xlsx"
)

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    valores_distintos.to_excel(writer, sheet_name="valores_distintos", index=False)
    df_con_texto.to_excel(writer, sheet_name="con_texto_SQL", index=False)
    df_placeholders.to_excel(writer, sheet_name="placeholders_a_NULL", index=False)
    df_largos.to_excel(writer, sheet_name="top_largos", index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)

print("\nColumna lista para migrar a ConceptoFinal.InsumosPendientes:", "InsumosPendientes_SQL")


Columnas candidatas para INSUMOS PENDIENTES (antes de filtrar): ['INSUMOS PENDIENTES ']
Usando columna de Excel: INSUMOS PENDIENTES 

=== Valores NORMALIZADOS (TXT_LIMPIO) top 40 ===
InsumosPendientes_TXT_LIMPIO
None                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1651064343.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["InsumosPendientes_RAW"] = df[col_insumos]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1651064343.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["InsumosPendientes_TXT_LIMPIO"] = df["InsumosPendientes_RAW"].apply(_limpiar_txt)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1651064343.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result


=== Resumen INSUMOS PENDIENTES (SQL nulo vs no nulo) ===
InsumosPendientes_SQL
True     19559
False     4496
Name: count, dtype: int64

=== Resumen estadístico longitudes (LEN) ===
count    24055.000000
mean        29.703637
std        203.930420
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      16831.000000
Name: InsumosPendientes_LEN, dtype: float64

Filas con texto real (SQL no nulo): 4496
Filas con placeholders (RAW no nulo pero SQL nulo): 81
Top largos (muestra): 200

Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\InsumosPendientes_revision.xlsx

Columna lista para migrar a ConceptoFinal.InsumosPendientes: InsumosPendientes_SQL


## FECHA DE ENTREGA A REVISOR

In [49]:
import pandas as pd
import numpy as np
import datetime as dt
import re

# ==========================================================
# FECHA DE ENTREGA A REVISOR  → ConceptoFinal.FechaEntregaARevisor (datetime NULL)
# Metodología:
# - FechaEntregaRevisor_RAW: copia exacta del Excel (NO se modifica)
# - FECHA_ENTREGA_REV_TXT: texto para inspección (sin perder RAW)
# - FECHA_ENTREGA_REV_TXT_LIMPIO: placeholders fuera + recorte de hora
# - FECHA_ENTREGA_REV_TXT_LIMPIO_FIX: corrección puntual (año 3 dígitos)
# - FechaEntregaARevisor_SQL: listo para migrar
# - Excel de revisión + invalidas + auditoría de correcciones
#
# AJUSTE SOLICITADO:
# - NO recarga df (NO df = df.copy(), NO df = pd.concat)
# - Re-ejecución segura: drop IN-PLACE de columnas derivadas de este bloque
# - Resto queda igual
# ==========================================================

# 0) Asegurar columna de fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Re-ejecución segura SIN recargar df: borrar columnas derivadas si existen (IN-PLACE)
cols_prev = [
    "FechaEntregaRevisor_RAW",
    "FECHA_ENTREGA_REV_TXT",
    "FECHA_ENTREGA_REV_TXT_LIMPIO",
    "FECHA_ENTREGA_REV_TXT_LIMPIO_FIX",
    "FechaEntregaARevisor_SQL",
    "MotivoFechaEntregaRevisorNula",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# 1) Detectar columna de Excel para FECHA DE ENTREGA A REVISOR
candidatas = [
    c for c in df.columns
    if ("ENTREGA" in str(c).upper() and "REVISOR" in str(c).upper())
]

print("Columnas candidatas para FECHA ENTREGA A REVISOR:", candidatas)

if not candidatas:
    raise ValueError("No se encontró la columna de FECHA DE ENTREGA A REVISOR en el DataFrame.")

# Priorizar nombre exacto
preferida = [c for c in candidatas if str(c).strip().upper() == "FECHA DE ENTREGA A REVISOR"]
col_fecha_entrega = preferida[0] if preferida else candidatas[0]

print("Usando columna:", col_fecha_entrega)

# 2) RAW (exacto)
df["FechaEntregaRevisor_RAW"] = df[col_fecha_entrega]

# 3) Convertir a texto (para inspección)
def a_texto_fecha(v):
    if pd.isna(v):
        return ""
    if isinstance(v, (pd.Timestamp, dt.datetime, dt.date)):
        return v.strftime("%d/%m/%Y")
    return str(v).strip()

df["FECHA_ENTREGA_REV_TXT"] = df["FechaEntregaRevisor_RAW"].apply(a_texto_fecha)

# 4) Limpiar texto: placeholders + recorte de hora
PLACEHOLDERS_NO_FECHA = {
    "NO", "NO.", "NO APLICA", "NO APLICA.", "NO REGISTRA", "NO REGISTRA.",
    "SIN FECHA", "SIN INFORME", "N/A", "NA", "NINGUNO", "NINGUNA", ""
}

def limpiar_texto_fecha_entrega(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip()
    if not s:
        return ""
    su = s.upper().strip()
    if su in PLACEHOLDERS_NO_FECHA:
        return ""
    # si viene "2024-04-25 00:00:00" o similar
    if " " in s:
        s = s.split()[0].strip()
    return s

df["FECHA_ENTREGA_REV_TXT_LIMPIO"] = df["FECHA_ENTREGA_REV_TXT"].apply(limpiar_texto_fecha_entrega)

# 5) Corrección puntual: año de 3 dígitos (ej: 10/12/205 -> 10/12/2025)
def corregir_anio_3_digitos(s: str) -> str:
    if not s:
        return s
    s = str(s).strip()

    m = re.fullmatch(r"(\d{1,2})/(\d{1,2})/(\d{3})", s)
    if not m:
        return s

    dd, mm, yyy = m.group(1), m.group(2), m.group(3)

    # 205 -> 2025 (interpretar 3 dígitos como "20" + (centena) + (último dígito)
    # Es decir: 2 0 5  => 2 0 2 5
    if yyy.startswith("20"):
        yyyy = "202" + yyy[-1]   # 205 -> 2025
        return f"{dd.zfill(2)}/{mm.zfill(2)}/{yyyy}"

    return s

df["FECHA_ENTREGA_REV_TXT_LIMPIO_FIX"] = df["FECHA_ENTREGA_REV_TXT_LIMPIO"].apply(corregir_anio_3_digitos)

# 6) Parseo a datetime seguro (día primero)
df["FechaEntregaARevisor_SQL"] = pd.to_datetime(
    df["FECHA_ENTREGA_REV_TXT_LIMPIO_FIX"],
    dayfirst=True,
    errors="coerce"
)

# 7) Motivo de nulos (para auditoría)
def motivo_fecha_nula(raw, txt_limpio_fix, fecha_sql):
    if pd.notna(fecha_sql):
        return None

    raw_na = (raw is None) or (isinstance(raw, float) and pd.isna(raw)) or pd.isna(raw)

    if raw_na and (txt_limpio_fix == "" or pd.isna(txt_limpio_fix)):
        return "NaN"

    if (txt_limpio_fix == "" or pd.isna(txt_limpio_fix)) and (not raw_na):
        return "Placeholder (NO/NO APLICA/NO REGISTRA/...)"

    if txt_limpio_fix not in ("", None) and not pd.isna(txt_limpio_fix):
        return "Otro texto raro (posible fecha mal escrita)"

    return "NaN"

df["MotivoFechaEntregaRevisorNula"] = df.apply(
    lambda r: motivo_fecha_nula(
        r["FechaEntregaRevisor_RAW"],
        r["FECHA_ENTREGA_REV_TXT_LIMPIO_FIX"],
        r["FechaEntregaARevisor_SQL"]
    ),
    axis=1
)

# 8) Auditoría REAL de correcciones (solo cambios reales)
mask_anio_3 = df["FECHA_ENTREGA_REV_TXT_LIMPIO"].astype(str).str.fullmatch(r"\d{1,2}/\d{1,2}/\d{3}", na=False)

corregidos = df.loc[
    mask_anio_3
    & (df["FECHA_ENTREGA_REV_TXT_LIMPIO_FIX"] != df["FECHA_ENTREGA_REV_TXT_LIMPIO"]),
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
     "FECHA_ENTREGA_REV_TXT_LIMPIO", "FECHA_ENTREGA_REV_TXT_LIMPIO_FIX"]
].copy()

print("\n=== Auditoría corrección año 3 dígitos ===")
print("Total con año 3 dígitos:", int(mask_anio_3.sum()))
print("Total corregidos (cambio real):", len(corregidos))
if not corregidos.empty:
    print("\nEjemplos (hasta 30):")
    print(corregidos.head(30).to_string(index=False))

# 9) Resúmenes
total = len(df)
validas = int(df["FechaEntregaARevisor_SQL"].notna().sum())
nulas = int(df["FechaEntregaARevisor_SQL"].isna().sum())

print(f"\nTotal filas: {total}")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas nulas/NaT       : {nulas}")

print("\n=== Resumen de motivos para FechaEntregaARevisor_SQL = NaT ===")
print(
    df.loc[df["FechaEntregaARevisor_SQL"].isna(), "MotivoFechaEntregaRevisorNula"]
      .value_counts(dropna=False)
)

# 10) Invalidas: texto presente pero no parsea
invalidas = df[
    df["FechaEntregaARevisor_SQL"].isna()
    & df["FECHA_ENTREGA_REV_TXT_LIMPIO_FIX"].notna()
    & (df["FECHA_ENTREGA_REV_TXT_LIMPIO_FIX"] != "")
].copy()

print("\n=== Ejemplos de FECHA ENTREGA A REVISOR con texto que no se pudo parsear ===")
if invalidas.empty:
    print("No hay textos inválidos pendientes.")
else:
    print(
        invalidas[[
            "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
            "FechaEntregaRevisor_RAW",
            "FECHA_ENTREGA_REV_TXT_LIMPIO",
            "FECHA_ENTREGA_REV_TXT_LIMPIO_FIX",
        ]].head(40).to_string(index=False)
    )

# 11) Exportar Excel de revisión (valores_distintos + invalidas + corregidos + muestra)
contexto = ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"]
contexto = [c for c in contexto if c in df.columns]

cols_det = contexto + [
    "FechaEntregaRevisor_RAW",
    "FECHA_ENTREGA_REV_TXT",
    "FECHA_ENTREGA_REV_TXT_LIMPIO",
    "FECHA_ENTREGA_REV_TXT_LIMPIO_FIX",
    "FechaEntregaARevisor_SQL",
    "MotivoFechaEntregaRevisorNula",
]

valores_distintos = (
    df[[
        "FechaEntregaRevisor_RAW",
        "FECHA_ENTREGA_REV_TXT_LIMPIO",
        "FECHA_ENTREGA_REV_TXT_LIMPIO_FIX",
        "FechaEntregaARevisor_SQL",
    ]]
    .drop_duplicates()
    .sort_values(
        by=["FECHA_ENTREGA_REV_TXT_LIMPIO_FIX"],
        key=lambda col: col.astype(str),
        na_position="last"
    )
)

muestra = df.loc[
    df["FechaEntregaRevisor_RAW"].notna() | df["FechaEntregaARevisor_SQL"].notna(),
    cols_det
].head(500).copy()

ruta_salida = (
    r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT"
    r"\Documentos\Migracion base de datos viablidad juridica\FechaEntregaARevisor_revision.xlsx"
)

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    valores_distintos.to_excel(writer, sheet_name="valores_distintos", index=False)
    muestra.to_excel(writer, sheet_name="muestra", index=False)
    invalidas[cols_det].to_excel(writer, sheet_name="invalidas_parseo", index=False)
    corregidos.to_excel(writer, sheet_name="corregidos_anio_3dig", index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)

print("\nColumna lista para migrar a ConceptoFinal.FechaEntregaARevisor:", "FechaEntregaARevisor_SQL")


Columnas candidatas para FECHA ENTREGA A REVISOR: ['FECHA DE ENTREGA A REVISOR', 'PLAZO PARA ENTREGA A REVISOR']
Usando columna: FECHA DE ENTREGA A REVISOR


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2317135340.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["FechaEntregaRevisor_RAW"] = df[col_fecha_entrega]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2317135340.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["FECHA_ENTREGA_REV_TXT"] = df["FechaEntregaRevisor_RAW"].apply(a_texto_fecha)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2317135340.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res


=== Auditoría corrección año 3 dígitos ===
Total con año 3 dígitos: 1
Total corregidos (cambio real): 1

Ejemplos (hasta 30):
 Fila_Excel_aprox      FMI DEPARTAMENTO MUNICIPIO FECHA_ENTREGA_REV_TXT_LIMPIO FECHA_ENTREGA_REV_TXT_LIMPIO_FIX
            24045 346-2802        SUCRE   CAIMITO                    10/12/205                       10/12/2025

Total filas: 24055
Fechas válidas (no NaT): 6224
Fechas nulas/NaT       : 17831

=== Resumen de motivos para FechaEntregaARevisor_SQL = NaT ===
MotivoFechaEntregaRevisorNula
NaN                                           17829
Placeholder (NO/NO APLICA/NO REGISTRA/...)        2
Name: count, dtype: int64

=== Ejemplos de FECHA ENTREGA A REVISOR con texto que no se pudo parsear ===
No hay textos inválidos pendientes.

Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\FechaEntregaARevisor_revision.xlsx

Columna lista para migrar a ConceptoF

*avance del excel*

In [50]:
import pandas as pd

# ======================================================================================
# AVANCE A EXCEL (SIN RECARGAR df) - CONSOLIDADO HASTA HOY
# - NO hace: df = df.copy()
# - NO hace: df = pd.concat([...]) reasignando df
# - NO aplica reglas nuevas ni reprocesa nada (solo USA resultados ya calculados)
# - Exporta:
#     1) RegistroProceso_avance
#     2) EstudioTerreno_avance
#     3) MedidaProcesal_listo_insertar (formato largo)
#     4) ConceptosPrevio_listo_insertar (formato largo)  -> usa IdRegistroProceso si existe, si no usa llaves fallback
#     5) ConceptoFinal_listo_insertar (formato largo)    -> usa IdRegistroProceso si existe, si no usa llaves fallback
#     6) Unificado_df (df completo con columnas trabajadas)
#     7) Diccionario (hoja->columnas)
# ======================================================================================

# 0) Asegurar fila Excel (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# --------------------------------------------------------------------------------------
# 1) Columnas contexto (solo si existen)
# --------------------------------------------------------------------------------------
cols_ctx = [
    "Fila_Excel_aprox",
    "IdPostulacion_SQL",
    "ID_SQL",
    "FMI", "FMI_SQL",
    "DEPARTAMENTO", "MUNICIPIO",
]
cols_ctx = [c for c in cols_ctx if c in df.columns]

# Llaves fallback (para tablas que dependen de RegistroProceso cuando aún no hay IdRegistroProceso_SQL)
keys_fallback = [c for c in ["IdPostulacion_SQL", "FMI_SQL"] if c in df.columns]
idrp_col = "IdRegistroProceso_SQL" if "IdRegistroProceso_SQL" in df.columns else None

# --------------------------------------------------------------------------------------
# 2) RegistroProceso_avance (solo columnas ya calculadas que existan)
# --------------------------------------------------------------------------------------
cols_registro = [
    "IdPostulacion_SQL",
    "FMI_SQL",
    "NumeroExpediente_SQL",
    "IdFuenteProceso",
    "IdFuenteProceso_SQL",
    "IdTipoProceso",
    "IdTipoProceso_SQL",
    "IdEtapaProcesal",
    "IdEtapaProcesal_SQL",
    "Dependencia_SQL",
    "RadicadoOrfeo_SQL",
]
cols_registro_exist = [c for c in cols_registro if c in df.columns]
df_registro = df[cols_ctx + cols_registro_exist].copy()

# --------------------------------------------------------------------------------------
# 3) EstudioTerreno_avance (solo columnas ya calculadas que existan)
# --------------------------------------------------------------------------------------
cols_estudio = [
    "IdRegistroProceso_SQL",
    "IdLocalizacion_SQL",
    "AreaRegistral_SQL",
    "CirculoRegistral_SQL",
    "AreaCalculada_SQL",
    "TipoPersonaTitular_SQL",
    "NombrePropietario_SQL",
    "ApellidoPropietario_SQL",
    "Identificacion_SQL_TXT_MIGRAR",
    "IdTipoDocumento_SQL_MIGRAR",
    "NaturalezaJuridica_SQL_FINAL",
    "AcreditacionPropiedad_SQL_FINAL",
]
cols_estudio_exist = [c for c in cols_estudio if c in df.columns]
df_estudio = df[cols_ctx + cols_estudio_exist].copy()

# --------------------------------------------------------------------------------------
# 4) MedidaProcesal_listo_insertar (formato largo) - SOLO LEE resultados ya calculados
# --------------------------------------------------------------------------------------
base_cols_mp = ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"]
base_cols_mp = [c for c in base_cols_mp if c in df.columns]

mp_parts = []

def add_mp(objeto: str, col_valor_final: str, col_anot: str, col_tipo: str = None,
           col_estado: str = None, estados_ok=None):
    """
    Agrega un bloque de MedidaProcesal en formato largo:
      base_cols + [Valor, Anotacion, TipoClasificacion, Objeto]
    Filtra:
      - Si existe col_estado y estados_ok, usa eso.
      - Si no, usa Valor notna().
    NO modifica df.
    """
    if col_valor_final not in df.columns:
        return
    if col_anot not in df.columns:
        return

    valor = df[col_valor_final]
    anot = df[col_anot]

    if (col_estado is not None) and (col_estado in df.columns) and estados_ok:
        mask = df[col_estado].isin(estados_ok)
    else:
        mask = valor.notna()

    if not mask.any():
        return

    out = df.loc[mask, base_cols_mp].copy()
    out["Valor"] = valor.loc[mask].values
    out["Anotacion"] = anot.loc[mask].values

    if col_tipo and (col_tipo in df.columns):
        out["TipoClasificacion"] = df.loc[mask, col_tipo].values
    else:
        out["TipoClasificacion"] = None

    out["Objeto"] = objeto
    mp_parts.append(out)

# Mismos objetos que venías usando
add_mp("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL", None, "HIPOTECA_ESTADO", ["ok", "sin_reg"])
add_mp("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL", None, "SERVIDUMBRES_ESTADO", ["ok", "sin_reg"])
add_mp("MEDIDAS CAUTELARES", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL", None, "MEDIDAS_CAUT_ESTADO", ["ok", "sin_reg"])
add_mp("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL", "RUPTA_ESTADO", ["ok", "sin_reg"])
add_mp("RTDAF LEY 1448", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL", None, "RTDAF_ESTADO", ["ok", "sin_reg"])
add_mp("OFERTA OTRAS ENTIDADES", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL", None, "OFERTA_ESTADO", ["ok", "sin_reg"])
add_mp("PROCESOS DE CLARIFICACIÓN", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL", None, "PROC_CLARIF_ESTADO", ["ok", "sin_reg"])

if mp_parts:
    df_medidaprocesal = pd.concat(mp_parts, ignore_index=True)
else:
    df_medidaprocesal = pd.DataFrame(columns=base_cols_mp + ["Valor", "Anotacion", "TipoClasificacion", "Objeto"])

sort_cols = [c for c in ["DEPARTAMENTO", "MUNICIPIO", "FMI", "Objeto"] if c in df_medidaprocesal.columns]
if sort_cols and not df_medidaprocesal.empty:
    df_medidaprocesal = df_medidaprocesal.sort_values(sort_cols).reset_index(drop=True)

# --------------------------------------------------------------------------------------
# 5) ConceptosPrevio_listo_insertar (NO reglas, SOLO resultados ya calculados)
# --------------------------------------------------------------------------------------
cols_cp_src = ["FechaInformePrevio_SQL", "ConceptoPrevio_SQL"]
cols_cp_src_exist = [c for c in cols_cp_src if c in df.columns]

if cols_cp_src_exist:
    mask_cp = pd.Series(False, index=df.index)
    for c in cols_cp_src_exist:
        mask_cp = mask_cp | df[c].notna()

    cols_cp = cols_ctx + (["IdRegistroProceso_SQL"] if idrp_col else []) + keys_fallback + cols_cp_src_exist
    cols_cp = list(dict.fromkeys([c for c in cols_cp if c in df.columns]))  # únicos + existentes

    df_conceptosprevio = df.loc[mask_cp, cols_cp].copy()

    ren_cp = {}
    if idrp_col and ("IdRegistroProceso_SQL" in df_conceptosprevio.columns):
        ren_cp["IdRegistroProceso_SQL"] = "IdRegistroProceso"
    if "FechaInformePrevio_SQL" in df_conceptosprevio.columns:
        ren_cp["FechaInformePrevio_SQL"] = "FechaInforme"
    if "ConceptoPrevio_SQL" in df_conceptosprevio.columns:
        ren_cp["ConceptoPrevio_SQL"] = "Concepto"

    df_conceptosprevio.rename(columns=ren_cp, inplace=True)
else:
    df_conceptosprevio = pd.DataFrame()

# --------------------------------------------------------------------------------------
# 6) ConceptoFinal_listo_insertar (NO reglas, SOLO resultados ya calculados)
#    >>> AQUÍ SE AGREGAN LOS ÚLTIMOS CAMPOS <<<
# --------------------------------------------------------------------------------------
cols_cf_map = [
    ("ConceptoActualDeViabilidadJuridica_SQL", "ConceptoActualDeViabilidadJuridica"),
    ("FechaInforme_SQL", "FechaInforme"),
    ("IdTipoViabilidad_SQL", "IdTipoViabilidad"),
    ("IdTipoInforme_SQL", "IdTipoInforme"),
    ("IdTipoNoViabilidad_SQL", "IdTipoNoViabilidad"),
    ("InsumosPendientes_SQL", "InsumosPendientes"),
    ("FechaEntregaARevisor_SQL", "FechaEntregaARevisor"),

    # ===== NUEVOS CAMPOS (ÚLTIMO AVANCE) =====
    ("AbogadoSustanciadorAsignado_SQL", "AbogadoSustanciadorAsignado"),
    ("AbogadoRevisorAsignado_SQL", "AbogadoRevisorAsignado"),
    ("NroReparto_SQL", "NroReparto"),
    ("FechaAsignacionReparto_SQL", "FechaAsignacionReparto"),
    ("FechaPlazoEntregaARevisor_SQL", "FechaPlazoEntregaARevisor"),

    # (Opcional) estados / auditoría si quieres trazabilidad (no afecta migración)
    ("Estado_NroReparto", "Estado_NroReparto"),
    ("Estado_FechaAsignacionReparto", "Estado_FechaAsignacionReparto"),
    ("Estado_FechaPlazoEntregaARevisor", "Estado_FechaPlazoEntregaARevisor"),
]
cols_cf_src_exist = [src for (src, _) in cols_cf_map if src in df.columns]

if cols_cf_src_exist:
    mask_cf = pd.Series(False, index=df.index)
    for c in cols_cf_src_exist:
        mask_cf = mask_cf | df[c].notna()

    cols_cf = cols_ctx + (["IdRegistroProceso_SQL"] if idrp_col else []) + keys_fallback + cols_cf_src_exist
    cols_cf = list(dict.fromkeys([c for c in cols_cf if c in df.columns]))

    df_conceptofinal = df.loc[mask_cf, cols_cf].copy()

    ren_cf = {}
    if idrp_col and ("IdRegistroProceso_SQL" in df_conceptofinal.columns):
        ren_cf["IdRegistroProceso_SQL"] = "IdRegistroProceso"
    for src, dst in cols_cf_map:
        if src in df_conceptofinal.columns:
            ren_cf[src] = dst

    df_conceptofinal.rename(columns=ren_cf, inplace=True)
else:
    df_conceptofinal = pd.DataFrame()

# --------------------------------------------------------------------------------------
# 7) Unificado_df (df completo: todas las columnas trabajadas)
# --------------------------------------------------------------------------------------
df_unificado = df.copy()

# --------------------------------------------------------------------------------------
# 8) Diccionario (hoja -> columnas)
# --------------------------------------------------------------------------------------
dicc = []
for c in cols_registro_exist:
    dicc.append(["RegistroProceso_avance", c])
for c in cols_estudio_exist:
    dicc.append(["EstudioTerreno_avance", c])

dicc.append(["MedidaProcesal_listo_insertar", "Fila_Excel_aprox"])
dicc.append(["MedidaProcesal_listo_insertar", "FMI"])
dicc.append(["MedidaProcesal_listo_insertar", "DEPARTAMENTO"])
dicc.append(["MedidaProcesal_listo_insertar", "MUNICIPIO"])
dicc.append(["MedidaProcesal_listo_insertar", "Valor"])
dicc.append(["MedidaProcesal_listo_insertar", "Anotacion"])
dicc.append(["MedidaProcesal_listo_insertar", "TipoClasificacion"])
dicc.append(["MedidaProcesal_listo_insertar", "Objeto"])

for c in df_conceptosprevio.columns:
    dicc.append(["ConceptosPrevio_listo_insertar", c])

for c in df_conceptofinal.columns:
    dicc.append(["ConceptoFinal_listo_insertar", c])

dicc.append(["Unificado_df", f"{len(df_unificado.columns)} columnas"])

df_dicc = pd.DataFrame(dicc, columns=["Hoja", "Columna"])

# --------------------------------------------------------------------------------------
# 9) Exportar Excel consolidado
# --------------------------------------------------------------------------------------
out_xlsx = "AVANCE_MIGRACION_HASTA_HOY_ACTUALIZADO.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    df_registro.to_excel(writer, index=False, sheet_name="RegistroProceso_avance")
    df_estudio.to_excel(writer, index=False, sheet_name="EstudioTerreno_avance")
    df_medidaprocesal.to_excel(writer, index=False, sheet_name="MedidaProcesal_listo_insertar")
    df_conceptosprevio.to_excel(writer, index=False, sheet_name="ConceptosPrevio_listo_insertar")
    df_conceptofinal.to_excel(writer, index=False, sheet_name="ConceptoFinal_listo_insertar")
    df_unificado.to_excel(writer, index=False, sheet_name="Unificado_df")
    df_dicc.to_excel(writer, index=False, sheet_name="Diccionario")

# --------------------------------------------------------------------------------------
# 10) Logs
# --------------------------------------------------------------------------------------
print("Excel creado:", out_xlsx)
print("Hojas: RegistroProceso_avance, EstudioTerreno_avance, MedidaProcesal_listo_insertar, "
      "ConceptosPrevio_listo_insertar, ConceptoFinal_listo_insertar, Unificado_df, Diccionario")
print("Filas RegistroProceso_avance:", len(df_registro))
print("Filas EstudioTerreno_avance:", len(df_estudio))
print("Filas MedidaProcesal_listo_insertar:", len(df_medidaprocesal))
print("Filas ConceptosPrevio_listo_insertar:", len(df_conceptosprevio))
print("Filas ConceptoFinal_listo_insertar:", len(df_conceptofinal))
print("Filas Unificado_df:", len(df_unificado))
print("Columnas Unificado_df:", len(df_unificado.columns))

# Listado explícito de "últimos campos" para confirmar si existen en df
ultimos = [
    "AbogadoSustanciadorAsignado_SQL",
    "AbogadoRevisorAsignado_SQL",
    "NroReparto_SQL",
    "FechaAsignacionReparto_SQL",
    "FechaPlazoEntregaARevisor_SQL",
    "Estado_NroReparto",
    "Estado_FechaAsignacionReparto",
    "Estado_FechaPlazoEntregaARevisor",
]
faltantes = [c for c in ultimos if c not in df.columns]
if faltantes:
    print("\n[OJO] Estos campos NO existen en df (por eso no salen en ConceptoFinal):")
    for c in faltantes:
        print(" -", c)

if idrp_col is None:
    print("\n[OJO] No existe 'IdRegistroProceso_SQL' en df. ConceptosPrevio/ConceptoFinal se exportan con llaves fallback "
          "(IdPostulacion_SQL / FMI_SQL) para cruce en migración.")


Excel creado: AVANCE_MIGRACION_HASTA_HOY_ACTUALIZADO.xlsx
Hojas: RegistroProceso_avance, EstudioTerreno_avance, MedidaProcesal_listo_insertar, ConceptosPrevio_listo_insertar, ConceptoFinal_listo_insertar, Unificado_df, Diccionario
Filas RegistroProceso_avance: 24055
Filas EstudioTerreno_avance: 24055
Filas MedidaProcesal_listo_insertar: 69662
Filas ConceptosPrevio_listo_insertar: 5869
Filas ConceptoFinal_listo_insertar: 24055
Filas Unificado_df: 24055
Columnas Unificado_df: 269

[OJO] Estos campos NO existen en df (por eso no salen en ConceptoFinal):
 - AbogadoSustanciadorAsignado_SQL
 - AbogadoRevisorAsignado_SQL
 - NroReparto_SQL
 - FechaAsignacionReparto_SQL
 - FechaPlazoEntregaARevisor_SQL
 - Estado_NroReparto
 - Estado_FechaAsignacionReparto
 - Estado_FechaPlazoEntregaARevisor

[OJO] No existe 'IdRegistroProceso_SQL' en df. ConceptosPrevio/ConceptoFinal se exportan con llaves fallback (IdPostulacion_SQL / FMI_SQL) para cruce en migración.


## ABOGADO  SUSTANCIADOR ASIGNADO


In [51]:
import pandas as pd
import numpy as np

# ============================================================
# ABOGADO SUSTANCIADOR ASIGNADO (SIN RECARGAR df)
# - NO hace: df = df.copy()
# - Re-ejecución segura: borra columnas derivadas previas in-place
# - Resto del flujo queda igual
# ============================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas previas (SIN recargar df)
cols_prev = [
    "AbogadoSustanciadorAsignado_RAW",
    "AbogadoSustanciadorAsignado_SQL",
    "AbogadoSustanciadorAsignado_Len",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# 1) Detectar la columna del Excel
candidatas = [
    c for c in df.columns
    if 'ABOGADO' in str(c).upper() and 'SUSTANCIADOR' in str(c).upper()
]

print("Columnas candidatas para ABOGADO SUSTANCIADOR:", candidatas)
if not candidatas:
    raise ValueError("No se encontró la columna de ABOGADO SUSTANCIADOR ASIGNADO en el DataFrame.")

col_abog = candidatas[0]
print("Usando columna:", col_abog)

# 2) Guardar valor crudo (tal cual viene del Excel)
df['AbogadoSustanciadorAsignado_RAW'] = df[col_abog]

# 3) Normalizar texto para la columna SQL
def normalizar_abogado_sust(v):
    if pd.isna(v):
        return None      # quedará como NULL en SQL
    t = str(v).strip()
    if t == "":
        return None
    # colapsar espacios internos múltiples
    t = " ".join(t.split())
    # aquí NO cambiamos a mayúsculas si quieres conservar el formato original de nombres
    # si prefieres todo en mayúscula, descomenta la siguiente línea:
    # t = t.upper()
    return t

df['AbogadoSustanciadorAsignado_SQL'] = df['AbogadoSustanciadorAsignado_RAW'].apply(normalizar_abogado_sust)

# 4) Longitud para verificar contra varchar(1000)
df['AbogadoSustanciadorAsignado_Len'] = df['AbogadoSustanciadorAsignado_SQL'].fillna("").astype(str).str.len()

print("=== Resumen longitudes ABOGADO SUSTANCIADOR ASIGNADO ===")
print(df['AbogadoSustanciadorAsignado_Len'].describe())

# 5) Registros que superen los 1000 caracteres (si existen)
mas_de_1000 = df[df['AbogadoSustanciadorAsignado_Len'] > 1000][
    [
        'Fila_Excel_aprox',
        'FMI',
        'DEPARTAMENTO',
        'MUNICIPIO',
        'AbogadoSustanciadorAsignado_RAW',
        'AbogadoSustanciadorAsignado_SQL',
        'AbogadoSustanciadorAsignado_Len'
    ]
]

if not mas_de_1000.empty:
    print("\n=== Registros que superan 1000 caracteres (revisar) ===")
    print(mas_de_1000.to_string(index=False))
else:
    print("\nNingún registro supera los 1000 caracteres (ok para varchar(1000)).")

# 6) Muestra rápida RAW vs SQL
muestra = df.loc[
    df['AbogadoSustanciadorAsignado_RAW'].notna(),
    [
        'Fila_Excel_aprox',
        'FMI',
        'DEPARTAMENTO',
        'MUNICIPIO',
        'AbogadoSustanciadorAsignado_RAW',
        'AbogadoSustanciadorAsignado_SQL',
        'AbogadoSustanciadorAsignado_Len'
    ]
].head(30)

print("\n=== Muestra ABOGADO SUSTANCIADOR (RAW vs SQL) ===")
print(muestra.to_string(index=False))

# 7) Excel de revisión (consolidado)
columnas_exportar = [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    'AbogadoSustanciadorAsignado_RAW',
    'AbogadoSustanciadorAsignado_SQL',
    'AbogadoSustanciadorAsignado_Len'
]

columnas_exportar = [c for c in columnas_exportar if c in df.columns]
df_export = df[columnas_exportar].copy()

ruta_salida = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\AbogadoSustanciador_revision.xlsx"
df_export.to_excel(ruta_salida, index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)


Columnas candidatas para ABOGADO SUSTANCIADOR: ['ABOGADO  SUSTANCIADOR ASIGNADO']
Usando columna: ABOGADO  SUSTANCIADOR ASIGNADO
=== Resumen longitudes ABOGADO SUSTANCIADOR ASIGNADO ===
count    24055.000000
mean        19.189981
std         12.901946
min          0.000000
25%          0.000000
50%         24.000000
75%         30.000000
max         37.000000
Name: AbogadoSustanciadorAsignado_Len, dtype: float64

Ningún registro supera los 1000 caracteres (ok para varchar(1000)).

=== Muestra ABOGADO SUSTANCIADOR (RAW vs SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO            MUNICIPIO   AbogadoSustanciadorAsignado_RAW   AbogadoSustanciadorAsignado_SQL  AbogadoSustanciadorAsignado_Len
                2  062-37357      BOLIVAR             ZAMBRANO   VICTOR EDUARDO TORRES HERNANDEZ   VICTOR EDUARDO TORRES HERNANDEZ                               31
                3  062-37359      BOLIVAR             ZAMBRANO            ALBEIRO ROJAS AHUMADA              ALBEIRO ROJAS AHUMADA      

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2787309641.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['AbogadoSustanciadorAsignado_RAW'] = df[col_abog]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2787309641.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['AbogadoSustanciadorAsignado_SQL'] = df['AbogadoSustanciadorAsignado_RAW'].apply(normalizar_abogado_sust)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2787309641.py:58: PerformanceWarning: DataFrame is highly fragmente


Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\AbogadoSustanciador_revision.xlsx


## ABOGADO REVISOR ASIGNADO


In [52]:
import pandas as pd
import numpy as np

# ============================================================
# ABOGADO REVISOR ASIGNADO (SIN RECARGAR df)
# - NO hace: df = df.copy()
# - Re-ejecución segura: borra columnas derivadas previas in-place
# - Resto queda igual
# ============================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas previas (SIN recargar df)
cols_prev = [
    "AbogadoRevisorAsignado_RAW",
    "AbogadoRevisorAsignado_SQL",
    "AbogadoRevisor_Len",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# 1) Detectar la columna del Excel
candidatas_rev = [
    c for c in df.columns
    if 'ABOGADO' in str(c).upper() and 'REVISOR' in str(c).upper()
]

print("Columnas candidatas para ABOGADO REVISOR:", candidatas_rev)
if not candidatas_rev:
    raise ValueError("No se encontró la columna de ABOGADO REVISOR ASIGNADO en el DataFrame.")

col_abog_rev = candidatas_rev[0]
print("Usando columna:", col_abog_rev)

# 2) Guardar valor crudo (tal cual viene del Excel)
df['AbogadoRevisorAsignado_RAW'] = df[col_abog_rev]

# 3) Normalizar texto para la columna SQL
def normalizar_abogado_revisor(v):
    if pd.isna(v):
        return None      # quedará como NULL en SQL
    t = str(v).strip()
    if t == "":
        return None
    # colapsar espacios internos múltiples
    t = " ".join(t.split())
    # NO cambiamos a mayúsculas para conservar el formato de nombres
    # si quisieras todo en mayúsculas, descomenta:
    # t = t.upper()
    return t

df['AbogadoRevisorAsignado_SQL'] = df['AbogadoRevisorAsignado_RAW'].apply(normalizar_abogado_revisor)

# 4) Longitud para verificar contra varchar(1000)
df['AbogadoRevisor_Len'] = df['AbogadoRevisorAsignado_SQL'].fillna("").astype(str).str.len()

print("=== Resumen longitudes ABOGADO REVISOR ASIGNADO ===")
print(df['AbogadoRevisor_Len'].describe())

# 5) Registros que superen los 1000 caracteres (si existen)
largos_rev = df[df['AbogadoRevisor_Len'] > 1000][
    [
        'Fila_Excel_aprox',
        'FMI',
        'DEPARTAMENTO',
        'MUNICIPIO',
        'AbogadoRevisorAsignado_RAW',
        'AbogadoRevisorAsignado_SQL',
        'AbogadoRevisor_Len'
    ]
]

if not largos_rev.empty:
    print("\n=== Registros que superan 1000 caracteres (REVISOR) ===")
    print(largos_rev.to_string(index=False))
else:
    print("\nNingún registro de ABOGADO REVISOR supera los 1000 caracteres (ok para varchar(1000)).")

# 6) Muestra rápida RAW vs SQL
muestra_rev = df.loc[
    df['AbogadoRevisorAsignado_RAW'].notna(),
    [
        'Fila_Excel_aprox',
        'FMI',
        'DEPARTAMENTO',
        'MUNICIPIO',
        'AbogadoRevisorAsignado_RAW',
        'AbogadoRevisorAsignado_SQL',
        'AbogadoRevisor_Len'
    ]
].head(30)

print("\n=== Muestra ABOGADO REVISOR (RAW vs SQL) ===")
print(muestra_rev.to_string(index=False))

# 7) Excel de revisión (consolidado)
columnas_exportar = [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    'AbogadoRevisorAsignado_RAW',
    'AbogadoRevisorAsignado_SQL',
    'AbogadoRevisor_Len'
]

columnas_exportar = [c for c in columnas_exportar if c in df.columns]
df_export = df[columnas_exportar].copy()

ruta_salida = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\AbogadoRevisor_revision.xlsx"
df_export.to_excel(ruta_salida, index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)


Columnas candidatas para ABOGADO REVISOR: ['ABOGADO REVISOR ASIGNADO']
Usando columna: ABOGADO REVISOR ASIGNADO
=== Resumen longitudes ABOGADO REVISOR ASIGNADO ===
count    24055.000000
mean        12.432052
std         14.495511
min          0.000000
25%          0.000000
50%          0.000000
75%         30.000000
max         33.000000
Name: AbogadoRevisor_Len, dtype: float64

Ningún registro de ABOGADO REVISOR supera los 1000 caracteres (ok para varchar(1000)).

=== Muestra ABOGADO REVISOR (RAW vs SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO           MUNICIPIO       AbogadoRevisorAsignado_RAW      AbogadoRevisorAsignado_SQL  AbogadoRevisor_Len
                2  062-37357      BOLIVAR            ZAMBRANO     CINDY ALEXANDRA MORA FAJARDO    CINDY ALEXANDRA MORA FAJARDO                  28
                3  062-37359      BOLIVAR            ZAMBRANO   JEYSON RAUL BERMUDEZ ECHEVERRI  JEYSON RAUL BERMUDEZ ECHEVERRI                  30
                4  062-37361      BOLIVAR   

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\489348053.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['AbogadoRevisorAsignado_RAW'] = df[col_abog_rev]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\489348053.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['AbogadoRevisorAsignado_SQL'] = df['AbogadoRevisorAsignado_RAW'].apply(normalizar_abogado_revisor)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\489348053.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is


Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\AbogadoRevisor_revision.xlsx


## # REPARTO


In [53]:
import pandas as pd
import numpy as np
import re

# =====================================================
# # REPARTO  → ConceptoFinal.NroReparto (int NULL)
# - NO recarga/reasigna df
# - Borra columnas previas para re-ejecución segura (IN-PLACE)
# - Resto queda igual
# =====================================================

# 0) Asegurar columna de fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas generadas antes (IN-PLACE)
cols_prev = [
    "NroReparto_RAW",
    "NroReparto_SQL",
    "Estado_NroReparto",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# 1) Ubicar la columna de Excel para # REPARTO
candidatas_rep = [c for c in df.columns if 'REPARTO' in str(c).upper()]
print("Columnas candidatas para # REPARTO:", candidatas_rep)

if not candidatas_rep:
    raise ValueError("No se encontró la columna de # REPARTO en el DataFrame.")

# Preferir exactamente "# REPARTO" si existe
preferida = [c for c in candidatas_rep if str(c).strip().upper() == "# REPARTO"]
col_rep = preferida[0] if preferida else candidatas_rep[0]
print("Usando columna:", col_rep)

# 2) Guardar valor crudo
df['NroReparto_RAW'] = df[col_rep]

# 3) Función para normalizar / parsear a entero
PLACEHOLDERS_REPARTO = {
    'NO APLICA', 'NO', 'N/A', 'NA',
    'NINGUNO', 'NINGUNA',
    'SIN REPARTO', 'SIN REGISTRO', 'NO REGISTRA', 'NO REGISTRA.'
}

def parse_nro_reparto(v):
    # Vacío o nulo
    if pd.isna(v):
        return (None, 'vacio')

    t = str(v).strip()
    if t == '':
        return (None, 'vacio')

    t_up = t.upper()

    # Textos que significan "no aplica / sin reparto"
    if t_up in PLACEHOLDERS_REPARTO:
        return (None, 'sin_reparto_texto')

    # Extraer solo dígitos (REGRA CONFIRMADA: "PA2" -> 2)
    solo_digitos = re.sub(r'[^0-9]', '', t)
    if solo_digitos == '':
        return (None, 'sin_digitos')

    try:
        n = int(solo_digitos)
    except Exception:
        return (None, 'error_conversion')

    # Validaciones
    if n <= 0:
        return (None, 'no_positivo')
    if n > 2_147_483_647:  # límite INT de SQL Server
        return (None, 'demasiado_grande')

    # Etiquetar si vino con letras (ej PA2) para auditoría
    if re.search(r'[A-Za-z]', t):
        return (n, 'ok_from_alphanum')

    return (n, 'ok')

# 4) Aplicar al DataFrame: NroReparto_SQL + Estado_NroReparto (SIN concat, IN-PLACE)
tmp = df['NroReparto_RAW'].apply(lambda v: pd.Series(parse_nro_reparto(v), index=["NroReparto_SQL", "Estado_NroReparto"]))
df["NroReparto_SQL"] = tmp["NroReparto_SQL"].values
df["Estado_NroReparto"] = tmp["Estado_NroReparto"].values

# 5) Asegurar tipo entero nullable para evitar 43.0, 2.0, etc.
df['NroReparto_SQL'] = df['NroReparto_SQL'].astype('Int64')

# 6) Resumen de estados
print("\n=== Conteo por estado de NroReparto ===")
print(df['Estado_NroReparto'].value_counts(dropna=False))

# 7) Estadísticos solo para los OK (los que se van a migrar)
ok_mask = df['Estado_NroReparto'].isin(['ok', 'ok_from_alphanum'])
print("\n=== Estadísticos NroReparto_SQL (solo estados OK) ===")
print(df.loc[ok_mask, 'NroReparto_SQL'].astype('float').describe())

# 8) Auditoría específica: cuántos vinieron como PA*/alfanum y se convirtieron
mask_alphanum_ok = df['Estado_NroReparto'].eq('ok_from_alphanum')
print("\n=== Auditoría conversiones alfanuméricas (ej: PA2 -> 2) ===")
print("Total convertidos desde alfanumérico:", int(mask_alphanum_ok.sum()))

if mask_alphanum_ok.any():
    ejemplos_alphanum = df.loc[mask_alphanum_ok, [
        'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
        'NroReparto_RAW', 'NroReparto_SQL'
    ]].head(30)
    print("\nEjemplos (hasta 30) ok_from_alphanum:")
    print(ejemplos_alphanum.to_string(index=False))
else:
    print("No hay casos alfanuméricos convertidos.")

# 9) Ejemplos OK (RAW vs SQL)
ej_ok = df.loc[ok_mask, [
    'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    'NroReparto_RAW', 'NroReparto_SQL', 'Estado_NroReparto'
]].head(30)

print("\n=== Ejemplos NroReparto OK (RAW vs SQL) ===")
print(ej_ok.to_string(index=False))

# 10) Ejemplos problemáticos por categoría distinta de OK y vacio
categorias_problema = df['Estado_NroReparto'].dropna().unique().tolist()
categorias_problema = [c for c in categorias_problema if c not in ['ok', 'ok_from_alphanum', 'vacio']]

for cat in categorias_problema:
    sub = df.loc[df['Estado_NroReparto'] == cat, [
        'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
        'NroReparto_RAW', 'Estado_NroReparto'
    ]].head(20)
    print(f"\n=== Ejemplos estado '{cat}' ===")
    if not sub.empty:
        print(sub.to_string(index=False))
    else:
        print("Sin ejemplos.")

# 11) (Opcional) Exportar Excel de revisión
cols_export = [
    'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    col_rep, 'NroReparto_RAW', 'NroReparto_SQL', 'Estado_NroReparto'
]
cols_export = [c for c in cols_export if c in df.columns]

ruta_salida = (
    r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT"
    r"\Documentos\Migracion base de datos viablidad juridica\NroReparto_revision.xlsx"
)

df.loc[:, cols_export].to_excel(ruta_salida, index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)

print("\nColumna lista para migrar a ConceptoFinal.NroReparto:", "NroReparto_SQL")


Columnas candidatas para # REPARTO: ['# REPARTO', 'FECHA ASIGNACIÓN REPARTO']
Usando columna: # REPARTO


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\351365265.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['NroReparto_RAW'] = df[col_rep]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\351365265.py:87: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["NroReparto_SQL"] = tmp["NroReparto_SQL"].values
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\351365265.py:88: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has


=== Conteo por estado de NroReparto ===
Estado_NroReparto
vacio               20388
ok                   3341
ok_from_alphanum      326
Name: count, dtype: int64

=== Estadísticos NroReparto_SQL (solo estados OK) ===
count    3667.000000
mean       33.199073
std        10.925212
min         1.000000
25%        29.000000
50%        36.000000
75%        40.000000
max        43.000000
Name: NroReparto_SQL, dtype: float64

=== Auditoría conversiones alfanuméricas (ej: PA2 -> 2) ===
Total convertidos desde alfanumérico: 326

Ejemplos (hasta 30) ok_from_alphanum:
 Fila_Excel_aprox        FMI DEPARTAMENTO                   MUNICIPIO NroReparto_RAW  NroReparto_SQL
               13  148-16873      CORDOBA                PLANETA RICA            PA2               2
               14  340-14462        SUCRE                  SAN ONOFRE            PA2               2
               22  340-14458        SUCRE                  SAN ONOFRE            PA2               2
               26  340-14459   

## FECHA ASIGNACIÓN REPARTO


In [54]:
import pandas as pd
import numpy as np
import re
import unicodedata
from datetime import datetime

# ==========================================================
# FECHA ASIGNACIÓN REPARTO  → ConceptoFinal.FechaAsignacionReparto (datetime NULL)
# Corrección:
# - NO recarga/reasigna df (NO df = df.copy(), NO df = pd.concat(...))
# - Re-ejecución segura: elimina columnas generadas antes (IN-PLACE)
# - Resto del flujo queda igual
# ==========================================================

# 0) Asegurar fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Si re-ejecutas, borra columnas previas (IN-PLACE) para no duplicar y evitar fragmentación acumulada
cols_prev = [
    "FechaAsignacionReparto_RAW",
    "FechaAsignacionReparto_TXT",
    "FechaAsignacionReparto_TXT_LIMPIO",
    "FechaAsignacionReparto_TXT_LIMPIO_FIX",
    "FechaAsignacionReparto_SQL",
    "Estado_FechaAsignacionReparto",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# ------------------------------
# 1) Buscar columna FECHA ASIGNACIÓN REPARTO (robusto)
# ------------------------------
def norm_header(s: str) -> str:
    s = str(s)
    s_norm = unicodedata.normalize("NFD", s)
    s_sin_tildes = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    return s_sin_tildes.upper()

candidatas_asig = []
for c in df.columns:
    nh = norm_header(c)
    if "FECHA" in nh and "ASIGNACION" in nh and "REPARTO" in nh:
        candidatas_asig.append(c)

print("Columnas candidatas para FECHA ASIGNACIÓN REPARTO:", candidatas_asig)

if not candidatas_asig:
    raise ValueError("No se encontró columna para FECHA ASIGNACIÓN REPARTO; revisa encabezado exacto.")
col_asig = candidatas_asig[0]
print("Usando columna:", repr(col_asig))

# ------------------------------
# 2) Series auxiliares (sin crear df nuevo ni concatenar al final)
# ------------------------------
raw = df[col_asig]

# 3) RAW -> texto base
def a_texto_fecha(v):
    if pd.isna(v):
        return ""
    if isinstance(v, (pd.Timestamp, datetime)):
        return v.strftime("%d/%m/%Y")
    return str(v).strip()

txt = raw.apply(a_texto_fecha)

# 4) Limpiar texto (placeholders, horas, formatos raros)
PLACEHOLDERS_NO_FECHA = {
    "NO", "NO.", "NO APLICA", "NO APLICA.", "NO REGISTRA", "NO REGISTRA.",
    "SIN FECHA", "SIN INFORME", "N/A", "NA", "NINGUNO", "NINGUNA"
}

def limpiar_texto_fecha_asig(s: str) -> str:
    s = "" if s is None else str(s).strip()
    if s == "":
        return ""
    su = s.upper()
    if su in PLACEHOLDERS_NO_FECHA:
        return ""
    # quitar hora tipo "2024-04-25 00:00:00"
    if " " in s:
        s = s.split()[0].strip()
    # compactar espacios/saltos
    s = re.sub(r"\s+", " ", s)

    # corregir "20/112024" -> "20/11/2024"
    m1 = re.fullmatch(r"(\d{1,2})/(\d{2})(\d{4})", s)
    if m1:
        d, mm, yyyy = m1.groups()
        return f"{d}/{mm}/{yyyy}"

    # corregir "2103/2024" -> "21/03/2024"
    m2 = re.fullmatch(r"(\d{2})(\d{2})/(\d{4})", s)
    if m2:
        d, mm, yyyy = m2.groups()
        return f"{d}/{mm}/{yyyy}"

    return s

txt_limpio = txt.apply(limpiar_texto_fecha_asig)

# ------------------------------
# 5) Corrección PUNTUAL (solo 1 registro)
#    Fila_Excel_aprox=7194, FMI=120-12319, valor=01/12/202 -> 01/12/2025
# ------------------------------
txt_fix = txt_limpio.copy()

mask_fix = (
    (df["Fila_Excel_aprox"] == 7194) &
    (df["FMI"].astype(str).str.strip() == "120-12319") &
    (txt_limpio.astype(str).str.strip() == "01/12/202")
)

print("\n=== Corrección puntual año truncado ===")
print("Casos a corregir (debería ser 1):", int(mask_fix.sum()))

txt_fix.loc[mask_fix] = "01/12/2025"

# ------------------------------
# 6) Parsear a datetime + estado (usando FIX)
# ------------------------------
def parse_fecha_asig(t_str: str):
    t_str = "" if t_str is None else str(t_str).strip()
    if t_str == "":
        return pd.NaT, "vacio"

    su = t_str.upper()
    if su in PLACEHOLDERS_NO_FECHA:
        return pd.NaT, "placeholder"

    fecha = pd.to_datetime(t_str, dayfirst=True, errors="coerce")
    if pd.isna(fecha):
        return pd.NaT, "no_parseable"

    return fecha, "ok"

parsed = txt_fix.apply(parse_fecha_asig)
parsed_df = parsed.apply(pd.Series)
parsed_df.columns = ["FechaAsignacionReparto_SQL", "Estado_FechaAsignacionReparto"]

# ------------------------------
# 7) Asignar columnas al df (IN-PLACE, sin concat)
# ------------------------------
df["FechaAsignacionReparto_RAW"] = raw
df["FechaAsignacionReparto_TXT"] = txt
df["FechaAsignacionReparto_TXT_LIMPIO"] = txt_limpio
df["FechaAsignacionReparto_TXT_LIMPIO_FIX"] = txt_fix
df["FechaAsignacionReparto_SQL"] = parsed_df["FechaAsignacionReparto_SQL"].values
df["Estado_FechaAsignacionReparto"] = parsed_df["Estado_FechaAsignacionReparto"].values

# ------------------------------
# 8) Resumen
# ------------------------------
total = len(df)
validas = df["FechaAsignacionReparto_SQL"].notna().sum()
nulas = df["FechaAsignacionReparto_SQL"].isna().sum()

print("\n=== Resumen FECHA ASIGNACIÓN REPARTO ===")
print("Total filas:", total)
print("Fechas válidas (no NaT):", validas)
print("Fechas nulas/NaT       :", nulas)

print("\n=== Conteo por Estado_FechaAsignacionReparto ===")
print(df["Estado_FechaAsignacionReparto"].value_counts(dropna=False))

# Validar el registro corregido
if mask_fix.any():
    print("\n=== Validación registro corregido ===")
    cols_val = [c for c in ["Fila_Excel_aprox","FMI","DEPARTAMENTO","MUNICIPIO"] if c in df.columns] + [
        "FechaAsignacionReparto_RAW",
        "FechaAsignacionReparto_TXT_LIMPIO",
        "FechaAsignacionReparto_TXT_LIMPIO_FIX",
        "FechaAsignacionReparto_SQL",
        "Estado_FechaAsignacionReparto"
    ]
    print(df.loc[mask_fix, cols_val].to_string(index=False))

# ------------------------------
# 9) Textos que no se pudieron parsear
# ------------------------------
mask_raras = (
    df["FechaAsignacionReparto_SQL"].isna()
    & (df["FechaAsignacionReparto_TXT_LIMPIO_FIX"].astype(str).str.strip() != "")
)

print("\n=== Valores FIX que producen NaT (top 20) ===")
print(
    df.loc[mask_raras, "FechaAsignacionReparto_TXT_LIMPIO_FIX"]
      .value_counts()
      .head(20)
)

# ------------------------------
# 10) Excels de revisión
# ------------------------------
base_path = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica"

cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]
cols_rev = cols_ctx + [
    "FechaAsignacionReparto_RAW",
    "FechaAsignacionReparto_TXT_LIMPIO",
    "FechaAsignacionReparto_TXT_LIMPIO_FIX",
    "FechaAsignacionReparto_SQL",
    "Estado_FechaAsignacionReparto"
]

df_revision = df.loc[:, cols_rev].copy()
ruta_rev = fr"{base_path}\FechaAsignacionReparto_revision.xlsx"
df_revision.to_excel(ruta_rev, index=False)
print("\nArchivo de revisión generado en:")
print(ruta_rev)

df_problemas = df.loc[mask_raras, cols_rev].copy()
ruta_prob = fr"{base_path}\FechaAsignacionReparto_problemas.xlsx"
df_problemas.to_excel(ruta_prob, index=False)
print("Archivo de problemáticos generado en:")
print(ruta_prob)

print("\nColumna lista para migrar a ConceptoFinal.FechaAsignacionReparto: FechaAsignacionReparto_SQL")


Columnas candidatas para FECHA ASIGNACIÓN REPARTO: ['FECHA ASIGNACIÓN REPARTO']
Usando columna: 'FECHA ASIGNACIÓN REPARTO'

=== Corrección puntual año truncado ===
Casos a corregir (debería ser 1): 1


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4145418570.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["FechaAsignacionReparto_RAW"] = raw
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4145418570.py:147: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["FechaAsignacionReparto_TXT"] = txt
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4145418570.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po


=== Resumen FECHA ASIGNACIÓN REPARTO ===
Total filas: 24055
Fechas válidas (no NaT): 9831
Fechas nulas/NaT       : 14224

=== Conteo por Estado_FechaAsignacionReparto ===
Estado_FechaAsignacionReparto
vacio    14224
ok        9831
Name: count, dtype: int64

=== Validación registro corregido ===
 Fila_Excel_aprox       FMI DEPARTAMENTO MUNICIPIO FechaAsignacionReparto_RAW FechaAsignacionReparto_TXT_LIMPIO FechaAsignacionReparto_TXT_LIMPIO_FIX FechaAsignacionReparto_SQL Estado_FechaAsignacionReparto
             7194 120-12319        CAUCA    TIMBIO                  01/12/202                         01/12/202                            01/12/2025                 2025-12-01                            ok

=== Valores FIX que producen NaT (top 20) ===
Series([], Name: count, dtype: int64)

Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\FechaAsignacionReparto_revision.xlsx
Archivo de

## PLAZO PARA ENTREGA A REVISOR


In [55]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# ============================================================
# PLAZO PARA ENTREGA A REVISOR  -> FechaPlazoEntregaARevisor_SQL (datetime)
# Destino: AnalisisJuridico.ConceptoFinal.FechaPlazoEntregaARevisor
#
# Corrección:
# - NO recarga/reasigna df (NO df = df.copy(), NO df = pd.concat(...))
# - Re-ejecución segura: elimina columnas generadas antes (IN-PLACE)
# - Resto del flujo queda igual
# ============================================================

# ===========================
# 0) Asegurar fila aproximada
# ===========================
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Si re-ejecutas, borra columnas previas (IN-PLACE) para no duplicar y evitar fragmentación acumulada
cols_prev = [
    "FechaPlazoEntregaARevisor_RAW",
    "FechaPlazoEntregaARevisor_TXT",
    "FechaPlazoEntregaARevisor_TXT_LIMPIO",
    "FechaPlazoEntregaARevisor_SQL",
    "Estado_FechaPlazoEntregaARevisor",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# ===========================
# 1) Localizar columna PLAZO PARA ENTREGA A REVISOR
# ===========================
candidatas_plazo = [
    c for c in df.columns
    if ("PLAZO" in str(c).upper()
        and "ENTREGA" in str(c).upper()
        and "REVISOR" in str(c).upper())
]

print("Columnas candidatas para PLAZO PARA ENTREGA A REVISOR:", candidatas_plazo)

if not candidatas_plazo:
    raise ValueError("No se encontró la columna de PLAZO PARA ENTREGA A REVISOR en el DataFrame.")

# Priorizar el encabezado exacto si existe
prior = [c for c in candidatas_plazo if str(c).strip().upper() == "PLAZO PARA ENTREGA A REVISOR"]
col_plazo = prior[0] if prior else candidatas_plazo[0]
print("Usando columna:", repr(col_plazo))

# ===========================
# 2) Series auxiliares (sin crear df nuevo ni concatenar)
# ===========================
raw = df[col_plazo]

# ===========================
# 3) RAW -> texto base
# ===========================
def a_texto_fecha(v):
    if pd.isna(v):
        return ""
    if isinstance(v, (pd.Timestamp, datetime)):
        return v.strftime("%d/%m/%Y")
    return str(v).strip()

txt = raw.apply(a_texto_fecha)

# ===========================
# 4) Limpieza de texto
# ===========================
PLACEHOLDERS_NO_FECHA = {
    "NO", "NO.", "NO APLICA", "NO APLICA.",
    "NO REGISTRA", "NO REGISTRA.",
    "SIN FECHA", "SIN INFORME",
    "N/A", "NA", "NINGUNO", "NINGUNA"
}

def limpiar_texto_fecha_plazo(s: str) -> str:
    s = "" if s is None else str(s).strip()
    if s == "":
        return ""

    su = s.upper()
    if su in PLACEHOLDERS_NO_FECHA:
        return ""

    # quitar hora: "2024-04-25 00:00:00" -> "2024-04-25"
    if " " in s:
        s = s.split()[0].strip()

    # compactar espacios/saltos
    s = re.sub(r"\s+", " ", s)

    # Caso 20/112024 -> 20/11/2024
    m1 = re.fullmatch(r"(\d{1,2})/(\d{2})(\d{4})", s)
    if m1:
        d, mm, yyyy = m1.groups()
        return f"{d}/{mm}/{yyyy}"

    # Caso 2103/2024 -> 21/03/2024
    m2 = re.fullmatch(r"(\d{2})(\d{2})/(\d{4})", s)
    if m2:
        d, mm, yyyy = m2.groups()
        return f"{d}/{mm}/{yyyy}"

    # Caso año truncado: 01/12/202 -> 01/12/2025  (solo si lo quieres forzar)
    # NOTA: si prefieres no asumir 2025, comenta este bloque.
    m3 = re.fullmatch(r"(\d{1,2})/(\d{1,2})/(\d{3})", s)
    if m3:
        d, mm, y3 = m3.groups()
        return f"{d.zfill(2)}/{mm.zfill(2)}/{y3}5"

    return s

txt_limpio = txt.apply(limpiar_texto_fecha_plazo)

# ===========================
# 5) Parse a datetime + estado
# ===========================
def parse_fecha_plazo(t_str: str):
    t_str = "" if t_str is None else str(t_str).strip()
    if t_str == "":
        return pd.NaT, "vacio"

    su = t_str.upper()
    if su in PLACEHOLDERS_NO_FECHA:
        return pd.NaT, "placeholder"

    fecha = pd.to_datetime(t_str, dayfirst=True, errors="coerce")
    if pd.isna(fecha):
        return pd.NaT, "no_parseable"

    return fecha, "ok"

parsed = txt_limpio.apply(parse_fecha_plazo)
parsed_df = parsed.apply(pd.Series)
parsed_df.columns = ["FechaPlazoEntregaARevisor_SQL", "Estado_FechaPlazoEntregaARevisor"]

# ===========================
# 6) Asignar al df principal (IN-PLACE, sin concat)
# ===========================
df["FechaPlazoEntregaARevisor_RAW"] = raw
df["FechaPlazoEntregaARevisor_TXT"] = txt
df["FechaPlazoEntregaARevisor_TXT_LIMPIO"] = txt_limpio
df["FechaPlazoEntregaARevisor_SQL"] = parsed_df["FechaPlazoEntregaARevisor_SQL"].values
df["Estado_FechaPlazoEntregaARevisor"] = parsed_df["Estado_FechaPlazoEntregaARevisor"].values

# ===========================
# 7) Resumen
# ===========================
total = len(df)
validas = df["FechaPlazoEntregaARevisor_SQL"].notna().sum()
nulas = df["FechaPlazoEntregaARevisor_SQL"].isna().sum()

print("\n=== Resumen PLAZO PARA ENTREGA A REVISOR ===")
print("Total filas:", total)
print("Fechas válidas (no NaT):", validas)
print("Fechas nulas/NaT       :", nulas)

print("\n=== Conteo por Estado_FechaPlazoEntregaARevisor ===")
print(df["Estado_FechaPlazoEntregaARevisor"].value_counts(dropna=False))

# ===========================
# 8) Ejemplos válidos
# ===========================
cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]
cols_muestra_ok = cols_ctx + [
    "FechaPlazoEntregaARevisor_RAW",
    "FechaPlazoEntregaARevisor_TXT_LIMPIO",
    "FechaPlazoEntregaARevisor_SQL",
]

ej_validas = df.loc[df["FechaPlazoEntregaARevisor_SQL"].notna(), cols_muestra_ok].head(30)

print("\n=== Ejemplos de fechas válidas (RAW vs TXT_LIMPIO vs SQL) ===")
print(ej_validas.to_string(index=False))

# ===========================
# 9) No parseables (para revisión)
# ===========================
mask_raras = (
    df["FechaPlazoEntregaARevisor_SQL"].isna()
    & (df["FechaPlazoEntregaARevisor_TXT_LIMPIO"].astype(str).str.strip() != "")
)

print("\n=== Valores de FechaPlazoEntregaARevisor_TXT_LIMPIO que producen NaT (top 20) ===")
print(
    df.loc[mask_raras, "FechaPlazoEntregaARevisor_TXT_LIMPIO"]
      .value_counts()
      .head(20)
)

cols_muestra_raras = cols_ctx + [
    "FechaPlazoEntregaARevisor_RAW",
    "FechaPlazoEntregaARevisor_TXT_LIMPIO",
    "Estado_FechaPlazoEntregaARevisor",
]

ej_raras = df.loc[mask_raras, cols_muestra_raras].head(50)

print("\n=== Ejemplos de filas con texto que no se pudo convertir a fecha ===")
if not ej_raras.empty:
    print(ej_raras.to_string(index=False))
else:
    print("No hay textos raros; sólo nulos o placeholders (NO APLICA / NO REGISTRA / etc.).")

# ===========================
# 10) Excels de revisión
# ===========================
base_path = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica"

df_revision = df.loc[:, cols_muestra_ok + ["Estado_FechaPlazoEntregaARevisor"]].copy()
ruta_rev = fr"{base_path}\FechaPlazoEntregaARevisor_revision.xlsx"
df_revision.to_excel(ruta_rev, index=False)
print("\nArchivo de revisión generado en:")
print(ruta_rev)

df_problemas = df.loc[mask_raras, cols_muestra_raras].copy()
ruta_prob = fr"{base_path}\FechaPlazoEntregaARevisor_problemas.xlsx"
df_problemas.to_excel(ruta_prob, index=False)
print("Archivo de problemáticos generado en:")
print(ruta_prob)

print("\nColumna lista para migrar a ConceptoFinal.FechaPlazoEntregaARevisor: FechaPlazoEntregaARevisor_SQL")


Columnas candidatas para PLAZO PARA ENTREGA A REVISOR: ['PLAZO PARA ENTREGA A REVISOR']
Usando columna: 'PLAZO PARA ENTREGA A REVISOR'


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4241581496.py:145: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["FechaPlazoEntregaARevisor_RAW"] = raw
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4241581496.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["FechaPlazoEntregaARevisor_TXT"] = txt
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4241581496.py:147: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which 


=== Resumen PLAZO PARA ENTREGA A REVISOR ===
Total filas: 24055
Fechas válidas (no NaT): 7493
Fechas nulas/NaT       : 16562

=== Conteo por Estado_FechaPlazoEntregaARevisor ===
Estado_FechaPlazoEntregaARevisor
vacio    16562
ok        7493
Name: count, dtype: int64

=== Ejemplos de fechas válidas (RAW vs TXT_LIMPIO vs SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO           MUNICIPIO FechaPlazoEntregaARevisor_RAW FechaPlazoEntregaARevisor_TXT_LIMPIO FechaPlazoEntregaARevisor_SQL
                2  062-37357      BOLIVAR            ZAMBRANO                    2024-10-17                           17/10/2024                    2024-10-17
                3  062-37359      BOLIVAR            ZAMBRANO                    2025-11-18                           18/11/2025                    2025-11-18
                4  062-37361      BOLIVAR            ZAMBRANO                    2024-10-01                           01/10/2024                    2024-10-01
                5  062-37354     

*avance de excel* pedir que lo realice ahi voy

In [56]:
import pandas as pd
import numpy as np

# ============================================================
# AVANCE A EXCEL (SIN RECARGAR df) - INCLUYE ÚLTIMOS CAMPOS
# - NO hace df = df.copy() / NO concatena a df
# - Solo construye dataframes AUXILIARES para exportar
# ============================================================

# 0) Asegurar fila Excel (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# -----------------------------
# Helpers
# -----------------------------
def pick_first_existing(cands):
    for c in cands:
        if c in df.columns:
            return c
    return None

def find_by_tokens(tokens, exclude_suffixes=(), exclude_prefixes=()):
    """
    Devuelve primera columna cuyo nombre contiene TODOS los tokens (en upper),
    excluyendo sufijos/prefijos.
    """
    toks = [t.upper() for t in tokens]
    for c in df.columns:
        cu = str(c).upper()
        if any(cu.endswith(suf.upper()) for suf in exclude_suffixes):
            continue
        if any(cu.startswith(pre.upper()) for pre in exclude_prefixes):
            continue
        if all(t in cu for t in toks):
            return c
    return None

# 1) Columna IdRegistroProceso para export (solo trazabilidad si falta la real)
id_reg_col = pick_first_existing(["IdRegistroProceso_SQL", "ID_SQL"])
if id_reg_col is None:
    print("[OJO] No existe IdRegistroProceso_SQL ni ID_SQL en df. ConceptosPrevio/ConceptoFinal pueden salir vacíos.")

# 2) Contexto
cols_ctx = [
    "Fila_Excel_aprox",
    "IdPostulacion_SQL",
    "ID_SQL",
    "IdRegistroProceso_SQL",
    "FMI", "FMI_SQL",
    "DEPARTAMENTO", "MUNICIPIO",
]
cols_ctx = [c for c in cols_ctx if c in df.columns]

# ------------------------------------------------------------
# A) RegistroProceso
# ------------------------------------------------------------
cols_registro = [
    "IdPostulacion_SQL",
    "FMI_SQL",
    "NumeroExpediente_SQL",
    "IdFuenteProceso", "IdFuenteProceso_SQL",
    "IdTipoProceso", "IdTipoProceso_SQL",
    "IdEtapaProcesal", "IdEtapaProcesal_SQL",
    "Dependencia_SQL",
    "RadicadoOrfeo_SQL",
]
cols_registro_exist = [c for c in cols_registro if c in df.columns]
df_registro = df[cols_ctx + cols_registro_exist].copy()

# ------------------------------------------------------------
# B) EstudioTerreno
# ------------------------------------------------------------
cols_estudio = [
    "IdRegistroProceso_SQL",
    "IdLocalizacion_SQL",
    "AreaRegistral_SQL",
    "CirculoRegistral_SQL",
    "AreaCalculada_SQL",
    "TipoPersonaTitular_SQL",
    "NombrePropietario_SQL",
    "ApellidoPropietario_SQL",
    "Identificacion_SQL_TXT_MIGRAR",
    "IdTipoDocumento_SQL_MIGRAR",
    "NaturalezaJuridica_SQL_FINAL",
    "AcreditacionPropiedad_SQL_FINAL",
]
cols_estudio_exist = [c for c in cols_estudio if c in df.columns]
df_estudio = df[cols_ctx + cols_estudio_exist].copy()

# ------------------------------------------------------------
# C) MedidaProcesal (consolidada en largo)
# ------------------------------------------------------------
base_cols_mp = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]
mp_parts = []

def add_mp(objeto, col_valor_final, col_anot, col_tipo=None, col_estado=None, estados_ok=None):
    if col_valor_final not in df.columns or col_anot not in df.columns:
        return
    valor = df[col_valor_final]
    anot = df[col_anot]

    if col_estado and (col_estado in df.columns) and estados_ok:
        mask = df[col_estado].isin(estados_ok)
    else:
        mask = valor.notna()

    if not mask.any():
        return

    out = df.loc[mask, base_cols_mp].copy()
    out["Valor"] = valor.loc[mask].values
    out["Anotacion"] = anot.loc[mask].values
    out["TipoClasificacion"] = df.loc[mask, col_tipo].values if (col_tipo and col_tipo in df.columns) else None
    out["Objeto"] = objeto
    mp_parts.append(out)

add_mp("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL", None, "HIPOTECA_ESTADO", ["ok", "sin_reg"])
add_mp("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL", None, "SERVIDUMBRES_ESTADO", ["ok", "sin_reg"])
add_mp("MEDIDAS CAUTELARES", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL", None, "MEDIDAS_CAUT_ESTADO", ["ok", "sin_reg"])
add_mp("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL", "RUPTA_ESTADO", ["ok", "sin_reg"])
add_mp("RTDAF LEY 1448", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL", None, "RTDAF_ESTADO", ["ok", "sin_reg"])
add_mp("OFERTA OTRAS ENTIDADES", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL", None, "OFERTA_ESTADO", ["ok", "sin_reg"])
add_mp("PROCESOS DE CLARIFICACIÓN", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL", None, "PROC_CLARIF_ESTADO", ["ok", "sin_reg"])

df_medidaprocesal = (
    pd.concat(mp_parts, ignore_index=True)
    if mp_parts else
    pd.DataFrame(columns=base_cols_mp + ["Valor", "Anotacion", "TipoClasificacion", "Objeto"])
)

# ------------------------------------------------------------
# D) ConceptosPrevio_listo_insertar (si existe)
#    (usa columnas que ya tengas creadas para previos)
# ------------------------------------------------------------
df_conceptosprevio = pd.DataFrame()
if id_reg_col is not None:
    # Si ya tienes estas columnas con esos nombres, las toma.
    # Si no, busca por tokens.
    col_prev_concepto = pick_first_existing(["ConceptoPrevio_SQL", "ConceptosPrevio_SQL"])
    if col_prev_concepto is None:
        col_prev_concepto = find_by_tokens(["CONCEPTO", "PREVIO"])

    col_prev_fecha = pick_first_existing(["FechaConceptoPrevio_SQL"])
    if col_prev_fecha is None:
        col_prev_fecha = find_by_tokens(["FECHA", "INFORME", "PREVIO"])

    cols_prev = [c for c in [id_reg_col, col_prev_fecha, col_prev_concepto] if c is not None and c in df.columns]
    if cols_prev:
        ctx_prev = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]
        df_conceptosprevio = df[ctx_prev + cols_prev].copy()
        df_conceptosprevio = df_conceptosprevio.rename(columns={id_reg_col: "IdRegistroProceso_SQL_EXPORT"})

# ------------------------------------------------------------
# E) ConceptoFinal_listo_insertar (INCLUYE LOS ÚLTIMOS)
# ------------------------------------------------------------
df_conceptofinal = pd.DataFrame()
if id_reg_col is not None:
    ctx_cf = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

    # Preferencia por nombres EXACTOS que venimos usando
    prefer_cf = [
        "ConceptoActualDeViabilidadJuridica_SQL",
        "FechaInforme_SQL",
        "IdTipoViabilidad_SQL",
        "IdTipoInforme_SQL",
        "IdTipoNoViabilidad_SQL",
        "InsumosPendientes_SQL",
        "FechaEntregaARevisor_SQL",
        "AbogadoSustanciadorAsignado_SQL",
        "AbogadoRevisorAsignado_SQL",
        "NroReparto_SQL",
        "FechaAsignacionReparto_SQL",
        "FechaPlazoEntregaARevisor_SQL",
    ]
    cols_cf_exist = [c for c in prefer_cf if c in df.columns]

    # Si alguna no está, intenta buscar por tokens (por si cambió el nombre)
    # (solo añade si la encuentra y NO está ya)
    token_map = {
        "ConceptoActualDeViabilidadJuridica_SQL": ["CONCEPTO", "VIABILIDAD"],
        "FechaInforme_SQL": ["FECHA", "INFORME"],
        "FechaEntregaARevisor_SQL": ["FECHA", "ENTREGA", "REVISOR"],
        "FechaAsignacionReparto_SQL": ["FECHA", "ASIGNACION", "REPARTO"],
        "FechaPlazoEntregaARevisor_SQL": ["PLAZO", "ENTREGA", "REVISOR"],
        "AbogadoSustanciadorAsignado_SQL": ["ABOGADO", "SUSTANCIADOR"],
        "AbogadoRevisorAsignado_SQL": ["ABOGADO", "REVISOR"],
        "NroReparto_SQL": ["REPARTO"],
        "InsumosPendientes_SQL": ["INSUMOS", "PENDIENT"],
        "IdTipoViabilidad_SQL": ["IDTIPOVIABILIDAD"],
        "IdTipoNoViabilidad_SQL": ["IDTIPONOVIABILIDAD"],
        "IdTipoInforme_SQL": ["IDTIPOINFORME"],
    }

    for desired, toks in token_map.items():
        if desired in cols_cf_exist:
            continue
        found = find_by_tokens(toks)
        if found and found in df.columns:
            cols_cf_exist.append(found)

    cols_cf_final = [id_reg_col] + cols_cf_exist
    cols_cf_final = [c for c in cols_cf_final if c in df.columns]

    if cols_cf_final:
        df_conceptofinal = df[ctx_cf + cols_cf_final].copy()
        df_conceptofinal = df_conceptofinal.rename(columns={id_reg_col: "IdRegistroProceso_SQL_EXPORT"})

# ------------------------------------------------------------
# F) Unificado_df + Diccionario
# ------------------------------------------------------------
df_unificado = df.copy()

dicc = []
for c in cols_registro_exist:
    dicc.append(["RegistroProceso_avance", c])
for c in cols_estudio_exist:
    dicc.append(["EstudioTerreno_avance", c])

for c in ["Valor", "Anotacion", "TipoClasificacion", "Objeto"]:
    dicc.append(["MedidaProcesal_listo_insertar", c])

if not df_conceptosprevio.empty:
    for c in df_conceptosprevio.columns:
        dicc.append(["ConceptosPrevio_listo_insertar", c])

if not df_conceptofinal.empty:
    for c in df_conceptofinal.columns:
        dicc.append(["ConceptoFinal_listo_insertar", c])

df_dicc = pd.DataFrame(dicc, columns=["Hoja", "Columna"])

# ------------------------------------------------------------
# Export final
# ------------------------------------------------------------
out_xlsx = "AVANCE_MIGRACION_HASTA_HOY_ACTUALIZADO.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    df_registro.to_excel(writer, index=False, sheet_name="RegistroProceso_avance")
    df_estudio.to_excel(writer, index=False, sheet_name="EstudioTerreno_avance")
    df_medidaprocesal.to_excel(writer, index=False, sheet_name="MedidaProcesal_listo_insertar")
    df_conceptosprevio.to_excel(writer, index=False, sheet_name="ConceptosPrevio_listo_insertar")
    df_conceptofinal.to_excel(writer, index=False, sheet_name="ConceptoFinal_listo_insertar")
    df_unificado.to_excel(writer, index=False, sheet_name="Unificado_df")
    df_dicc.to_excel(writer, index=False, sheet_name="Diccionario")

print("Excel creado:", out_xlsx)
print("Filas ConceptosPrevio_listo_insertar:", len(df_conceptosprevio))
print("Filas ConceptoFinal_listo_insertar  :", len(df_conceptofinal))
print("Columnas ConceptoFinal_listo_insertar:", list(df_conceptofinal.columns)[:30], "..." if df_conceptofinal.shape[1] > 30 else "")
print("Columnas faltantes esperadas (si aplica):")
for c in [
    "AbogadoSustanciadorAsignado_SQL", "AbogadoRevisorAsignado_SQL",
    "NroReparto_SQL", "FechaAsignacionReparto_SQL", "FechaPlazoEntregaARevisor_SQL"
]:
    if c not in df.columns:
        print("  - NO existe en df:", c)


Excel creado: AVANCE_MIGRACION_HASTA_HOY_ACTUALIZADO.xlsx
Filas ConceptosPrevio_listo_insertar: 24055
Filas ConceptoFinal_listo_insertar  : 24055
Columnas ConceptoFinal_listo_insertar: ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO', 'IdRegistroProceso_SQL_EXPORT', 'ConceptoActualDeViabilidadJuridica_SQL', 'FechaInforme_SQL', 'IdTipoViabilidad_SQL', 'IdTipoInforme_SQL', 'IdTipoNoViabilidad_SQL', 'InsumosPendientes_SQL', 'FechaEntregaARevisor_SQL', 'AbogadoSustanciadorAsignado_SQL', 'AbogadoRevisorAsignado_SQL', 'NroReparto_SQL', 'FechaAsignacionReparto_SQL', 'FechaPlazoEntregaARevisor_SQL'] 
Columnas faltantes esperadas (si aplica):


## "ESTADO REVISION APROBADO / DEVUELTO / EN REVISION"


In [57]:
import pandas as pd
import numpy as np
import unicodedata
import re

# ============================================================
# ESTADO REVISION (SIN RECARGAR df)
# - NO hace: df = df.copy()
# - Re-ejecución segura: elimina columnas derivadas previas in-place
# - Evita fragmentación: crea columnas en DataFrame auxiliar y añade en un solo paso
# - Resto del flujo queda igual
# ============================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Re-ejecución segura (SIN recargar df): borrar columnas derivadas previas
cols_prev = [
    "EstadoRevision_RAW",
    "EstadoRevision_TXT_LIMPIO",
    "IdTipoEstadoRevision_SQL",
    "EstadoRevision_NOMBRE_SQL",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# =========================
# 1) Detectar columna ESTADO REVISION
# =========================
candidatas_estado = [
    c for c in df.columns
    if ("ESTADO" in str(c).upper()) and ("REVISION" in str(c).upper())
]
print("Columnas candidatas para ESTADO REVISION:", candidatas_estado)

if not candidatas_estado:
    raise ValueError("No se encontró la columna de ESTADO REVISION en el DataFrame.")

# (opcional) priorizar encabezado exacto si existe
prior = [c for c in candidatas_estado if str(c).strip().upper() == "ESTADO REVISION"]
col_estado = prior[0] if prior else candidatas_estado[0]
print("Usando columna:", repr(col_estado))

# =========================
# 2) Auxiliar
# =========================
nuevas = pd.DataFrame(index=df.index)
nuevas["EstadoRevision_RAW"] = df[col_estado]

# =========================
# 3) Limpieza texto (sin tildes, mayúsculas, espacios)
# =========================
def limpiar_estado_revision(v):
    if pd.isna(v):
        return None
    if isinstance(v, pd.Timestamp):
        return v.strftime("%Y-%m-%d")
    t = str(v).strip()
    if t == "":
        return None

    t_norm = unicodedata.normalize("NFD", t)
    t_sin = "".join(ch for ch in t_norm if unicodedata.category(ch) != "Mn")
    t_sin = re.sub(r"\s+", " ", t_sin).strip().upper()
    return t_sin

nuevas["EstadoRevision_TXT_LIMPIO"] = nuevas["EstadoRevision_RAW"].apply(limpiar_estado_revision)

print("\n=== Valores CRUDOS ESTADO REVISION (top 30) ===")
print(nuevas["EstadoRevision_RAW"].value_counts(dropna=False).head(30))

print("\n=== Valores NORMALIZADOS ESTADO REVISION (top 30) ===")
print(nuevas["EstadoRevision_TXT_LIMPIO"].value_counts(dropna=False).head(30))

# =========================
# 4) Clasificación (catálogo SQL)
# 1 -> Aprobado
# 2 -> Devuelto
# 3 -> En revisión
# =========================
def es_fecha_parseable(txt: str) -> bool:
    if txt is None:
        return False
    s = str(txt).strip()
    if s == "":
        return False
    if re.match(r"^\d{4}-\d{2}-\d{2}", s):
        d = pd.to_datetime(s, errors="coerce")
        return pd.notna(d)
    return False

def clasificar_estado_revision(t):
    if t is None or t == "":
        return (None, None)

    # ---- REGLAS PUNTUALES ----
    if t == "PREDIO DAE":
        return (1, "APROBADO")

    if t == "REVISION":
        return (3, "EN REVISION")

    if t == "CORRECCIONES":
        return (2, "DEVUELTO")

    # ---- Reglas generales ----
    if es_fecha_parseable(t):
        return (1, "APROBADO")

    if "DEVUELT" in t or "DEVOLUCION" in t or "SE DEVUELVE" in t:
        return (2, "DEVUELTO")

    if "EN REVISION" in t:
        return (3, "EN REVISION")

    if "APROBAD" in t:
        return (1, "APROBADO")

    if "FIRMAD" in t:
        return (1, "APROBADO")

    if "PENDIENTE" in t:
        return (3, "EN REVISION")

    if "SE CAMBIA A NO VIABLE" in t or ("CAMBIA" in t and "NO VIABLE" in t):
        return (3, "EN REVISION")

    if any(k in t for k in [
        "FUE ENVIADO", "MEDIANTE CORREO", "CORREO", "REALICE EL ESTUDIO",
        "ESCRITURA", "PREDIO ACREDITA", "PROPIEDAD PRIVADA", "SE INDICO EN CORREO"
    ]):
        return (1, "APROBADO")

    if len(t) >= 60 and not any(k in t for k in ["DEVUELT", "DEVOLUCION", "PENDIENTE", "EN REVISION", "REVISION"]):
        return (1, "APROBADO")

    return (None, "SIN_CLASIFICAR")

clasif = nuevas["EstadoRevision_TXT_LIMPIO"].apply(clasificar_estado_revision)
clasif_df = clasif.apply(pd.Series)
clasif_df.columns = ["IdTipoEstadoRevision_SQL", "EstadoRevision_NOMBRE_SQL"]

# (evita otro concat grande al final)
nuevas[clasif_df.columns] = clasif_df

# =========================
# 5) Unir al df (SIN df = pd.concat(...))
# =========================
df[nuevas.columns] = nuevas

# =========================
# 6) Resumen
# =========================
total = len(df)
no_nulos_id = df["IdTipoEstadoRevision_SQL"].notna().sum()
nulos_id = df["IdTipoEstadoRevision_SQL"].isna().sum()

print("\n=== Resumen IdTipoEstadoRevision_SQL ===")
print("Total filas:", total)
print("Con IdTipoEstadoRevision_SQL NO nulo:", no_nulos_id)
print("Con IdTipoEstadoRevision_SQL nulo   :", nulos_id)

print("\nDistribución IdTipoEstadoRevision_SQL:")
print(df["IdTipoEstadoRevision_SQL"].value_counts(dropna=False).sort_index())

print("\n=== Valores normalizados EstadoRevision_NOMBRE_SQL (top 20) ===")
print(df["EstadoRevision_NOMBRE_SQL"].value_counts(dropna=False).head(20))

# =========================
# 7) Revisión SIN_CLASIFICAR
# =========================
mask_sin_clasif = (
    df["EstadoRevision_TXT_LIMPIO"].notna()
    & (df["EstadoRevision_TXT_LIMPIO"] != "")
    & df["IdTipoEstadoRevision_SQL"].isna()
)

cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

ej_sin_clasif = df.loc[
    mask_sin_clasif,
    cols_ctx + ["EstadoRevision_RAW", "EstadoRevision_TXT_LIMPIO", "EstadoRevision_NOMBRE_SQL"]
].head(50)

print("\n=== Ejemplos de ESTADO REVISION con TEXTO pero SIN clasificar (post-reglas) ===")
if not ej_sin_clasif.empty:
    print(ej_sin_clasif.to_string(index=False))
else:
    print("No quedan textos sin clasificar; sólo nulos/vacíos.")

# =========================
# 8) Excels de revisión
# =========================
base_path = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica"

df_revision = df[cols_ctx + [
    "EstadoRevision_RAW",
    "EstadoRevision_TXT_LIMPIO",
    "IdTipoEstadoRevision_SQL",
    "EstadoRevision_NOMBRE_SQL"
]].copy()

ruta_rev = fr"{base_path}\EstadoRevision_raw_y_normalizado.xlsx"
df_revision.to_excel(ruta_rev, index=False)
print("\nArchivo de revisión generado en:")
print(ruta_rev)

df_sin_clasif = df.loc[mask_sin_clasif, cols_ctx + [
    "EstadoRevision_RAW",
    "EstadoRevision_TXT_LIMPIO",
    "EstadoRevision_NOMBRE_SQL"
]].copy()

ruta_sin = fr"{base_path}\EstadoRevision_sin_clasificar.xlsx"
df_sin_clasif.to_excel(ruta_sin, index=False)
print("Archivo de no clasificados generado en:")
print(ruta_sin)

print("\nColumna lista para migrar a ConceptoFinal.IdTipoEstadoRevision:", "IdTipoEstadoRevision_SQL")


Columnas candidatas para ESTADO REVISION: ['ESTADO REVISION\nAPROBADO / DEVUELTO / EN REVISION']
Usando columna: 'ESTADO REVISION\nAPROBADO / DEVUELTO / EN REVISION'

=== Valores CRUDOS ESTADO REVISION (top 30) ===
EstadoRevision_RAW
NaN                                                                                                                                                                                                                                                18964
APROBADO                                                                                                                                                                                                                                            4691
DEVUELTO                                                                                                                                                                                                                                             331
PENDIENTE          

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2475361649.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2475361649.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2475361649.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi


=== Resumen IdTipoEstadoRevision_SQL ===
Total filas: 24055
Con IdTipoEstadoRevision_SQL NO nulo: 5091
Con IdTipoEstadoRevision_SQL nulo   : 18964

Distribución IdTipoEstadoRevision_SQL:
IdTipoEstadoRevision_SQL
1.0     4712
2.0      333
3.0       46
NaN    18964
Name: count, dtype: int64

=== Valores normalizados EstadoRevision_NOMBRE_SQL (top 20) ===
EstadoRevision_NOMBRE_SQL
None           18964
APROBADO        4712
DEVUELTO         333
EN REVISION       46
Name: count, dtype: int64

=== Ejemplos de ESTADO REVISION con TEXTO pero SIN clasificar (post-reglas) ===
No quedan textos sin clasificar; sólo nulos/vacíos.

Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\EstadoRevision_raw_y_normalizado.xlsx
Archivo de no clasificados generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\EstadoRevision_si

## OBSERVACIONES REVISOR

In [58]:
import pandas as pd
import numpy as np
import unicodedata
import re

# ============================================================
# OBSERVACIONES REVISOR (SIN RECARGAR df)
# - NO hace: df = df.copy()
# - Re-ejecución segura: elimina columnas derivadas previas in-place
# - Evita fragmentación: crea columnas en DataFrame auxiliar y asigna en un solo paso
# - Resto del flujo queda igual
# ============================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Re-ejecución segura (SIN recargar df): borrar columnas derivadas previas
cols_prev = [
    "ObservacionesRevisor_RAW",
    "ObservacionesRevisor_SQL",
    "ObservacionesRevisor_Len",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

def normalizar_header(s):
    if s is None:
        return ""
    s_norm = unicodedata.normalize("NFD", str(s))
    s_sin_tildes = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    return s_sin_tildes.upper().strip()

# 1) Detectar columna
candidatas_obs = [
    c for c in df.columns
    if "OBSERVACION" in normalizar_header(c) and "REVISOR" in normalizar_header(c)
]
print("Columnas candidatas para OBSERVACIONES REVISOR:", candidatas_obs)
if not candidatas_obs:
    raise ValueError("No se encontró la columna de OBSERVACIONES REVISOR en el DataFrame.")

# (opcional) priorizar encabezado exacto si existe
prior = [c for c in candidatas_obs if normalizar_header(c) == "OBSERVACIONES REVISOR"]
col_obs = prior[0] if prior else candidatas_obs[0]
print("Usando columna:", repr(col_obs))

# 2) Auxiliar
nuevas = pd.DataFrame(index=df.index)
nuevas["ObservacionesRevisor_RAW"] = df[col_obs]

# 3) Limpieza texto (robusta)
def limpiar_observacion(v):
    if pd.isna(v):
        return None
    t = str(v)

    # reemplazar NBSP y caracteres invisibles comunes
    t = t.replace("\u00A0", " ")  # NBSP
    t = t.replace("\u200B", "")   # zero width space
    t = t.replace("\ufeff", "")   # BOM

    t = t.strip()
    if t == "":
        return None

    # colapsar espacios y saltos a un solo espacio
    t = re.sub(r"\s+", " ", t).strip()
    return t

nuevas["ObservacionesRevisor_SQL"] = nuevas["ObservacionesRevisor_RAW"].apply(limpiar_observacion)
nuevas["ObservacionesRevisor_Len"] = nuevas["ObservacionesRevisor_SQL"].fillna("").astype(str).str.len()

# 4) Unir al df (SIN df = pd.concat(...))
df[nuevas.columns] = nuevas

# 5) Resumen
total = len(df)
con_texto = df["ObservacionesRevisor_SQL"].notna().sum()
sin_texto = df["ObservacionesRevisor_SQL"].isna().sum()
max_len = df["ObservacionesRevisor_Len"].max()

print("\n=== Resumen OBSERVACIONES REVISOR ===")
print(f"Total registros: {total}")
print(f"Con texto (SQL no nulo): {con_texto}")
print(f"Sin texto (SQL nulo)    : {sin_texto}")
print(f"Longitud máxima observada: {max_len}")

# 6) Umbral orientativo (solo auditoría)
UMBRAL = 4000
largas = df[df["ObservacionesRevisor_Len"] > UMBRAL]
if not largas.empty:
    print(f"\n=== ObservacionesRevisor que superan {UMBRAL} caracteres ===")
    cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]
    print(largas[cols_ctx + ["ObservacionesRevisor_Len", "ObservacionesRevisor_SQL"]].to_string(index=False))
else:
    print(f"\nNinguna observación supera los {UMBRAL} caracteres (ok para varchar(MAX)).")

# 7) Excel de revisión
base_path = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica"
ruta_excel = fr"{base_path}\ObservacionesRevisor_raw_y_normalizado.xlsx"

cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]
df_export = df[cols_ctx + [col_obs, "ObservacionesRevisor_RAW", "ObservacionesRevisor_SQL", "ObservacionesRevisor_Len"]].copy()
df_export.to_excel(ruta_excel, index=False)
print("\nArchivo de revisión generado en:")
print(ruta_excel)

print("\nColumna lista para migrar a ConceptoFinal.ObservacionesRevisor:", "ObservacionesRevisor_SQL")


Columnas candidatas para OBSERVACIONES REVISOR: ['OBSERVACIONES REVISOR']
Usando columna: 'OBSERVACIONES REVISOR'

=== Resumen OBSERVACIONES REVISOR ===
Total registros: 24055
Con texto (SQL no nulo): 4398
Sin texto (SQL nulo)    : 19657
Longitud máxima observada: 2586

Ninguna observación supera los 4000 caracteres (ok para varchar(MAX)).


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\604303789.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\604303789.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\604303789.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo


Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\ObservacionesRevisor_raw_y_normalizado.xlsx

Columna lista para migrar a ConceptoFinal.ObservacionesRevisor: ObservacionesRevisor_SQL


## "ENTREGÓ CARPETA DE SOPORTE (SI / NO)"


In [59]:
import pandas as pd
import numpy as np
import unicodedata
import re

# ============================================================
# ENTREGÓ CARPETA DE SOPORTES (SI/NO) (SIN RECARGAR df)
# - NO hace: df = ...
# - NO hace: df = df.loc[:, ...].copy()
# - Re-ejecución segura: elimina columnas derivadas previas in-place
# - El resto del flujo queda igual (mismas reglas y salidas)
# ============================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas derivadas previas (SIN recargar df)
cols_prev = [
    "EntregoCarpeta_RAW",
    "EntregoCarpeta_TXT_LIMPIO",
    "EntregoCarpetaSoportes_SQL",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# 0.2) (Opcional) detectar duplicados de columnas y reportar (SIN modificar df)
dup_cols = df.columns[df.columns.duplicated(keep="first")]
if len(dup_cols) > 0:
    print("[OJO] Hay columnas duplicadas en df (no se eliminan aquí):", list(dict.fromkeys(dup_cols)))

# =========================
# 1) Normalizar encabezados solo para BUSCAR la columna (NO cambia el RAW)
# =========================
def normalizar_header(s):
    if s is None:
        return ""
    s_norm = unicodedata.normalize("NFD", str(s))
    s_sin_tildes = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    s_sin_tildes = re.sub(r"\s+", " ", s_sin_tildes)
    return s_sin_tildes.upper().strip()

# 2) Ubicar la columna ENTREGÓ CARPETA DE SOPORTES (SI / NO)
candidatas = []
for c in df.columns:
    norm = normalizar_header(c)
    if "ENTREGO" in norm and "CARPETA" in norm and "SOPORTES" in norm:
        candidatas.append(c)

print("Columnas candidatas para ENTREGÓ CARPETA DE SOPORTES:", candidatas)
if not candidatas:
    raise ValueError("No se encontró la columna de ENTREGÓ CARPETA DE SOPORTES en el DataFrame.")

# priorizar encabezado exacto si existe
prior = [c for c in candidatas if normalizar_header(c) == "ENTREGO CARPETA DE SOPORTES (SI / NO)"]
col_entrega = prior[0] if prior else candidatas[0]
print("Usando columna:", repr(col_entrega))

# =========================
# 3) Construir columnas nuevas en auxiliar (evita fragmentación)
# =========================
nuevas = pd.DataFrame(index=df.index)

# RAW
nuevas["EntregoCarpeta_RAW"] = df[col_entrega]

# =========================
# 4) Texto limpio (robusto: NBSP + espacios/saltos)
# =========================
def limpiar_basico(v):
    if pd.isna(v):
        return None
    t = str(v)
    t = t.replace("\u00A0", " ").replace("\u200B", "").replace("\ufeff", "")
    t = t.strip()
    if t == "":
        return None
    t = re.sub(r"\s+", " ", t)
    return t

nuevas["EntregoCarpeta_TXT_LIMPIO"] = nuevas["EntregoCarpeta_RAW"].apply(limpiar_basico)

# =========================
# 5) Normalizar a SI / NO / PENDIENTE según reglas
# =========================
def quitar_tildes(v):
    if v is None:
        return None
    s_norm = unicodedata.normalize("NFD", str(v))
    return "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")

def es_fecha_iso(txt: str) -> bool:
    if txt is None:
        return False
    s = str(txt).strip()
    if s == "":
        return False
    return bool(re.match(r"^\d{4}-\d{2}-\d{2}(\s+\d{2}:\d{2}:\d{2})?$", s))

def normalizar_entrego(v):
    """
    Reglas acordadas:
    - NO SE REALIZÓ INFORME / NO SE GENERÓ INFORME -> NO
    - SIS -> PENDIENTE
    - SE REMITE PARA FIRMA -> SI
    - Fechas ISO (YYYY-MM-DD ...):
        * todas -> SI
        * excepto '2026-10-06 00:00:00' -> PENDIENTE
    - SI/NO/PENDIENTE (variantes) -> estándar
    - Cualquier otro texto -> NULL
    """
    if v is None:
        return None

    t0 = str(v).strip()
    if t0 == "":
        return None

    t = quitar_tildes(t0)
    t = re.sub(r"\s+", " ", t).strip().upper()

    if t in ("NO SE REALIZO INFORME", "NO SE GENERO INFORME"):
        return "NO"

    if t == "SIS":
        return "PENDIENTE"

    if t == "SE REMITE PARA FIRMA":
        return "SI"

    if es_fecha_iso(t0):
        if t0 == "2026-10-06 00:00:00":
            return "PENDIENTE"
        return "SI"

    if t in ("SI", "S", "SI."):
        return "SI"
    if t in ("NO", "N", "NO."):
        return "NO"
    if "PEND" in t:
        return "PENDIENTE"

    return None

nuevas["EntregoCarpetaSoportes_SQL"] = nuevas["EntregoCarpeta_TXT_LIMPIO"].apply(normalizar_entrego)

# =========================
# 6) Asignar al df (SIN df = concat / copy)
# =========================
df[nuevas.columns] = nuevas

# =========================
# 7) Resúmenes
# =========================
print("\n=== Valores CRUDOS ENTREGÓ CARPETA (top 30) ===")
print(df["EntregoCarpeta_RAW"].value_counts(dropna=False).head(30))

print("\n=== Valores NORMALIZADOS ENTREGÓ CARPETA (para SQL) ===")
print(df["EntregoCarpetaSoportes_SQL"].value_counts(dropna=False))

total = len(df)
no_nulos_sql = df["EntregoCarpetaSoportes_SQL"].notna().sum()
nulos_sql = df["EntregoCarpetaSoportes_SQL"].isna().sum()

print("\n=== Resumen nulos / no nulos EntregoCarpetaSoportes_SQL ===")
print(f"Total registros           : {total}")
print(f"Con valor en SQL (no nulo): {no_nulos_sql}")
print(f"Sin valor en SQL (nulo)   : {nulos_sql}")

# =========================
# 8) Auditoría: textos que quedan NULL (para ver qué falta)
# =========================
cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

mask_texto_y_null = (
    df["EntregoCarpeta_TXT_LIMPIO"].notna()
    & df["EntregoCarpeta_TXT_LIMPIO"].astype(str).str.strip().ne("")
    & df["EntregoCarpetaSoportes_SQL"].isna()
)

pendientes_rev = df.loc[
    mask_texto_y_null,
    cols_ctx + ["EntregoCarpeta_RAW", "EntregoCarpeta_TXT_LIMPIO"]
].copy()

print("\n=== Registros con TEXTO pero que quedan NULL (post-reglas) ===")
if pendientes_rev.empty:
    print("No quedan textos residuales; sólo SI/NO/PENDIENTE o nulos.")
else:
    print(pendientes_rev.head(80).to_string(index=False))
    print("\nTotal textos residuales en NULL:", len(pendientes_rev))

print("\nColumna lista para migrar a ConceptoFinal.EntregoCarpetaSoportes:", "EntregoCarpetaSoportes_SQL")


Columnas candidatas para ENTREGÓ CARPETA DE SOPORTES: ['ENTREGÓ CARPETA DE SOPORTES\n(SI / NO)']
Usando columna: 'ENTREGÓ CARPETA DE SOPORTES\n(SI / NO)'

=== Valores CRUDOS ENTREGÓ CARPETA (top 30) ===
EntregoCarpeta_RAW
NaN                       21393
SI                         2432
SI                          135
NO                           74
si                            9
2025-07-29 00:00:00           1
NO SE REALIZÓ INFORME         1
2025-10-03 00:00:00           1
2025-08-12 00:00:00           1
2026-10-06 00:00:00           1
2025-09-02 00:00:00           1
                              1
SIS                           1
SE REMITE PARA FIRMA          1
PENDIENTE                     1
NO SE GENERÓ INFORME          1
Si                            1
Name: count, dtype: int64

=== Valores NORMALIZADOS ENTREGÓ CARPETA (para SQL) ===
EntregoCarpetaSoportes_SQL
None         21394
SI            2582
NO              76
PENDIENTE        3
Name: count, dtype: int64

=== Resumen nulos / n

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1134591118.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1134591118.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1134591118.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi

## FECHA DE ENVÍO A COORDINACIÓN

In [60]:
import pandas as pd
import numpy as np
import unicodedata
import re

# ======================================================
# FECHA DE ENVÍO A COORDINACIÓN (SIN RECARGAR df)
# - NO hace: df = ...
# - Re-ejecución segura: borra columnas derivadas previas in-place
# - Mantiene el resto del flujo igual (misma lógica/prints/exports)
# ======================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas derivadas previas (SIN recargar df)
cols_prev = [
    "FechaEnvioCoord_RAW",
    "FechaEnvioCoord_TXT_LIMPIO",
    "FechaEnvioACoordinacion_SQL",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# 0.2) (Opcional) detectar duplicados de columnas y reportar (SIN modificar df)
dup_cols = df.columns[df.columns.duplicated(keep="first")]
if len(dup_cols) > 0:
    print("[OJO] Hay columnas duplicadas en df (no se eliminan aquí):", list(dict.fromkeys(dup_cols)))

# ======================================================
# 1) Funciones auxiliares
# ======================================================
def normalizar_header(s: str) -> str:
    if s is None:
        return ""
    s_norm = unicodedata.normalize("NFD", str(s))
    s_sin_tildes = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    s_sin_tildes = re.sub(r"\s+", " ", s_sin_tildes)
    return s_sin_tildes.upper().strip()

def limpiar_fecha_texto(v):
    """
    Limpia texto de fecha: maneja NO / NO REGISTRA / vacíos / NaT.
    Si trae texto que NO es fecha (SI, PENDIENTE, NO VIABLE, Enviado a firma, etc) -> None.
    """
    if pd.isna(v):
        return None

    t = str(v)
    t = t.replace("\u00A0", " ").replace("\u200B", "").replace("\ufeff", "")
    t = t.strip()
    if t == "":
        return None

    # Versión sin tildes y en mayúscula para detectar basura
    t_up = unicodedata.normalize("NFD", t)
    t_up = "".join(ch for ch in t_up if unicodedata.category(ch) != "Mn")
    t_up = re.sub(r"\s+", " ", t_up).strip().upper()

    # Casos que significan 'sin fecha'
    if t_up in ("NAT", "NONE"):
        return None
    if re.fullmatch(r"NO(\s+REGISTRA| APLICA| INFORMA)?\.?", t_up):
        return None
    if "SIN INFORMACION" in t_up or "SIN INFORMACIÓN" in t_up or "SIN REGISTRO" in t_up:
        return None

    # ===== REGLA NUEVA: si NO es fecha, se deja como None =====
    if t_up in ("SI", "PENDIENTE", "NO VIABLE", "NO/VIABLE", "ENVIADO A FIRMA", "ENVIADO/A/FIRMA"):
        return None

    # Normalizar separadores de fecha
    t_norm = t.replace("\\", "/").replace("-", "/").replace(".", "/")
    t_norm = re.sub(r"\s+DE\s+", "/", t_norm, flags=re.IGNORECASE)
    t_norm = re.sub(r"\s+", "/", t_norm)

    return t_norm

def parsear_fecha_flexible(txt):
    """Intenta convertir distintos formatos a Timestamp; si no, devuelve NaT."""
    if txt is None:
        return pd.NaT

    t = str(txt).strip()
    if t == "" or t.upper() == "NAT":
        return pd.NaT

    # 1) Intento directo
    fecha = pd.to_datetime(t, dayfirst=True, errors="coerce")
    if pd.notna(fecha):
        return fecha

    # 2) Formato sólo dígitos: DDMMYYYY
    solo_dig = re.sub(r"\D", "", t)
    if len(solo_dig) == 8:
        try:
            d = int(solo_dig[0:2])
            m = int(solo_dig[2:4])
            y = int(solo_dig[4:8])
            if 1 <= d <= 31 and 1 <= m <= 12:
                return pd.Timestamp(year=y, month=m, day=d)
        except ValueError:
            pass

    return pd.NaT

# ======================================================
# 2) Localizar la columna en el Excel
# ======================================================
candidatas_envio = []
for c in df.columns:
    norm = normalizar_header(c)
    if "FECHA" in norm and "ENVIO" in norm and "COORDINACION" in norm:
        candidatas_envio.append(c)

print("Columnas candidatas para FECHA DE ENVÍO A COORDINACIÓN:", candidatas_envio)
if not candidatas_envio:
    raise ValueError("No se encontró la columna de FECHA DE ENVÍO A COORDINACIÓN.")

col_envio = candidatas_envio[0]
print("Usando columna:", repr(col_envio))

# ======================================================
# 3) Crear columnas RAW / LIMPIA / SQL (en auxiliar y asignar)
# ======================================================
nuevas = pd.DataFrame(index=df.index)
nuevas["FechaEnvioCoord_RAW"] = df[col_envio]
nuevas["FechaEnvioCoord_TXT_LIMPIO"] = nuevas["FechaEnvioCoord_RAW"].apply(limpiar_fecha_texto)
nuevas["FechaEnvioACoordinacion_SQL"] = nuevas["FechaEnvioCoord_TXT_LIMPIO"].apply(parsear_fecha_flexible)

# Asignación sin recargar df
df[nuevas.columns] = nuevas

# ======================================================
# 4) Resumen de calidad
# ======================================================
total = len(df)
validas = df["FechaEnvioACoordinacion_SQL"].notna().sum()
invalidas_con_texto = (
    df["FechaEnvioACoordinacion_SQL"].isna()
    & df["FechaEnvioCoord_TXT_LIMPIO"].notna()
    & (df["FechaEnvioCoord_TXT_LIMPIO"].astype(str).str.strip() != "")
).sum()
sin_fecha = total - validas - invalidas_con_texto

print("\n=== Resumen FECHA DE ENVÍO A COORDINACIÓN ===")
print(f"Total filas: {total}")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto}")
print(f"Sin fecha (vacío o 'NO/NO REGISTRA/etc.'): {sin_fecha}")

# ======================================================
# 5) Auditoría: textos que aún quedan como texto
# ======================================================
cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

cols_muestra = cols_ctx + [
    "FechaEnvioCoord_RAW",
    "FechaEnvioCoord_TXT_LIMPIO",
    "FechaEnvioACoordinacion_SQL",
]

mask_invalidos = (
    df["FechaEnvioACoordinacion_SQL"].isna()
    & df["FechaEnvioCoord_TXT_LIMPIO"].notna()
    & (df["FechaEnvioCoord_TXT_LIMPIO"].astype(str).str.strip() != "")
)

invalidos = df.loc[mask_invalidos, cols_muestra].copy()

print("\n=== Filas con TEXTO que NO se pudo convertir (debería quedar vacío) ===")
if invalidos.empty:
    print("OK: no quedan textos raros, todo lo no-fecha quedó como NULL.")
else:
    print(invalidos.to_string(index=False))
    print("\nTotal filas con texto no parseado:", len(invalidos))

# ======================================================
# 6) Exportar a Excel para inspección visual
# ======================================================
df[cols_muestra].to_excel("FechaEnvioACoordinacion_todas_filas.xlsx", index=False)

if not invalidos.empty:
    invalidos.to_excel("FechaEnvioACoordinacion_invalidas_texto.xlsx", index=False)

print("\nArchivos generados:")
print(" - FechaEnvioACoordinacion_todas_filas.xlsx  (crudo + limpio + SQL)")
if not invalidos.empty:
    print(" - FechaEnvioACoordinacion_invalidas_texto.xlsx  (textos residuales)")

print("\nColumna lista para migrar a ConceptoFinal.FechaEnvioACoordinacion:", "FechaEnvioACoordinacion_SQL")


Columnas candidatas para FECHA DE ENVÍO A COORDINACIÓN: ['FECHA DE ENVÍO A COORDINACIÓN']
Usando columna: 'FECHA DE ENVÍO A COORDINACIÓN'


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2000989479.py:91: UserWarning: Parsing dates in %Y/%m/%d/%H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  fecha = pd.to_datetime(t, dayfirst=True, errors="coerce")
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2000989479.py:134: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2000989479.py:134: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fram


=== Resumen FECHA DE ENVÍO A COORDINACIÓN ===
Total filas: 24055
Fechas válidas (no NaT): 2544
Fechas inválidas (NaT con texto no vacío): 0
Sin fecha (vacío o 'NO/NO REGISTRA/etc.'): 21511

=== Filas con TEXTO que NO se pudo convertir (debería quedar vacío) ===
OK: no quedan textos raros, todo lo no-fecha quedó como NULL.

Archivos generados:
 - FechaEnvioACoordinacion_todas_filas.xlsx  (crudo + limpio + SQL)

Columna lista para migrar a ConceptoFinal.FechaEnvioACoordinacion: FechaEnvioACoordinacion_SQL


## ESTADO APROBACIÓN COORDINADORA


In [61]:
import pandas as pd
import numpy as np
import unicodedata
import re

# ======================================================
# ESTADO APROBACIÓN COORDINADORA (SIN RECARGAR df)
# - NO hace: df = ...
# - Re-ejecución segura: borra columnas derivadas previas in-place
# - Resto del flujo queda igual (misma lógica/prints/export)
# ======================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución: borrar columnas derivadas previas (SIN recargar df)
cols_prev = [
    "EstadoAprobCoord_RAW",
    "EstadoAprobacionCoordinadora_SQL",
    "EstadoAprobCoord_LEN",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# 0.2) (Opcional) detectar duplicados de columnas y reportar (SIN modificar df)
dup_cols = df.columns[df.columns.duplicated(keep="first")]
if len(dup_cols) > 0:
    print("[OJO] Hay columnas duplicadas en df (no se eliminan aquí):", list(dict.fromkeys(dup_cols)))

# ======================================================
# 1) Funciones auxiliares
# ======================================================
def normalizar_header(s: str) -> str:
    if s is None:
        return ""
    s_norm = unicodedata.normalize("NFD", str(s))
    s_sin_tildes = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    s_sin_tildes = re.sub(r"\s+", " ", s_sin_tildes)
    return s_sin_tildes.upper().strip()

def limpiar_estado_texto(v):
    """
    - NaN / vacío -> None
    - Texto -> sin tildes, espacios colapsados, MAYÚSCULAS
    - NO se mapea a catálogo (se migra como varchar(100))
    """
    if pd.isna(v):
        return None

    t = str(v)
    t = t.replace("\u00A0", " ").replace("\u200B", "").replace("\ufeff", "")
    t = t.strip()
    if t == "":
        return None

    t_norm = unicodedata.normalize("NFD", t)
    t_norm = "".join(ch for ch in t_norm if unicodedata.category(ch) != "Mn")
    t_norm = re.sub(r"\s+", " ", t_norm).strip().upper()

    return t_norm

# ======================================================
# 2) Localizar la columna "ESTADO APROBACIÓN COORDINADORA"
# ======================================================
norm_cols = {c: normalizar_header(c) for c in df.columns}

candidatas_estado_coord = [
    c for c, norm in norm_cols.items()
    if "ESTADO" in norm and "APROB" in norm and "COORDIN" in norm
]

print("Columnas candidatas para ESTADO APROBACIÓN COORDINADORA:", candidatas_estado_coord)
if not candidatas_estado_coord:
    raise ValueError("No se encontró la columna de ESTADO APROBACIÓN COORDINADORA en el DataFrame.")

col_estado = candidatas_estado_coord[0]
print("Usando columna:", repr(col_estado))

# ======================================================
# 3) Crear columnas RAW / SQL / LEN (en bloque para evitar fragmentación)
# ======================================================
nuevas = pd.DataFrame(index=df.index)

nuevas["EstadoAprobCoord_RAW"] = df[col_estado]
nuevas["EstadoAprobacionCoordinadora_SQL"] = nuevas["EstadoAprobCoord_RAW"].apply(limpiar_estado_texto)
nuevas["EstadoAprobCoord_LEN"] = nuevas["EstadoAprobacionCoordinadora_SQL"].fillna("").astype(str).str.len()

# Asignación sin recargar df
df[nuevas.columns] = nuevas

# ======================================================
# 4) Resúmenes
# ======================================================
print("\n=== Valores CRUDOS (top 30) ESTADO APROBACIÓN COORDINADORA ===")
print(df["EstadoAprobCoord_RAW"].value_counts(dropna=False).head(30))

print("\n=== Valores NORMALIZADOS (top 30, MAYÚSCULAS) ===")
print(df["EstadoAprobacionCoordinadora_SQL"].value_counts(dropna=False).head(30))

# Validación varchar(100)
largos = df[df["EstadoAprobCoord_LEN"] > 100]
if not largos.empty:
    cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]
    print("\n=== Filas donde el texto supera 100 caracteres (REVISAR / RECORTAR) ===")
    print(
        largos[
            cols_ctx
            + ["EstadoAprobCoord_RAW", "EstadoAprobacionCoordinadora_SQL", "EstadoAprobCoord_LEN"]
        ].to_string(index=False)
    )
else:
    print("\nNingún valor supera 100 caracteres (ok para varchar(100)).")

# Conteo nulos / no nulos
total = len(df)
no_nulos = df["EstadoAprobacionCoordinadora_SQL"].notna().sum()
nulos = df["EstadoAprobacionCoordinadora_SQL"].isna().sum()

print("\n=== Resumen nulos / no nulos EstadoAprobacionCoordinadora_SQL ===")
print(f"Total registros           : {total}")
print(f"Con valor en SQL (no nulo): {no_nulos}")
print(f"Sin valor en SQL (nulo)   : {nulos}")

# ======================================================
# 5) Exportar a Excel para inspección visual
# ======================================================
cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]
cols_excel = cols_ctx + [
    "EstadoAprobCoord_RAW",
    "EstadoAprobacionCoordinadora_SQL",
    "EstadoAprobCoord_LEN",
]

df[cols_excel].to_excel("EstadoAprobacionCoordinadora_sin_catalogo.xlsx", index=False)

print("\nArchivo generado:")
print(" - EstadoAprobacionCoordinadora_sin_catalogo.xlsx")

print("\nColumna lista para migrar a ConceptoFinal.EstadoAprobacionCoordinadora:",
      "EstadoAprobacionCoordinadora_SQL")


Columnas candidatas para ESTADO APROBACIÓN COORDINADORA: ['ESTADO APROBACIÓN COORDINADORA']
Usando columna: 'ESTADO APROBACIÓN COORDINADORA'

=== Valores CRUDOS (top 30) ESTADO APROBACIÓN COORDINADORA ===
EstadoAprobCoord_RAW
NaN                                  20653
APROBACIÓN COORDINACIÓN               1182
FIRMADO Y CARGADO EN ORFEO             634
APROBADO Y FIRMADO                     451
FIRMADO Y ENVIADO A CARGUE ORFEO       234
Enviado a cargue ORFEO                  85
FIRMADO Y ENVIADO A CARGUE              66
FIRMADO Y ENVIADO 21/11/24              58
APROBADO - CARGADO SIT 28/11/2024       41
FIRMADO Y ENVIADO 23/08/2024            38
FIRMADO Y ENVIADO  21/11/2024           38
FIRMADO Y ENVIADO                       37
APROBADO - CARGADO SIT 21/11/2024       31
FIRMADO Y ENVIADO 14/11/2024            20
FIRMADO Y ENVIADO  20/11/2024           15
FIRMADO Y ENVIADO 24/11/2024            14
02/12/2024: FIRMADO Y ENVIADO           13
FIRMADO Y ENVIADO PARA CARGUE           12


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2209380394.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2209380394.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2209380394.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider


Archivo generado:
 - EstadoAprobacionCoordinadora_sin_catalogo.xlsx

Columna lista para migrar a ConceptoFinal.EstadoAprobacionCoordinadora: EstadoAprobacionCoordinadora_SQL


## FECHA DE REMISIÓN SOPORTES A GESTORA DOCUMENTAL

In [62]:
import pandas as pd
import unicodedata
import re
from datetime import date, datetime

# ============================================================
# FECHA REMISIÓN SOPORTES A GESTORA DOCUMENTAL (SIN RECARGAR df)
# - NO hace: df = ...
# - Re-ejecución segura: borra columnas derivadas previas in-place
# - Resto del flujo queda igual (misma lógica/prints/export)
# ============================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Re-ejecución segura: borrar columnas derivadas previas (SIN recargar df)
cols_prev = [
    "FechaRemSoportes_RAW",
    "FechaRemSoportes_TXT_LIMPIO",
    "FechaRemisionSoportesAGestoraDocumental_SQL",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# 0.2) (Opcional) detectar duplicados de columnas y reportar (SIN modificar df)
dup_cols = df.columns[df.columns.duplicated(keep="first")]
if len(dup_cols) > 0:
    print("[OJO] Hay columnas duplicadas en df (no se eliminan aquí):", list(dict.fromkeys(dup_cols)))

# ============================================
# 1) Normalizar encabezados (solo para buscar)
# ============================================
def normalizar_header(s: str) -> str:
    if s is None:
        return ""
    s_norm = unicodedata.normalize("NFD", str(s))
    s_sin_tildes = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    s_sin_tildes = re.sub(r"\s+", " ", s_sin_tildes)
    return s_sin_tildes.upper().strip()

# ============================================
# 2) Localizar la columna FECHA REMISIÓN SOPORTES A GESTORA DOCUMENTAL
# ============================================
norm_cols = {c: normalizar_header(c) for c in df.columns}

candidatas_rem = [
    c for c, norm in norm_cols.items()
    if "FECHA" in norm and "SOPORTES" in norm and "GESTORA" in norm
]

if not candidatas_rem:
    candidatas_rem = [
        c for c, norm in norm_cols.items()
        if "FECHA" in norm and "REMISION" in norm and "SOPORTES" in norm
    ]

print("Columnas candidatas para FECHA REMISIÓN SOPORTES A GESTORA DOCUMENTAL:", candidatas_rem)
if not candidatas_rem:
    raise ValueError("No se encontró la columna de FECHA REMISIÓN SOPORTES A GESTORA DOCUMENTAL en el DataFrame.")

col_rem = candidatas_rem[0]
print("Usando columna:", repr(col_rem))

# ============================================
# 3) Limpieza base + correcciones puntuales acordadas
# ============================================
def corregir_formato_raro_fecha(t: str) -> str:
    """
    Corrige casos tipo:
      - '20/112024' -> '20/11/2024'
      - '2103/2024' -> '21/03/2024'
    """
    t = t.strip()

    m = re.match(r"^(\d{1,2})/(\d{2})(\d{4})$", t)  # dd/mm + aaaa pegado
    if m:
        d, mm, yyyy = m.groups()
        return f"{d}/{mm}/{yyyy}"

    m = re.match(r"^(\d{2})(\d{2})/(\d{4})$", t)  # ddmm/aaaa
    if m:
        dd, mm, yyyy = m.groups()
        return f"{dd}/{mm}/{yyyy}"

    return t

def limpiar_texto_fecha_soportes(v):
    """
    Reglas:
    - NaN/vacío -> None
    - 'FIRMADO Y CARGADO EN ORFEO' -> None (vacío)
    - Quita NBSP, apóstrofes raros y espacios
    - Corrige formatos raros + correcciones puntuales:
        * 20/08/20025  -> 20/08/2025
        * 19/08/20285  -> 19/08/2025
        * 07/04/205    -> 07/04/2025
        * ´19/08/2025  -> 19/08/2025
        * 20/08/22025  -> 20/08/2025
        * 01/0/2025    -> 01/01/2025
        * 01/08/205    -> 01/08/2025
        * 19/03/20225  -> 19/03/2025
        * 04/12/20255  -> 04/12/2025
    - Textos tipo NO REGISTRA/NO APLICA/etc -> None
    """
    if pd.isna(v):
        return None

    if isinstance(v, (pd.Timestamp, datetime, date)):
        return v

    t = str(v)
    t = t.replace("\u00A0", " ").replace("\u200B", "").replace("\ufeff", "")
    t = t.replace("´", "").replace("`", "").replace("’", "").replace("‘", "").replace("'", "")
    t = re.sub(r"\s+", " ", t).strip()

    if t == "":
        return None

    t_up = unicodedata.normalize("NFD", t)
    t_up = "".join(ch for ch in t_up if unicodedata.category(ch) != "Mn")
    t_up = re.sub(r"\s+", " ", t_up).strip().upper()

    # texto que se deja vacío
    if t_up == "FIRMADO Y CARGADO EN ORFEO":
        return None

    patrones_sin_fecha = [
        "NO REGISTRA",
        "NO APLICA",
        "NO APLICA.",
        "NO REPORTA",
        "NO TIENE",
        "NO",
        "N/A",
        "NA",
        "SIN FECHA",
        "SIN DATO",
        "NO INFORMA",
    ]
    if any(pat in t_up for pat in patrones_sin_fecha):
        return None

    # Correcciones puntuales del caso (antes de parsear)
    fixes = {
        "20/08/20025": "20/08/2025",
        "19/08/20285": "19/08/2025",
        "07/04/205":   "07/04/2025",
        "20/08/22025": "20/08/2025",
        "01/0/2025":   "01/01/2025",
        "01/08/205":   "01/08/2025",
        "19/03/20225": "19/03/2025",
        "04/12/20255": "04/12/2025",
    }
    if t in fixes:
        t = fixes[t]

    # Corrección general de formatos raros conocidos
    t = corregir_formato_raro_fecha(t)

    return t

# ============================================
# 4) Columnas RAW / LIMPIA / SQL (en bloque)
# ============================================
nuevas = pd.DataFrame(index=df.index)

nuevas["FechaRemSoportes_RAW"] = df[col_rem]
nuevas["FechaRemSoportes_TXT_LIMPIO"] = nuevas["FechaRemSoportes_RAW"].apply(limpiar_texto_fecha_soportes)

nuevas["FechaRemisionSoportesAGestoraDocumental_SQL"] = pd.to_datetime(
    nuevas["FechaRemSoportes_TXT_LIMPIO"],
    errors="coerce",
    dayfirst=True
)

# Asignación sin recargar df
df[nuevas.columns] = nuevas

# ============================================
# 5) Resumen (incluye SIN FECHA)
# ============================================
total = len(df)
validas = df["FechaRemisionSoportesAGestoraDocumental_SQL"].notna().sum()

mask_texto_no_vacio = (
    df["FechaRemSoportes_TXT_LIMPIO"].notna()
    & (df["FechaRemSoportes_TXT_LIMPIO"].astype(str).str.strip() != "")
)

invalidas_con_texto = (
    df["FechaRemisionSoportesAGestoraDocumental_SQL"].isna()
    & mask_texto_no_vacio
).sum()

sin_fecha = (
    df["FechaRemSoportes_TXT_LIMPIO"].isna()
    | (df["FechaRemSoportes_TXT_LIMPIO"].astype(str).str.strip() == "")
).sum()

print("\n=== Resumen FECHA DE REMISIÓN SOPORTES A GESTORA DOCUMENTAL ===")
print(f"Total filas: {total}")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto}")
print(f"Sin fecha (vacío o texto filtrado): {sin_fecha}")

# ============================================
# 6) Ejemplos de filas todavía inválidas
# ============================================
mask_invalid = (
    df["FechaRemisionSoportesAGestoraDocumental_SQL"].isna()
    & mask_texto_no_vacio
)

cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

ej_invalidas = df.loc[
    mask_invalid,
    cols_ctx + ["FechaRemSoportes_RAW", "FechaRemSoportes_TXT_LIMPIO"]
].head(120)

print("\n=== Filas con texto que NO se pudo parsear (post-correcciones) ===")
if ej_invalidas.empty:
    print("No hay filas con texto 'raro': todo lo que tiene texto se pudo convertir a fecha o quedó en vacío.")
else:
    print(ej_invalidas.to_string(index=False))
    print("\nTotal inválidas con texto:", int(mask_invalid.sum()))

# ============================================
# 7) Exportar a Excel
# ============================================
cols_ver = cols_ctx + [
    "FechaRemSoportes_RAW",
    "FechaRemSoportes_TXT_LIMPIO",
    "FechaRemisionSoportesAGestoraDocumental_SQL",
]

nombre_excel = "FechaRemisionSoportesGestora_raw_limpio_sql.xlsx"
df[cols_ver].to_excel(nombre_excel, index=False)

print("\nArchivo creado:")
print(nombre_excel)

if not ej_invalidas.empty:
    nombre_excel_inv = "FechaRemisionSoportesGestora_invalidas.xlsx"
    df.loc[mask_invalid, cols_ver].to_excel(nombre_excel_inv, index=False)
    print("Archivo con sólo filas inválidas:")
    print(nombre_excel_inv)

print("\nColumna lista para migrar a ConceptoFinal.FechaRemisionSoportesAGestoraDocumental:",
      "FechaRemisionSoportesAGestoraDocumental_SQL")


Columnas candidatas para FECHA REMISIÓN SOPORTES A GESTORA DOCUMENTAL: ['FECHA DE REMISIÓN SOPORTES A GESTORA DOCUMENTAL']
Usando columna: 'FECHA DE REMISIÓN SOPORTES A GESTORA DOCUMENTAL'

=== Resumen FECHA DE REMISIÓN SOPORTES A GESTORA DOCUMENTAL ===
Total filas: 24055
Fechas válidas (no NaT): 1318
Fechas inválidas (NaT con texto no vacío): 0
Sin fecha (vacío o texto filtrado): 22737

=== Filas con texto que NO se pudo parsear (post-correcciones) ===
No hay filas con texto 'raro': todo lo que tiene texto se pudo convertir a fecha o quedó en vacío.


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2816907875.py:179: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2816907875.py:179: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\2816907875.py:179: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi


Archivo creado:
FechaRemisionSoportesGestora_raw_limpio_sql.xlsx

Columna lista para migrar a ConceptoFinal.FechaRemisionSoportesAGestoraDocumental: FechaRemisionSoportesAGestoraDocumental_SQL


## FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL

In [63]:
import pandas as pd
import unicodedata
import re
from datetime import date, datetime

# ======================================================
# FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL (SIN RECARGAR df)
# - NO hace: df = ...
# - Re-ejecución segura: borra columnas derivadas previas in-place
# - Resto del flujo queda igual (misma lógica/prints/export)
# ======================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Re-ejecución segura: borrar columnas derivadas previas (SIN recargar df)
cols_prev = [
    "FechaRemInforme_RAW",
    "FechaRemInforme_TXT_LIMPIO",
    "FechaRemisionInformeAGestoraDocumental_SQL",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# 0.2) (Opcional) detectar duplicados de columnas y reportar (SIN modificar df)
dup_cols = df.columns[df.columns.duplicated(keep="first")]
if len(dup_cols) > 0:
    print("[OJO] Hay columnas duplicadas en df (no se eliminan aquí):", list(dict.fromkeys(dup_cols)))

# ============================================
# 1) Función para normalizar encabezados
# ============================================
def normalizar_header(s: str) -> str:
    if s is None:
        return ""
    s_norm = unicodedata.normalize("NFD", str(s))
    s_sin_tildes = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    s_sin_tildes = re.sub(r"\s+", " ", s_sin_tildes)
    return s_sin_tildes.upper().strip()

# ============================================
# 2) Localizar la columna
# ============================================
norm_cols = {c: normalizar_header(c) for c in df.columns}

candidatas_inf = [
    c for c, norm in norm_cols.items()
    if "FECHA" in norm and "INFORME" in norm and "GESTORA" in norm
]

# Plan B
if not candidatas_inf:
    candidatas_inf = [
        c for c, norm in norm_cols.items()
        if "FECHA" in norm and "INFORME" in norm
    ]

print("Columnas candidatas para FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL:", candidatas_inf)

if not candidatas_inf:
    raise ValueError("No se encontró la columna de FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL en el DataFrame.")

col_inf = candidatas_inf[0]
print("Usando columna:", repr(col_inf))

# ============================================
# 3) Funciones de limpieza
# ============================================
def corregir_formato_raro_fecha(t: str) -> str:
    """
    Corrige casos tipo:
      - '20/112024' -> '20/11/2024'
      - '17/052024' -> '17/05/2024'
      - '25/042024' -> '25/04/2024'
      - '2103/2024' -> '21/03/2024'
      - '26/085/2024' -> '26/08/2024'   (caso puntual)
      - 'dd/0mm/aaaa' (3 dígitos en mes con 0 adelante) -> recorta a 2 dígitos
    """
    t = t.strip()

    # Caso puntual exacto detectado
    if t == "26/085/2024":
        return "26/08/2024"

    # dd/mm + aaaa pegado: 20/112024 -> 20/11/2024
    m = re.match(r"^(\d{1,2})/(\d{2})(\d{4})$", t)
    if m:
        d, mm, yyyy = m.groups()
        return f"{d}/{mm}/{yyyy}"

    # ddmm/aaaa: 2103/2024 -> 21/03/2024
    m = re.match(r"^(\d{2})(\d{2})/(\d{4})$", t)
    if m:
        dd, mm, yyyy = m.groups()
        return f"{dd}/{mm}/{yyyy}"

    # Mes con 3 dígitos donde inicia con 0 (ej: 26/085/2024)
    # 085 -> 08
    m = re.match(r"^(\d{1,2})/(0\d{2})/(\d{4})$", t)
    if m:
        d, mm3, yyyy = m.groups()
        return f"{d}/{mm3[:2]}/{yyyy}"

    return t

def limpiar_texto_fecha_generica(v):
    """
    Limpieza estándar de fechas:
      - NaN -> None
      - Timestamps -> se devuelven tal cual
      - Textos tipo 'NO REGISTRA', 'NO APLICA', etc. -> None
      - Quita saltos de línea / espacios
      - Aplica corrección de formatos raros
    """
    if pd.isna(v):
        return None

    if isinstance(v, (pd.Timestamp, datetime, date)):
        return v

    t = str(v).replace("\u00A0", " ").strip()
    if t == "":
        return None

    t_upper = t.upper()

    patrones_sin_fecha = [
        "NO REGISTRA",
        "NO APLICA",
        "NO APLICA.",
        "NO REPORTA",
        "NO TIENE",
        "NO",
        "N/A",
        "NA",
        "SIN FECHA",
        "SIN DATO",
        "NO INFORMA",
        "NAT",
        "NONE",
    ]
    if any(pat in t_upper for pat in patrones_sin_fecha):
        return None

    t_limpio = re.sub(r"\s+", " ", t).strip()
    t_limpio = t_limpio.replace("\n", " ").replace("\t", " ").strip()

    t_limpio = corregir_formato_raro_fecha(t_limpio)
    return t_limpio

# ============================================
# 4) Crear columnas RAW / LIMPIO / SQL (en bloque, SIN concat a df)
# ============================================
nuevas = pd.DataFrame(index=df.index)

nuevas["FechaRemInforme_RAW"] = df[col_inf]
nuevas["FechaRemInforme_TXT_LIMPIO"] = nuevas["FechaRemInforme_RAW"].apply(limpiar_texto_fecha_generica)

nuevas["FechaRemisionInformeAGestoraDocumental_SQL"] = pd.to_datetime(
    nuevas["FechaRemInforme_TXT_LIMPIO"],
    errors="coerce",
    dayfirst=True
)

# Asignación sin recargar df
df[nuevas.columns] = nuevas

# ============================================
# 5) Resumen de calidad
# ============================================
total = len(df)
validas = df["FechaRemisionInformeAGestoraDocumental_SQL"].notna().sum()
invalidas_con_texto = (
    df["FechaRemisionInformeAGestoraDocumental_SQL"].isna()
    & df["FechaRemInforme_TXT_LIMPIO"].notna()
    & (df["FechaRemInforme_TXT_LIMPIO"].astype(str).str.strip() != "")
).sum()
sin_fecha = total - validas - invalidas_con_texto

print("\n=== Resumen FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL ===")
print(f"Total filas: {total}")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto}")
print(f"Sin fecha (vacío o 'NO/NO REGISTRA/etc.'): {sin_fecha}")

# ============================================
# 6) Mostrar inválidas (si quedan)
# ============================================
cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]
cols_ver = cols_ctx + [
    "FechaRemInforme_RAW",
    "FechaRemInforme_TXT_LIMPIO",
    "FechaRemisionInformeAGestoraDocumental_SQL",
]

mask_invalid = (
    df["FechaRemisionInformeAGestoraDocumental_SQL"].isna()
    & df["FechaRemInforme_TXT_LIMPIO"].notna()
    & (df["FechaRemInforme_TXT_LIMPIO"].astype(str).str.strip() != "")
)

invalidas = df.loc[mask_invalid, cols_ver]

print("\n=== Filas con texto que NO se pudo parsear (post-correcciones) ===")
if invalidas.empty:
    print("No hay filas con texto 'raro': todo lo que tiene texto se pudo convertir a fecha.")
else:
    print(invalidas.head(80).to_string(index=False))
    print("\nTotal filas con texto no parseado:", len(invalidas))

# ============================================
# 7) Exportar a Excel (todo + inválidas si existen)
# ============================================
base_path = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica"
ruta_todo = fr"{base_path}\FechaRemisionInformeGestora_raw_limpio_sql.xlsx"
df[cols_ver].to_excel(ruta_todo, index=False)
print("\nArchivo creado:")
print(ruta_todo)

if not invalidas.empty:
    ruta_inv = fr"{base_path}\FechaRemisionInformeGestora_invalidas.xlsx"
    invalidas.to_excel(ruta_inv, index=False)
    print("\nArchivo con sólo filas inválidas:")
    print(ruta_inv)

print("\nColumna lista para migrar a ConceptoFinal.FechaRemisiónInformeAGestoraDocumental:",
      "FechaRemisionInformeAGestoraDocumental_SQL")


Columnas candidatas para FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL: ['FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL']
Usando columna: 'FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL'

=== Resumen FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL ===
Total filas: 24055
Fechas válidas (no NaT): 1654
Fechas inválidas (NaT con texto no vacío): 0
Sin fecha (vacío o 'NO/NO REGISTRA/etc.'): 22401

=== Filas con texto que NO se pudo parsear (post-correcciones) ===
No hay filas con texto 'raro': todo lo que tiene texto se pudo convertir a fecha.


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3249787259.py:168: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3249787259.py:168: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nuevas.columns] = nuevas
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\3249787259.py:168: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi


Archivo creado:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\FechaRemisionInformeGestora_raw_limpio_sql.xlsx

Columna lista para migrar a ConceptoFinal.FechaRemisiónInformeAGestoraDocumental: FechaRemisionInformeAGestoraDocumental_SQL


*avance excel*

In [64]:
import pandas as pd

# ======================================================================================
# AVANCE A EXCEL (SIN RECARGAR df) - CONSOLIDADO HASTA HOY (incluye lo último trabajado)
# - NO hace: df = df.copy()
# - NO hace: df = pd.concat([...]) reasignando df
# - NO reprocesa nada: SOLO USA columnas YA calculadas en df (si existen)
# - Exporta:
#     1) RegistroProceso_avance
#     2) EstudioTerreno_avance
#     3) MedidaProcesal_listo_insertar (formato largo)
#     4) ConceptosPrevio_listo_insertar
#     5) ConceptoFinal_listo_insertar (incluye nuevos campos si existen)
#     6) Unificado_df (df completo)
#     7) Diccionario
# ======================================================================================

# 0) Asegurar fila Excel (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# --------------------------------------------------------------------------------------
# 1) Columnas contexto (solo si existen)
# --------------------------------------------------------------------------------------
cols_ctx = [
    "Fila_Excel_aprox",
    "IdPostulacion_SQL",
    "ID_SQL",
    "FMI", "FMI_SQL",
    "DEPARTAMENTO", "MUNICIPIO",
]
cols_ctx = [c for c in cols_ctx if c in df.columns]

# Llaves fallback (para cruce si aún no hay IdRegistroProceso_SQL)
keys_fallback = [c for c in ["IdPostulacion_SQL", "FMI_SQL"] if c in df.columns]
idrp_col = "IdRegistroProceso_SQL" if "IdRegistroProceso_SQL" in df.columns else None

# --------------------------------------------------------------------------------------
# 2) RegistroProceso_avance
# --------------------------------------------------------------------------------------
cols_registro = [
    "IdPostulacion_SQL",
    "FMI_SQL",
    "NumeroExpediente_SQL",
    "IdFuenteProceso",
    "IdFuenteProceso_SQL",
    "IdTipoProceso",
    "IdTipoProceso_SQL",
    "IdEtapaProcesal",
    "IdEtapaProcesal_SQL",
    "Dependencia_SQL",
    "RadicadoOrfeo_SQL",
]
cols_registro_exist = [c for c in cols_registro if c in df.columns]
df_registro = df[cols_ctx + cols_registro_exist].copy()

# --------------------------------------------------------------------------------------
# 3) EstudioTerreno_avance
# --------------------------------------------------------------------------------------
cols_estudio = [
    "IdRegistroProceso_SQL",
    "IdLocalizacion_SQL",
    "AreaRegistral_SQL",
    "CirculoRegistral_SQL",
    "AreaCalculada_SQL",
    "TipoPersonaTitular_SQL",
    "NombrePropietario_SQL",
    "ApellidoPropietario_SQL",
    "Identificacion_SQL_TXT_MIGRAR",
    "IdTipoDocumento_SQL_MIGRAR",
    "NaturalezaJuridica_SQL_FINAL",
    "AcreditacionPropiedad_SQL_FINAL",
]
cols_estudio_exist = [c for c in cols_estudio if c in df.columns]
df_estudio = df[cols_ctx + cols_estudio_exist].copy()

# --------------------------------------------------------------------------------------
# 4) MedidaProcesal_listo_insertar (formato largo) - SOLO LEE resultados ya calculados
# --------------------------------------------------------------------------------------
base_cols_mp = ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"]
base_cols_mp = [c for c in base_cols_mp if c in df.columns]

mp_parts = []

def add_mp(objeto: str, col_valor_final: str, col_anot: str, col_tipo: str = None,
           col_estado: str = None, estados_ok=None):
    if col_valor_final not in df.columns:
        return
    if col_anot not in df.columns:
        return

    valor = df[col_valor_final]
    anot = df[col_anot]

    if (col_estado is not None) and (col_estado in df.columns) and estados_ok:
        mask = df[col_estado].isin(estados_ok)
    else:
        mask = valor.notna()

    if not mask.any():
        return

    out = df.loc[mask, base_cols_mp].copy()
    out["Valor"] = valor.loc[mask].values
    out["Anotacion"] = anot.loc[mask].values

    if col_tipo and (col_tipo in df.columns):
        out["TipoClasificacion"] = df.loc[mask, col_tipo].values
    else:
        out["TipoClasificacion"] = None

    out["Objeto"] = objeto
    mp_parts.append(out)

# Objetos MedidaProcesal (los que ya vienes usando)
add_mp("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL", None, "HIPOTECA_ESTADO", ["ok", "sin_reg"])
add_mp("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL", None, "SERVIDUMBRES_ESTADO", ["ok", "sin_reg"])
add_mp("MEDIDAS CAUTELARES", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL", None, "MEDIDAS_CAUT_ESTADO", ["ok", "sin_reg"])
add_mp("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL", "RUPTA_ESTADO", ["ok", "sin_reg"])
add_mp("RTDAF LEY 1448", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL", None, "RTDAF_ESTADO", ["ok", "sin_reg"])
add_mp("OFERTA OTRAS ENTIDADES", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL", None, "OFERTA_ESTADO", ["ok", "sin_reg"])
add_mp("PROCESOS DE CLARIFICACIÓN", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL", None, "PROC_CLARIF_ESTADO", ["ok", "sin_reg"])

if mp_parts:
    df_medidaprocesal = pd.concat(mp_parts, ignore_index=True)
else:
    df_medidaprocesal = pd.DataFrame(columns=base_cols_mp + ["Valor", "Anotacion", "TipoClasificacion", "Objeto"])

sort_cols = [c for c in ["DEPARTAMENTO", "MUNICIPIO", "FMI", "Objeto"] if c in df_medidaprocesal.columns]
if sort_cols and not df_medidaprocesal.empty:
    df_medidaprocesal = df_medidaprocesal.sort_values(sort_cols).reset_index(drop=True)

# --------------------------------------------------------------------------------------
# 5) ConceptosPrevio_listo_insertar (SOLO resultados ya calculados)
# --------------------------------------------------------------------------------------
cols_cp_src = ["FechaInformePrevio_SQL", "ConceptoPrevio_SQL"]
cols_cp_src_exist = [c for c in cols_cp_src if c in df.columns]

if cols_cp_src_exist:
    mask_cp = pd.Series(False, index=df.index)
    for c in cols_cp_src_exist:
        mask_cp = mask_cp | df[c].notna()

    cols_cp = cols_ctx + (["IdRegistroProceso_SQL"] if idrp_col else []) + keys_fallback + cols_cp_src_exist
    cols_cp = list(dict.fromkeys([c for c in cols_cp if c in df.columns]))

    df_conceptosprevio = df.loc[mask_cp, cols_cp].copy()

    ren_cp = {}
    if idrp_col and ("IdRegistroProceso_SQL" in df_conceptosprevio.columns):
        ren_cp["IdRegistroProceso_SQL"] = "IdRegistroProceso"
    if "FechaInformePrevio_SQL" in df_conceptosprevio.columns:
        ren_cp["FechaInformePrevio_SQL"] = "FechaInforme"
    if "ConceptoPrevio_SQL" in df_conceptosprevio.columns:
        ren_cp["ConceptoPrevio_SQL"] = "Concepto"

    df_conceptosprevio.rename(columns=ren_cp, inplace=True)
else:
    df_conceptosprevio = pd.DataFrame()

# --------------------------------------------------------------------------------------
# 6) ConceptoFinal_listo_insertar (incluye lo último trabajado si existe en df)
# --------------------------------------------------------------------------------------
cols_cf_map = [
    # Base ConceptoFinal (ya venías exportando)
    ("ConceptoActualDeViabilidadJuridica_SQL", "ConceptoActualDeViabilidadJuridica"),
    ("FechaInforme_SQL", "FechaInforme"),
    ("IdTipoViabilidad_SQL", "IdTipoViabilidad"),
    ("IdTipoInforme_SQL", "IdTipoInforme"),
    ("IdTipoNoViabilidad_SQL", "IdTipoNoViabilidad"),
    ("InsumosPendientes_SQL", "InsumosPendientes"),
    ("FechaEntregaARevisor_SQL", "FechaEntregaARevisor"),

    # NUEVOS (los últimos trabajados)
    ("AbogadoSustanciadorAsignado_SQL", "AbogadoSustanciadorAsignado"),
    ("AbogadoRevisorAsignado_SQL", "AbogadoRevisorAsignado"),
    ("NroReparto_SQL", "NroReparto"),
    ("FechaAsignacionReparto_SQL", "FechaAsignacionReparto"),
    ("FechaPlazoEntregaARevisor_SQL", "FechaPlazoEntregaARevisor"),
    ("IdTipoEstadoRevision_SQL", "IdTipoEstadoRevision"),
    ("ObservacionesRevisor_SQL", "ObservacionesRevisor"),
    ("EntregoCarpetaSoportes_SQL", "EntregoCarpetaSoportes"),
    ("FechaEnvioACoordinacion_SQL", "FechaEnvioACoordinacion"),
    ("EstadoAprobacionCoordinadora_SQL", "EstadoAprobacionCoordinadora"),
    ("FechaRemisionSoportesAGestoraDocumental_SQL", "FechaRemisionSoportesAGestoraDocumental"),
    ("FechaRemisionInformeAGestoraDocumental_SQL", "FechaRemisionInformeAGestoraDocumental"),
]

cols_cf_src_exist = [src for (src, _) in cols_cf_map if src in df.columns]

if cols_cf_src_exist:
    mask_cf = pd.Series(False, index=df.index)
    for c in cols_cf_src_exist:
        mask_cf = mask_cf | df[c].notna()

    cols_cf = cols_ctx + (["IdRegistroProceso_SQL"] if idrp_col else []) + keys_fallback + cols_cf_src_exist
    cols_cf = list(dict.fromkeys([c for c in cols_cf if c in df.columns]))

    df_conceptofinal = df.loc[mask_cf, cols_cf].copy()

    ren_cf = {}
    if idrp_col and ("IdRegistroProceso_SQL" in df_conceptofinal.columns):
        ren_cf["IdRegistroProceso_SQL"] = "IdRegistroProceso"
    for src, dst in cols_cf_map:
        if src in df_conceptofinal.columns:
            ren_cf[src] = dst

    df_conceptofinal.rename(columns=ren_cf, inplace=True)
else:
    df_conceptofinal = pd.DataFrame()

# --------------------------------------------------------------------------------------
# 7) Unificado_df (df completo para trazabilidad)
# --------------------------------------------------------------------------------------
df_unificado = df  # referencia directa; NO recarga df

# --------------------------------------------------------------------------------------
# 8) Diccionario (hoja -> columnas)
# --------------------------------------------------------------------------------------
dicc = []
for c in cols_registro_exist:
    dicc.append(["RegistroProceso_avance", c])
for c in cols_estudio_exist:
    dicc.append(["EstudioTerreno_avance", c])

# MedidaProcesal
for c in (base_cols_mp + ["Valor", "Anotacion", "TipoClasificacion", "Objeto"]):
    dicc.append(["MedidaProcesal_listo_insertar", c])

# ConceptosPrevio
for c in df_conceptosprevio.columns:
    dicc.append(["ConceptosPrevio_listo_insertar", c])

# ConceptoFinal
for c in df_conceptofinal.columns:
    dicc.append(["ConceptoFinal_listo_insertar", c])

# Unificado
dicc.append(["Unificado_df", f"{len(df_unificado.columns)} columnas"])

df_dicc = pd.DataFrame(dicc, columns=["Hoja", "Columna"])

# --------------------------------------------------------------------------------------
# 9) Exportar Excel consolidado
# --------------------------------------------------------------------------------------
out_xlsx = "AVANCE_MIGRACION_HASTA_HOY_ACTUALIZADO.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    df_registro.to_excel(writer, index=False, sheet_name="RegistroProceso_avance")
    df_estudio.to_excel(writer, index=False, sheet_name="EstudioTerreno_avance")
    df_medidaprocesal.to_excel(writer, index=False, sheet_name="MedidaProcesal_listo_insertar")
    df_conceptosprevio.to_excel(writer, index=False, sheet_name="ConceptosPrevio_listo_insertar")
    df_conceptofinal.to_excel(writer, index=False, sheet_name="ConceptoFinal_listo_insertar")
    df_unificado.to_excel(writer, index=False, sheet_name="Unificado_df")
    df_dicc.to_excel(writer, index=False, sheet_name="Diccionario")

# --------------------------------------------------------------------------------------
# 10) Logs + faltantes (para que sepas qué no está en df)
# --------------------------------------------------------------------------------------
print("Excel creado:", out_xlsx)
print("Hojas: RegistroProceso_avance, EstudioTerreno_avance, MedidaProcesal_listo_insertar, "
      "ConceptosPrevio_listo_insertar, ConceptoFinal_listo_insertar, Unificado_df, Diccionario")
print("Filas RegistroProceso_avance:", len(df_registro))
print("Filas EstudioTerreno_avance:", len(df_estudio))
print("Filas MedidaProcesal_listo_insertar:", len(df_medidaprocesal))
print("Filas ConceptosPrevio_listo_insertar:", len(df_conceptosprevio))
print("Filas ConceptoFinal_listo_insertar:", len(df_conceptofinal))
print("Filas Unificado_df:", len(df_unificado))
print("Columnas Unificado_df:", len(df_unificado.columns))

# Lista clara de campos esperados en ConceptoFinal y cuáles faltan en df
esperados_cf = [src for (src, _) in cols_cf_map]
faltantes_cf = [c for c in esperados_cf if c not in df.columns]
if faltantes_cf:
    print("\n[OJO] Estos campos NO existen en df (por eso no salen en ConceptoFinal):")
    for c in faltantes_cf:
        print(" -", c)

if idrp_col is None:
    print("\n[OJO] No existe 'IdRegistroProceso_SQL' en df. ConceptosPrevio/ConceptoFinal se exportan con llaves "
          "fallback (IdPostulacion_SQL / FMI_SQL) para cruce en migración.")


Excel creado: AVANCE_MIGRACION_HASTA_HOY_ACTUALIZADO.xlsx
Hojas: RegistroProceso_avance, EstudioTerreno_avance, MedidaProcesal_listo_insertar, ConceptosPrevio_listo_insertar, ConceptoFinal_listo_insertar, Unificado_df, Diccionario
Filas RegistroProceso_avance: 24055
Filas EstudioTerreno_avance: 24055
Filas MedidaProcesal_listo_insertar: 69662
Filas ConceptosPrevio_listo_insertar: 5869
Filas ConceptoFinal_listo_insertar: 24055
Filas Unificado_df: 24055
Columnas Unificado_df: 311

[OJO] No existe 'IdRegistroProceso_SQL' en df. ConceptosPrevio/ConceptoFinal se exportan con llaves fallback (IdPostulacion_SQL / FMI_SQL) para cruce en migración.


## FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO

In [65]:
import numpy as np
import pandas as pd
import re

# ============================================================
# FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO (SIN RECARGAR df)
# - NO hace: df = df.copy()
# - Evita duplicados por re-ejecución: borra columnas derivadas previas in-place
# - Resto del flujo queda igual (misma lógica, mismos excels)
# ============================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución (SIN recargar df)
cols_prev = [
    "FechaCargueInformeOrfeo_RAW",
    "FechaCargueInformeOrfeo_TXT_LIMPIO",
    "FechaCargueInformeOrfeo_TXT_UP",
    "FechaCargueInformeJuridicoExpOrfeo_SQL",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# ============================================
# 1) Ubicar la columna de FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO
# ============================================
candidatas_cargue = [
    c for c in df.columns
    if ('FECHA'    in str(c).upper()
        and 'CARG' in str(c).upper()     # CARGUE / CARGADO
        and 'INFORME' in str(c).upper()
        and 'ORFEO' in str(c).upper())
]

print("Columnas candidatas para FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO:", candidatas_cargue)

if candidatas_cargue:
    col_cargue_orfeo = candidatas_cargue[0]
else:
    # Ajusta este nombre si en tu df.columns ves algo distinto
    col_cargue_orfeo = 'FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO'

print("Usando columna:", col_cargue_orfeo)

# ============================================
# 2) Columnas auxiliares RAW / TEXTO LIMPIO
# ============================================
df['FechaCargueInformeOrfeo_RAW'] = df[col_cargue_orfeo]

df['FechaCargueInformeOrfeo_TXT_LIMPIO'] = (
    df['FechaCargueInformeOrfeo_RAW']
      .astype(str)
      .str.strip()
)

# Normalizar vacíos típicos
mask_vacio = df['FechaCargueInformeOrfeo_TXT_LIMPIO'].str.upper().isin(['', 'NAN', 'NAT'])
df.loc[mask_vacio, 'FechaCargueInformeOrfeo_TXT_LIMPIO'] = np.nan

df['FechaCargueInformeOrfeo_TXT_UP'] = (
    df['FechaCargueInformeOrfeo_TXT_LIMPIO']
      .astype(str)
      .str.strip()
      .str.upper()
)

# ============================================
# 3) Función de parseo a datetime
# ============================================
def parse_fecha_cargue_orfeo(valor):
    """
    Convierte a datetime valores tipo fecha.
    Deja como NaT textos como 'NO REGISTRA', 'NO APLICA',
    'FIRMADO Y CARGADO EN ORFEO', etc.
    """
    if pd.isna(valor):
        return pd.NaT

    txt = str(valor).strip()
    if txt == "":
        return pd.NaT

    tu = txt.upper()

    # Textos claramente NO fecha -> NaT
    if tu in ("NO", "NO REGISTRA", "NO APLICA", "N/A"):
        return pd.NaT
    if "FIRMADO Y CARGADO EN ORFEO" in tu:
        return pd.NaT
    if "CARGADO EN ORFEO" in tu and not any(ch.isdigit() for ch in tu):
        return pd.NaT

    # Unificar separadores raros a "/"
    t = re.sub(r"[-.]", "/", txt)
    t = re.sub(r"\s+", "/", t)  # "2024 04 25" -> "2024/04/25"

    # Intento genérico con dayfirst
    fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')

    return fecha  # si no puede, queda NaT

# ============================================
# 4) Columna final para SQL
# ============================================
df['FechaCargueInformeJuridicoExpOrfeo_SQL'] = (
    df['FechaCargueInformeOrfeo_TXT_LIMPIO'].apply(parse_fecha_cargue_orfeo)
)

# ============================================
# 5) Resumen y muestras (válidos e inválidos)
# ============================================
mask_valida = df['FechaCargueInformeJuridicoExpOrfeo_SQL'].notna()

mask_con_texto = (
    df['FechaCargueInformeOrfeo_TXT_LIMPIO'].notna()
    & (df['FechaCargueInformeOrfeo_TXT_LIMPIO'].astype(str).str.strip() != '')
)

mask_invalida_con_texto = ~mask_valida & mask_con_texto

validas = int(mask_valida.sum())
invalidas_con_texto = int(mask_invalida_con_texto.sum())

print("\n=== Resumen FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO ===")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto}")

# Muestra de filas listas para migrar (válidas)
cols_ctx = [c for c in ['Fila_Excel_aprox','FMI','DEPARTAMENTO','MUNICIPIO'] if c in df.columns]

cols_validos = cols_ctx + [
    col_cargue_orfeo,
    'FechaCargueInformeOrfeo_TXT_LIMPIO',
    'FechaCargueInformeJuridicoExpOrfeo_SQL'
]
cols_validos = [c for c in cols_validos if c in df.columns]

ej_validos = df.loc[mask_valida, cols_validos].head(30)

print("\n=== Ejemplos de filas LISTAS PARA MIGRAR (fechas válidas) ===")
if not ej_validos.empty:
    print(ej_validos.to_string(index=False))
else:
    print("No hay fechas válidas para mostrar.")

# Muestra de filas con texto que NO se pudo convertir
cols_invalidos = cols_ctx + [
    'FechaCargueInformeOrfeo_RAW',
    'FechaCargueInformeOrfeo_TXT_LIMPIO'
]
cols_invalidos = [c for c in cols_invalidos if c in df.columns]

ej_invalidos = df.loc[mask_invalida_con_texto, cols_invalidos].head(50)

print("\n=== Ejemplos de filas con TEXTO que NO se pudo convertir (para revisar) ===")
if not ej_invalidos.empty:
    print(ej_invalidos.to_string(index=False))
else:
    print("No hay filas con texto no vacío que sigan inválidas.")

# ============================================
# 6) Exportar Excels
# ============================================

# 6.1. Excel con TODOS los registros (crudo vs limpio vs SQL)
cols_export_todos = cols_ctx + [
    col_cargue_orfeo,                          # columna original del Excel
    'FechaCargueInformeOrfeo_RAW',             # valor crudo
    'FechaCargueInformeOrfeo_TXT_LIMPIO',      # texto limpio
    'FechaCargueInformeJuridicoExpOrfeo_SQL'   # fecha final para SQL
]
cols_export_todos = [c for c in cols_export_todos if c in df.columns]

df_export_todos = df[cols_export_todos].copy()
df_export_todos.to_excel(
    'FechaCargueInforme_Juridico_ORFEO_raw_vs_sql.xlsx',
    index=False
)
print("\nArchivo creado (todos): FechaCargueInforme_Juridico_ORFEO_raw_vs_sql.xlsx")

# 6.2. Excel SOLO con registros inválidos (texto que no se pudo convertir)
df_export_invalidos = df.loc[mask_invalida_con_texto, cols_export_todos].copy()
df_export_invalidos.to_excel(
    'FechaCargueInforme_Juridico_ORFEO_invalidos.xlsx',
    index=False
)
print("Archivo creado (inválidos): FechaCargueInforme_Juridico_ORFEO_invalidos.xlsx")

print("\nColumna lista para migrar a ConceptoFinal.FechaCargueInformeJurídicoExpOrfeo:",
      "FechaCargueInformeJuridicoExpOrfeo_SQL")


Columnas candidatas para FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO: ['FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO']
Usando columna: FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4123126883.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaCargueInformeOrfeo_RAW'] = df[col_cargue_orfeo]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4123126883.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaCargueInformeOrfeo_TXT_LIMPIO'] = (
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4123126883.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man


=== Resumen FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO ===
Fechas válidas (no NaT): 3372
Fechas inválidas (NaT con texto no vacío): 0

=== Ejemplos de filas LISTAS PARA MIGRAR (fechas válidas) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO                   MUNICIPIO FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO FechaCargueInformeOrfeo_TXT_LIMPIO FechaCargueInformeJuridicoExpOrfeo_SQL
                4  062-37361      BOLIVAR                    ZAMBRANO                          2024-11-21 00:00:00                2024-11-21 00:00:00                             2024-11-21
                5  062-37354      BOLIVAR                    ZAMBRANO                          2025-02-25 00:00:00                2025-02-25 00:00:00                             2025-02-25
                7  062-37360      BOLIVAR                    ZAMBRANO                          2024-11-06 00:00:00                2024-11-06 00:00:00                             2024-06-11
                8  062-37356      BOLIVAR   

## FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO

In [66]:
import numpy as np
import pandas as pd
import re

# ============================================================
# FECHA DE CARGUE DOCS Y SOPORTES EN EXPDTE ORFEO (SIN RECARGAR df)
# - NO hace: df = df.copy()
# - Re-ejecución segura: borra columnas derivadas previas in-place
# - Resto queda igual (misma lógica, mismos excels)
# ============================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución (SIN recargar df)
cols_prev = [
    "FechaCargueDocsSoportesOrfeo_RAW",
    "FechaCargueDocsSoportesOrfeo_TXT_LIMPIO",
    "FechaCargueDocsSoportesOrfeo_TXT_UP",
    "FechaDeCargueDocsYSoportesExpOrfeo_SQL",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# ============================================
# 1) Ubicar la columna de FECHA DE CARGUE DOCS Y SOPORTES EN EXPDTE ORFEO
# ============================================
candidatas_docs_orfeo = [
    c for c in df.columns
    if ('FECHA'   in str(c).upper()
        and 'CARG'   in str(c).upper()        # CARGUE / CARGADO
        and ('DOC'   in str(c).upper() or 'SOPORT' in str(c).upper())
        and 'ORFEO'  in str(c).upper())
]

print(
    "Columnas candidatas para FECHA DE CARGUE DOCUMENTOS Y SOPORTES EN EXPDTE ORFEO:",
    candidatas_docs_orfeo
)

if candidatas_docs_orfeo:
    col_cargue_docs_orfeo = candidatas_docs_orfeo[0]
else:
    # Ajusta este nombre si en tu df.columns ves algo distinto
    col_cargue_docs_orfeo = 'FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO'

print("Usando columna:", col_cargue_docs_orfeo)

# ============================================
# 2) Columnas auxiliares RAW / TEXTO LIMPIO
# ============================================
df['FechaCargueDocsSoportesOrfeo_RAW'] = df[col_cargue_docs_orfeo]

df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'] = (
    df['FechaCargueDocsSoportesOrfeo_RAW']
      .astype(str)
      .str.strip()
)

# Normalizar vacíos típicos
mask_vacio = df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'].str.upper().isin(['', 'NAN', 'NAT'])
df.loc[mask_vacio, 'FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'] = np.nan

df['FechaCargueDocsSoportesOrfeo_TXT_UP'] = (
    df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO']
      .astype(str)
      .str.strip()
      .str.upper()
)

# ============================================
# 3) Función de parseo a datetime
# ============================================
def parse_fecha_cargue_docs_orfeo(valor):
    """
    Convierte a datetime valores tipo fecha para
    FECHA DE CARGUE DOCUMENTOS Y SOPORTES EN EXPDTE ORFEO.
    Deja como NaT textos como 'NO REGISTRA', 'NO APLICA',
    'FIRMADO Y CARGADO EN ORFEO', etc.
    """
    if pd.isna(valor):
        return pd.NaT

    txt = str(valor).strip()
    if txt == "":
        return pd.NaT

    tu = txt.upper()

    # Textos claramente NO fecha -> NaT
    if tu in ("NO", "NO REGISTRA", "NO APLICA", "N/A"):
        return pd.NaT
    if "FIRMADO Y CARGADO EN ORFEO" in tu:
        return pd.NaT
    if "CARGADO EN ORFEO" in tu and not any(ch.isdigit() for ch in tu):
        return pd.NaT

    # Unificar separadores raros a "/"
    t = re.sub(r"[-.]", "/", txt)
    t = re.sub(r"\s+", "/", t)  # "2024 04 25" -> "2024/04/25"

    # Intento genérico con dayfirst
    fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')

    return fecha  # si no puede, queda NaT

# ============================================
# 4) Columna final para SQL
# ============================================
df['FechaDeCargueDocsYSoportesExpOrfeo_SQL'] = (
    df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'].apply(parse_fecha_cargue_docs_orfeo)
)

# ============================================
# 5) Resumen y muestras (válidos e inválidos)
# ============================================
mask_valida = df['FechaDeCargueDocsYSoportesExpOrfeo_SQL'].notna()

mask_con_texto = (
    df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'].notna()
    & (df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'].astype(str).str.strip() != '')
)

mask_invalida_con_texto = ~mask_valida & mask_con_texto

validas = int(mask_valida.sum())
invalidas_con_texto = int(mask_invalida_con_texto.sum())

print("\n=== Resumen FECHA DE CARGUE DOCUMENTOS Y SOPORTES EN EXPDTE ORFEO ===")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto}")

# Muestra de filas listas para migrar (válidas)
cols_ctx = [c for c in ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO'] if c in df.columns]

cols_validos = cols_ctx + [
    col_cargue_docs_orfeo,
    'FechaCargueDocsSoportesOrfeo_TXT_LIMPIO',
    'FechaDeCargueDocsYSoportesExpOrfeo_SQL'
]
cols_validos = [c for c in cols_validos if c in df.columns]

ej_validos = df.loc[mask_valida, cols_validos].head(30)

print("\n=== Ejemplos de filas LISTAS PARA MIGRAR (fechas válidas) ===")
if not ej_validos.empty:
    print(ej_validos.to_string(index=False))
else:
    print("No hay fechas válidas para mostrar.")

# Muestra de filas con texto que NO se pudo convertir
cols_invalidos = cols_ctx + [
    'FechaCargueDocsSoportesOrfeo_RAW',
    'FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'
]
cols_invalidos = [c for c in cols_invalidos if c in df.columns]

ej_invalidos = df.loc[mask_invalida_con_texto, cols_invalidos].head(50)

print("\n=== Ejemplos de filas con TEXTO que NO se pudo convertir (para revisar) ===")
if not ej_invalidos.empty:
    print(ej_invalidos.to_string(index=False))
else:
    print("No hay filas con texto no vacío que sigan inválidas.")

# ============================================
# 6) Exportar Excels (ORFEO, docs + soportes)
# ============================================

cols_export_todos = cols_ctx + [
    col_cargue_docs_orfeo,                       # columna original del Excel
    'FechaCargueDocsSoportesOrfeo_RAW',          # valor crudo
    'FechaCargueDocsSoportesOrfeo_TXT_LIMPIO',   # texto limpio
    'FechaDeCargueDocsYSoportesExpOrfeo_SQL'     # fecha final para SQL
]
cols_export_todos = [c for c in cols_export_todos if c in df.columns]

# 6.1. Excel con TODOS los registros (crudo vs limpio vs SQL)
df_export_todos = df[cols_export_todos].copy()
df_export_todos.to_excel(
    'FechaCargue_Docs_Soportes_ORFEO_raw_vs_sql.xlsx',
    index=False
)
print("\nArchivo creado (todos): FechaCargue_Docs_Soportes_ORFEO_raw_vs_sql.xlsx")

# 6.2. Excel SOLO con registros inválidos (texto que no se pudo convertir)
df_export_invalidos = df.loc[mask_invalida_con_texto, cols_export_todos].copy()
df_export_invalidos.to_excel(
    'FechaCargue_Docs_Soportes_ORFEO_invalidos.xlsx',
    index=False
)
print("Archivo creado (inválidos): FechaCargue_Docs_Soportes_ORFEO_invalidos.xlsx")

print("\nColumna lista para migrar a ConceptoFinal.FechaDeCargueDocsYSoportesExpOrfeo:",
      "FechaDeCargueDocsYSoportesExpOrfeo_SQL")


Columnas candidatas para FECHA DE CARGUE DOCUMENTOS Y SOPORTES EN EXPDTE ORFEO: ['FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO']
Usando columna: FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4269181712.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaCargueDocsSoportesOrfeo_RAW'] = df[col_cargue_docs_orfeo]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4269181712.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'] = (
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\4269181712.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fr


=== Resumen FECHA DE CARGUE DOCUMENTOS Y SOPORTES EN EXPDTE ORFEO ===
Fechas válidas (no NaT): 2486
Fechas inválidas (NaT con texto no vacío): 0

=== Ejemplos de filas LISTAS PARA MIGRAR (fechas válidas) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO                   MUNICIPIO FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO FechaCargueDocsSoportesOrfeo_TXT_LIMPIO FechaDeCargueDocsYSoportesExpOrfeo_SQL
                4  062-37361      BOLIVAR                    ZAMBRANO                                            2024-12-21                              2024-12-21                             2024-12-21
                7  062-37360      BOLIVAR                    ZAMBRANO                                            2024-12-21                              2024-12-21                             2024-12-21
                8  062-37356      BOLIVAR                    ZAMBRANO                                            2024-12-21                              2024-12-21                   

## FECHA GESTION DE ETAPA SIT

In [67]:
import numpy as np
import pandas as pd
import re

# ============================================================
# FECHA GESTION ETAPA SIT (SIN RECARGAR df)
# - NO hace: df = df.copy()
# - Re-ejecución segura: borra columnas derivadas previas in-place
# - Resto queda igual (misma lógica, mismos excels)
# ============================================================

# 0) Asegurar fila aproximada (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 0.1) Evitar duplicados por re-ejecución (SIN recargar df)
cols_prev = [
    "FechaGestionEtapaSIT_RAW",
    "FechaGestionEtapaSIT_TXT_LIMPIO",
    "FechaGestionEtapaSIT_TXT_UP",
    "FechaGestionEtapaSIT_SQL",
]
cols_prev_exist = [c for c in cols_prev if c in df.columns]
if cols_prev_exist:
    df.drop(columns=cols_prev_exist, inplace=True, errors="ignore")

# ============================================
# 1) Ubicar la columna de FECHA GESTION ETAPA SIT en el Excel
# ============================================
candidatas_sit = [
    c for c in df.columns
    if ('FECHA' in str(c).upper()
        and 'SIT' in str(c).upper()
        and ('GEST' in str(c).upper() or 'ETAPA' in str(c).upper()))
]

print("Columnas candidatas para FECHA GESTION DE ETAPA SIT:", candidatas_sit)

if candidatas_sit:
    col_sit = candidatas_sit[0]
else:
    # Ajusta este nombre exacto si ves otro en df.columns
    col_sit = 'FECHA GESTION DE ETAPA SIT'

print("Usando columna:", col_sit)

# ============================================
# 2) Columnas auxiliares RAW / TEXTO LIMPIO
# ============================================
df['FechaGestionEtapaSIT_RAW'] = df[col_sit]

df['FechaGestionEtapaSIT_TXT_LIMPIO'] = (
    df['FechaGestionEtapaSIT_RAW']
      .astype(str)
      .str.strip()
)

# Normalizar vacíos típicos
mask_vacio_sit = df['FechaGestionEtapaSIT_TXT_LIMPIO'].str.upper().isin(['', 'NAN', 'NAT'])
df.loc[mask_vacio_sit, 'FechaGestionEtapaSIT_TXT_LIMPIO'] = np.nan

df['FechaGestionEtapaSIT_TXT_UP'] = (
    df['FechaGestionEtapaSIT_TXT_LIMPIO']
      .astype(str)
      .str.strip()
      .str.upper()
)

# ============================================
# 3) Función de parseo a datetime (SIT)
# ============================================
def parse_fecha_sit(valor):
    """
    Convierte a datetime los valores para
    FECHA GESTION DE ETAPA SIT.
    Deja como NaT textos tipo 'NO REGISTRA', 'NO APLICA',
    'NO', 'N/A', etc.
    """
    if pd.isna(valor):
        return pd.NaT

    txt = str(valor).strip()
    if txt == "":
        return pd.NaT

    tu = txt.upper()

    # Textos claramente NO fecha -> NaT
    if tu in ("NO", "NO REGISTRA", "NO APLICA", "N/A"):
        return pd.NaT

    # Unificar separadores a "/"
    t = re.sub(r"[-.]", "/", txt)
    t = re.sub(r"\s+", "/", t)  # "2024 04 25" -> "2024/04/25"

    # Intento genérico con dayfirst
    fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')

    return fecha  # si no puede, NaT

# ============================================
# 4) Columna final para SQL
# ============================================
df['FechaGestionEtapaSIT_SQL'] = (
    df['FechaGestionEtapaSIT_TXT_LIMPIO'].apply(parse_fecha_sit)
)

# ============================================
# 5) Resumen y muestras (válidos e inválidos)
# ============================================
mask_valida_sit = df['FechaGestionEtapaSIT_SQL'].notna()

mask_con_texto_sit = (
    df['FechaGestionEtapaSIT_TXT_LIMPIO'].notna()
    & (df['FechaGestionEtapaSIT_TXT_LIMPIO'].astype(str).str.strip() != '')
)

mask_invalida_con_texto_sit = ~mask_valida_sit & mask_con_texto_sit

validas_sit = int(mask_valida_sit.sum())
invalidas_con_texto_sit = int(mask_invalida_con_texto_sit.sum())

print("\n=== Resumen FECHA GESTION DE ETAPA SIT ===")
print(f"Fechas válidas (no NaT): {validas_sit}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto_sit}")

# Muestra de filas listas para migrar (válidas)
cols_ctx = [c for c in ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO'] if c in df.columns]

cols_validos_sit = cols_ctx + [
    col_sit,
    'FechaGestionEtapaSIT_TXT_LIMPIO',
    'FechaGestionEtapaSIT_SQL'
]
cols_validos_sit = [c for c in cols_validos_sit if c in df.columns]

ej_validos_sit = df.loc[mask_valida_sit, cols_validos_sit].head(30)

print("\n=== Ejemplos de filas LISTAS PARA MIGRAR (SIT, fechas válidas) ===")
if not ej_validos_sit.empty:
    print(ej_validos_sit.to_string(index=False))
else:
    print("No hay fechas válidas para mostrar.")

# Muestra de filas con texto que NO se pudo convertir
cols_invalidos_sit = cols_ctx + [
    'FechaGestionEtapaSIT_RAW',
    'FechaGestionEtapaSIT_TXT_LIMPIO'
]
cols_invalidos_sit = [c for c in cols_invalidos_sit if c in df.columns]

ej_invalidos_sit = df.loc[mask_invalida_con_texto_sit, cols_invalidos_sit].head(50)

print("\n=== Ejemplos de filas con TEXTO que NO se pudo convertir (SIT, para revisar) ===")
if not ej_invalidos_sit.empty:
    print(ej_invalidos_sit.to_string(index=False))
else:
    print("No hay filas con texto no vacío que sigan inválidas.")

# ============================================
# 6) Exportar Excels (SIT)
# ============================================
cols_export_todos_sit = cols_ctx + [
    col_sit,                              # columna original del Excel
    'FechaGestionEtapaSIT_RAW',           # valor crudo
    'FechaGestionEtapaSIT_TXT_LIMPIO',    # texto limpio
    'FechaGestionEtapaSIT_SQL'            # fecha final para SQL
]
cols_export_todos_sit = [c for c in cols_export_todos_sit if c in df.columns]

# 6.1. Excel con TODOS los registros (crudo vs limpio vs SQL)
df_export_todos_sit = df[cols_export_todos_sit].copy()
df_export_todos_sit.to_excel(
    'FechaGestion_Etapa_SIT_raw_vs_sql.xlsx',
    index=False
)
print("\nArchivo creado (todos): FechaGestion_Etapa_SIT_raw_vs_sql.xlsx")

# 6.2. Excel SOLO con registros inválidos (texto que no se pudo convertir)
df_export_invalidos_sit = df.loc[mask_invalida_con_texto_sit, cols_export_todos_sit].copy()
df_export_invalidos_sit.to_excel(
    'FechaGestion_Etapa_SIT_invalidos.xlsx',
    index=False
)
print("Archivo creado (inválidos): FechaGestion_Etapa_SIT_invalidos.xlsx")

print("\nColumna lista para migrar a ConceptoFinal.FechaGestionEtapaSIT:",
      "FechaGestionEtapaSIT_SQL")


Columnas candidatas para FECHA GESTION DE ETAPA SIT: ['FECHA GESTION DE ETAPA SIT']
Usando columna: FECHA GESTION DE ETAPA SIT


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1018648054.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaGestionEtapaSIT_RAW'] = df[col_sit]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1018648054.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaGestionEtapaSIT_TXT_LIMPIO'] = (
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_23180\1018648054.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which 


=== Resumen FECHA GESTION DE ETAPA SIT ===
Fechas válidas (no NaT): 423
Fechas inválidas (NaT con texto no vacío): 0

=== Ejemplos de filas LISTAS PARA MIGRAR (SIT, fechas válidas) ===
 Fila_Excel_aprox        FMI       DEPARTAMENTO              MUNICIPIO FECHA GESTION DE ETAPA SIT FechaGestionEtapaSIT_TXT_LIMPIO FechaGestionEtapaSIT_SQL
               29   068-2956            BOLIVAR                MORALES                 2025-10-28                      2025-10-28               2025-10-28
               76  214-19727         LA GUAJIRA     SAN JUAN DEL CESAR                 2025-10-28                      2025-10-28               2025-10-28
              111  170-35015       CUNDINAMARCA                  PACHO                 2025-10-29                      2025-10-29               2025-10-29
              112  170-35016       CUNDINAMARCA                  PACHO                 2025-10-28                      2025-10-28               2025-10-28
              347  140-79422           

*Excel final*

In [68]:
import pandas as pd

# ======================================================================================
# AVANCE A EXCEL (SIN RECARGAR df) - CONSOLIDADO HASTA HOY (incluye lo último trabajado)
# - NO hace: df = df.copy()
# - NO hace: df = pd.concat([...]) reasignando df
# - NO reprocesa reglas: SOLO USA columnas ya calculadas en df
# - Exporta hojas:
#     1) RegistroProceso_avance
#     2) EstudioTerreno_avance
#     3) MedidaProcesal_listo_insertar
#     4) ConceptosPrevio_listo_insertar
#     5) ConceptoFinal_listo_insertar   (incluye campos nuevos: estado revisión, observaciones, fechas, ORFEO, etc.)
#     6) Unificado_df
#     7) Diccionario
#     8) _Debug_Faltantes  (qué columnas esperadas NO existen en df)
# ======================================================================================

# 0) Asegurar fila Excel (sin sobrescribir)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# --------------------------------------------------------------------------------------
# 1) Columnas contexto (solo si existen)
# --------------------------------------------------------------------------------------
cols_ctx = [
    "Fila_Excel_aprox",
    "IdPostulacion_SQL",
    "ID_SQL",
    "FMI", "FMI_SQL",
    "DEPARTAMENTO", "MUNICIPIO",
]
cols_ctx = [c for c in cols_ctx if c in df.columns]

# Llaves fallback para cruce mientras NO exista IdRegistroProceso_SQL
keys_fallback = [c for c in ["IdPostulacion_SQL", "FMI_SQL"] if c in df.columns]
idrp_col = "IdRegistroProceso_SQL" if "IdRegistroProceso_SQL" in df.columns else None

# --------------------------------------------------------------------------------------
# 2) RegistroProceso_avance (solo columnas ya calculadas que existan)
# --------------------------------------------------------------------------------------
cols_registro = [
    "IdPostulacion_SQL",
    "FMI_SQL",
    "NumeroExpediente_SQL",
    "IdFuenteProceso",
    "IdFuenteProceso_SQL",
    "IdTipoProceso",
    "IdTipoProceso_SQL",
    "IdEtapaProcesal",
    "IdEtapaProcesal_SQL",
    "Dependencia_SQL",
    "RadicadoOrfeo_SQL",
]
cols_registro_exist = [c for c in cols_registro if c in df.columns]
df_registro = df[cols_ctx + cols_registro_exist].copy()

# --------------------------------------------------------------------------------------
# 3) EstudioTerreno_avance (solo columnas ya calculadas que existan)
# --------------------------------------------------------------------------------------
cols_estudio = [
    "IdRegistroProceso_SQL",
    "IdLocalizacion_SQL",
    "AreaRegistral_SQL",
    "CirculoRegistral_SQL",
    "AreaCalculada_SQL",
    "TipoPersonaTitular_SQL",
    "NombrePropietario_SQL",
    "ApellidoPropietario_SQL",
    "Identificacion_SQL_TXT_MIGRAR",
    "IdTipoDocumento_SQL_MIGRAR",
    "NaturalezaJuridica_SQL_FINAL",
    "AcreditacionPropiedad_SQL_FINAL",
]
cols_estudio_exist = [c for c in cols_estudio if c in df.columns]
df_estudio = df[cols_ctx + cols_estudio_exist].copy()

# --------------------------------------------------------------------------------------
# 4) MedidaProcesal_listo_insertar (formato largo) - SOLO LEE resultados ya calculados
# --------------------------------------------------------------------------------------
base_cols_mp = ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"]
base_cols_mp = [c for c in base_cols_mp if c in df.columns]

mp_parts = []

def add_mp(objeto: str, col_valor_final: str, col_anot: str, col_tipo: str = None,
           col_estado: str = None, estados_ok=None):
    """
    Agrega un bloque de MedidaProcesal en formato largo:
      base_cols + [Valor, Anotacion, TipoClasificacion, Objeto]
    Filtra:
      - Si existe col_estado y estados_ok, usa eso.
      - Si no, usa Valor notna().
    NO modifica df.
    """
    if col_valor_final not in df.columns:
        return
    if col_anot not in df.columns:
        return

    valor = df[col_valor_final]
    anot = df[col_anot]

    if (col_estado is not None) and (col_estado in df.columns) and estados_ok:
        mask = df[col_estado].isin(estados_ok)
    else:
        mask = valor.notna()

    if not mask.any():
        return

    out = df.loc[mask, base_cols_mp].copy()
    out["Valor"] = valor.loc[mask].values
    out["Anotacion"] = anot.loc[mask].values

    if col_tipo and (col_tipo in df.columns):
        out["TipoClasificacion"] = df.loc[mask, col_tipo].values
    else:
        out["TipoClasificacion"] = None

    out["Objeto"] = objeto
    mp_parts.append(out)

# Objetos ya trabajados
add_mp("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL", None, "HIPOTECA_ESTADO", ["ok", "sin_reg"])
add_mp("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL", None, "SERVIDUMBRES_ESTADO", ["ok", "sin_reg"])
add_mp("MEDIDAS CAUTELARES", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL", None, "MEDIDAS_CAUT_ESTADO", ["ok", "sin_reg"])
add_mp("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL", "RUPTA_ESTADO", ["ok", "sin_reg"])
add_mp("RTDAF LEY 1448", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL", None, "RTDAF_ESTADO", ["ok", "sin_reg"])
add_mp("OFERTA OTRAS ENTIDADES", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL", None, "OFERTA_ESTADO", ["ok", "sin_reg"])
add_mp("PROCESOS DE CLARIFICACIÓN", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL", None, "PROC_CLARIF_ESTADO", ["ok", "sin_reg"])

if mp_parts:
    df_medidaprocesal = pd.concat(mp_parts, ignore_index=True)
else:
    df_medidaprocesal = pd.DataFrame(columns=base_cols_mp + ["Valor", "Anotacion", "TipoClasificacion", "Objeto"])

sort_cols = [c for c in ["DEPARTAMENTO", "MUNICIPIO", "FMI", "Objeto"] if c in df_medidaprocesal.columns]
if sort_cols and not df_medidaprocesal.empty:
    df_medidaprocesal = df_medidaprocesal.sort_values(sort_cols).reset_index(drop=True)

# --------------------------------------------------------------------------------------
# 5) ConceptosPrevio_listo_insertar (SOLO resultados ya calculados)
#    Destino: AnalisisJuridico.ConceptosPrevio
#      - IdRegistroProceso (obligatorio en BD) -> si no existe aún, exporta llaves fallback
#      - FechaInforme -> FechaInformePrevio_SQL
#      - Concepto -> ConceptoPrevio_SQL
# --------------------------------------------------------------------------------------
cols_cp_src = ["FechaInformePrevio_SQL", "ConceptoPrevio_SQL"]
cols_cp_src_exist = [c for c in cols_cp_src if c in df.columns]

if cols_cp_src_exist:
    mask_cp = pd.Series(False, index=df.index)
    for c in cols_cp_src_exist:
        mask_cp = mask_cp | df[c].notna()

    cols_cp = cols_ctx + (["IdRegistroProceso_SQL"] if idrp_col else []) + keys_fallback + cols_cp_src_exist
    cols_cp = list(dict.fromkeys([c for c in cols_cp if c in df.columns]))  # únicos + existentes

    df_conceptosprevio = df.loc[mask_cp, cols_cp].copy()

    ren_cp = {}
    if idrp_col and ("IdRegistroProceso_SQL" in df_conceptosprevio.columns):
        ren_cp["IdRegistroProceso_SQL"] = "IdRegistroProceso"
    if "FechaInformePrevio_SQL" in df_conceptosprevio.columns:
        ren_cp["FechaInformePrevio_SQL"] = "FechaInforme"
    if "ConceptoPrevio_SQL" in df_conceptosprevio.columns:
        ren_cp["ConceptoPrevio_SQL"] = "Concepto"

    df_conceptosprevio.rename(columns=ren_cp, inplace=True)
else:
    df_conceptosprevio = pd.DataFrame()

# --------------------------------------------------------------------------------------
# 6) ConceptoFinal_listo_insertar (SOLO resultados ya calculados)
#    Destino: AnalisisJuridico.ConceptoFinal (según DDL)
#    Incluye también lo último trabajado:
#      - IdTipoEstadoRevision_SQL
#      - ObservacionesRevisor_SQL
#      - EntregoCarpetaSoportes_SQL
#      - FechaEnvioACoordinacion_SQL
#      - EstadoAprobacionCoordinadora_SQL
#      - FechaRemisionSoportesAGestoraDocumental_SQL
#      - FechaRemisionInformeAGestoraDocumental_SQL
#      - FechaCargueInformeJuridicoExpOrfeo_SQL
#      - FechaDeCargueDocsYSoportesExpOrfeo_SQL
#      - FechaGestionEtapaSIT_SQL
#      - (y los de antes: viabilidad, tipo informe, no viabilidad, reparto, fechas reparto/revisor, etc.)
# --------------------------------------------------------------------------------------
cols_cf_map = [
    ("ConceptoActualDeViabilidadJuridica_SQL", "ConceptoActualDeViabilidadJuridica"),
    ("FechaInforme_SQL", "FechaInforme"),
    ("IdTipoViabilidad_SQL", "IdTipoViabilidad"),
    ("IdTipoInforme_SQL", "IdTipoInforme"),
    ("IdTipoNoViabilidad_SQL", "IdTipoNoViabilidad"),
    ("InsumosPendientes_SQL", "InsumosPendientes"),
    ("FechaEntregaARevisor_SQL", "FechaEntregaARevisor"),
    ("AbogadoSustanciadorAsignado_SQL", "AbogadoSustanciadorAsignado"),
    ("AbogadoRevisorAsignado_SQL", "AbogadoRevisorAsignado"),
    ("NroReparto_SQL", "NroReparto"),
    ("FechaAsignacionReparto_SQL", "FechaAsignacionReparto"),
    ("FechaPlazoEntregaARevisor_SQL", "FechaPlazoEntregaARevisor"),
    ("IdTipoEstadoRevision_SQL", "IdTipoEstadoRevision"),
    ("ObservacionesRevisor_SQL", "ObservacionesRevisor"),
    ("EntregoCarpetaSoportes_SQL", "EntregoCarpetaSoportes"),
    ("FechaEnvioACoordinacion_SQL", "FechaEnvioACoordinacion"),
    ("EstadoAprobacionCoordinadora_SQL", "EstadoAprobacionCoordinadora"),
    ("FechaRemisionSoportesAGestoraDocumental_SQL", "FechaRemisionSoportesAGestoraDocumental"),
    ("FechaRemisionInformeAGestoraDocumental_SQL", "FechaRemisionInformeAGestoraDocumental"),
    ("FechaCargueInformeJuridicoExpOrfeo_SQL", "FechaCargueInformeJuridicoExpOrfeo"),
    ("FechaDeCargueDocsYSoportesExpOrfeo_SQL", "FechaDeCargueDocsYSoportesExpOrfeo"),
    ("FechaGestionEtapaSIT_SQL", "FechaGestionEtapaSIT"),
]
cols_cf_src_exist = [src for (src, _) in cols_cf_map if src in df.columns]

if cols_cf_src_exist:
    mask_cf = pd.Series(False, index=df.index)
    for c in cols_cf_src_exist:
        mask_cf = mask_cf | df[c].notna()

    cols_cf = cols_ctx + (["IdRegistroProceso_SQL"] if idrp_col else []) + keys_fallback + cols_cf_src_exist
    cols_cf = list(dict.fromkeys([c for c in cols_cf if c in df.columns]))

    df_conceptofinal = df.loc[mask_cf, cols_cf].copy()

    ren_cf = {}
    if idrp_col and ("IdRegistroProceso_SQL" in df_conceptofinal.columns):
        ren_cf["IdRegistroProceso_SQL"] = "IdRegistroProceso"
    for src, dst in cols_cf_map:
        if src in df_conceptofinal.columns:
            ren_cf[src] = dst

    df_conceptofinal.rename(columns=ren_cf, inplace=True)
else:
    df_conceptofinal = pd.DataFrame()

# --------------------------------------------------------------------------------------
# 7) Unificado_df (df completo con todas las columnas trabajadas)
# --------------------------------------------------------------------------------------
df_unificado = df.copy()

# --------------------------------------------------------------------------------------
# 8) Debug de faltantes (para que veas por qué no aparecen en el Excel)
# --------------------------------------------------------------------------------------
esperadas_cf = [src for (src, _) in cols_cf_map]
faltantes_cf = [c for c in esperadas_cf if c not in df.columns]

esperadas_cp = cols_cp_src
faltantes_cp = [c for c in esperadas_cp if c not in df.columns]

debug_rows = []
for c in faltantes_cp:
    debug_rows.append(["ConceptosPrevio", c, "NO existe en df"])
for c in faltantes_cf:
    debug_rows.append(["ConceptoFinal", c, "NO existe en df"])
if idrp_col is None:
    debug_rows.append(["LLAVE", "IdRegistroProceso_SQL", "NO existe en df (se usa fallback IdPostulacion_SQL/FMI_SQL)"])

df_debug = pd.DataFrame(debug_rows, columns=["Seccion", "Columna", "Estado"])

# --------------------------------------------------------------------------------------
# 9) Diccionario (hoja -> columnas)
# --------------------------------------------------------------------------------------
dicc = []
for c in cols_registro_exist:
    dicc.append(["RegistroProceso_avance", c])
for c in cols_estudio_exist:
    dicc.append(["EstudioTerreno_avance", c])

for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", "Valor", "Anotacion", "TipoClasificacion", "Objeto"]:
    dicc.append(["MedidaProcesal_listo_insertar", c])

for c in df_conceptosprevio.columns:
    dicc.append(["ConceptosPrevio_listo_insertar", c])

for c in df_conceptofinal.columns:
    dicc.append(["ConceptoFinal_listo_insertar", c])

dicc.append(["Unificado_df", f"{len(df_unificado.columns)} columnas"])

df_dicc = pd.DataFrame(dicc, columns=["Hoja", "Columna"])

# --------------------------------------------------------------------------------------
# 10) Exportar Excel consolidado
# --------------------------------------------------------------------------------------
out_xlsx = "AVANCE_MIGRACION_HASTA_HOY_ACTUALIZADO.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    df_registro.to_excel(writer, index=False, sheet_name="RegistroProceso_avance")
    df_estudio.to_excel(writer, index=False, sheet_name="EstudioTerreno_avance")
    df_medidaprocesal.to_excel(writer, index=False, sheet_name="MedidaProcesal_listo_insertar")
    df_conceptosprevio.to_excel(writer, index=False, sheet_name="ConceptosPrevio_listo_insertar")
    df_conceptofinal.to_excel(writer, index=False, sheet_name="ConceptoFinal_listo_insertar")
    df_unificado.to_excel(writer, index=False, sheet_name="Unificado_df")
    df_dicc.to_excel(writer, index=False, sheet_name="Diccionario")
    df_debug.to_excel(writer, index=False, sheet_name="_Debug_Faltantes")

# --------------------------------------------------------------------------------------
# 11) Logs
# --------------------------------------------------------------------------------------
print("Excel creado:", out_xlsx)
print("Hojas: RegistroProceso_avance, EstudioTerreno_avance, MedidaProcesal_listo_insertar, "
      "ConceptosPrevio_listo_insertar, ConceptoFinal_listo_insertar, Unificado_df, Diccionario, _Debug_Faltantes")
print("Filas RegistroProceso_avance:", len(df_registro))
print("Filas EstudioTerreno_avance:", len(df_estudio))
print("Filas MedidaProcesal_listo_insertar:", len(df_medidaprocesal))
print("Filas ConceptosPrevio_listo_insertar:", len(df_conceptosprevio))
print("Filas ConceptoFinal_listo_insertar:", len(df_conceptofinal))
print("Filas Unificado_df:", len(df_unificado))
print("Columnas Unificado_df:", len(df_unificado.columns))

if idrp_col is None:
    print("\n[OJO] No existe 'IdRegistroProceso_SQL' en df. ConceptosPrevio/ConceptoFinal van con llaves fallback (IdPostulacion_SQL / FMI_SQL).")

if not df_debug.empty:
    print("\n[DEBUG] Ver hoja '_Debug_Faltantes' para columnas esperadas que NO existen en df.")


Excel creado: AVANCE_MIGRACION_HASTA_HOY_ACTUALIZADO.xlsx
Hojas: RegistroProceso_avance, EstudioTerreno_avance, MedidaProcesal_listo_insertar, ConceptosPrevio_listo_insertar, ConceptoFinal_listo_insertar, Unificado_df, Diccionario, _Debug_Faltantes
Filas RegistroProceso_avance: 24055
Filas EstudioTerreno_avance: 24055
Filas MedidaProcesal_listo_insertar: 69662
Filas ConceptosPrevio_listo_insertar: 5869
Filas ConceptoFinal_listo_insertar: 24055
Filas Unificado_df: 24055
Columnas Unificado_df: 323

[OJO] No existe 'IdRegistroProceso_SQL' en df. ConceptosPrevio/ConceptoFinal van con llaves fallback (IdPostulacion_SQL / FMI_SQL).

[DEBUG] Ver hoja '_Debug_Faltantes' para columnas esperadas que NO existen en df.


## Migración

In [69]:
import urllib.parse
from sqlalchemy import create_engine, text
import pandas as pd

# =====================================================
# 0) CONEXIÓN (tal cual la que pasaste)
# =====================================================
server = r"172.23.92.97\MSSQLSERVER2022"
database = "ViabilidadJuridica"
user = "andres.garciav"
password = "4ndr3s.g4rc14v_3610@"

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={user};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# =====================================================
# 1) CONFIG: 1 SOLO REGISTRO (por Fila_Excel_aprox)
# =====================================================
FILA_EXCEL = 2  # <-- cambia esto a la fila que quieras probar

# =====================================================
# 2) Asegurar Fila_Excel_aprox (solo si no existe)
# =====================================================
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# =====================================================
# 3) Tomar SOLO 1 fila del DF (sin recargar df)
# =====================================================
r = df.loc[df["Fila_Excel_aprox"] == FILA_EXCEL]
if r.empty:
    raise ValueError(f"No existe Fila_Excel_aprox={FILA_EXCEL} en df.")
r = r.iloc[0]

def get_value(col_name):
    if col_name not in df.columns:
        return None
    v = r[col_name]
    if pd.isna(v):
        return None
    s = str(v).strip()
    if s == "" or s.upper() in ("NAN", "NAT", "NONE"):
        return None
    return s

def to_int_or_none(x):
    if x is None:
        return None
    try:
        return int(float(x))
    except:
        return None

# =====================================================
# 4) Mapear EXACTO a STG_RegistroProceso
# =====================================================
Fila_Excel_aprox = int(r["Fila_Excel_aprox"])

IdPostulacion = get_value("IdPostulacion_SQL")
FMI = get_value("FMI_SQL") or get_value("FMI")

if not IdPostulacion:
    raise ValueError("IdPostulacion_SQL está vacío en esta fila. No se puede insertar.")
if not FMI:
    raise ValueError("FMI_SQL/FMI está vacío en esta fila. No se puede insertar.")

NumeroExpediente = get_value("NumeroExpediente_SQL")

IdFuenteProceso = to_int_or_none(get_value("IdFuenteProceso_SQL") or get_value("IdFuenteProceso"))
IdTipoProceso = to_int_or_none(get_value("IdTipoProceso_SQL") or get_value("IdTipoProceso"))
IdEtapaProcesal = to_int_or_none(get_value("IdEtapaProcesal_SQL") or get_value("IdEtapaProcesal"))

Dependencia = get_value("Dependencia_SQL")
RadicadoOrfeo = get_value("RadicadoOrfeo_SQL")

payload = {
    "Fila_Excel_aprox": Fila_Excel_aprox,
    "IdPostulacion": IdPostulacion,
    "FMI": FMI,
    "NumeroExpediente": NumeroExpediente,
    "IdFuenteProceso": IdFuenteProceso,
    "IdTipoProceso": IdTipoProceso,
    "IdEtapaProcesal": IdEtapaProcesal,
    "Dependencia": Dependencia,
    "RadicadoOrfeo": RadicadoOrfeo,
}

print("Payload a insertar en STG_RegistroProceso:")
print(payload)

# =====================================================
# 5) INSERT 1 REGISTRO (SQLAlchemy)
# =====================================================
sql_insert = text("""
INSERT INTO ViabilidadJuridica.dbo.STG_RegistroProceso
(
  Fila_Excel_aprox,
  IdPostulacion,
  FMI,
  NumeroExpediente,
  IdFuenteProceso,
  IdTipoProceso,
  IdEtapaProcesal,
  Dependencia,
  RadicadoOrfeo
)
VALUES
(
  :Fila_Excel_aprox,
  :IdPostulacion,
  :FMI,
  :NumeroExpediente,
  :IdFuenteProceso,
  :IdTipoProceso,
  :IdEtapaProcesal,
  :Dependencia,
  :RadicadoOrfeo
);
""")

with engine.begin() as conn:  # begin() = commit automático si no hay error
    conn.execute(sql_insert, payload)

print(f"OK: Insertado 1 registro en STG_RegistroProceso (Fila_Excel_aprox={Fila_Excel_aprox})")

# =====================================================
# 6) VERIFICACIÓN RÁPIDA (traer lo insertado)
# =====================================================
sql_check = text("""
SELECT TOP 5 *
FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox = :Fila_Excel_aprox
ORDER BY Fila_Excel_aprox DESC;
""")

with engine.connect() as conn:
    df_check = pd.read_sql(sql_check, conn, params={"Fila_Excel_aprox": Fila_Excel_aprox})

print("\n=== Verificación (lo insertado) ===")
print(df_check.to_string(index=False))


Payload a insertar en STG_RegistroProceso:
{'Fila_Excel_aprox': 2, 'IdPostulacion': 'ID8099B', 'FMI': '062-37357', 'NumeroExpediente': None, 'IdFuenteProceso': 49, 'IdTipoProceso': None, 'IdEtapaProcesal': None, 'Dependencia': None, 'RadicadoOrfeo': None}
OK: Insertado 1 registro en STG_RegistroProceso (Fila_Excel_aprox=2)

=== Verificación (lo insertado) ===
 Fila_Excel_aprox IdPostulacion       FMI NumeroExpediente  IdFuenteProceso IdTipoProceso IdEtapaProcesal Dependencia RadicadoOrfeo
                2       ID8099B 062-37357             None               49          None            None        None          None


In [71]:
import urllib.parse
from sqlalchemy import create_engine, text
import pandas as pd

# =====================================================
# 0) CONEXIÓN (igual a la tuya)
# =====================================================
server = r"172.23.92.97\MSSQLSERVER2022"
database = "ViabilidadJuridica"
user = "andres.garciav"
password = "4ndr3s.g4rc14v_3610@"

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={user};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# =====================================================
# 1) CONFIG: cuál fila del STG vas a pasar a la tabla final
# =====================================================
FILA_EXCEL = 2

# =====================================================
# 2) Tomar 1 registro desde STG_RegistroProceso
# =====================================================
sql_get_stg = text("""
SELECT TOP 1
    Fila_Excel_aprox,
    IdPostulacion,
    FMI,
    NumeroExpediente,
    IdFuenteProceso,
    IdTipoProceso,
    IdEtapaProcesal,
    Dependencia,
    RadicadoOrfeo
FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox = :fila;
""")

with engine.connect() as conn:
    stg = pd.read_sql(sql_get_stg, conn, params={"fila": FILA_EXCEL})

if stg.empty:
    raise ValueError(f"No existe Fila_Excel_aprox={FILA_EXCEL} en STG_RegistroProceso")

row = stg.iloc[0].to_dict()

# Validación mínima (en tabla final NO existe Fila_Excel_aprox)
if not row.get("IdPostulacion") or not str(row["IdPostulacion"]).strip():
    raise ValueError("STG.IdPostulacion viene vacío. No se puede migrar a RegistroProceso.")
if not row.get("FMI") or not str(row["FMI"]).strip():
    raise ValueError("STG.FMI viene vacío. No se puede migrar a RegistroProceso.")

print("Registro encontrado en STG (se migrará a tabla final):")
print(row)

# =====================================================
# 3) Insertar en Postulacion.RegistroProceso (tabla final)
#    y capturar IdRegistroProceso generado (SIN multi-sentencias)
# =====================================================
sql_insert_final = text("""
INSERT INTO ViabilidadJuridica.Postulacion.RegistroProceso
(
    IdPostulacion,
    FMI,
    NumeroExpediente,
    IdFuenteProceso,
    IdTipoProceso,
    IdEtapaProcesal,
    Dependencia,
    RadicadoOrfeo
)
OUTPUT inserted.IdRegistroProceso
VALUES
(
    :IdPostulacion,
    :FMI,
    :NumeroExpediente,
    :IdFuenteProceso,
    :IdTipoProceso,
    :IdEtapaProcesal,
    :Dependencia,
    :RadicadoOrfeo
);
""")

payload = {
    "IdPostulacion": str(row["IdPostulacion"]).strip(),
    "FMI": str(row["FMI"]).strip(),
    "NumeroExpediente": row.get("NumeroExpediente"),
    "IdFuenteProceso": row.get("IdFuenteProceso"),
    "IdTipoProceso": row.get("IdTipoProceso"),
    "IdEtapaProcesal": row.get("IdEtapaProcesal"),
    "Dependencia": row.get("Dependencia"),
    "RadicadoOrfeo": row.get("RadicadoOrfeo"),
}

with engine.begin() as conn:
    res = conn.execute(sql_insert_final, payload)
    new_id = res.fetchone()[0]  # IdRegistroProceso

print(f"\nOK: Migrado a Postulacion.RegistroProceso. IdRegistroProceso generado = {new_id}")

# =====================================================
# 4) Verificación rápida (tabla final)
# =====================================================
sql_check_final = text("""
SELECT TOP 5
    IdRegistroProceso, IdPostulacion, FMI,
    NumeroExpediente, IdFuenteProceso, IdTipoProceso, IdEtapaProcesal,
    Dependencia, RadicadoOrfeo
FROM ViabilidadJuridica.Postulacion.RegistroProceso
WHERE IdRegistroProceso = :idrp;
""")

with engine.connect() as conn:
    df_check = pd.read_sql(sql_check_final, conn, params={"idrp": new_id})

print("\n=== Verificación (tabla final) ===")
print(df_check.to_string(index=False))



Registro encontrado en STG (se migrará a tabla final):
{'Fila_Excel_aprox': 2, 'IdPostulacion': 'ID8099B', 'FMI': '062-37357', 'NumeroExpediente': None, 'IdFuenteProceso': 49, 'IdTipoProceso': None, 'IdEtapaProcesal': None, 'Dependencia': None, 'RadicadoOrfeo': None}

OK: Migrado a Postulacion.RegistroProceso. IdRegistroProceso generado = 2

=== Verificación (tabla final) ===
 IdRegistroProceso IdPostulacion       FMI NumeroExpediente  IdFuenteProceso IdTipoProceso IdEtapaProcesal Dependencia RadicadoOrfeo
                 2       ID8099B 062-37357             None               49          None            None        None          None


In [72]:
from sqlalchemy import text
import pandas as pd

# =====================================================
# 5) MIGRAR 1 REGISTRO A Postulacion.EstudioTerreno (desde df)
#     - Usa new_id (IdRegistroProceso ya creado)
#     - Captura IdEstudioTerreno con OUTPUT
# =====================================================

# Asegurar Fila_Excel_aprox en df (solo si no existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# Tomar la misma fila del df
r = df.loc[df["Fila_Excel_aprox"] == FILA_EXCEL]
if r.empty:
    raise ValueError(f"No existe Fila_Excel_aprox={FILA_EXCEL} en df.")
r = r.iloc[0]

def getv(col):
    if col not in df.columns:
        return None
    v = r[col]
    if pd.isna(v):
        return None
    s = str(v).strip()
    if s == "" or s.upper() in ("NAN", "NAT", "NONE"):
        return None
    return s

def to_int(x):
    if x is None:
        return None
    try:
        return int(float(x))
    except:
        return None

def to_float(x):
    if x is None:
        return None
    try:
        return float(str(x).replace(",", "."))
    except:
        return None

# --- Mapear columnas ya trabajadas en tu df ---
IdLocalizacion = to_int(getv("IdLocalizacion_SQL"))
AreaRegistral  = to_float(getv("AreaRegistral_SQL"))
AreaCalculada  = to_float(getv("AreaCalculada_SQL"))
Circulo        = getv("CirculoRegistral_SQL")

TipoPersona    = getv("TipoPersonaTitular_SQL")
NombreProp     = getv("NombrePropietario_SQL")
ApellidoProp   = getv("ApellidoPropietario_SQL")

# IMPORTANTE: tú cambiaste Identificacion a varchar para migración
Identificacion_txt = getv("Identificacion_SQL_TXT_MIGRAR")

IdTipoDoc      = to_int(getv("IdTipoDocumento_SQL_MIGRAR"))
Naturaleza     = getv("NaturalezaJuridica_SQL_FINAL")
Acreditacion   = getv("AcreditacionPropiedad_SQL_FINAL")

# Validaciones mínimas del DDL (obligatorios)
if IdLocalizacion is None:
    raise ValueError("IdLocalizacion_SQL está vacío. EstudioTerreno requiere IdLocalizacion (NOT NULL).")
if AreaCalculada is None:
    raise ValueError("AreaCalculada_SQL está vacío. EstudioTerreno requiere AreaCalculada (NOT NULL).")

payload_est = {
    "IdRegistroProceso": int(new_id),
    "IdLocalizacion": IdLocalizacion,
    "AreaRegistral": AreaRegistral,
    "CirculoRegistral": Circulo,
    "AreaCalculada": AreaCalculada,
    "TipoPersonaTitular": TipoPersona,
    "NombrePropietario": NombreProp,
    "ApellidoPropietario": ApellidoProp,
    "IdTipoDocumento": IdTipoDoc,
    "Identificacion": Identificacion_txt,   # <-- varchar (según tu cambio)
    "NaturalezaJuridica": Naturaleza,
    "AcreditacionPropiedad": Acreditacion,
}

print("\nPayload EstudioTerreno:")
print(payload_est)

sql_insert_est = text("""
INSERT INTO ViabilidadJuridica.Postulacion.EstudioTerreno
(
    IdRegistroProceso,
    IdLocalizacion,
    AreaRegistral,
    CirculoRegistral,
    AreaCalculada,
    TipoPersonaTitular,
    NombrePropietario,
    ApellidoPropietario,
    IdTipoDocumento,
    Identificacion,
    NaturalezaJuridica,
    AcreditacionPropiedad
)
OUTPUT inserted.IdEstudioTerreno
VALUES
(
    :IdRegistroProceso,
    :IdLocalizacion,
    :AreaRegistral,
    :CirculoRegistral,
    :AreaCalculada,
    :TipoPersonaTitular,
    :NombrePropietario,
    :ApellidoPropietario,
    :IdTipoDocumento,
    :Identificacion,
    :NaturalezaJuridica,
    :AcreditacionPropiedad
);
""")

with engine.begin() as conn:
    res = conn.execute(sql_insert_est, payload_est)
    new_id_estudio = res.fetchone()[0]

print(f"\nOK: Migrado a Postulacion.EstudioTerreno. IdEstudioTerreno generado = {new_id_estudio}")

# Verificación rápida
sql_check_est = text("""
SELECT TOP 5 *
FROM ViabilidadJuridica.Postulacion.EstudioTerreno
WHERE IdEstudioTerreno = :id;
""")
with engine.connect() as conn:
    df_est = pd.read_sql(sql_check_est, conn, params={"id": new_id_estudio})

print("\n=== Verificación (EstudioTerreno) ===")
print(df_est.to_string(index=False))



Payload EstudioTerreno:
{'IdRegistroProceso': 2, 'IdLocalizacion': 195, 'AreaRegistral': 3.0034, 'CirculoRegistral': None, 'AreaCalculada': 2.9061, 'TipoPersonaTitular': 'Jurídica', 'NombrePropietario': 'PALMARES DEL RIO SAS', 'ApellidoPropietario': None, 'IdTipoDocumento': None, 'Identificacion': '9005427093', 'NaturalezaJuridica': 'PRIVADO', 'AcreditacionPropiedad': 'FÓRMULA TRANSACCIONAL'}

OK: Migrado a Postulacion.EstudioTerreno. IdEstudioTerreno generado = 1

=== Verificación (EstudioTerreno) ===
 IdEstudioTerreno  IdRegistroProceso  IdLocalizacion  AreaRegistral CirculoRegistral  AreaCalculada TipoPersonaTitular    NombrePropietario ApellidoPropietario IdTipoDocumento NaturalezaJuridica AcreditacionPropiedad Identificacion
                1                  2             195         3.0034             None         2.9061           Jurídica PALMARES DEL RIO SAS                None            None            PRIVADO FÓRMULA TRANSACCIONAL     9005427093


In [74]:
from sqlalchemy import text
import pandas as pd

# =====================================================
# MIGRAR 1 REGISTRO A AnalisisJuridico.ConceptoFinal (desde df)
# Requiere:
#   - new_id (IdRegistroProceso ya creado en RegistroProceso)
#   - FILA_EXCEL (para ubicar la fila en df)
# =====================================================

# Asegurar Fila_Excel_aprox en df (solo si no existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# Tomar la misma fila del df
r = df.loc[df["Fila_Excel_aprox"] == FILA_EXCEL]
if r.empty:
    raise ValueError(f"No existe Fila_Excel_aprox={FILA_EXCEL} en df.")
r = r.iloc[0]

def get_raw(col):
    if col not in df.columns:
        return None
    v = r[col]
    if pd.isna(v):
        return None
    return v

def get_str(col):
    v = get_raw(col)
    if v is None:
        return None
    s = str(v).strip()
    if s == "" or s.upper() in ("NAN", "NAT", "NONE"):
        return None
    return s

def to_int(col):
    v = get_str(col)
    if v is None:
        return None
    try:
        return int(float(v))
    except:
        return None

def to_dt(col):
    v = get_raw(col)
    if v is None:
        return None
    # ya debería venir Timestamp/datetime si lo trabajaste como *_SQL
    try:
        ts = pd.to_datetime(v, errors="coerce")
        if pd.isna(ts):
            return None
        return ts.to_pydatetime()
    except:
        return None

# -----------------------------
# Mapear columnas del df -> ConceptoFinal (según tu DDL)
# (usa los nombres *_SQL que ya trabajaste)
# -----------------------------
payload_cf = {
    "IdRegistroProceso": int(new_id),

    "ConceptoActualDeViabilidadJuridica": get_str("ConceptoActualDeViabilidadJuridica_SQL"),
    "FechaInforme": to_dt("FechaInforme_SQL"),

    "IdTipoViabilidad": to_int("IdTipoViabilidad_SQL"),
    "IdTipoInforme": to_int("IdTipoInforme_SQL"),
    "IdTipoNoViabilidad": to_int("IdTipoNoViabilidad_SQL"),

    "InsumosPendientes": get_str("InsumosPendientes_SQL"),
    "FechaEntregaARevisor": to_dt("FechaEntregaARevisor_SQL"),

    "AbogadoSustanciadorAsignado": get_str("AbogadoSustanciadorAsignado_SQL"),
    "AbogadoRevisorAsignado": get_str("AbogadoRevisorAsignado_SQL"),

    "NroReparto": to_int("NroReparto_SQL"),
    "FechaAsignacionReparto": to_dt("FechaAsignacionReparto_SQL"),
    "FechaPlazoEntregaARevisor": to_dt("FechaPlazoEntregaARevisor_SQL"),

    "IdTipoEstadoRevision": to_int("IdTipoEstadoRevision_SQL"),
    "ObservacionesRevisor": get_str("ObservacionesRevisor_SQL"),
    "EntregoCarpetaSoportes": get_str("EntregoCarpetaSoportes_SQL"),

    "FechaEnvioACoordinacion": to_dt("FechaEnvioACoordinacion_SQL"),
    "EstadoAprobacionCoordinadora": get_str("EstadoAprobacionCoordinadora_SQL"),

    "FechaRemisionSoportesAGestoraDocumental": to_dt("FechaRemisionSoportesAGestoraDocumental_SQL"),
    "FechaRemisionInformeAGestoraDocumental": to_dt("FechaRemisionInformeAGestoraDocumental_SQL"),

    "FechaCargueInformeJuridicoExpOrfeo": to_dt("FechaCargueInformeJuridicoExpOrfeo_SQL"),
    "FechaDeCargueDocsYSoportesExpOrfeo": to_dt("FechaDeCargueDocsYSoportesExpOrfeo_SQL"),
    "FechaGestionEtapaSIT": to_dt("FechaGestionEtapaSIT_SQL"),
}

# Quitar keys con None para saber si realmente hay algo que insertar (aparte del IdRegistroProceso)
payload_sin_id = {k: v for k, v in payload_cf.items() if k != "IdRegistroProceso" and v is not None}

print("\nPayload ConceptoFinal (preview):")
print(payload_cf)

if not payload_sin_id:
    print("\nEsta fila NO tiene datos de ConceptoFinal (todo viene vacío). No se inserta.")
else:
    sql_insert_cf = text("""
    INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
    (
        IdRegistroProceso,
        ConceptoActualDeViabilidadJuridica,
        FechaInforme,
        IdTipoViabilidad,
        IdTipoInforme,
        IdTipoNoViabilidad,
        InsumosPendientes,
        FechaEntregaARevisor,
        AbogadoSustanciadorAsignado,
        AbogadoRevisorAsignado,
        NroReparto,
        FechaAsignacionReparto,
        FechaPlazoEntregaARevisor,
        IdTipoEstadoRevision,
        ObservacionesRevisor,
        EntregoCarpetaSoportes,
        FechaEnvioACoordinacion,
        EstadoAprobacionCoordinadora,
        FechaRemisionSoportesAGestoraDocumental,
        [FechaRemisiónInformeAGestoraDocumental],
        [FechaCargueInformeJurídicoExpOrfeo],
        FechaDeCargueDocsYSoportesExpOrfeo,
        FechaGestionEtapaSIT
    )
    OUTPUT inserted.IdConceptoFinal
    VALUES
    (
        :IdRegistroProceso,
        :ConceptoActualDeViabilidadJuridica,
        :FechaInforme,
        :IdTipoViabilidad,
        :IdTipoInforme,
        :IdTipoNoViabilidad,
        :InsumosPendientes,
        :FechaEntregaARevisor,
        :AbogadoSustanciadorAsignado,
        :AbogadoRevisorAsignado,
        :NroReparto,
        :FechaAsignacionReparto,
        :FechaPlazoEntregaARevisor,
        :IdTipoEstadoRevision,
        :ObservacionesRevisor,
        :EntregoCarpetaSoportes,
        :FechaEnvioACoordinacion,
        :EstadoAprobacionCoordinadora,
        :FechaRemisionSoportesAGestoraDocumental,
        :FechaRemisionInformeAGestoraDocumental,
        :FechaCargueInformeJuridicoExpOrfeo,
        :FechaDeCargueDocsYSoportesExpOrfeo,
        :FechaGestionEtapaSIT
    );
    """)


    with engine.begin() as conn:
        res = conn.execute(sql_insert_cf, payload_cf)
        new_id_cf = res.fetchone()[0]

    print(f"\nOK: Migrado a AnalisisJuridico.ConceptoFinal. IdConceptoFinal generado = {new_id_cf}")

    # Verificación rápida
    sql_check_cf = text("""
    SELECT TOP 5 *
    FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
    WHERE IdConceptoFinal = :id;
    """)
    with engine.connect() as conn:
        df_cf = pd.read_sql(sql_check_cf, conn, params={"id": new_id_cf})

    print("\n=== Verificación (ConceptoFinal) ===")
    print(df_cf.to_string(index=False))



Payload ConceptoFinal (preview):
{'IdRegistroProceso': 2, 'ConceptoActualDeViabilidadJuridica': 'I.\tIDENTIFICACIÓN Y DENOMINACIÓN\n\nEl presente concepto se efectúa conforme el análisis jurídico realizado sobre el inmueble RURAL denominado LT 5 A, ubicado en el municipio de ZAMBRANO, departamento de BOLIVAR, identificado con el Folio de Matrícula Inmobiliaria No. 062-37357, de la Oficina de Registro de Instrumentos Públicos de EL CARMEN DE BOLIVAR, no reporta cédula catastral. \n\nII.\tNATURALEZA JURIDICA\n\nEl artículo 48 de la Ley 160 de 1994 indica que la propiedad privada se acredita de dos formas, la primera, con un título originario expedido por el Estado que no haya perdido su eficacia legal y, la segunda, mediante títulos, debidamente inscritos, otorgados con anterioridad a la vigencia de la Ley 160 de 1994, en que consten tradiciones de dominio por un lapso no menor al término que señalan las leyes para la prescripción extraordinaria, esto es, 20 años contados a partir de la

In [75]:
from sqlalchemy import text
import pandas as pd

# Requiere: engine, df, FILA_EXCEL, new_id (IdRegistroProceso)
# new_id = IdRegistroProceso ya creado en Postulacion.RegistroProceso

if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

r = df.loc[df["Fila_Excel_aprox"] == FILA_EXCEL]
if r.empty:
    raise ValueError(f"No existe Fila_Excel_aprox={FILA_EXCEL} en df.")
r = r.iloc[0]

def get_raw(col):
    if col not in df.columns:
        return None
    v = r[col]
    if pd.isna(v):
        return None
    return v

def get_str(col):
    v = get_raw(col)
    if v is None:
        return None
    s = str(v).strip()
    if s == "" or s.upper() in ("NAN","NAT","NONE"):
        return None
    return s

def to_dt(col):
    v = get_raw(col)
    if v is None:
        return None
    ts = pd.to_datetime(v, errors="coerce")
    if pd.isna(ts):
        return None
    return ts.to_pydatetime()

payload_cp = {
    "IdRegistroProceso": int(new_id),
    "FechaInforme": to_dt("FechaInformePrevio_SQL"),
    "Concepto": get_str("ConceptoPrevio_SQL"),
}

# Si no hay nada, no insertes
if payload_cp["FechaInforme"] is None and payload_cp["Concepto"] is None:
    print("Esta fila no tiene ConceptosPrevio. No se inserta.")
else:
    sql_cp = text("""
    INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
    (
        IdRegistroProceso,
        FechaInforme,
        Concepto
    )
    OUTPUT inserted.IdConceptoPrevio
    VALUES
    (
        :IdRegistroProceso,
        :FechaInforme,
        :Concepto
    );
    """)
    with engine.begin() as conn:
        res = conn.execute(sql_cp, payload_cp)
        new_id_cp = res.fetchone()[0]

    print(f"OK: Insertado ConceptosPrevio. IdConceptoPrevio = {new_id_cp}")


OK: Insertado ConceptosPrevio. IdConceptoPrevio = 1


In [79]:
from sqlalchemy import text
import pandas as pd

# =====================================================
# MIGRAR MedidaProcesal (1 SOLO REGISTRO) desde df
# Requiere:
#   - engine (ya creado)
#   - df (ya cargado)
#   - FILA_EXCEL (la fila que estás probando, p.ej. 2)
#   - new_id_et (IdEstudioTerreno ya creado en Postulacion.EstudioTerreno)
# =====================================================

# >>> IMPORTANTE: define el IdEstudioTerreno generado en el paso anterior <<<
# En tu caso salió: "IdEstudioTerreno generado = 1"
new_id_et = 1  # <-- AJUSTA si tu IdEstudioTerreno fue otro

# Asegurar Fila_Excel_aprox en df (solo si no existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# Tomar la misma fila del df
r = df.loc[df["Fila_Excel_aprox"] == FILA_EXCEL]
if r.empty:
    raise ValueError(f"No existe Fila_Excel_aprox={FILA_EXCEL} en df.")
r = r.iloc[0]

def get_str_cell(col):
    if col not in df.columns:
        return None
    v = r[col]
    if pd.isna(v):
        return None
    s = str(v).strip()
    if s == "" or s.upper() in ("NAN", "NAT", "NONE"):
        return None
    return s

def add_mp_row(objeto, col_valor, col_anot, col_tipo=None):
    """
    Retorna dict listo para INSERT si hay algo que insertar.
    No inventa "SI": si Valor es None pero hay anotación, deja Valor en None.
    """
    valor = get_str_cell(col_valor)
    anot = get_str_cell(col_anot)

    # Si ambos vacíos, no inserta
    if valor is None and anot is None:
        return None

    row = {
        "IdEstudioTerreno": int(new_id_et),
        "Objeto": objeto,
        "Valor": valor,  # <-- NO inventa "SI"
        "Anotacion": anot,
        "TipoClasificacion": get_str_cell(col_tipo) if col_tipo else None,
    }
    return row

rows = []

# HIPOTECA
x = add_mp_row("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL")
if x: rows.append(x)

# SERVIDUMBRES
x = add_mp_row("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL")
if x: rows.append(x)

# MEDIDAS CAUTELARES
x = add_mp_row("MEDIDAS CAUTELARES", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL")
if x: rows.append(x)

# RUPTA (con TipoClasificacion si aplica)
x = add_mp_row("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL")
if x: rows.append(x)

# RTDAF LEY 1448
x = add_mp_row("RTDAF LEY 1448", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL")
if x: rows.append(x)

# OFERTA OTRAS ENTIDADES
x = add_mp_row("OFERTA OTRAS ENTIDADES", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL")
if x: rows.append(x)

# PROCESOS DE CLARIFICACIÓN
x = add_mp_row("PROCESOS DE CLARIFICACIÓN", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL")
if x: rows.append(x)

print("Filas MedidaProcesal a insertar:", len(rows))
if rows:
    sql_mp = text("""
    INSERT INTO ViabilidadJuridica.Postulacion.MedidaProcesal
    (
        IdEstudioTerreno,
        Objeto,
        Valor,
        Anotacion,
        TipoClasificacion
    )
    VALUES
    (
        :IdEstudioTerreno,
        :Objeto,
        :Valor,
        :Anotacion,
        :TipoClasificacion
    );
    """)
    with engine.begin() as conn:
        conn.execute(sql_mp, rows)  # executemany

    print("OK: Insertadas MedidaProcesal (todas las que aplican en esa fila).")
else:
    print("No hay MedidaProcesal para insertar en esta fila.")


Filas MedidaProcesal a insertar: 7
OK: Insertadas MedidaProcesal (todas las que aplican en esa fila).


## CARGAR 10 RESGISTROS

In [80]:
import urllib.parse
from sqlalchemy import create_engine, text
import pandas as pd

# =====================================================
# 0) CONEXIÓN (igual a la tuya)
# =====================================================
server = r"172.23.92.97\MSSQLSERVER2022"
database = "ViabilidadJuridica"
user = "andres.garciav"
password = "4ndr3s.g4rc14v_3610@"

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={user};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# =====================================================
# 1) CONFIG: siguientes 5 filas a probar
# =====================================================
FILAS_EXCEL = [3, 4, 5, 6, 7]  # siguientes 5 después de la 2

# =====================================================
# 2) Helpers DF
# =====================================================
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

def get_row_df(fila_excel: int):
    r = df.loc[df["Fila_Excel_aprox"] == fila_excel]
    if r.empty:
        raise ValueError(f"No existe Fila_Excel_aprox={fila_excel} en df.")
    return r.iloc[0]

def get_raw(r, col):
    if col not in df.columns:
        return None
    v = r[col]
    if pd.isna(v):
        return None
    return v

def get_str(r, col):
    v = get_raw(r, col)
    if v is None:
        return None
    s = str(v).strip()
    if s == "" or s.upper() in ("NAN", "NAT", "NONE"):
        return None
    return s

def to_int_value(x):
    if x is None:
        return None
    try:
        return int(float(x))
    except:
        return None

def to_int(r, col):
    return to_int_value(get_str(r, col))

def to_dt(r, col):
    v = get_raw(r, col)
    if v is None:
        return None
    ts = pd.to_datetime(v, errors="coerce")
    if pd.isna(ts):
        return None
    return ts.to_pydatetime()

# =====================================================
# 3) Helpers DB: columnas reales (evitar errores de tildes)
# =====================================================
def get_table_cols(schema: str, table: str):
    sql = text("""
    SELECT c.name
    FROM sys.columns c
    JOIN sys.tables t ON c.object_id = t.object_id
    JOIN sys.schemas s ON t.schema_id = s.schema_id
    WHERE s.name = :schema AND t.name = :table
    ORDER BY c.column_id;
    """)
    with engine.connect() as conn:
        rows = conn.execute(sql, {"schema": schema, "table": table}).fetchall()
    return [x[0] for x in rows]

# Cache columnas reales
COLS_CONCEPTOFINAL = set(get_table_cols("AnalisisJuridico", "ConceptoFinal"))
COLS_CONCEPTOSPREVIO = set(get_table_cols("AnalisisJuridico", "ConceptosPrevio"))
COLS_REGISTROPROCESO = set(get_table_cols("Postulacion", "RegistroProceso"))
COLS_ESTUDIOTERRENO  = set(get_table_cols("Postulacion", "EstudioTerreno"))

# =====================================================
# 4) SQL statements
# =====================================================
sql_insert_stg = text("""
INSERT INTO ViabilidadJuridica.dbo.STG_RegistroProceso
(
  Fila_Excel_aprox,
  IdPostulacion,
  FMI,
  NumeroExpediente,
  IdFuenteProceso,
  IdTipoProceso,
  IdEtapaProcesal,
  Dependencia,
  RadicadoOrfeo
)
VALUES
(
  :Fila_Excel_aprox,
  :IdPostulacion,
  :FMI,
  :NumeroExpediente,
  :IdFuenteProceso,
  :IdTipoProceso,
  :IdEtapaProcesal,
  :Dependencia,
  :RadicadoOrfeo
);
""")

sql_get_stg = text("""
SELECT TOP 1
    Fila_Excel_aprox,
    IdPostulacion,
    FMI,
    NumeroExpediente,
    IdFuenteProceso,
    IdTipoProceso,
    IdEtapaProcesal,
    Dependencia,
    RadicadoOrfeo
FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox = :fila;
""")

sql_insert_registro = text("""
INSERT INTO ViabilidadJuridica.Postulacion.RegistroProceso
(
    IdPostulacion, FMI, NumeroExpediente,
    IdFuenteProceso, IdTipoProceso, IdEtapaProcesal,
    Dependencia, RadicadoOrfeo
)
OUTPUT inserted.IdRegistroProceso
VALUES
(
    :IdPostulacion, :FMI, :NumeroExpediente,
    :IdFuenteProceso, :IdTipoProceso, :IdEtapaProcesal,
    :Dependencia, :RadicadoOrfeo
);
""")

sql_insert_estudio = text("""
INSERT INTO ViabilidadJuridica.Postulacion.EstudioTerreno
(
    IdRegistroProceso,
    IdLocalizacion,
    AreaRegistral,
    CirculoRegistral,
    AreaCalculada,
    TipoPersonaTitular,
    NombrePropietario,
    ApellidoPropietario,
    IdTipoDocumento,
    Identificacion,
    NaturalezaJuridica,
    AcreditacionPropiedad
)
OUTPUT inserted.IdEstudioTerreno
VALUES
(
    :IdRegistroProceso,
    :IdLocalizacion,
    :AreaRegistral,
    :CirculoRegistral,
    :AreaCalculada,
    :TipoPersonaTitular,
    :NombrePropietario,
    :ApellidoPropietario,
    :IdTipoDocumento,
    :Identificacion,
    :NaturalezaJuridica,
    :AcreditacionPropiedad
);
""")

sql_insert_cp = text("""
INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
(
    IdRegistroProceso,
    FechaInforme,
    Concepto
)
OUTPUT inserted.IdConceptoPrevio
VALUES
(
    :IdRegistroProceso,
    :FechaInforme,
    :Concepto
);
""")

sql_insert_mp = text("""
INSERT INTO ViabilidadJuridica.Postulacion.MedidaProcesal
(
    IdEstudioTerreno,
    Objeto,
    Valor,
    Anotacion,
    TipoClasificacion
)
VALUES
(
    :IdEstudioTerreno,
    :Objeto,
    :Valor,
    :Anotacion,
    :TipoClasificacion
);
""")

# =====================================================
# 5) Funciones de armado de payloads
# =====================================================
def payload_stg_from_dfrow(r):
    fila = int(r["Fila_Excel_aprox"])
    idpost = get_str(r, "IdPostulacion_SQL")
    fmi = get_str(r, "FMI_SQL") or get_str(r, "FMI")
    if not idpost:
        raise ValueError("IdPostulacion_SQL vacío.")
    if not fmi:
        raise ValueError("FMI_SQL/FMI vacío.")
    return {
        "Fila_Excel_aprox": fila,
        "IdPostulacion": idpost,
        "FMI": fmi,
        "NumeroExpediente": get_str(r, "NumeroExpediente_SQL"),
        "IdFuenteProceso": to_int_value(get_str(r, "IdFuenteProceso_SQL") or get_str(r, "IdFuenteProceso")),
        "IdTipoProceso": to_int_value(get_str(r, "IdTipoProceso_SQL") or get_str(r, "IdTipoProceso")),
        "IdEtapaProcesal": to_int_value(get_str(r, "IdEtapaProcesal_SQL") or get_str(r, "IdEtapaProcesal")),
        "Dependencia": get_str(r, "Dependencia_SQL"),
        "RadicadoOrfeo": get_str(r, "RadicadoOrfeo_SQL"),
    }

def payload_estudio_from_dfrow(r, id_registro):
    # OJO: Identificacion ahora es texto en BD (según tu cambio)
    return {
        "IdRegistroProceso": int(id_registro),
        "IdLocalizacion": to_int_value(get_str(r, "IdLocalizacion_SQL")),
        "AreaRegistral": get_raw(r, "AreaRegistral_SQL"),
        "CirculoRegistral": get_str(r, "CirculoRegistral_SQL"),
        "AreaCalculada": get_raw(r, "AreaCalculada_SQL"),
        "TipoPersonaTitular": get_str(r, "TipoPersonaTitular_SQL"),
        "NombrePropietario": get_str(r, "NombrePropietario_SQL"),
        "ApellidoPropietario": get_str(r, "ApellidoPropietario_SQL"),
        "IdTipoDocumento": to_int_value(get_str(r, "IdTipoDocumento_SQL_MIGRAR")),
        "Identificacion": get_str(r, "Identificacion_SQL_TXT_MIGRAR"),
        "NaturalezaJuridica": get_str(r, "NaturalezaJuridica_SQL_FINAL"),
        "AcreditacionPropiedad": get_str(r, "AcreditacionPropiedad_SQL_FINAL"),
    }

def payload_conceptosprevio_from_dfrow(r, id_registro):
    return {
        "IdRegistroProceso": int(id_registro),
        "FechaInforme": to_dt(r, "FechaInformePrevio_SQL"),
        "Concepto": get_str(r, "ConceptoPrevio_SQL"),
    }

def build_cf_payload_dfrow(r, id_registro):
    # Intentar cubrir ambas variantes (con y sin tilde) leyendo columnas reales.
    # Mapeo df -> posibles nombres de BD:
    candidates = {
        "ConceptoActualDeViabilidadJuridica": get_str(r, "ConceptoActualDeViabilidadJuridica_SQL"),
        "FechaInforme": to_dt(r, "FechaInforme_SQL"),
        "IdTipoViabilidad": to_int(r, "IdTipoViabilidad_SQL"),
        "IdTipoInforme": to_int(r, "IdTipoInforme_SQL"),
        "IdTipoNoViabilidad": to_int(r, "IdTipoNoViabilidad_SQL"),
        "InsumosPendientes": get_str(r, "InsumosPendientes_SQL"),
        "FechaEntregaARevisor": to_dt(r, "FechaEntregaARevisor_SQL"),
        "AbogadoSustanciadorAsignado": get_str(r, "AbogadoSustanciadorAsignado_SQL"),
        "AbogadoRevisorAsignado": get_str(r, "AbogadoRevisorAsignado_SQL"),
        "NroReparto": to_int(r, "NroReparto_SQL"),
        "FechaAsignacionReparto": to_dt(r, "FechaAsignacionReparto_SQL"),
        "FechaPlazoEntregaARevisor": to_dt(r, "FechaPlazoEntregaARevisor_SQL"),
        "IdTipoEstadoRevision": to_int(r, "IdTipoEstadoRevision_SQL"),
        "ObservacionesRevisor": get_str(r, "ObservacionesRevisor_SQL"),
        "EntregoCarpetaSoportes": get_str(r, "EntregoCarpetaSoportes_SQL"),
        "FechaEnvioACoordinacion": to_dt(r, "FechaEnvioACoordinacion_SQL"),
        "EstadoAprobacionCoordinadora": get_str(r, "EstadoAprobacionCoordinadora_SQL"),
        "FechaRemisionSoportesAGestoraDocumental": to_dt(r, "FechaRemisionSoportesAGestoraDocumental_SQL"),
        "FechaRemisionInformeAGestoraDocumental": to_dt(r, "FechaRemisionInformeAGestoraDocumental_SQL"),
        "FechaCargueInformeJuridicoExpOrfeo": to_dt(r, "FechaCargueInformeJuridicoExpOrfeo_SQL"),
        "FechaDeCargueDocsYSoportesExpOrfeo": to_dt(r, "FechaDeCargueDocsYSoportesExpOrfeo_SQL"),
        "FechaGestionEtapaSIT": to_dt(r, "FechaGestionEtapaSIT_SQL"),
    }

    # Ajuste de columna con tilde si existe en BD:
    if "FechaRemisiónInformeAGestoraDocumental" in COLS_CONCEPTOFINAL:
        candidates["FechaRemisiónInformeAGestoraDocumental"] = candidates.pop("FechaRemisionInformeAGestoraDocumental", None)
    if "FechaCargueInformeJurídicoExpOrfeo" in COLS_CONCEPTOFINAL and "FechaCargueInformeJuridicoExpOrfeo" in candidates:
        candidates["FechaCargueInformeJurídicoExpOrfeo"] = candidates.pop("FechaCargueInformeJuridicoExpOrfeo", None)

    payload = {"IdRegistroProceso": int(id_registro)}
    for k, v in candidates.items():
        if k in COLS_CONCEPTOFINAL:
            payload[k] = v

    return payload

def insert_conceptofinal(conn, payload):
    # No insertar si todo viene vacío (excepto IdRegistroProceso)
    has_data = any(v is not None for k, v in payload.items() if k != "IdRegistroProceso")
    if not has_data:
        return None

    cols = [c for c in payload.keys()]  # ya filtrados por columnas reales
    params_cols = [f":{c}" for c in cols]

    sql = f"""
    INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
    ({", ".join(cols)})
    OUTPUT inserted.IdConceptoFinal
    VALUES
    ({", ".join(params_cols)});
    """
    res = conn.execute(text(sql), payload)
    return res.fetchone()[0]

def build_mp_rows(r, id_estudio):
    def cell(col):
        return get_str(r, col)

    def add(objeto, col_valor, col_anot, col_tipo=None):
        valor = cell(col_valor)
        anot = cell(col_anot)
        if valor is None and anot is None:
            return None
        return {
            "IdEstudioTerreno": int(id_estudio),
            "Objeto": objeto,
            "Valor": (valor or "SI"),
            "Anotacion": anot,
            "TipoClasificacion": cell(col_tipo) if col_tipo else None,
        }

    rows = []
    for spec in [
        ("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL", None),
        ("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL", None),
        ("MEDIDAS CAUTELARES", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL", None),
        ("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL"),
        ("RTDAF LEY 1448", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL", None),
        ("OFERTA OTRAS ENTIDADES", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL", None),
        ("PROCESOS DE CLARIFICACIÓN", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL", None),
    ]:
        row = add(*spec)
        if row:
            rows.append(row)
    return rows

# =====================================================
# 6) Migración 5 filas con transacción por fila
# =====================================================
resultados = []

for fila in FILAS_EXCEL:
    try:
        r = get_row_df(fila)
        stg_payload = payload_stg_from_dfrow(r)

        with engine.begin() as conn:
            # 1) STG
            conn.execute(sql_insert_stg, stg_payload)

            # 2) Traer STG
            stg = pd.read_sql(sql_get_stg, conn, params={"fila": fila})
            if stg.empty:
                raise ValueError("No se encontró STG recién insertado.")
            stg_row = stg.iloc[0].to_dict()

            # 3) RegistroProceso final
            payload_reg = {
                "IdPostulacion": str(stg_row["IdPostulacion"]).strip(),
                "FMI": str(stg_row["FMI"]).strip(),
                "NumeroExpediente": stg_row.get("NumeroExpediente"),
                "IdFuenteProceso": stg_row.get("IdFuenteProceso"),
                "IdTipoProceso": stg_row.get("IdTipoProceso"),
                "IdEtapaProcesal": stg_row.get("IdEtapaProcesal"),
                "Dependencia": stg_row.get("Dependencia"),
                "RadicadoOrfeo": stg_row.get("RadicadoOrfeo"),
            }
            new_id_rp = conn.execute(sql_insert_registro, payload_reg).fetchone()[0]

            # 4) EstudioTerreno
            payload_et = payload_estudio_from_dfrow(r, new_id_rp)
            new_id_et = conn.execute(sql_insert_estudio, payload_et).fetchone()[0]

            # 5) ConceptoFinal
            payload_cf = build_cf_payload_dfrow(r, new_id_rp)
            new_id_cf = insert_conceptofinal(conn, payload_cf)

            # 6) ConceptosPrevio
            payload_cp = payload_conceptosprevio_from_dfrow(r, new_id_rp)
            has_cp = (payload_cp["FechaInforme"] is not None) or (payload_cp["Concepto"] is not None)
            new_id_cp = None
            if has_cp:
                new_id_cp = conn.execute(sql_insert_cp, payload_cp).fetchone()[0]

            # 7) MedidaProcesal (0..7)
            mp_rows = build_mp_rows(r, new_id_et)
            if mp_rows:
                conn.execute(sql_insert_mp, mp_rows)

        resultados.append({
            "Fila_Excel_aprox": fila,
            "IdRegistroProceso": new_id_rp,
            "IdEstudioTerreno": new_id_et,
            "IdConceptoFinal": new_id_cf,
            "IdConceptoPrevio": new_id_cp,
            "MedidaProcesal_insertadas": len(mp_rows),
            "Estado": "OK"
        })
        print(f"[OK] Fila {fila} -> RP={new_id_rp}, ET={new_id_et}, CF={new_id_cf}, CP={new_id_cp}, MP={len(mp_rows)}")

    except Exception as e:
        resultados.append({
            "Fila_Excel_aprox": fila,
            "Estado": "ERROR",
            "Error": str(e)[:4000]
        })
        print(f"[ERROR] Fila {fila} -> {e}")

print("\n=== RESUMEN ===")
print(pd.DataFrame(resultados).to_string(index=False))


[OK] Fila 3 -> RP=3, ET=2, CF=2, CP=2, MP=7
[OK] Fila 4 -> RP=4, ET=3, CF=3, CP=3, MP=7
[OK] Fila 5 -> RP=5, ET=4, CF=4, CP=4, MP=7
[OK] Fila 6 -> RP=6, ET=5, CF=5, CP=None, MP=7
[OK] Fila 7 -> RP=7, ET=6, CF=6, CP=5, MP=0

=== RESUMEN ===
 Fila_Excel_aprox  IdRegistroProceso  IdEstudioTerreno  IdConceptoFinal  IdConceptoPrevio  MedidaProcesal_insertadas Estado
                3                  3                 2                2               2.0                          7     OK
                4                  4                 3                3               3.0                          7     OK
                5                  5                 4                4               4.0                          7     OK
                6                  6                 5                5               NaN                          7     OK
                7                  7                 6                6               5.0                          0     OK


## VALIDACIONES ANTES DE MIGRAR

In [81]:
from sqlalchemy import text
import pandas as pd

# =====================================================
# VALIDACIONES POST-MIGRACIÓN (SQL)
# - Duplicados en RegistroProceso por llave natural
# - Integridad FK: EstudioTerreno / ConceptoFinal / ConceptosPrevio
# - Resumen de conteos
# Requiere: engine ya creado
# =====================================================

# -----------------------------
# 0) Helper para ejecutar y mostrar
# -----------------------------
def run_df(sql: str, params: dict | None = None) -> pd.DataFrame:
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn, params=params or {})

def show(title: str, df_: pd.DataFrame, max_rows: int = 20):
    print("\n" + "="*90)
    print(title)
    print("="*90)
    if df_.empty:
        print("(OK) Sin hallazgos.")
    else:
        print(df_.head(max_rows).to_string(index=False))
        if len(df_) > max_rows:
            print(f"... ({len(df_)} filas, mostrando {max_rows})")

# -----------------------------
# 1) Duplicados en RegistroProceso por llave natural
#    (Ajusta la llave si decides otra: aquí uso IdPostulacion + FMI)
# -----------------------------
sql_dup_rp = """
SELECT
    IdPostulacion,
    FMI,
    COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.RegistroProceso
GROUP BY IdPostulacion, FMI
HAVING COUNT(*) > 1
ORDER BY N DESC, IdPostulacion, FMI;
"""
df_dup_rp = run_df(sql_dup_rp)
show("1) Duplicados en Postulacion.RegistroProceso por (IdPostulacion, FMI)", df_dup_rp)

# (Opcional) ver los registros completos de los duplicados (si existen)
if not df_dup_rp.empty:
    sql_dup_rp_det = """
    SELECT rp.*
    FROM ViabilidadJuridica.Postulacion.RegistroProceso rp
    JOIN (
        SELECT IdPostulacion, FMI
        FROM ViabilidadJuridica.Postulacion.RegistroProceso
        GROUP BY IdPostulacion, FMI
        HAVING COUNT(*) > 1
    ) d ON d.IdPostulacion = rp.IdPostulacion AND d.FMI = rp.FMI
    ORDER BY rp.IdPostulacion, rp.FMI, rp.IdRegistroProceso;
    """
    df_dup_rp_det = run_df(sql_dup_rp_det)
    show("1.1) Detalle duplicados RegistroProceso", df_dup_rp_det, max_rows=50)

# -----------------------------
# 2) Integridad FK: EstudioTerreno -> RegistroProceso
# -----------------------------
sql_fk_et = """
SELECT TOP 100
    et.IdEstudioTerreno,
    et.IdRegistroProceso
FROM ViabilidadJuridica.Postulacion.EstudioTerreno et
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = et.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY et.IdEstudioTerreno DESC;
"""
df_fk_et = run_df(sql_fk_et)
show("2) Huérfanos: EstudioTerreno.IdRegistroProceso sin RegistroProceso", df_fk_et)

# -----------------------------
# 3) Integridad FK: ConceptoFinal -> RegistroProceso
# -----------------------------
sql_fk_cf = """
SELECT TOP 100
    cf.IdConceptoFinal,
    cf.IdRegistroProceso
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal cf
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = cf.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY cf.IdConceptoFinal DESC;
"""
df_fk_cf = run_df(sql_fk_cf)
show("3) Huérfanos: ConceptoFinal.IdRegistroProceso sin RegistroProceso", df_fk_cf)

# -----------------------------
# 4) Integridad FK: ConceptosPrevio -> RegistroProceso
# -----------------------------
sql_fk_cp = """
SELECT TOP 100
    cp.IdConceptoPrevio,
    cp.IdRegistroProceso
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio cp
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = cp.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY cp.IdConceptoPrevio DESC;
"""
df_fk_cp = run_df(sql_fk_cp)
show("4) Huérfanos: ConceptosPrevio.IdRegistroProceso sin RegistroProceso", df_fk_cp)

# -----------------------------
# 5) Integridad FK: MedidaProcesal -> EstudioTerreno
# -----------------------------
sql_fk_mp = """
SELECT TOP 100
    mp.IdMedidasProcesal,
    mp.IdEstudioTerreno
FROM ViabilidadJuridica.Postulacion.MedidaProcesal mp
LEFT JOIN ViabilidadJuridica.Postulacion.EstudioTerreno et
    ON et.IdEstudioTerreno = mp.IdEstudioTerreno
WHERE et.IdEstudioTerreno IS NULL
ORDER BY mp.IdMedidasProcesal DESC;
"""
df_fk_mp = run_df(sql_fk_mp)
show("5) Huérfanos: MedidaProcesal.IdEstudioTerreno sin EstudioTerreno", df_fk_mp)

# -----------------------------
# 6) Conteos rápidos (sanity check)
# -----------------------------
sql_counts = """
SELECT 'RegistroProceso'   AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.Postulacion.RegistroProceso
UNION ALL
SELECT 'EstudioTerreno'    AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.Postulacion.EstudioTerreno
UNION ALL
SELECT 'ConceptoFinal'     AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
UNION ALL
SELECT 'ConceptosPrevio'   AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
UNION ALL
SELECT 'MedidaProcesal'    AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.Postulacion.MedidaProcesal;
"""
df_counts = run_df(sql_counts)
show("6) Conteos actuales por tabla", df_counts, max_rows=100)

# -----------------------------
# 7) (Opcional) Conteos SOLO del bloque que migraste
#     Si migraste filas 2..7, ajusta el rango aquí.
#     Se identifica por IdRegistroProceso (asumiendo BD limpia antes) o por join a STG si aún existe.
# -----------------------------
IDRP_MIN = 2
IDRP_MAX = 7

sql_counts_block = """
SELECT 'RegistroProceso' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.RegistroProceso
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'EstudioTerreno' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.EstudioTerreno
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'ConceptoFinal' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'ConceptosPrevio' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'MedidaProcesal' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.MedidaProcesal mp
JOIN ViabilidadJuridica.Postulacion.EstudioTerreno et
  ON et.IdEstudioTerreno = mp.IdEstudioTerreno
WHERE et.IdRegistroProceso BETWEEN :a AND :b;
"""
df_counts_block = run_df(sql_counts_block, params={"a": IDRP_MIN, "b": IDRP_MAX})
show(f"7) Conteos SOLO para IdRegistroProceso {IDRP_MIN}..{IDRP_MAX}", df_counts_block, max_rows=100)

print("\nListo. Si todo sale en (OK) sin hallazgos en 2..5 y sin duplicados en 1, podemos masificar.")



1) Duplicados en Postulacion.RegistroProceso por (IdPostulacion, FMI)
(OK) Sin hallazgos.

2) Huérfanos: EstudioTerreno.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

3) Huérfanos: ConceptoFinal.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

4) Huérfanos: ConceptosPrevio.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

5) Huérfanos: MedidaProcesal.IdEstudioTerreno sin EstudioTerreno
(OK) Sin hallazgos.

6) Conteos actuales por tabla
          Tabla  N
RegistroProceso  6
 EstudioTerreno  6
  ConceptoFinal  6
ConceptosPrevio  5
 MedidaProcesal 35

7) Conteos SOLO para IdRegistroProceso 2..7
          Tabla  N
RegistroProceso  6
 EstudioTerreno  6
  ConceptoFinal  6
ConceptosPrevio  5
 MedidaProcesal 35

Listo. Si todo sale en (OK) sin hallazgos en 2..5 y sin duplicados en 1, podemos masificar.


## Migracion de los perimeros 2mil

In [82]:
import urllib.parse
from sqlalchemy import create_engine, text
import pandas as pd

# =====================================================
# 0) CONEXIÓN (usa tus mismas variables ya definidas)
# =====================================================
# server = r"172.23.92.97\MSSQLSERVER2022"
# database = "ViabilidadJuridica"
# user = "andres.garciav"
# password = "..."

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={user};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# =====================================================
# 1) CONFIG: migrar por lotes hasta 2000 filas (ajusta)
# =====================================================
START_FILA = 8
END_FILA = 2001   # 2..2000 inclusive
FILAS_EXCEL = list(range(START_FILA, END_FILA))

# Archivos de log
OUT_OK_CSV = "log_migracion_OK.csv"
OUT_ERR_CSV = "log_migracion_ERRORES.csv"

# =====================================================
# 2) Helpers DF
# =====================================================
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

def get_row_df(fila_excel: int):
    r = df.loc[df["Fila_Excel_aprox"] == fila_excel]
    if r.empty:
        raise ValueError(f"No existe Fila_Excel_aprox={fila_excel} en df.")
    return r.iloc[0]

def get_raw(r, col):
    if col not in df.columns:
        return None
    v = r[col]
    if pd.isna(v):
        return None
    return v

def get_str(r, col):
    v = get_raw(r, col)
    if v is None:
        return None
    s = str(v).strip()
    if s == "" or s.upper() in ("NAN", "NAT", "NONE"):
        return None
    return s

def to_int_value(x):
    if x is None:
        return None
    try:
        return int(float(x))
    except:
        return None

def to_int(r, col):
    return to_int_value(get_str(r, col))

def to_dt(r, col):
    v = get_raw(r, col)
    if v is None:
        return None
    ts = pd.to_datetime(v, errors="coerce")
    if pd.isna(ts):
        return None
    return ts.to_pydatetime()

# =====================================================
# 3) Helpers DB: columnas reales (evita errores por tildes)
# =====================================================
def get_table_cols(schema: str, table: str):
    sql = text("""
    SELECT c.name
    FROM sys.columns c
    JOIN sys.tables t ON c.object_id = t.object_id
    JOIN sys.schemas s ON t.schema_id = s.schema_id
    WHERE s.name = :schema AND t.name = :table
    ORDER BY c.column_id;
    """)
    with engine.connect() as conn:
        rows = conn.execute(sql, {"schema": schema, "table": table}).fetchall()
    return [x[0] for x in rows]

COLS_CONCEPTOFINAL = set(get_table_cols("AnalisisJuridico", "ConceptoFinal"))

# =====================================================
# 4) SQL statements
# =====================================================
sql_insert_stg = text("""
INSERT INTO ViabilidadJuridica.dbo.STG_RegistroProceso
(
  Fila_Excel_aprox,
  IdPostulacion,
  FMI,
  NumeroExpediente,
  IdFuenteProceso,
  IdTipoProceso,
  IdEtapaProcesal,
  Dependencia,
  RadicadoOrfeo
)
VALUES
(
  :Fila_Excel_aprox,
  :IdPostulacion,
  :FMI,
  :NumeroExpediente,
  :IdFuenteProceso,
  :IdTipoProceso,
  :IdEtapaProcesal,
  :Dependencia,
  :RadicadoOrfeo
);
""")

sql_get_stg = text("""
SELECT TOP 1
    Fila_Excel_aprox,
    IdPostulacion,
    FMI,
    NumeroExpediente,
    IdFuenteProceso,
    IdTipoProceso,
    IdEtapaProcesal,
    Dependencia,
    RadicadoOrfeo
FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox = :fila;
""")

# Idempotencia SOLO por IdPostulacion:
sql_get_rp_by_idpost = text("""
SELECT TOP 1 IdRegistroProceso
FROM ViabilidadJuridica.Postulacion.RegistroProceso
WHERE IdPostulacion = :IdPostulacion;
""")

sql_insert_rp = text("""
INSERT INTO ViabilidadJuridica.Postulacion.RegistroProceso
(
    IdPostulacion, FMI, NumeroExpediente,
    IdFuenteProceso, IdTipoProceso, IdEtapaProcesal,
    Dependencia, RadicadoOrfeo
)
OUTPUT inserted.IdRegistroProceso
VALUES
(
    :IdPostulacion, :FMI, :NumeroExpediente,
    :IdFuenteProceso, :IdTipoProceso, :IdEtapaProcesal,
    :Dependencia, :RadicadoOrfeo
);
""")

# EstudioTerreno 1:1 por IdRegistroProceso (idempotente)
sql_get_et_by_rp = text("""
SELECT TOP 1 IdEstudioTerreno
FROM ViabilidadJuridica.Postulacion.EstudioTerreno
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

sql_insert_et = text("""
INSERT INTO ViabilidadJuridica.Postulacion.EstudioTerreno
(
    IdRegistroProceso,
    IdLocalizacion,
    AreaRegistral,
    CirculoRegistral,
    AreaCalculada,
    TipoPersonaTitular,
    NombrePropietario,
    ApellidoPropietario,
    IdTipoDocumento,
    Identificacion,
    NaturalezaJuridica,
    AcreditacionPropiedad
)
OUTPUT inserted.IdEstudioTerreno
VALUES
(
    :IdRegistroProceso,
    :IdLocalizacion,
    :AreaRegistral,
    :CirculoRegistral,
    :AreaCalculada,
    :TipoPersonaTitular,
    :NombrePropietario,
    :ApellidoPropietario,
    :IdTipoDocumento,
    :Identificacion,
    :NaturalezaJuridica,
    :AcreditacionPropiedad
);
""")

# ConceptoFinal 1:1 por IdRegistroProceso (idempotente)
sql_get_cf_by_rp = text("""
SELECT TOP 1 IdConceptoFinal
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

# ConceptosPrevio: aquí lo tratamos como 1:1 por IdRegistroProceso (idempotente)
sql_get_cp_by_rp = text("""
SELECT TOP 1 IdConceptoPrevio
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

sql_insert_cp = text("""
INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
(
    IdRegistroProceso,
    FechaInforme,
    Concepto
)
OUTPUT inserted.IdConceptoPrevio
VALUES
(
    :IdRegistroProceso,
    :FechaInforme,
    :Concepto
);
""")

# MedidaProcesal: si ya hay alguna para ese EstudioTerreno, no insertamos (idempotente simple)
sql_mp_exists = text("""
SELECT TOP 1 1
FROM ViabilidadJuridica.Postulacion.MedidaProcesal
WHERE IdEstudioTerreno = :IdEstudioTerreno;
""")

sql_insert_mp = text("""
INSERT INTO ViabilidadJuridica.Postulacion.MedidaProcesal
(
    IdEstudioTerreno,
    Objeto,
    Valor,
    Anotacion,
    TipoClasificacion
)
VALUES
(
    :IdEstudioTerreno,
    :Objeto,
    :Valor,
    :Anotacion,
    :TipoClasificacion
);
""")

# =====================================================
# 5) Armado de payloads
# =====================================================
def payload_stg_from_dfrow(r):
    fila = int(r["Fila_Excel_aprox"])
    idpost = get_str(r, "IdPostulacion_SQL")
    fmi = get_str(r, "FMI_SQL") or get_str(r, "FMI")
    if not idpost:
        raise ValueError("IdPostulacion_SQL vacío.")
    if not fmi:
        raise ValueError("FMI_SQL/FMI vacío.")
    return {
        "Fila_Excel_aprox": fila,
        "IdPostulacion": idpost,
        "FMI": fmi,
        "NumeroExpediente": get_str(r, "NumeroExpediente_SQL"),
        "IdFuenteProceso": to_int_value(get_str(r, "IdFuenteProceso_SQL") or get_str(r, "IdFuenteProceso")),
        "IdTipoProceso": to_int_value(get_str(r, "IdTipoProceso_SQL") or get_str(r, "IdTipoProceso")),
        "IdEtapaProcesal": to_int_value(get_str(r, "IdEtapaProcesal_SQL") or get_str(r, "IdEtapaProcesal")),
        "Dependencia": get_str(r, "Dependencia_SQL"),
        "RadicadoOrfeo": get_str(r, "RadicadoOrfeo_SQL"),
    }

def payload_et_from_dfrow(r, id_registro):
    return {
        "IdRegistroProceso": int(id_registro),
        "IdLocalizacion": to_int_value(get_str(r, "IdLocalizacion_SQL")),
        "AreaRegistral": get_raw(r, "AreaRegistral_SQL"),
        "CirculoRegistral": get_str(r, "CirculoRegistral_SQL"),
        "AreaCalculada": get_raw(r, "AreaCalculada_SQL"),
        "TipoPersonaTitular": get_str(r, "TipoPersonaTitular_SQL"),
        "NombrePropietario": get_str(r, "NombrePropietario_SQL"),
        "ApellidoPropietario": get_str(r, "ApellidoPropietario_SQL"),
        "IdTipoDocumento": to_int_value(get_str(r, "IdTipoDocumento_SQL_MIGRAR")),
        "Identificacion": get_str(r, "Identificacion_SQL_TXT_MIGRAR"),
        "NaturalezaJuridica": get_str(r, "NaturalezaJuridica_SQL_FINAL"),
        "AcreditacionPropiedad": get_str(r, "AcreditacionPropiedad_SQL_FINAL"),
    }

def payload_cp_from_dfrow(r, id_registro):
    return {
        "IdRegistroProceso": int(id_registro),
        "FechaInforme": to_dt(r, "FechaInformePrevio_SQL"),
        "Concepto": get_str(r, "ConceptoPrevio_SQL"),
    }

def build_cf_payload_dfrow(r, id_registro):
    candidates = {
        "ConceptoActualDeViabilidadJuridica": get_str(r, "ConceptoActualDeViabilidadJuridica_SQL"),
        "FechaInforme": to_dt(r, "FechaInforme_SQL"),
        "IdTipoViabilidad": to_int(r, "IdTipoViabilidad_SQL"),
        "IdTipoInforme": to_int(r, "IdTipoInforme_SQL"),
        "IdTipoNoViabilidad": to_int(r, "IdTipoNoViabilidad_SQL"),
        "InsumosPendientes": get_str(r, "InsumosPendientes_SQL"),
        "FechaEntregaARevisor": to_dt(r, "FechaEntregaARevisor_SQL"),
        "AbogadoSustanciadorAsignado": get_str(r, "AbogadoSustanciadorAsignado_SQL"),
        "AbogadoRevisorAsignado": get_str(r, "AbogadoRevisorAsignado_SQL"),
        "NroReparto": to_int(r, "NroReparto_SQL"),
        "FechaAsignacionReparto": to_dt(r, "FechaAsignacionReparto_SQL"),
        "FechaPlazoEntregaARevisor": to_dt(r, "FechaPlazoEntregaARevisor_SQL"),
        "IdTipoEstadoRevision": to_int(r, "IdTipoEstadoRevision_SQL"),
        "ObservacionesRevisor": get_str(r, "ObservacionesRevisor_SQL"),
        "EntregoCarpetaSoportes": get_str(r, "EntregoCarpetaSoportes_SQL"),
        "FechaEnvioACoordinacion": to_dt(r, "FechaEnvioACoordinacion_SQL"),
        "EstadoAprobacionCoordinadora": get_str(r, "EstadoAprobacionCoordinadora_SQL"),
        "FechaRemisionSoportesAGestoraDocumental": to_dt(r, "FechaRemisionSoportesAGestoraDocumental_SQL"),
        "FechaRemisionInformeAGestoraDocumental": to_dt(r, "FechaRemisionInformeAGestoraDocumental_SQL"),
        "FechaCargueInformeJuridicoExpOrfeo": to_dt(r, "FechaCargueInformeJuridicoExpOrfeo_SQL"),
        "FechaDeCargueDocsYSoportesExpOrfeo": to_dt(r, "FechaDeCargueDocsYSoportesExpOrfeo_SQL"),
        "FechaGestionEtapaSIT": to_dt(r, "FechaGestionEtapaSIT_SQL"),
    }

    # Ajustes por tildes si existen en BD
    if "FechaRemisiónInformeAGestoraDocumental" in COLS_CONCEPTOFINAL:
        candidates["FechaRemisiónInformeAGestoraDocumental"] = candidates.pop("FechaRemisionInformeAGestoraDocumental", None)
    if "FechaCargueInformeJurídicoExpOrfeo" in COLS_CONCEPTOFINAL and "FechaCargueInformeJuridicoExpOrfeo" in candidates:
        candidates["FechaCargueInformeJurídicoExpOrfeo"] = candidates.pop("FechaCargueInformeJuridicoExpOrfeo", None)

    payload = {"IdRegistroProceso": int(id_registro)}
    for k, v in candidates.items():
        if k in COLS_CONCEPTOFINAL:
            payload[k] = v
    return payload

def insert_conceptofinal_dynamic(conn, payload):
    has_data = any(v is not None for k, v in payload.items() if k != "IdRegistroProceso")
    if not has_data:
        return None

    cols = list(payload.keys())
    params_cols = [f":{c}" for c in cols]
    sql = f"""
    INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
    ({", ".join(cols)})
    OUTPUT inserted.IdConceptoFinal
    VALUES
    ({", ".join(params_cols)});
    """
    res = conn.execute(text(sql), payload)
    return res.fetchone()[0]

def build_mp_rows(r, id_estudio):
    def cell(col):
        return get_str(r, col)

    def add(objeto, col_valor, col_anot, col_tipo=None):
        valor = cell(col_valor)
        anot = cell(col_anot)
        if valor is None and anot is None:
            return None
        return {
            "IdEstudioTerreno": int(id_estudio),
            "Objeto": objeto,
            "Valor": (valor or "SI"),
            "Anotacion": anot,
            "TipoClasificacion": cell(col_tipo) if col_tipo else None,
        }

    specs = [
        ("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL", None),
        ("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL", None),
        ("MEDIDAS CAUTELARES", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL", None),
        ("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL"),
        ("RTDAF LEY 1448", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL", None),
        ("OFERTA OTRAS ENTIDADES", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL", None),
        ("PROCESOS DE CLARIFICACIÓN", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL", None),
    ]

    rows = []
    for spec in specs:
        row = add(*spec)
        if row:
            rows.append(row)
    return rows

# =====================================================
# 6) MIGRACIÓN (transacción por fila + logs)
# - Idempotencia pedida: RegistroProceso por IdPostulacion (SIN usar FMI).
# - Además (para evitar duplicados en re-ejecución): ET/CF/CP/MP también idempotentes.
# =====================================================
oks = []
errs = []

for fila in FILAS_EXCEL:
    try:
        r = get_row_df(fila)
        stg_payload = payload_stg_from_dfrow(r)

        with engine.begin() as conn:
            # A) Insertar en STG (puede repetirse; no afecta RP por idempotencia)
            conn.execute(sql_insert_stg, stg_payload)

            # B) Leer STG recién insertado
            stg = pd.read_sql(sql_get_stg, conn, params={"fila": fila})
            if stg.empty:
                raise ValueError("No se encontró STG recién insertado.")
            stg_row = stg.iloc[0].to_dict()

            idpost = str(stg_row["IdPostulacion"]).strip()

            # C) RegistroProceso FINAL (idempotente por IdPostulacion)
            rp_exist = conn.execute(sql_get_rp_by_idpost, {"IdPostulacion": idpost}).fetchone()
            if rp_exist:
                new_id_rp = int(rp_exist[0])
                rp_accion = "EXISTE"
            else:
                payload_reg = {
                    "IdPostulacion": idpost,
                    "FMI": str(stg_row["FMI"]).strip() if stg_row.get("FMI") is not None else None,
                    "NumeroExpediente": stg_row.get("NumeroExpediente"),
                    "IdFuenteProceso": stg_row.get("IdFuenteProceso"),
                    "IdTipoProceso": stg_row.get("IdTipoProceso"),
                    "IdEtapaProcesal": stg_row.get("IdEtapaProcesal"),
                    "Dependencia": stg_row.get("Dependencia"),
                    "RadicadoOrfeo": stg_row.get("RadicadoOrfeo"),
                }
                new_id_rp = conn.execute(sql_insert_rp, payload_reg).fetchone()[0]
                rp_accion = "INSERT"

            # D) EstudioTerreno (idempotente por IdRegistroProceso)
            et_exist = conn.execute(sql_get_et_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if et_exist:
                new_id_et = int(et_exist[0])
                et_accion = "EXISTE"
            else:
                payload_et = payload_et_from_dfrow(r, new_id_rp)
                new_id_et = conn.execute(sql_insert_et, payload_et).fetchone()[0]
                et_accion = "INSERT"

            # E) ConceptoFinal (idempotente por IdRegistroProceso)
            cf_exist = conn.execute(sql_get_cf_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if cf_exist:
                new_id_cf = int(cf_exist[0])
                cf_accion = "EXISTE"
            else:
                payload_cf = build_cf_payload_dfrow(r, new_id_rp)
                new_id_cf = insert_conceptofinal_dynamic(conn, payload_cf)
                cf_accion = "INSERT" if new_id_cf is not None else "SKIP_VACIO"

            # F) ConceptosPrevio (idempotente por IdRegistroProceso)
            cp_exist = conn.execute(sql_get_cp_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if cp_exist:
                new_id_cp = int(cp_exist[0])
                cp_accion = "EXISTE"
            else:
                payload_cp = payload_cp_from_dfrow(r, new_id_rp)
                has_cp = (payload_cp["FechaInforme"] is not None) or (payload_cp["Concepto"] is not None)
                if has_cp:
                    new_id_cp = conn.execute(sql_insert_cp, payload_cp).fetchone()[0]
                    cp_accion = "INSERT"
                else:
                    new_id_cp = None
                    cp_accion = "SKIP_VACIO"

            # G) MedidaProcesal (idempotente simple: si ya hay algo para ese ET, no repetir)
            mp_already = conn.execute(sql_mp_exists, {"IdEstudioTerreno": new_id_et}).fetchone()
            if mp_already:
                mp_ins = 0
                mp_accion = "EXISTE"
            else:
                mp_rows = build_mp_rows(r, new_id_et)
                if mp_rows:
                    conn.execute(sql_insert_mp, mp_rows)  # executemany
                    mp_ins = len(mp_rows)
                    mp_accion = "INSERT"
                else:
                    mp_ins = 0
                    mp_accion = "SKIP_VACIO"

        oks.append({
            "Fila_Excel_aprox": fila,
            "IdPostulacion": idpost,
            "IdRegistroProceso": int(new_id_rp),
            "IdEstudioTerreno": int(new_id_et),
            "IdConceptoFinal": None if new_id_cf is None else int(new_id_cf),
            "IdConceptoPrevio": None if new_id_cp is None else int(new_id_cp),
            "MedidaProcesal_insertadas": int(mp_ins),
            "RP_accion": rp_accion,
            "ET_accion": et_accion,
            "CF_accion": cf_accion,
            "CP_accion": cp_accion,
            "MP_accion": mp_accion,
            "Estado": "OK",
        })

        if fila % 50 == 0:
            print(f"[OK] Hasta fila {fila} (última: RP={new_id_rp}, RP={rp_accion}, ET={et_accion}, CF={cf_accion}, CP={cp_accion}, MP={mp_accion})")

    except Exception as e:
        errs.append({
            "Fila_Excel_aprox": fila,
            "Estado": "ERROR",
            "Error": str(e)[:4000],
        })
        print(f"[ERROR] Fila {fila} -> {e}")

# =====================================================
# 7) Exportar logs a CSV (sin frenar toda la carga)
# =====================================================
df_ok = pd.DataFrame(oks)
df_err = pd.DataFrame(errs)

df_ok.to_csv(OUT_OK_CSV, index=False, encoding="utf-8-sig")
df_err.to_csv(OUT_ERR_CSV, index=False, encoding="utf-8-sig")

print("\n=== RESUMEN ===")
print("OK:", len(df_ok), "| ERRORES:", len(df_err))
print("Logs:", OUT_OK_CSV, "y", OUT_ERR_CSV)

# Vista rápida
if not df_err.empty:
    print("\n=== Primeros 10 ERRORES ===")
    print(df_err.head(10).to_string(index=False))


[OK] Hasta fila 50 (última: RP=50, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 100 (última: RP=100, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=INSERT)
[OK] Hasta fila 150 (última: RP=150, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 200 (última: RP=200, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=INSERT)
[OK] Hasta fila 250 (última: RP=250, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=SKIP_VACIO)
[OK] Hasta fila 300 (última: RP=300, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=INSERT)
[OK] Hasta fila 350 (última: RP=350, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=INSERT)
[OK] Hasta fila 400 (última: RP=400, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=INSERT)
[OK] Hasta fila 450 (última: RP=450, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 500 (última: RP=500, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 550 (última: RP=550, RP=INSERT, ET=INS

## VALIDACION DE LOS PRIMEROS 2MIL

In [83]:
from sqlalchemy import text
import pandas as pd

# =====================================================
# VALIDACIONES POST-MIGRACIÓN (SQL)
# - Duplicados en RegistroProceso por llave natural (IdPostulacion)
# - Integridad FK: EstudioTerreno / ConceptoFinal / ConceptosPrevio
# - Resumen de conteos
# Requiere: engine ya creado
# =====================================================

def run_df(sql: str, params: dict | None = None) -> pd.DataFrame:
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn, params=params or {})

def show(title: str, df_: pd.DataFrame, max_rows: int = 20):
    print("\n" + "="*90)
    print(title)
    print("="*90)
    if df_.empty:
        print("(OK) Sin hallazgos.")
    else:
        print(df_.head(max_rows).to_string(index=False))
        if len(df_) > max_rows:
            print(f"... ({len(df_)} filas, mostrando {max_rows})")

# -----------------------------
# 1) Duplicados en RegistroProceso por llave natural (IdPostulacion)
# -----------------------------
sql_dup_rp = """
SELECT
    IdPostulacion,
    COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.RegistroProceso
GROUP BY IdPostulacion
HAVING COUNT(*) > 1
ORDER BY N DESC, IdPostulacion;
"""
df_dup_rp = run_df(sql_dup_rp)
show("1) Duplicados en Postulacion.RegistroProceso por (IdPostulacion)", df_dup_rp)

# Detalle duplicados (si existen)
if not df_dup_rp.empty:
    sql_dup_rp_det = """
    SELECT rp.*
    FROM ViabilidadJuridica.Postulacion.RegistroProceso rp
    JOIN (
        SELECT IdPostulacion
        FROM ViabilidadJuridica.Postulacion.RegistroProceso
        GROUP BY IdPostulacion
        HAVING COUNT(*) > 1
    ) d ON d.IdPostulacion = rp.IdPostulacion
    ORDER BY rp.IdPostulacion, rp.IdRegistroProceso;
    """
    df_dup_rp_det = run_df(sql_dup_rp_det)
    show("1.1) Detalle duplicados RegistroProceso", df_dup_rp_det, max_rows=50)

# -----------------------------
# 2) Integridad FK: EstudioTerreno -> RegistroProceso
# -----------------------------
sql_fk_et = """
SELECT TOP 100
    et.IdEstudioTerreno,
    et.IdRegistroProceso
FROM ViabilidadJuridica.Postulacion.EstudioTerreno et
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = et.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY et.IdEstudioTerreno DESC;
"""
df_fk_et = run_df(sql_fk_et)
show("2) Huérfanos: EstudioTerreno.IdRegistroProceso sin RegistroProceso", df_fk_et)

# -----------------------------
# 3) Integridad FK: ConceptoFinal -> RegistroProceso
# -----------------------------
sql_fk_cf = """
SELECT TOP 100
    cf.IdConceptoFinal,
    cf.IdRegistroProceso
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal cf
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = cf.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY cf.IdConceptoFinal DESC;
"""
df_fk_cf = run_df(sql_fk_cf)
show("3) Huérfanos: ConceptoFinal.IdRegistroProceso sin RegistroProceso", df_fk_cf)

# -----------------------------
# 4) Integridad FK: ConceptosPrevio -> RegistroProceso
# -----------------------------
sql_fk_cp = """
SELECT TOP 100
    cp.IdConceptoPrevio,
    cp.IdRegistroProceso
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio cp
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = cp.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY cp.IdConceptoPrevio DESC;
"""
df_fk_cp = run_df(sql_fk_cp)
show("4) Huérfanos: ConceptosPrevio.IdRegistroProceso sin RegistroProceso", df_fk_cp)

# -----------------------------
# 5) Integridad FK: MedidaProcesal -> EstudioTerreno
# -----------------------------
sql_fk_mp = """
SELECT TOP 100
    mp.IdMedidasProcesal,
    mp.IdEstudioTerreno
FROM ViabilidadJuridica.Postulacion.MedidaProcesal mp
LEFT JOIN ViabilidadJuridica.Postulacion.EstudioTerreno et
    ON et.IdEstudioTerreno = mp.IdEstudioTerreno
WHERE et.IdEstudioTerreno IS NULL
ORDER BY mp.IdMedidasProcesal DESC;
"""
df_fk_mp = run_df(sql_fk_mp)
show("5) Huérfanos: MedidaProcesal.IdEstudioTerreno sin EstudioTerreno", df_fk_mp)

# -----------------------------
# 6) Conteos globales
# -----------------------------
sql_counts = """
SELECT 'RegistroProceso'   AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.Postulacion.RegistroProceso
UNION ALL
SELECT 'EstudioTerreno'    AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.Postulacion.EstudioTerreno
UNION ALL
SELECT 'ConceptoFinal'     AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
UNION ALL
SELECT 'ConceptosPrevio'   AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
UNION ALL
SELECT 'MedidaProcesal'    AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.Postulacion.MedidaProcesal;
"""
df_counts = run_df(sql_counts)
show("6) Conteos actuales por tabla", df_counts, max_rows=100)

# -----------------------------
# 7) Conteos SOLO del bloque migrado (2..2000)
# -----------------------------
IDRP_MIN = 2
IDRP_MAX = 2000

sql_counts_block = """
SELECT 'RegistroProceso' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.RegistroProceso
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'EstudioTerreno' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.EstudioTerreno
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'ConceptoFinal' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'ConceptosPrevio' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'MedidaProcesal' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.MedidaProcesal mp
JOIN ViabilidadJuridica.Postulacion.EstudioTerreno et
  ON et.IdEstudioTerreno = mp.IdEstudioTerreno
WHERE et.IdRegistroProceso BETWEEN :a AND :b;
"""
df_counts_block = run_df(sql_counts_block, params={"a": IDRP_MIN, "b": IDRP_MAX})
show(f"7) Conteos SOLO para IdRegistroProceso {IDRP_MIN}..{IDRP_MAX}", df_counts_block, max_rows=100)

print("\nListo. Si todo sale en (OK) sin hallazgos en 2..5 y sin duplicados en 1, puedes seguir con el siguiente lote.")



1) Duplicados en Postulacion.RegistroProceso por (IdPostulacion)
(OK) Sin hallazgos.

2) Huérfanos: EstudioTerreno.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

3) Huérfanos: ConceptoFinal.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

4) Huérfanos: ConceptosPrevio.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

5) Huérfanos: MedidaProcesal.IdEstudioTerreno sin EstudioTerreno
(OK) Sin hallazgos.

6) Conteos actuales por tabla
          Tabla    N
RegistroProceso 1999
 EstudioTerreno 1999
  ConceptoFinal 1999
ConceptosPrevio 1486
 MedidaProcesal 6647

7) Conteos SOLO para IdRegistroProceso 2..2000
          Tabla    N
RegistroProceso 1999
 EstudioTerreno 1999
  ConceptoFinal 1999
ConceptosPrevio 1486
 MedidaProcesal 6647

Listo. Si todo sale en (OK) sin hallazgos en 2..5 y sin duplicados en 1, puedes seguir con el siguiente lote.


## Segunda migracion 2000 a 4000

In [84]:
import urllib.parse
from sqlalchemy import create_engine, text
import pandas as pd

# =====================================================
# 0) CONEXIÓN (usa tus mismas variables ya definidas)
# =====================================================
# server = r"172.23.92.97\MSSQLSERVER2022"
# database = "ViabilidadJuridica"
# user = "andres.garciav"
# password = "..."

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={user};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# =====================================================
# 1) CONFIG: migrar por lotes hasta 2000 filas (ajusta)
# =====================================================
START_FILA = 2001
END_FILA = 4001   # 2001..4000 inclusive
FILAS_EXCEL = list(range(START_FILA, END_FILA))


# Archivos de log
OUT_OK_CSV = "log_migracion_OK.csv"
OUT_ERR_CSV = "log_migracion_ERRORES.csv"

# =====================================================
# 2) Helpers DF
# =====================================================
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

def get_row_df(fila_excel: int):
    r = df.loc[df["Fila_Excel_aprox"] == fila_excel]
    if r.empty:
        raise ValueError(f"No existe Fila_Excel_aprox={fila_excel} en df.")
    return r.iloc[0]

def get_raw(r, col):
    if col not in df.columns:
        return None
    v = r[col]
    if pd.isna(v):
        return None
    return v

def get_str(r, col):
    v = get_raw(r, col)
    if v is None:
        return None
    s = str(v).strip()
    if s == "" or s.upper() in ("NAN", "NAT", "NONE"):
        return None
    return s

def to_int_value(x):
    if x is None:
        return None
    try:
        return int(float(x))
    except:
        return None

def to_int(r, col):
    return to_int_value(get_str(r, col))

def to_dt(r, col):
    v = get_raw(r, col)
    if v is None:
        return None
    ts = pd.to_datetime(v, errors="coerce")
    if pd.isna(ts):
        return None
    return ts.to_pydatetime()

# =====================================================
# 3) Helpers DB: columnas reales (evita errores por tildes)
# =====================================================
def get_table_cols(schema: str, table: str):
    sql = text("""
    SELECT c.name
    FROM sys.columns c
    JOIN sys.tables t ON c.object_id = t.object_id
    JOIN sys.schemas s ON t.schema_id = s.schema_id
    WHERE s.name = :schema AND t.name = :table
    ORDER BY c.column_id;
    """)
    with engine.connect() as conn:
        rows = conn.execute(sql, {"schema": schema, "table": table}).fetchall()
    return [x[0] for x in rows]

COLS_CONCEPTOFINAL = set(get_table_cols("AnalisisJuridico", "ConceptoFinal"))

# =====================================================
# 4) SQL statements
# =====================================================
sql_insert_stg = text("""
INSERT INTO ViabilidadJuridica.dbo.STG_RegistroProceso
(
  Fila_Excel_aprox,
  IdPostulacion,
  FMI,
  NumeroExpediente,
  IdFuenteProceso,
  IdTipoProceso,
  IdEtapaProcesal,
  Dependencia,
  RadicadoOrfeo
)
VALUES
(
  :Fila_Excel_aprox,
  :IdPostulacion,
  :FMI,
  :NumeroExpediente,
  :IdFuenteProceso,
  :IdTipoProceso,
  :IdEtapaProcesal,
  :Dependencia,
  :RadicadoOrfeo
);
""")

sql_get_stg = text("""
SELECT TOP 1
    Fila_Excel_aprox,
    IdPostulacion,
    FMI,
    NumeroExpediente,
    IdFuenteProceso,
    IdTipoProceso,
    IdEtapaProcesal,
    Dependencia,
    RadicadoOrfeo
FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox = :fila;
""")

# Idempotencia SOLO por IdPostulacion:
sql_get_rp_by_idpost = text("""
SELECT TOP 1 IdRegistroProceso
FROM ViabilidadJuridica.Postulacion.RegistroProceso
WHERE IdPostulacion = :IdPostulacion;
""")

sql_insert_rp = text("""
INSERT INTO ViabilidadJuridica.Postulacion.RegistroProceso
(
    IdPostulacion, FMI, NumeroExpediente,
    IdFuenteProceso, IdTipoProceso, IdEtapaProcesal,
    Dependencia, RadicadoOrfeo
)
OUTPUT inserted.IdRegistroProceso
VALUES
(
    :IdPostulacion, :FMI, :NumeroExpediente,
    :IdFuenteProceso, :IdTipoProceso, :IdEtapaProcesal,
    :Dependencia, :RadicadoOrfeo
);
""")

# EstudioTerreno 1:1 por IdRegistroProceso (idempotente)
sql_get_et_by_rp = text("""
SELECT TOP 1 IdEstudioTerreno
FROM ViabilidadJuridica.Postulacion.EstudioTerreno
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

sql_insert_et = text("""
INSERT INTO ViabilidadJuridica.Postulacion.EstudioTerreno
(
    IdRegistroProceso,
    IdLocalizacion,
    AreaRegistral,
    CirculoRegistral,
    AreaCalculada,
    TipoPersonaTitular,
    NombrePropietario,
    ApellidoPropietario,
    IdTipoDocumento,
    Identificacion,
    NaturalezaJuridica,
    AcreditacionPropiedad
)
OUTPUT inserted.IdEstudioTerreno
VALUES
(
    :IdRegistroProceso,
    :IdLocalizacion,
    :AreaRegistral,
    :CirculoRegistral,
    :AreaCalculada,
    :TipoPersonaTitular,
    :NombrePropietario,
    :ApellidoPropietario,
    :IdTipoDocumento,
    :Identificacion,
    :NaturalezaJuridica,
    :AcreditacionPropiedad
);
""")

# ConceptoFinal 1:1 por IdRegistroProceso (idempotente)
sql_get_cf_by_rp = text("""
SELECT TOP 1 IdConceptoFinal
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

# ConceptosPrevio: aquí lo tratamos como 1:1 por IdRegistroProceso (idempotente)
sql_get_cp_by_rp = text("""
SELECT TOP 1 IdConceptoPrevio
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

sql_insert_cp = text("""
INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
(
    IdRegistroProceso,
    FechaInforme,
    Concepto
)
OUTPUT inserted.IdConceptoPrevio
VALUES
(
    :IdRegistroProceso,
    :FechaInforme,
    :Concepto
);
""")

# MedidaProcesal: si ya hay alguna para ese EstudioTerreno, no insertamos (idempotente simple)
sql_mp_exists = text("""
SELECT TOP 1 1
FROM ViabilidadJuridica.Postulacion.MedidaProcesal
WHERE IdEstudioTerreno = :IdEstudioTerreno;
""")

sql_insert_mp = text("""
INSERT INTO ViabilidadJuridica.Postulacion.MedidaProcesal
(
    IdEstudioTerreno,
    Objeto,
    Valor,
    Anotacion,
    TipoClasificacion
)
VALUES
(
    :IdEstudioTerreno,
    :Objeto,
    :Valor,
    :Anotacion,
    :TipoClasificacion
);
""")

# =====================================================
# 5) Armado de payloads
# =====================================================
def payload_stg_from_dfrow(r):
    fila = int(r["Fila_Excel_aprox"])
    idpost = get_str(r, "IdPostulacion_SQL")
    fmi = get_str(r, "FMI_SQL") or get_str(r, "FMI")
    if not idpost:
        raise ValueError("IdPostulacion_SQL vacío.")
    if not fmi:
        raise ValueError("FMI_SQL/FMI vacío.")
    return {
        "Fila_Excel_aprox": fila,
        "IdPostulacion": idpost,
        "FMI": fmi,
        "NumeroExpediente": get_str(r, "NumeroExpediente_SQL"),
        "IdFuenteProceso": to_int_value(get_str(r, "IdFuenteProceso_SQL") or get_str(r, "IdFuenteProceso")),
        "IdTipoProceso": to_int_value(get_str(r, "IdTipoProceso_SQL") or get_str(r, "IdTipoProceso")),
        "IdEtapaProcesal": to_int_value(get_str(r, "IdEtapaProcesal_SQL") or get_str(r, "IdEtapaProcesal")),
        "Dependencia": get_str(r, "Dependencia_SQL"),
        "RadicadoOrfeo": get_str(r, "RadicadoOrfeo_SQL"),
    }

def payload_et_from_dfrow(r, id_registro):
    return {
        "IdRegistroProceso": int(id_registro),
        "IdLocalizacion": to_int_value(get_str(r, "IdLocalizacion_SQL")),
        "AreaRegistral": get_raw(r, "AreaRegistral_SQL"),
        "CirculoRegistral": get_str(r, "CirculoRegistral_SQL"),
        "AreaCalculada": get_raw(r, "AreaCalculada_SQL"),
        "TipoPersonaTitular": get_str(r, "TipoPersonaTitular_SQL"),
        "NombrePropietario": get_str(r, "NombrePropietario_SQL"),
        "ApellidoPropietario": get_str(r, "ApellidoPropietario_SQL"),
        "IdTipoDocumento": to_int_value(get_str(r, "IdTipoDocumento_SQL_MIGRAR")),
        "Identificacion": get_str(r, "Identificacion_SQL_TXT_MIGRAR"),
        "NaturalezaJuridica": get_str(r, "NaturalezaJuridica_SQL_FINAL"),
        "AcreditacionPropiedad": get_str(r, "AcreditacionPropiedad_SQL_FINAL"),
    }

def payload_cp_from_dfrow(r, id_registro):
    return {
        "IdRegistroProceso": int(id_registro),
        "FechaInforme": to_dt(r, "FechaInformePrevio_SQL"),
        "Concepto": get_str(r, "ConceptoPrevio_SQL"),
    }

def build_cf_payload_dfrow(r, id_registro):
    candidates = {
        "ConceptoActualDeViabilidadJuridica": get_str(r, "ConceptoActualDeViabilidadJuridica_SQL"),
        "FechaInforme": to_dt(r, "FechaInforme_SQL"),
        "IdTipoViabilidad": to_int(r, "IdTipoViabilidad_SQL"),
        "IdTipoInforme": to_int(r, "IdTipoInforme_SQL"),
        "IdTipoNoViabilidad": to_int(r, "IdTipoNoViabilidad_SQL"),
        "InsumosPendientes": get_str(r, "InsumosPendientes_SQL"),
        "FechaEntregaARevisor": to_dt(r, "FechaEntregaARevisor_SQL"),
        "AbogadoSustanciadorAsignado": get_str(r, "AbogadoSustanciadorAsignado_SQL"),
        "AbogadoRevisorAsignado": get_str(r, "AbogadoRevisorAsignado_SQL"),
        "NroReparto": to_int(r, "NroReparto_SQL"),
        "FechaAsignacionReparto": to_dt(r, "FechaAsignacionReparto_SQL"),
        "FechaPlazoEntregaARevisor": to_dt(r, "FechaPlazoEntregaARevisor_SQL"),
        "IdTipoEstadoRevision": to_int(r, "IdTipoEstadoRevision_SQL"),
        "ObservacionesRevisor": get_str(r, "ObservacionesRevisor_SQL"),
        "EntregoCarpetaSoportes": get_str(r, "EntregoCarpetaSoportes_SQL"),
        "FechaEnvioACoordinacion": to_dt(r, "FechaEnvioACoordinacion_SQL"),
        "EstadoAprobacionCoordinadora": get_str(r, "EstadoAprobacionCoordinadora_SQL"),
        "FechaRemisionSoportesAGestoraDocumental": to_dt(r, "FechaRemisionSoportesAGestoraDocumental_SQL"),
        "FechaRemisionInformeAGestoraDocumental": to_dt(r, "FechaRemisionInformeAGestoraDocumental_SQL"),
        "FechaCargueInformeJuridicoExpOrfeo": to_dt(r, "FechaCargueInformeJuridicoExpOrfeo_SQL"),
        "FechaDeCargueDocsYSoportesExpOrfeo": to_dt(r, "FechaDeCargueDocsYSoportesExpOrfeo_SQL"),
        "FechaGestionEtapaSIT": to_dt(r, "FechaGestionEtapaSIT_SQL"),
    }

    # Ajustes por tildes si existen en BD
    if "FechaRemisiónInformeAGestoraDocumental" in COLS_CONCEPTOFINAL:
        candidates["FechaRemisiónInformeAGestoraDocumental"] = candidates.pop("FechaRemisionInformeAGestoraDocumental", None)
    if "FechaCargueInformeJurídicoExpOrfeo" in COLS_CONCEPTOFINAL and "FechaCargueInformeJuridicoExpOrfeo" in candidates:
        candidates["FechaCargueInformeJurídicoExpOrfeo"] = candidates.pop("FechaCargueInformeJuridicoExpOrfeo", None)

    payload = {"IdRegistroProceso": int(id_registro)}
    for k, v in candidates.items():
        if k in COLS_CONCEPTOFINAL:
            payload[k] = v
    return payload

def insert_conceptofinal_dynamic(conn, payload):
    has_data = any(v is not None for k, v in payload.items() if k != "IdRegistroProceso")
    if not has_data:
        return None

    cols = list(payload.keys())
    params_cols = [f":{c}" for c in cols]
    sql = f"""
    INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
    ({", ".join(cols)})
    OUTPUT inserted.IdConceptoFinal
    VALUES
    ({", ".join(params_cols)});
    """
    res = conn.execute(text(sql), payload)
    return res.fetchone()[0]

def build_mp_rows(r, id_estudio):
    def cell(col):
        return get_str(r, col)

    def add(objeto, col_valor, col_anot, col_tipo=None):
        valor = cell(col_valor)
        anot = cell(col_anot)
        if valor is None and anot is None:
            return None
        return {
            "IdEstudioTerreno": int(id_estudio),
            "Objeto": objeto,
            "Valor": (valor or "SI"),
            "Anotacion": anot,
            "TipoClasificacion": cell(col_tipo) if col_tipo else None,
        }

    specs = [
        ("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL", None),
        ("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL", None),
        ("MEDIDAS CAUTELARES", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL", None),
        ("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL"),
        ("RTDAF LEY 1448", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL", None),
        ("OFERTA OTRAS ENTIDADES", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL", None),
        ("PROCESOS DE CLARIFICACIÓN", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL", None),
    ]

    rows = []
    for spec in specs:
        row = add(*spec)
        if row:
            rows.append(row)
    return rows

# =====================================================
# 6) MIGRACIÓN (transacción por fila + logs)
# - Idempotencia pedida: RegistroProceso por IdPostulacion (SIN usar FMI).
# - Además (para evitar duplicados en re-ejecución): ET/CF/CP/MP también idempotentes.
# =====================================================
oks = []
errs = []

for fila in FILAS_EXCEL:
    try:
        r = get_row_df(fila)
        stg_payload = payload_stg_from_dfrow(r)

        with engine.begin() as conn:
            # A) Insertar en STG (puede repetirse; no afecta RP por idempotencia)
            conn.execute(sql_insert_stg, stg_payload)

            # B) Leer STG recién insertado
            stg = pd.read_sql(sql_get_stg, conn, params={"fila": fila})
            if stg.empty:
                raise ValueError("No se encontró STG recién insertado.")
            stg_row = stg.iloc[0].to_dict()

            idpost = str(stg_row["IdPostulacion"]).strip()

            # C) RegistroProceso FINAL (idempotente por IdPostulacion)
            rp_exist = conn.execute(sql_get_rp_by_idpost, {"IdPostulacion": idpost}).fetchone()
            if rp_exist:
                new_id_rp = int(rp_exist[0])
                rp_accion = "EXISTE"
            else:
                payload_reg = {
                    "IdPostulacion": idpost,
                    "FMI": str(stg_row["FMI"]).strip() if stg_row.get("FMI") is not None else None,
                    "NumeroExpediente": stg_row.get("NumeroExpediente"),
                    "IdFuenteProceso": stg_row.get("IdFuenteProceso"),
                    "IdTipoProceso": stg_row.get("IdTipoProceso"),
                    "IdEtapaProcesal": stg_row.get("IdEtapaProcesal"),
                    "Dependencia": stg_row.get("Dependencia"),
                    "RadicadoOrfeo": stg_row.get("RadicadoOrfeo"),
                }
                new_id_rp = conn.execute(sql_insert_rp, payload_reg).fetchone()[0]
                rp_accion = "INSERT"

            # D) EstudioTerreno (idempotente por IdRegistroProceso)
            et_exist = conn.execute(sql_get_et_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if et_exist:
                new_id_et = int(et_exist[0])
                et_accion = "EXISTE"
            else:
                payload_et = payload_et_from_dfrow(r, new_id_rp)
                new_id_et = conn.execute(sql_insert_et, payload_et).fetchone()[0]
                et_accion = "INSERT"

            # E) ConceptoFinal (idempotente por IdRegistroProceso)
            cf_exist = conn.execute(sql_get_cf_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if cf_exist:
                new_id_cf = int(cf_exist[0])
                cf_accion = "EXISTE"
            else:
                payload_cf = build_cf_payload_dfrow(r, new_id_rp)
                new_id_cf = insert_conceptofinal_dynamic(conn, payload_cf)
                cf_accion = "INSERT" if new_id_cf is not None else "SKIP_VACIO"

            # F) ConceptosPrevio (idempotente por IdRegistroProceso)
            cp_exist = conn.execute(sql_get_cp_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if cp_exist:
                new_id_cp = int(cp_exist[0])
                cp_accion = "EXISTE"
            else:
                payload_cp = payload_cp_from_dfrow(r, new_id_rp)
                has_cp = (payload_cp["FechaInforme"] is not None) or (payload_cp["Concepto"] is not None)
                if has_cp:
                    new_id_cp = conn.execute(sql_insert_cp, payload_cp).fetchone()[0]
                    cp_accion = "INSERT"
                else:
                    new_id_cp = None
                    cp_accion = "SKIP_VACIO"

            # G) MedidaProcesal (idempotente simple: si ya hay algo para ese ET, no repetir)
            mp_already = conn.execute(sql_mp_exists, {"IdEstudioTerreno": new_id_et}).fetchone()
            if mp_already:
                mp_ins = 0
                mp_accion = "EXISTE"
            else:
                mp_rows = build_mp_rows(r, new_id_et)
                if mp_rows:
                    conn.execute(sql_insert_mp, mp_rows)  # executemany
                    mp_ins = len(mp_rows)
                    mp_accion = "INSERT"
                else:
                    mp_ins = 0
                    mp_accion = "SKIP_VACIO"

        oks.append({
            "Fila_Excel_aprox": fila,
            "IdPostulacion": idpost,
            "IdRegistroProceso": int(new_id_rp),
            "IdEstudioTerreno": int(new_id_et),
            "IdConceptoFinal": None if new_id_cf is None else int(new_id_cf),
            "IdConceptoPrevio": None if new_id_cp is None else int(new_id_cp),
            "MedidaProcesal_insertadas": int(mp_ins),
            "RP_accion": rp_accion,
            "ET_accion": et_accion,
            "CF_accion": cf_accion,
            "CP_accion": cp_accion,
            "MP_accion": mp_accion,
            "Estado": "OK",
        })

        if fila % 50 == 0:
            print(f"[OK] Hasta fila {fila} (última: RP={new_id_rp}, RP={rp_accion}, ET={et_accion}, CF={cf_accion}, CP={cp_accion}, MP={mp_accion})")

    except Exception as e:
        errs.append({
            "Fila_Excel_aprox": fila,
            "Estado": "ERROR",
            "Error": str(e)[:4000],
        })
        print(f"[ERROR] Fila {fila} -> {e}")

# =====================================================
# 7) Exportar logs a CSV (sin frenar toda la carga)
# =====================================================
df_ok = pd.DataFrame(oks)
df_err = pd.DataFrame(errs)

df_ok.to_csv(OUT_OK_CSV, index=False, encoding="utf-8-sig")
df_err.to_csv(OUT_ERR_CSV, index=False, encoding="utf-8-sig")

print("\n=== RESUMEN ===")
print("OK:", len(df_ok), "| ERRORES:", len(df_err))
print("Logs:", OUT_OK_CSV, "y", OUT_ERR_CSV)

# Vista rápida
if not df_err.empty:
    print("\n=== Primeros 10 ERRORES ===")
    print(df_err.head(10).to_string(index=False))


[OK] Hasta fila 2050 (última: RP=2050, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 2100 (última: RP=2100, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 2150 (última: RP=2150, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 2200 (última: RP=2200, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 2250 (última: RP=2250, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 2300 (última: RP=2300, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=SKIP_VACIO)
[OK] Hasta fila 2350 (última: RP=2350, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 2400 (última: RP=2400, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 2450 (última: RP=2450, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 2500 (última: RP=2500, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 2550 (última: RP=2550, RP=INSERT, ET=INS

## validacion

In [85]:
from sqlalchemy import text
import pandas as pd

# =====================================================
# VALIDACIONES POST-MIGRACIÓN (SQL)
# - Duplicados en RegistroProceso por llave natural (IdPostulacion)
# - Integridad FK: EstudioTerreno / ConceptoFinal / ConceptosPrevio
# - Resumen de conteos
# Requiere: engine ya creado
# =====================================================

def run_df(sql: str, params: dict | None = None) -> pd.DataFrame:
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn, params=params or {})

def show(title: str, df_: pd.DataFrame, max_rows: int = 20):
    print("\n" + "="*90)
    print(title)
    print("="*90)
    if df_.empty:
        print("(OK) Sin hallazgos.")
    else:
        print(df_.head(max_rows).to_string(index=False))
        if len(df_) > max_rows:
            print(f"... ({len(df_)} filas, mostrando {max_rows})")

# -----------------------------
# 1) Duplicados en RegistroProceso por llave natural (IdPostulacion)
# -----------------------------
sql_dup_rp = """
SELECT
    IdPostulacion,
    COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.RegistroProceso
GROUP BY IdPostulacion
HAVING COUNT(*) > 1
ORDER BY N DESC, IdPostulacion;
"""
df_dup_rp = run_df(sql_dup_rp)
show("1) Duplicados en Postulacion.RegistroProceso por (IdPostulacion)", df_dup_rp)

# Detalle duplicados (si existen)
if not df_dup_rp.empty:
    sql_dup_rp_det = """
    SELECT rp.*
    FROM ViabilidadJuridica.Postulacion.RegistroProceso rp
    JOIN (
        SELECT IdPostulacion
        FROM ViabilidadJuridica.Postulacion.RegistroProceso
        GROUP BY IdPostulacion
        HAVING COUNT(*) > 1
    ) d ON d.IdPostulacion = rp.IdPostulacion
    ORDER BY rp.IdPostulacion, rp.IdRegistroProceso;
    """
    df_dup_rp_det = run_df(sql_dup_rp_det)
    show("1.1) Detalle duplicados RegistroProceso", df_dup_rp_det, max_rows=50)

# -----------------------------
# 2) Integridad FK: EstudioTerreno -> RegistroProceso
# -----------------------------
sql_fk_et = """
SELECT TOP 100
    et.IdEstudioTerreno,
    et.IdRegistroProceso
FROM ViabilidadJuridica.Postulacion.EstudioTerreno et
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = et.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY et.IdEstudioTerreno DESC;
"""
df_fk_et = run_df(sql_fk_et)
show("2) Huérfanos: EstudioTerreno.IdRegistroProceso sin RegistroProceso", df_fk_et)

# -----------------------------
# 3) Integridad FK: ConceptoFinal -> RegistroProceso
# -----------------------------
sql_fk_cf = """
SELECT TOP 100
    cf.IdConceptoFinal,
    cf.IdRegistroProceso
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal cf
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = cf.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY cf.IdConceptoFinal DESC;
"""
df_fk_cf = run_df(sql_fk_cf)
show("3) Huérfanos: ConceptoFinal.IdRegistroProceso sin RegistroProceso", df_fk_cf)

# -----------------------------
# 4) Integridad FK: ConceptosPrevio -> RegistroProceso
# -----------------------------
sql_fk_cp = """
SELECT TOP 100
    cp.IdConceptoPrevio,
    cp.IdRegistroProceso
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio cp
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = cp.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY cp.IdConceptoPrevio DESC;
"""
df_fk_cp = run_df(sql_fk_cp)
show("4) Huérfanos: ConceptosPrevio.IdRegistroProceso sin RegistroProceso", df_fk_cp)

# -----------------------------
# 5) Integridad FK: MedidaProcesal -> EstudioTerreno
# -----------------------------
sql_fk_mp = """
SELECT TOP 100
    mp.IdMedidasProcesal,
    mp.IdEstudioTerreno
FROM ViabilidadJuridica.Postulacion.MedidaProcesal mp
LEFT JOIN ViabilidadJuridica.Postulacion.EstudioTerreno et
    ON et.IdEstudioTerreno = mp.IdEstudioTerreno
WHERE et.IdEstudioTerreno IS NULL
ORDER BY mp.IdMedidasProcesal DESC;
"""
df_fk_mp = run_df(sql_fk_mp)
show("5) Huérfanos: MedidaProcesal.IdEstudioTerreno sin EstudioTerreno", df_fk_mp)

# -----------------------------
# 6) Conteos globales
# -----------------------------
sql_counts = """
SELECT 'RegistroProceso'   AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.Postulacion.RegistroProceso
UNION ALL
SELECT 'EstudioTerreno'    AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.Postulacion.EstudioTerreno
UNION ALL
SELECT 'ConceptoFinal'     AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
UNION ALL
SELECT 'ConceptosPrevio'   AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
UNION ALL
SELECT 'MedidaProcesal'    AS Tabla, COUNT(*) AS N FROM ViabilidadJuridica.Postulacion.MedidaProcesal;
"""
df_counts = run_df(sql_counts)
show("6) Conteos actuales por tabla", df_counts, max_rows=100)

# -----------------------------
# 7) Conteos SOLO del bloque migrado (2..2000)
# -----------------------------
IDRP_MIN = 2001
IDRP_MAX = 4000

sql_counts_block = """
SELECT 'RegistroProceso' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.RegistroProceso
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'EstudioTerreno' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.EstudioTerreno
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'ConceptoFinal' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'ConceptosPrevio' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
WHERE IdRegistroProceso BETWEEN :a AND :b
UNION ALL
SELECT 'MedidaProcesal' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.MedidaProcesal mp
JOIN ViabilidadJuridica.Postulacion.EstudioTerreno et
  ON et.IdEstudioTerreno = mp.IdEstudioTerreno
WHERE et.IdRegistroProceso BETWEEN :a AND :b;
"""
df_counts_block = run_df(sql_counts_block, params={"a": IDRP_MIN, "b": IDRP_MAX})
show(f"7) Conteos SOLO para IdRegistroProceso {IDRP_MIN}..{IDRP_MAX}", df_counts_block, max_rows=100)

print("\nListo. Si todo sale en (OK) sin hallazgos en 2..5 y sin duplicados en 1, puedes seguir con el siguiente lote.")



1) Duplicados en Postulacion.RegistroProceso por (IdPostulacion)
(OK) Sin hallazgos.

2) Huérfanos: EstudioTerreno.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

3) Huérfanos: ConceptoFinal.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

4) Huérfanos: ConceptosPrevio.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

5) Huérfanos: MedidaProcesal.IdEstudioTerreno sin EstudioTerreno
(OK) Sin hallazgos.

6) Conteos actuales por tabla
          Tabla     N
RegistroProceso  3999
 EstudioTerreno  3999
  ConceptoFinal  3999
ConceptosPrevio  3235
 MedidaProcesal 12579

7) Conteos SOLO para IdRegistroProceso 2001..4000
          Tabla    N
RegistroProceso 2000
 EstudioTerreno 2000
  ConceptoFinal 2000
ConceptosPrevio 1749
 MedidaProcesal 5932

Listo. Si todo sale en (OK) sin hallazgos en 2..5 y sin duplicados en 1, puedes seguir con el siguiente lote.


## Tercera migracion

In [86]:
import urllib.parse
from sqlalchemy import create_engine, text
import pandas as pd

# =====================================================
# 0) CONEXIÓN (usa tus mismas variables ya definidas)
# =====================================================
# server = r"172.23.92.97\MSSQLSERVER2022"
# database = "ViabilidadJuridica"
# user = "andres.garciav"
# password = "..."

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={user};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# =====================================================
# 1) CONFIG: migrar por lotes
# =====================================================
START_FILA = 4001
END_FILA = 10001  # 4001..10000 inclusive (range excluye el END)
FILAS_EXCEL = list(range(START_FILA, END_FILA))

# Archivos de log
OUT_OK_CSV = "log_migracion_OK.csv"
OUT_ERR_CSV = "log_migracion_ERRORES.csv"

# =====================================================
# 2) Helpers DF
# =====================================================
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

def get_row_df(fila_excel: int):
    r = df.loc[df["Fila_Excel_aprox"] == fila_excel]
    if r.empty:
        raise ValueError(f"No existe Fila_Excel_aprox={fila_excel} en df.")
    return r.iloc[0]

def get_raw(r, col):
    if col not in df.columns:
        return None
    v = r[col]
    if pd.isna(v):
        return None
    return v

def get_str(r, col):
    v = get_raw(r, col)
    if v is None:
        return None
    s = str(v).strip()
    if s == "" or s.upper() in ("NAN", "NAT", "NONE"):
        return None
    return s

def to_int_value(x):
    if x is None:
        return None
    try:
        return int(float(x))
    except:
        return None

def to_int(r, col):
    return to_int_value(get_str(r, col))

def to_dt(r, col):
    v = get_raw(r, col)
    if v is None:
        return None
    ts = pd.to_datetime(v, errors="coerce")
    if pd.isna(ts):
        return None
    return ts.to_pydatetime()

# =====================================================
# 3) Helpers DB: columnas reales (evita errores por tildes)
# =====================================================
def get_table_cols(schema: str, table: str):
    sql = text("""
    SELECT c.name
    FROM sys.columns c
    JOIN sys.tables t ON c.object_id = t.object_id
    JOIN sys.schemas s ON t.schema_id = s.schema_id
    WHERE s.name = :schema AND t.name = :table
    ORDER BY c.column_id;
    """)
    with engine.connect() as conn:
        rows = conn.execute(sql, {"schema": schema, "table": table}).fetchall()
    return [x[0] for x in rows]

COLS_CONCEPTOFINAL = set(get_table_cols("AnalisisJuridico", "ConceptoFinal"))

# =====================================================
# 4) SQL statements
# =====================================================

# (FIX) Para que STG sea determinística por fila: borramos la fila antes de insertar.
sql_delete_stg_by_fila = text("""
DELETE FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox = :Fila_Excel_aprox;
""")

sql_insert_stg = text("""
INSERT INTO ViabilidadJuridica.dbo.STG_RegistroProceso
(
  Fila_Excel_aprox,
  IdPostulacion,
  FMI,
  NumeroExpediente,
  IdFuenteProceso,
  IdTipoProceso,
  IdEtapaProcesal,
  Dependencia,
  RadicadoOrfeo
)
VALUES
(
  :Fila_Excel_aprox,
  :IdPostulacion,
  :FMI,
  :NumeroExpediente,
  :IdFuenteProceso,
  :IdTipoProceso,
  :IdEtapaProcesal,
  :Dependencia,
  :RadicadoOrfeo
);
""")

sql_get_stg = text("""
SELECT TOP 1
    Fila_Excel_aprox,
    IdPostulacion,
    FMI,
    NumeroExpediente,
    IdFuenteProceso,
    IdTipoProceso,
    IdEtapaProcesal,
    Dependencia,
    RadicadoOrfeo
FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox = :fila;
""")

# Idempotencia SOLO por IdPostulacion:
sql_get_rp_by_idpost = text("""
SELECT TOP 1 IdRegistroProceso
FROM ViabilidadJuridica.Postulacion.RegistroProceso
WHERE IdPostulacion = :IdPostulacion;
""")

sql_insert_rp = text("""
INSERT INTO ViabilidadJuridica.Postulacion.RegistroProceso
(
    IdPostulacion, FMI, NumeroExpediente,
    IdFuenteProceso, IdTipoProceso, IdEtapaProcesal,
    Dependencia, RadicadoOrfeo
)
OUTPUT inserted.IdRegistroProceso
VALUES
(
    :IdPostulacion, :FMI, :NumeroExpediente,
    :IdFuenteProceso, :IdTipoProceso, :IdEtapaProcesal,
    :Dependencia, :RadicadoOrfeo
);
""")

# EstudioTerreno 1:1 por IdRegistroProceso (idempotente)
sql_get_et_by_rp = text("""
SELECT TOP 1 IdEstudioTerreno
FROM ViabilidadJuridica.Postulacion.EstudioTerreno
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

sql_insert_et = text("""
INSERT INTO ViabilidadJuridica.Postulacion.EstudioTerreno
(
    IdRegistroProceso,
    IdLocalizacion,
    AreaRegistral,
    CirculoRegistral,
    AreaCalculada,
    TipoPersonaTitular,
    NombrePropietario,
    ApellidoPropietario,
    IdTipoDocumento,
    Identificacion,
    NaturalezaJuridica,
    AcreditacionPropiedad
)
OUTPUT inserted.IdEstudioTerreno
VALUES
(
    :IdRegistroProceso,
    :IdLocalizacion,
    :AreaRegistral,
    :CirculoRegistral,
    :AreaCalculada,
    :TipoPersonaTitular,
    :NombrePropietario,
    :ApellidoPropietario,
    :IdTipoDocumento,
    :Identificacion,
    :NaturalezaJuridica,
    :AcreditacionPropiedad
);
""")

# ConceptoFinal 1:1 por IdRegistroProceso (idempotente)
sql_get_cf_by_rp = text("""
SELECT TOP 1 IdConceptoFinal
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

# ConceptosPrevio: 1:1 por IdRegistroProceso (idempotente)
sql_get_cp_by_rp = text("""
SELECT TOP 1 IdConceptoPrevio
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

sql_insert_cp = text("""
INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
(
    IdRegistroProceso,
    FechaInforme,
    Concepto
)
OUTPUT inserted.IdConceptoPrevio
VALUES
(
    :IdRegistroProceso,
    :FechaInforme,
    :Concepto
);
""")

# MedidaProcesal: si ya hay alguna para ese EstudioTerreno, no insertamos (idempotente simple)
sql_mp_exists = text("""
SELECT TOP 1 1
FROM ViabilidadJuridica.Postulacion.MedidaProcesal
WHERE IdEstudioTerreno = :IdEstudioTerreno;
""")

sql_insert_mp = text("""
INSERT INTO ViabilidadJuridica.Postulacion.MedidaProcesal
(
    IdEstudioTerreno,
    Objeto,
    Valor,
    Anotacion,
    TipoClasificacion
)
VALUES
(
    :IdEstudioTerreno,
    :Objeto,
    :Valor,
    :Anotacion,
    :TipoClasificacion
);
""")

# =====================================================
# 5) Armado de payloads
# =====================================================
def payload_stg_from_dfrow(r):
    fila = int(r["Fila_Excel_aprox"])
    idpost = get_str(r, "IdPostulacion_SQL")
    fmi = get_str(r, "FMI_SQL") or get_str(r, "FMI")
    if not idpost:
        raise ValueError("IdPostulacion_SQL vacío.")
    if not fmi:
        raise ValueError("FMI_SQL/FMI vacío.")
    return {
        "Fila_Excel_aprox": fila,
        "IdPostulacion": idpost,
        "FMI": fmi,
        "NumeroExpediente": get_str(r, "NumeroExpediente_SQL"),
        "IdFuenteProceso": to_int_value(get_str(r, "IdFuenteProceso_SQL") or get_str(r, "IdFuenteProceso")),
        "IdTipoProceso": to_int_value(get_str(r, "IdTipoProceso_SQL") or get_str(r, "IdTipoProceso")),
        "IdEtapaProcesal": to_int_value(get_str(r, "IdEtapaProcesal_SQL") or get_str(r, "IdEtapaProcesal")),
        "Dependencia": get_str(r, "Dependencia_SQL"),
        "RadicadoOrfeo": get_str(r, "RadicadoOrfeo_SQL"),
    }

def payload_et_from_dfrow(r, id_registro):
    return {
        "IdRegistroProceso": int(id_registro),
        "IdLocalizacion": to_int_value(get_str(r, "IdLocalizacion_SQL")),
        "AreaRegistral": get_raw(r, "AreaRegistral_SQL"),
        "CirculoRegistral": get_str(r, "CirculoRegistral_SQL"),
        "AreaCalculada": get_raw(r, "AreaCalculada_SQL"),
        "TipoPersonaTitular": get_str(r, "TipoPersonaTitular_SQL"),
        "NombrePropietario": get_str(r, "NombrePropietario_SQL"),
        "ApellidoPropietario": get_str(r, "ApellidoPropietario_SQL"),
        "IdTipoDocumento": to_int_value(get_str(r, "IdTipoDocumento_SQL_MIGRAR")),
        "Identificacion": get_str(r, "Identificacion_SQL_TXT_MIGRAR"),
        "NaturalezaJuridica": get_str(r, "NaturalezaJuridica_SQL_FINAL"),
        "AcreditacionPropiedad": get_str(r, "AcreditacionPropiedad_SQL_FINAL"),
    }

def payload_cp_from_dfrow(r, id_registro):
    return {
        "IdRegistroProceso": int(id_registro),
        "FechaInforme": to_dt(r, "FechaInformePrevio_SQL"),
        "Concepto": get_str(r, "ConceptoPrevio_SQL"),
    }

def build_cf_payload_dfrow(r, id_registro):
    candidates = {
        "ConceptoActualDeViabilidadJuridica": get_str(r, "ConceptoActualDeViabilidadJuridica_SQL"),
        "FechaInforme": to_dt(r, "FechaInforme_SQL"),
        "IdTipoViabilidad": to_int(r, "IdTipoViabilidad_SQL"),
        "IdTipoInforme": to_int(r, "IdTipoInforme_SQL"),
        "IdTipoNoViabilidad": to_int(r, "IdTipoNoViabilidad_SQL"),
        "InsumosPendientes": get_str(r, "InsumosPendientes_SQL"),
        "FechaEntregaARevisor": to_dt(r, "FechaEntregaARevisor_SQL"),
        "AbogadoSustanciadorAsignado": get_str(r, "AbogadoSustanciadorAsignado_SQL"),
        "AbogadoRevisorAsignado": get_str(r, "AbogadoRevisorAsignado_SQL"),
        "NroReparto": to_int(r, "NroReparto_SQL"),
        "FechaAsignacionReparto": to_dt(r, "FechaAsignacionReparto_SQL"),
        "FechaPlazoEntregaARevisor": to_dt(r, "FechaPlazoEntregaARevisor_SQL"),
        "IdTipoEstadoRevision": to_int(r, "IdTipoEstadoRevision_SQL"),
        "ObservacionesRevisor": get_str(r, "ObservacionesRevisor_SQL"),
        "EntregoCarpetaSoportes": get_str(r, "EntregoCarpetaSoportes_SQL"),
        "FechaEnvioACoordinacion": to_dt(r, "FechaEnvioACoordinacion_SQL"),
        "EstadoAprobacionCoordinadora": get_str(r, "EstadoAprobacionCoordinadora_SQL"),
        "FechaRemisionSoportesAGestoraDocumental": to_dt(r, "FechaRemisionSoportesAGestoraDocumental_SQL"),
        "FechaRemisionInformeAGestoraDocumental": to_dt(r, "FechaRemisionInformeAGestoraDocumental_SQL"),
        "FechaCargueInformeJuridicoExpOrfeo": to_dt(r, "FechaCargueInformeJuridicoExpOrfeo_SQL"),
        "FechaDeCargueDocsYSoportesExpOrfeo": to_dt(r, "FechaDeCargueDocsYSoportesExpOrfeo_SQL"),
        "FechaGestionEtapaSIT": to_dt(r, "FechaGestionEtapaSIT_SQL"),
    }

    # Ajustes por tildes si existen en BD
    if "FechaRemisiónInformeAGestoraDocumental" in COLS_CONCEPTOFINAL:
        candidates["FechaRemisiónInformeAGestoraDocumental"] = candidates.pop("FechaRemisionInformeAGestoraDocumental", None)
    if "FechaCargueInformeJurídicoExpOrfeo" in COLS_CONCEPTOFINAL and "FechaCargueInformeJuridicoExpOrfeo" in candidates:
        candidates["FechaCargueInformeJurídicoExpOrfeo"] = candidates.pop("FechaCargueInformeJuridicoExpOrfeo", None)

    payload = {"IdRegistroProceso": int(id_registro)}
    for k, v in candidates.items():
        if k in COLS_CONCEPTOFINAL:
            payload[k] = v
    return payload

def insert_conceptofinal_dynamic(conn, payload):
    has_data = any(v is not None for k, v in payload.items() if k != "IdRegistroProceso")
    if not has_data:
        return None

    cols = list(payload.keys())
    params_cols = [f":{c}" for c in cols]
    sql = f"""
    INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
    ({", ".join(cols)})
    OUTPUT inserted.IdConceptoFinal
    VALUES
    ({", ".join(params_cols)});
    """
    res = conn.execute(text(sql), payload)
    return res.fetchone()[0]

def build_mp_rows(r, id_estudio):
    def cell(col):
        return get_str(r, col)

    def add(objeto, col_valor, col_anot, col_tipo=None):
        valor = cell(col_valor)
        anot = cell(col_anot)
        if valor is None and anot is None:
            return None
        return {
            "IdEstudioTerreno": int(id_estudio),
            "Objeto": objeto,
            "Valor": (valor or "SI"),
            "Anotacion": anot,
            "TipoClasificacion": cell(col_tipo) if col_tipo else None,
        }

    specs = [
        ("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL", None),
        ("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL", None),
        ("MEDIDAS CAUTELARES", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL", None),
        ("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL"),
        ("RTDAF LEY 1448", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL", None),
        ("OFERTA OTRAS ENTIDADES", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL", None),
        ("PROCESOS DE CLARIFICACIÓN", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL", None),
    ]

    rows = []
    for spec in specs:
        row = add(*spec)
        if row:
            rows.append(row)
    return rows

# =====================================================
# 6) MIGRACIÓN (transacción por fila + logs)
# =====================================================
oks = []
errs = []

for fila in FILAS_EXCEL:
    try:
        r = get_row_df(fila)
        stg_payload = payload_stg_from_dfrow(r)

        with engine.begin() as conn:
            # A) STG determinística: borrar-poner por fila
            conn.execute(sql_delete_stg_by_fila, {"Fila_Excel_aprox": stg_payload["Fila_Excel_aprox"]})
            conn.execute(sql_insert_stg, stg_payload)

            # B) Leer STG recién insertado
            stg = pd.read_sql(sql_get_stg, conn, params={"fila": fila})
            if stg.empty:
                raise ValueError("No se encontró STG recién insertado.")
            stg_row = stg.iloc[0].to_dict()

            idpost = str(stg_row["IdPostulacion"]).strip()

            # C) RegistroProceso (idempotente por IdPostulacion)
            rp_exist = conn.execute(sql_get_rp_by_idpost, {"IdPostulacion": idpost}).fetchone()
            if rp_exist:
                new_id_rp = int(rp_exist[0])
                rp_accion = "EXISTE"
            else:
                payload_reg = {
                    "IdPostulacion": idpost,
                    "FMI": str(stg_row["FMI"]).strip() if stg_row.get("FMI") is not None else None,
                    "NumeroExpediente": stg_row.get("NumeroExpediente"),
                    "IdFuenteProceso": stg_row.get("IdFuenteProceso"),
                    "IdTipoProceso": stg_row.get("IdTipoProceso"),
                    "IdEtapaProcesal": stg_row.get("IdEtapaProcesal"),
                    "Dependencia": stg_row.get("Dependencia"),
                    "RadicadoOrfeo": stg_row.get("RadicadoOrfeo"),
                }
                new_id_rp = conn.execute(sql_insert_rp, payload_reg).fetchone()[0]
                rp_accion = "INSERT"

            # D) EstudioTerreno (idempotente por IdRegistroProceso)
            et_exist = conn.execute(sql_get_et_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if et_exist:
                new_id_et = int(et_exist[0])
                et_accion = "EXISTE"
            else:
                payload_et = payload_et_from_dfrow(r, new_id_rp)
                new_id_et = conn.execute(sql_insert_et, payload_et).fetchone()[0]
                et_accion = "INSERT"

            # E) ConceptoFinal (idempotente por IdRegistroProceso)
            cf_exist = conn.execute(sql_get_cf_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if cf_exist:
                new_id_cf = int(cf_exist[0])
                cf_accion = "EXISTE"
            else:
                payload_cf = build_cf_payload_dfrow(r, new_id_rp)
                new_id_cf = insert_conceptofinal_dynamic(conn, payload_cf)
                cf_accion = "INSERT" if new_id_cf is not None else "SKIP_VACIO"

            # F) ConceptosPrevio (idempotente por IdRegistroProceso)
            cp_exist = conn.execute(sql_get_cp_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if cp_exist:
                new_id_cp = int(cp_exist[0])
                cp_accion = "EXISTE"
            else:
                payload_cp = payload_cp_from_dfrow(r, new_id_rp)
                has_cp = (payload_cp["FechaInforme"] is not None) or (payload_cp["Concepto"] is not None)
                if has_cp:
                    new_id_cp = conn.execute(sql_insert_cp, payload_cp).fetchone()[0]
                    cp_accion = "INSERT"
                else:
                    new_id_cp = None
                    cp_accion = "SKIP_VACIO"

            # G) MedidaProcesal (idempotente simple)
            mp_already = conn.execute(sql_mp_exists, {"IdEstudioTerreno": new_id_et}).fetchone()
            if mp_already:
                mp_ins = 0
                mp_accion = "EXISTE"
            else:
                mp_rows = build_mp_rows(r, new_id_et)
                if mp_rows:
                    conn.execute(sql_insert_mp, mp_rows)  # executemany
                    mp_ins = len(mp_rows)
                    mp_accion = "INSERT"
                else:
                    mp_ins = 0
                    mp_accion = "SKIP_VACIO"

        oks.append({
            "Fila_Excel_aprox": fila,
            "IdPostulacion": idpost,
            "IdRegistroProceso": int(new_id_rp),
            "IdEstudioTerreno": int(new_id_et),
            "IdConceptoFinal": None if new_id_cf is None else int(new_id_cf),
            "IdConceptoPrevio": None if new_id_cp is None else int(new_id_cp),
            "MedidaProcesal_insertadas": int(mp_ins),
            "RP_accion": rp_accion,
            "ET_accion": et_accion,
            "CF_accion": cf_accion,
            "CP_accion": cp_accion,
            "MP_accion": mp_accion,
            "Estado": "OK",
        })

        if fila % 50 == 0:
            print(f"[OK] Hasta fila {fila} (última: RP={new_id_rp}, RP={rp_accion}, ET={et_accion}, CF={cf_accion}, CP={cp_accion}, MP={mp_accion})")

    except Exception as e:
        errs.append({
            "Fila_Excel_aprox": fila,
            "Estado": "ERROR",
            "Error": str(e)[:4000],
        })
        print(f"[ERROR] Fila {fila} -> {e}")

# =====================================================
# 7) Exportar logs a CSV
# =====================================================
df_ok = pd.DataFrame(oks)
df_err = pd.DataFrame(errs)

df_ok.to_csv(OUT_OK_CSV, index=False, encoding="utf-8-sig")
df_err.to_csv(OUT_ERR_CSV, index=False, encoding="utf-8-sig")

print("\n=== RESUMEN ===")
print("OK:", len(df_ok), "| ERRORES:", len(df_err))
print("Logs:", OUT_OK_CSV, "y", OUT_ERR_CSV)

if not df_err.empty:
    print("\n=== Primeros 10 ERRORES ===")
    print(df_err.head(10).to_string(index=False))


[OK] Hasta fila 4050 (última: RP=4050, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 4100 (última: RP=4100, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=SKIP_VACIO)
[OK] Hasta fila 4150 (última: RP=4150, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 4200 (última: RP=4200, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 4250 (última: RP=4250, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 4300 (última: RP=4300, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=SKIP_VACIO)
[OK] Hasta fila 4350 (última: RP=4350, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=SKIP_VACIO)
[OK] Hasta fila 4400 (última: RP=4400, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=SKIP_VACIO)
[OK] Hasta fila 4450 (última: RP=4450, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=SKIP_VACIO)
[OK] Hasta fila 4500 (última: RP=4500, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=SKIP_VACIO)
[OK] Hasta fila 4550 (última: RP

In [87]:
from sqlalchemy import text
import pandas as pd

# =====================================================
# VALIDACIONES POST-MIGRACIÓN (LOTE POR FILA_EXCEL + IdPostulacion)
# Objetivo:
#  - Sin duplicados por llave natural (IdPostulacion)
#  - Integridad FK (huérfanos)
#  - Cobertura del lote: filas esperadas vs filas realmente migradas
#  - 1:1 donde aplica (RP<->ET, RP<->CF, RP<->CP opcional)
#  - MedidaProcesal coherente (cada MP pertenece a un ET del lote)
#
# Requiere: engine ya creado
# Parámetros del lote:
#   START_FILA = 4001
#   END_FILA   = 10001   # range excluye END_FILA -> 4001..10000
# =====================================================

START_FILA = 4001
END_FILA   = 10001  # 4001..10000 inclusive

# --- helpers
def run_df(sql: str, params: dict | None = None) -> pd.DataFrame:
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn, params=params or {})

def show(title: str, df_: pd.DataFrame, max_rows: int = 20):
    print("\n" + "="*110)
    print(title)
    print("="*110)
    if df_.empty:
        print("(OK) Sin hallazgos.")
    else:
        print(df_.head(max_rows).to_string(index=False))
        if len(df_) > max_rows:
            print(f"... ({len(df_)} filas, mostrando {max_rows})")

params_lote = {"a": int(START_FILA), "b": int(END_FILA - 1)}

# =====================================================
# 0) Resumen rápido del lote en STG (fuente de trazabilidad por fila)
# =====================================================
sql_stg_summary = """
SELECT
    COUNT(*) AS STG_rows,
    COUNT(DISTINCT Fila_Excel_aprox) AS STG_filas_distintas,
    COUNT(DISTINCT IdPostulacion) AS STG_idpost_distintos,
    MIN(Fila_Excel_aprox) AS STG_min_fila,
    MAX(Fila_Excel_aprox) AS STG_max_fila
FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox BETWEEN :a AND :b;
"""
df_stg_summary = run_df(sql_stg_summary, params_lote)
show("0) STG resumen del lote (fila_excel e IdPostulacion)", df_stg_summary, max_rows=10)

# (CRÍTICO) Duplicados en STG por Fila_Excel_aprox
sql_stg_dup_fila = """
SELECT TOP 200
    Fila_Excel_aprox,
    COUNT(*) AS N
FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox BETWEEN :a AND :b
GROUP BY Fila_Excel_aprox
HAVING COUNT(*) > 1
ORDER BY N DESC, Fila_Excel_aprox;
"""
df_stg_dup_fila = run_df(sql_stg_dup_fila, params_lote)
show("0.1) Duplicados en STG por Fila_Excel_aprox (debe ser 0)", df_stg_dup_fila)

# (CRÍTICO) Duplicados en STG por IdPostulacion dentro del lote
sql_stg_dup_idpost = """
SELECT TOP 200
    IdPostulacion,
    COUNT(*) AS N
FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox BETWEEN :a AND :b
GROUP BY IdPostulacion
HAVING COUNT(*) > 1
ORDER BY N DESC, IdPostulacion;
"""
df_stg_dup_idpost = run_df(sql_stg_dup_idpost, params_lote)
show("0.2) Duplicados en STG por IdPostulacion dentro del lote (revisar si >0)", df_stg_dup_idpost)

# =====================================================
# 1) Duplicados en RegistroProceso por llave natural (IdPostulacion)
# =====================================================
sql_rp_dup = """
SELECT TOP 200
    IdPostulacion,
    COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.RegistroProceso
GROUP BY IdPostulacion
HAVING COUNT(*) > 1
ORDER BY N DESC, IdPostulacion;
"""
df_rp_dup = run_df(sql_rp_dup)
show("1) Duplicados en RegistroProceso por IdPostulacion (debe ser 0)", df_rp_dup)

# =====================================================
# 2) Integridad FK (huérfanos) global
# =====================================================
sql_fk_et = """
SELECT TOP 100
    et.IdEstudioTerreno,
    et.IdRegistroProceso
FROM ViabilidadJuridica.Postulacion.EstudioTerreno et
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = et.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY et.IdEstudioTerreno DESC;
"""
show("2) Huérfanos: EstudioTerreno.IdRegistroProceso sin RegistroProceso", run_df(sql_fk_et))

sql_fk_cf = """
SELECT TOP 100
    cf.IdConceptoFinal,
    cf.IdRegistroProceso
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal cf
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = cf.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY cf.IdConceptoFinal DESC;
"""
show("3) Huérfanos: ConceptoFinal.IdRegistroProceso sin RegistroProceso", run_df(sql_fk_cf))

sql_fk_cp = """
SELECT TOP 100
    cp.IdConceptoPrevio,
    cp.IdRegistroProceso
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio cp
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdRegistroProceso = cp.IdRegistroProceso
WHERE rp.IdRegistroProceso IS NULL
ORDER BY cp.IdConceptoPrevio DESC;
"""
show("4) Huérfanos: ConceptosPrevio.IdRegistroProceso sin RegistroProceso", run_df(sql_fk_cp))

sql_fk_mp = """
SELECT TOP 100
    mp.IdMedidasProcesal,
    mp.IdEstudioTerreno
FROM ViabilidadJuridica.Postulacion.MedidaProcesal mp
LEFT JOIN ViabilidadJuridica.Postulacion.EstudioTerreno et
    ON et.IdEstudioTerreno = mp.IdEstudioTerreno
WHERE et.IdEstudioTerreno IS NULL
ORDER BY mp.IdMedidasProcesal DESC;
"""
show("5) Huérfanos: MedidaProcesal.IdEstudioTerreno sin EstudioTerreno", run_df(sql_fk_mp))

# =====================================================
# 3) Cobertura del LOTE: STG -> RegistroProceso
#    Verifica que cada IdPostulacion del lote exista en RP
# =====================================================
sql_missing_rp = """
SELECT TOP 200
    s.Fila_Excel_aprox,
    s.IdPostulacion
FROM ViabilidadJuridica.dbo.STG_RegistroProceso s
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdPostulacion = s.IdPostulacion
WHERE s.Fila_Excel_aprox BETWEEN :a AND :b
  AND rp.IdRegistroProceso IS NULL
ORDER BY s.Fila_Excel_aprox;
"""
df_missing_rp = run_df(sql_missing_rp, params_lote)
show("6) LOTE: Filas STG cuyo IdPostulacion NO está en RegistroProceso (debe ser 0)", df_missing_rp)

# Resumen conteo cobertura (debe cuadrar)
sql_cov = """
SELECT
    COUNT(*) AS STG_rows,
    COUNT(DISTINCT s.IdPostulacion) AS STG_idpost_distintos,
    COUNT(DISTINCT rp.IdRegistroProceso) AS RP_encontrados
FROM ViabilidadJuridica.dbo.STG_RegistroProceso s
LEFT JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
    ON rp.IdPostulacion = s.IdPostulacion
WHERE s.Fila_Excel_aprox BETWEEN :a AND :b;
"""
df_cov = run_df(sql_cov, params_lote)
show("6.1) LOTE: Cobertura STG(IdPostulacion) -> RegistroProceso", df_cov, max_rows=10)

# =====================================================
# 4) 1:1 EstudioTerreno por RP del lote (no faltantes, no duplicados)
# =====================================================
sql_et_card = """
WITH lote AS (
    SELECT DISTINCT rp.IdRegistroProceso
    FROM ViabilidadJuridica.dbo.STG_RegistroProceso s
    JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
      ON rp.IdPostulacion = s.IdPostulacion
    WHERE s.Fila_Excel_aprox BETWEEN :a AND :b
)
SELECT TOP 200
    l.IdRegistroProceso,
    COUNT(et.IdEstudioTerreno) AS N_ET
FROM lote l
LEFT JOIN ViabilidadJuridica.Postulacion.EstudioTerreno et
  ON et.IdRegistroProceso = l.IdRegistroProceso
GROUP BY l.IdRegistroProceso
HAVING COUNT(et.IdEstudioTerreno) <> 1
ORDER BY N_ET, l.IdRegistroProceso;
"""
df_et_card = run_df(sql_et_card, params_lote)
show("7) LOTE: Cardinalidad RP->ET (debe ser EXACTAMENTE 1 por RP). Hallazgos = problemas", df_et_card)

# =====================================================
# 5) 1:1 ConceptoFinal por RP del lote (no faltantes, no duplicados)
#    Nota: si ConceptoFinal es obligatorio en tu lógica, esto debe ser 1.
# =====================================================
sql_cf_card = """
WITH lote AS (
    SELECT DISTINCT rp.IdRegistroProceso
    FROM ViabilidadJuridica.dbo.STG_RegistroProceso s
    JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
      ON rp.IdPostulacion = s.IdPostulacion
    WHERE s.Fila_Excel_aprox BETWEEN :a AND :b
)
SELECT TOP 200
    l.IdRegistroProceso,
    COUNT(cf.IdConceptoFinal) AS N_CF
FROM lote l
LEFT JOIN ViabilidadJuridica.AnalisisJuridico.ConceptoFinal cf
  ON cf.IdRegistroProceso = l.IdRegistroProceso
GROUP BY l.IdRegistroProceso
HAVING COUNT(cf.IdConceptoFinal) <> 1
ORDER BY N_CF, l.IdRegistroProceso;
"""
df_cf_card = run_df(sql_cf_card, params_lote)
show("8) LOTE: Cardinalidad RP->ConceptoFinal (esperado 1). Hallazgos = problemas", df_cf_card)

# =====================================================
# 6) ConceptosPrevio por RP del lote (opcional)
#    En tu lógica puede ser 0 o 1. Detecta duplicados (>1) y listo.
# =====================================================
sql_cp_dup_in_lote = """
WITH lote AS (
    SELECT DISTINCT rp.IdRegistroProceso
    FROM ViabilidadJuridica.dbo.STG_RegistroProceso s
    JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
      ON rp.IdPostulacion = s.IdPostulacion
    WHERE s.Fila_Excel_aprox BETWEEN :a AND :b
)
SELECT TOP 200
    l.IdRegistroProceso,
    COUNT(cp.IdConceptoPrevio) AS N_CP
FROM lote l
LEFT JOIN ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio cp
  ON cp.IdRegistroProceso = l.IdRegistroProceso
GROUP BY l.IdRegistroProceso
HAVING COUNT(cp.IdConceptoPrevio) > 1
ORDER BY N_CP DESC, l.IdRegistroProceso;
"""
df_cp_dup_lote = run_df(sql_cp_dup_in_lote, params_lote)
show("9) LOTE: Duplicados RP->ConceptosPrevio (si sale algo, es problema)", df_cp_dup_lote)

# =====================================================
# 7) MedidaProcesal coherencia en el lote:
#    A) Ninguna MP del lote debe apuntar a ET que NO sea del lote (por join es implícito)
#    B) Duplicados por (IdEstudioTerreno, Objeto) (si esperas máximo 1 por objeto)
# =====================================================
sql_mp_dup_obj = """
WITH lote_et AS (
    SELECT et.IdEstudioTerreno
    FROM ViabilidadJuridica.dbo.STG_RegistroProceso s
    JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
      ON rp.IdPostulacion = s.IdPostulacion
    JOIN ViabilidadJuridica.Postulacion.EstudioTerreno et
      ON et.IdRegistroProceso = rp.IdRegistroProceso
    WHERE s.Fila_Excel_aprox BETWEEN :a AND :b
)
SELECT TOP 200
    mp.IdEstudioTerreno,
    mp.Objeto,
    COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.MedidaProcesal mp
JOIN lote_et le ON le.IdEstudioTerreno = mp.IdEstudioTerreno
GROUP BY mp.IdEstudioTerreno, mp.Objeto
HAVING COUNT(*) > 1
ORDER BY N DESC, mp.IdEstudioTerreno, mp.Objeto;
"""
df_mp_dup_obj = run_df(sql_mp_dup_obj, params_lote)
show("10) LOTE: Duplicados de MedidaProcesal por (IdEstudioTerreno, Objeto) (si sale algo, revisar)", df_mp_dup_obj)

# =====================================================
# 8) Conteos del lote por tablas (basado en STG->RP)
# =====================================================
sql_counts_lote = """
WITH lote_rp AS (
    SELECT DISTINCT rp.IdRegistroProceso
    FROM ViabilidadJuridica.dbo.STG_RegistroProceso s
    JOIN ViabilidadJuridica.Postulacion.RegistroProceso rp
      ON rp.IdPostulacion = s.IdPostulacion
    WHERE s.Fila_Excel_aprox BETWEEN :a AND :b
),
lote_et AS (
    SELECT et.IdEstudioTerreno, et.IdRegistroProceso
    FROM ViabilidadJuridica.Postulacion.EstudioTerreno et
    JOIN lote_rp lr ON lr.IdRegistroProceso = et.IdRegistroProceso
)
SELECT 'RegistroProceso (lote)' AS Tabla, COUNT(*) AS N FROM lote_rp
UNION ALL
SELECT 'EstudioTerreno (lote)'  AS Tabla, COUNT(*) AS N FROM lote_et
UNION ALL
SELECT 'ConceptoFinal (lote)'   AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal cf
JOIN lote_rp lr ON lr.IdRegistroProceso = cf.IdRegistroProceso
UNION ALL
SELECT 'ConceptosPrevio (lote)' AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio cp
JOIN lote_rp lr ON lr.IdRegistroProceso = cp.IdRegistroProceso
UNION ALL
SELECT 'MedidaProcesal (lote)'  AS Tabla, COUNT(*) AS N
FROM ViabilidadJuridica.Postulacion.MedidaProcesal mp
JOIN lote_et le ON le.IdEstudioTerreno = mp.IdEstudioTerreno;
"""
df_counts_lote = run_df(sql_counts_lote, params_lote)
show("11) Conteos del lote (derivados de STG->RP)", df_counts_lote, max_rows=50)

print("\nFIN. Para estar 'perfecto':")
print("- 0.1 y 1 deben salir vacíos (sin duplicados).")
print("- 2..5 deben salir vacíos (sin huérfanos).")
print("- 6 debe salir vacío y 6.1 debe mostrar RP_encontrados == STG_idpost_distintos.")
print("- 7 y 8 deben salir vacíos (cardinalidad 1:1).")
print("- 9 y 10 deben salir vacíos (sin duplicados en CP y MP por objeto).")



0) STG resumen del lote (fila_excel e IdPostulacion)
 STG_rows  STG_filas_distintas  STG_idpost_distintos  STG_min_fila  STG_max_fila
     6000                 6000                  6000          4001         10000

0.1) Duplicados en STG por Fila_Excel_aprox (debe ser 0)
(OK) Sin hallazgos.

0.2) Duplicados en STG por IdPostulacion dentro del lote (revisar si >0)
(OK) Sin hallazgos.

1) Duplicados en RegistroProceso por IdPostulacion (debe ser 0)
(OK) Sin hallazgos.

2) Huérfanos: EstudioTerreno.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

3) Huérfanos: ConceptoFinal.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

4) Huérfanos: ConceptosPrevio.IdRegistroProceso sin RegistroProceso
(OK) Sin hallazgos.

5) Huérfanos: MedidaProcesal.IdEstudioTerreno sin EstudioTerreno
(OK) Sin hallazgos.

6) LOTE: Filas STG cuyo IdPostulacion NO está en RegistroProceso (debe ser 0)
(OK) Sin hallazgos.

6.1) LOTE: Cobertura STG(IdPostulacion) -> RegistroProceso
 STG_rows  STG_idp

## Ultima migracion

In [88]:
import pandas as pd
from sqlalchemy import text

# Asegura Fila_Excel_aprox
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

def norm_str(x):
    if x is None or pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.upper() in ("NAN","NAT","NONE"):
        return None
    return s

# Columnas fuente
COL_IDPOST = "IdPostulacion_SQL"
COL_FMI_1  = "FMI_SQL"
COL_FMI_2  = "FMI"

# Normaliza IdPostulacion y FMI en columnas auxiliares (no daña tu df)
tmp = df[["Fila_Excel_aprox"]].copy()
tmp["IdPost"] = df[COL_IDPOST].map(norm_str) if COL_IDPOST in df.columns else None
tmp["FMI"] = None
if COL_FMI_1 in df.columns:
    tmp["FMI"] = df[COL_FMI_1].map(norm_str)
if tmp["FMI"].isna().all() and (COL_FMI_2 in df.columns):
    tmp["FMI"] = df[COL_FMI_2].map(norm_str)
else:
    # si hay mezcla FMI_SQL y FMI, completa vacíos con FMI
    if COL_FMI_2 in df.columns:
        tmp["FMI"] = tmp["FMI"].fillna(df[COL_FMI_2].map(norm_str))

# Regla de “fila válida para migración”
# (mínimo: IdPostulacion y FMI presentes; si alguno falta => no migrar)
mask_valid = tmp["IdPost"].notna() & tmp["FMI"].notna()
tmp_valid = tmp[mask_valid].copy()

total_valid = len(tmp_valid)
min_fila = int(tmp_valid["Fila_Excel_aprox"].min()) if total_valid else None
max_fila = int(tmp_valid["Fila_Excel_aprox"].max()) if total_valid else None

print("=== EXCEL (df) ===")
print("Filas válidas (IdPostulacion+FMI):", total_valid)
print("Rango filas válidas:", min_fila, "..", max_fila)

# ¿Cuántas filas vacías hay después de la última válida? (para confirmar que no migres “de más”)
if max_fila is not None:
    after = tmp[tmp["Fila_Excel_aprox"] > max_fila]
    after_valid = after[after["IdPost"].notna() | after["FMI"].notna()]
    print("Filas con algún dato después de la última válida:", len(after_valid))

# Trae IdPostulacion ya migradas a SQL (llave natural)
sql_rp_idpost = text("""
SELECT rp.IdPostulacion
FROM ViabilidadJuridica.Postulacion.RegistroProceso rp;
""")
with engine.connect() as conn:
    rp = pd.read_sql(sql_rp_idpost, conn)

rp["IdPost"] = rp["IdPostulacion"].map(norm_str)
rp_set = set(rp["IdPost"].dropna().unique())

tmp_valid["Ya_en_SQL"] = tmp_valid["IdPost"].isin(rp_set)
pend = tmp_valid[~tmp_valid["Ya_en_SQL"]].copy()

print("\n=== SQL vs EXCEL ===")
print("IdPostulacion ya en SQL:", len(rp_set))
print("Filas válidas pendientes por migrar (según IdPostulacion):", len(pend))

# Opcional: muestra primeras pendientes
print("\nPrimeras 10 pendientes (Fila_Excel_aprox, IdPost):")
print(pend[["Fila_Excel_aprox","IdPost"]].head(10).to_string(index=False))


=== EXCEL (df) ===
Filas válidas (IdPostulacion+FMI): 24055
Rango filas válidas: 2 .. 24056
Filas con algún dato después de la última válida: 0

=== SQL vs EXCEL ===
IdPostulacion ya en SQL: 9999
Filas válidas pendientes por migrar (según IdPostulacion): 14056

Primeras 10 pendientes (Fila_Excel_aprox, IdPost):
 Fila_Excel_aprox  IdPost
            10001 ID16800
            10002 ID16806
            10003 ID16807
            10004 ID16803
            10005 ID16805
            10006 ID16811
            10007 ID16809
            10008 ID16810
            10009 ID16812
            10010 ID16813


In [89]:
import urllib.parse
from sqlalchemy import create_engine, text
import pandas as pd

# =====================================================
# 0) CONEXIÓN (usa tus mismas variables ya definidas)
# =====================================================
# server = r"172.23.92.97\MSSQLSERVER2022"
# database = "ViabilidadJuridica"
# user = "andres.garciav"
# password = "..."

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={user};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# =====================================================
# 1) CONFIG: migrar SOLO lo faltante por rango real
# =====================================================
START_FILA = 10001
END_FILA = 24057  # incluye 24056
FILAS_EXCEL = list(range(START_FILA, END_FILA))

# Logs por bloque para no sobreescribir
OUT_OK_CSV = f"log_migracion_OK_{START_FILA}_{END_FILA-1}.csv"
OUT_ERR_CSV = f"log_migracion_ERR_{START_FILA}_{END_FILA-1}.csv"


# =====================================================
# 2) Helpers DF
# =====================================================
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

def get_row_df(fila_excel: int):
    r = df.loc[df["Fila_Excel_aprox"] == fila_excel]
    if r.empty:
        raise ValueError(f"No existe Fila_Excel_aprox={fila_excel} en df.")
    return r.iloc[0]

def get_raw(r, col):
    if col not in df.columns:
        return None
    v = r[col]
    if pd.isna(v):
        return None
    return v

def get_str(r, col):
    v = get_raw(r, col)
    if v is None:
        return None
    s = str(v).strip()
    if s == "" or s.upper() in ("NAN", "NAT", "NONE"):
        return None
    return s

def to_int_value(x):
    if x is None:
        return None
    try:
        return int(float(x))
    except:
        return None

def to_int(r, col):
    return to_int_value(get_str(r, col))

def to_dt(r, col):
    v = get_raw(r, col)
    if v is None:
        return None
    ts = pd.to_datetime(v, errors="coerce")
    if pd.isna(ts):
        return None
    return ts.to_pydatetime()

# =====================================================
# 3) Helpers DB: columnas reales (evita errores por tildes)
# =====================================================
def get_table_cols(schema: str, table: str):
    sql = text("""
    SELECT c.name
    FROM sys.columns c
    JOIN sys.tables t ON c.object_id = t.object_id
    JOIN sys.schemas s ON t.schema_id = s.schema_id
    WHERE s.name = :schema AND t.name = :table
    ORDER BY c.column_id;
    """)
    with engine.connect() as conn:
        rows = conn.execute(sql, {"schema": schema, "table": table}).fetchall()
    return [x[0] for x in rows]

COLS_CONCEPTOFINAL = set(get_table_cols("AnalisisJuridico", "ConceptoFinal"))

# =====================================================
# 4) SQL statements
# =====================================================

# (FIX) Para que STG sea determinística por fila: borramos la fila antes de insertar.
sql_delete_stg_by_fila = text("""
DELETE FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox = :Fila_Excel_aprox;
""")

sql_insert_stg = text("""
INSERT INTO ViabilidadJuridica.dbo.STG_RegistroProceso
(
  Fila_Excel_aprox,
  IdPostulacion,
  FMI,
  NumeroExpediente,
  IdFuenteProceso,
  IdTipoProceso,
  IdEtapaProcesal,
  Dependencia,
  RadicadoOrfeo
)
VALUES
(
  :Fila_Excel_aprox,
  :IdPostulacion,
  :FMI,
  :NumeroExpediente,
  :IdFuenteProceso,
  :IdTipoProceso,
  :IdEtapaProcesal,
  :Dependencia,
  :RadicadoOrfeo
);
""")

sql_get_stg = text("""
SELECT TOP 1
    Fila_Excel_aprox,
    IdPostulacion,
    FMI,
    NumeroExpediente,
    IdFuenteProceso,
    IdTipoProceso,
    IdEtapaProcesal,
    Dependencia,
    RadicadoOrfeo
FROM ViabilidadJuridica.dbo.STG_RegistroProceso
WHERE Fila_Excel_aprox = :fila;
""")

# Idempotencia SOLO por IdPostulacion:
sql_get_rp_by_idpost = text("""
SELECT TOP 1 IdRegistroProceso
FROM ViabilidadJuridica.Postulacion.RegistroProceso
WHERE IdPostulacion = :IdPostulacion;
""")

sql_insert_rp = text("""
INSERT INTO ViabilidadJuridica.Postulacion.RegistroProceso
(
    IdPostulacion, FMI, NumeroExpediente,
    IdFuenteProceso, IdTipoProceso, IdEtapaProcesal,
    Dependencia, RadicadoOrfeo
)
OUTPUT inserted.IdRegistroProceso
VALUES
(
    :IdPostulacion, :FMI, :NumeroExpediente,
    :IdFuenteProceso, :IdTipoProceso, :IdEtapaProcesal,
    :Dependencia, :RadicadoOrfeo
);
""")

# EstudioTerreno 1:1 por IdRegistroProceso (idempotente)
sql_get_et_by_rp = text("""
SELECT TOP 1 IdEstudioTerreno
FROM ViabilidadJuridica.Postulacion.EstudioTerreno
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

sql_insert_et = text("""
INSERT INTO ViabilidadJuridica.Postulacion.EstudioTerreno
(
    IdRegistroProceso,
    IdLocalizacion,
    AreaRegistral,
    CirculoRegistral,
    AreaCalculada,
    TipoPersonaTitular,
    NombrePropietario,
    ApellidoPropietario,
    IdTipoDocumento,
    Identificacion,
    NaturalezaJuridica,
    AcreditacionPropiedad
)
OUTPUT inserted.IdEstudioTerreno
VALUES
(
    :IdRegistroProceso,
    :IdLocalizacion,
    :AreaRegistral,
    :CirculoRegistral,
    :AreaCalculada,
    :TipoPersonaTitular,
    :NombrePropietario,
    :ApellidoPropietario,
    :IdTipoDocumento,
    :Identificacion,
    :NaturalezaJuridica,
    :AcreditacionPropiedad
);
""")

# ConceptoFinal 1:1 por IdRegistroProceso (idempotente)
sql_get_cf_by_rp = text("""
SELECT TOP 1 IdConceptoFinal
FROM ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

# ConceptosPrevio: 1:1 por IdRegistroProceso (idempotente)
sql_get_cp_by_rp = text("""
SELECT TOP 1 IdConceptoPrevio
FROM ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
WHERE IdRegistroProceso = :IdRegistroProceso;
""")

sql_insert_cp = text("""
INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptosPrevio
(
    IdRegistroProceso,
    FechaInforme,
    Concepto
)
OUTPUT inserted.IdConceptoPrevio
VALUES
(
    :IdRegistroProceso,
    :FechaInforme,
    :Concepto
);
""")

# MedidaProcesal: si ya hay alguna para ese EstudioTerreno, no insertamos (idempotente simple)
sql_mp_exists = text("""
SELECT TOP 1 1
FROM ViabilidadJuridica.Postulacion.MedidaProcesal
WHERE IdEstudioTerreno = :IdEstudioTerreno;
""")

sql_insert_mp = text("""
INSERT INTO ViabilidadJuridica.Postulacion.MedidaProcesal
(
    IdEstudioTerreno,
    Objeto,
    Valor,
    Anotacion,
    TipoClasificacion
)
VALUES
(
    :IdEstudioTerreno,
    :Objeto,
    :Valor,
    :Anotacion,
    :TipoClasificacion
);
""")

# =====================================================
# 5) Armado de payloads
# =====================================================
def payload_stg_from_dfrow(r):
    fila = int(r["Fila_Excel_aprox"])
    idpost = get_str(r, "IdPostulacion_SQL")
    fmi = get_str(r, "FMI_SQL") or get_str(r, "FMI")
    if not idpost:
        raise ValueError("IdPostulacion_SQL vacío.")
    if not fmi:
        raise ValueError("FMI_SQL/FMI vacío.")
    return {
        "Fila_Excel_aprox": fila,
        "IdPostulacion": idpost,
        "FMI": fmi,
        "NumeroExpediente": get_str(r, "NumeroExpediente_SQL"),
        "IdFuenteProceso": to_int_value(get_str(r, "IdFuenteProceso_SQL") or get_str(r, "IdFuenteProceso")),
        "IdTipoProceso": to_int_value(get_str(r, "IdTipoProceso_SQL") or get_str(r, "IdTipoProceso")),
        "IdEtapaProcesal": to_int_value(get_str(r, "IdEtapaProcesal_SQL") or get_str(r, "IdEtapaProcesal")),
        "Dependencia": get_str(r, "Dependencia_SQL"),
        "RadicadoOrfeo": get_str(r, "RadicadoOrfeo_SQL"),
    }

def payload_et_from_dfrow(r, id_registro):
    return {
        "IdRegistroProceso": int(id_registro),
        "IdLocalizacion": to_int_value(get_str(r, "IdLocalizacion_SQL")),
        "AreaRegistral": get_raw(r, "AreaRegistral_SQL"),
        "CirculoRegistral": get_str(r, "CirculoRegistral_SQL"),
        "AreaCalculada": get_raw(r, "AreaCalculada_SQL"),
        "TipoPersonaTitular": get_str(r, "TipoPersonaTitular_SQL"),
        "NombrePropietario": get_str(r, "NombrePropietario_SQL"),
        "ApellidoPropietario": get_str(r, "ApellidoPropietario_SQL"),
        "IdTipoDocumento": to_int_value(get_str(r, "IdTipoDocumento_SQL_MIGRAR")),
        "Identificacion": get_str(r, "Identificacion_SQL_TXT_MIGRAR"),
        "NaturalezaJuridica": get_str(r, "NaturalezaJuridica_SQL_FINAL"),
        "AcreditacionPropiedad": get_str(r, "AcreditacionPropiedad_SQL_FINAL"),
    }

def payload_cp_from_dfrow(r, id_registro):
    return {
        "IdRegistroProceso": int(id_registro),
        "FechaInforme": to_dt(r, "FechaInformePrevio_SQL"),
        "Concepto": get_str(r, "ConceptoPrevio_SQL"),
    }

def build_cf_payload_dfrow(r, id_registro):
    candidates = {
        "ConceptoActualDeViabilidadJuridica": get_str(r, "ConceptoActualDeViabilidadJuridica_SQL"),
        "FechaInforme": to_dt(r, "FechaInforme_SQL"),
        "IdTipoViabilidad": to_int(r, "IdTipoViabilidad_SQL"),
        "IdTipoInforme": to_int(r, "IdTipoInforme_SQL"),
        "IdTipoNoViabilidad": to_int(r, "IdTipoNoViabilidad_SQL"),
        "InsumosPendientes": get_str(r, "InsumosPendientes_SQL"),
        "FechaEntregaARevisor": to_dt(r, "FechaEntregaARevisor_SQL"),
        "AbogadoSustanciadorAsignado": get_str(r, "AbogadoSustanciadorAsignado_SQL"),
        "AbogadoRevisorAsignado": get_str(r, "AbogadoRevisorAsignado_SQL"),
        "NroReparto": to_int(r, "NroReparto_SQL"),
        "FechaAsignacionReparto": to_dt(r, "FechaAsignacionReparto_SQL"),
        "FechaPlazoEntregaARevisor": to_dt(r, "FechaPlazoEntregaARevisor_SQL"),
        "IdTipoEstadoRevision": to_int(r, "IdTipoEstadoRevision_SQL"),
        "ObservacionesRevisor": get_str(r, "ObservacionesRevisor_SQL"),
        "EntregoCarpetaSoportes": get_str(r, "EntregoCarpetaSoportes_SQL"),
        "FechaEnvioACoordinacion": to_dt(r, "FechaEnvioACoordinacion_SQL"),
        "EstadoAprobacionCoordinadora": get_str(r, "EstadoAprobacionCoordinadora_SQL"),
        "FechaRemisionSoportesAGestoraDocumental": to_dt(r, "FechaRemisionSoportesAGestoraDocumental_SQL"),
        "FechaRemisionInformeAGestoraDocumental": to_dt(r, "FechaRemisionInformeAGestoraDocumental_SQL"),
        "FechaCargueInformeJuridicoExpOrfeo": to_dt(r, "FechaCargueInformeJuridicoExpOrfeo_SQL"),
        "FechaDeCargueDocsYSoportesExpOrfeo": to_dt(r, "FechaDeCargueDocsYSoportesExpOrfeo_SQL"),
        "FechaGestionEtapaSIT": to_dt(r, "FechaGestionEtapaSIT_SQL"),
    }

    # Ajustes por tildes si existen en BD
    if "FechaRemisiónInformeAGestoraDocumental" in COLS_CONCEPTOFINAL:
        candidates["FechaRemisiónInformeAGestoraDocumental"] = candidates.pop("FechaRemisionInformeAGestoraDocumental", None)
    if "FechaCargueInformeJurídicoExpOrfeo" in COLS_CONCEPTOFINAL and "FechaCargueInformeJuridicoExpOrfeo" in candidates:
        candidates["FechaCargueInformeJurídicoExpOrfeo"] = candidates.pop("FechaCargueInformeJuridicoExpOrfeo", None)

    payload = {"IdRegistroProceso": int(id_registro)}
    for k, v in candidates.items():
        if k in COLS_CONCEPTOFINAL:
            payload[k] = v
    return payload

def insert_conceptofinal_dynamic(conn, payload):
    has_data = any(v is not None for k, v in payload.items() if k != "IdRegistroProceso")
    if not has_data:
        return None

    cols = list(payload.keys())
    params_cols = [f":{c}" for c in cols]
    sql = f"""
    INSERT INTO ViabilidadJuridica.AnalisisJuridico.ConceptoFinal
    ({", ".join(cols)})
    OUTPUT inserted.IdConceptoFinal
    VALUES
    ({", ".join(params_cols)});
    """
    res = conn.execute(text(sql), payload)
    return res.fetchone()[0]

def build_mp_rows(r, id_estudio):
    def cell(col):
        return get_str(r, col)

    def add(objeto, col_valor, col_anot, col_tipo=None):
        valor = cell(col_valor)
        anot = cell(col_anot)
        if valor is None and anot is None:
            return None
        return {
            "IdEstudioTerreno": int(id_estudio),
            "Objeto": objeto,
            "Valor": (valor or "SI"),
            "Anotacion": anot,
            "TipoClasificacion": cell(col_tipo) if col_tipo else None,
        }

    specs = [
        ("HIPOTECA", "HIPOTECA_VALOR_SQL", "HIPOTECA_ANOT_SQL", None),
        ("SERVIDUMBRES", "SERVIDUMBRES_VALOR_SQL_FINAL", "SERVIDUMBRES_ANOT_SQL", None),
        ("MEDIDAS CAUTELARES", "MEDIDAS_CAUT_VALOR_SQL_FINAL", "MEDIDAS_CAUT_ANOT_SQL", None),
        ("RUPTA", "RUPTA_VALOR_SQL_FINAL", "RUPTA_ANOT_SQL", "RUPTA_TIPOCLASIF_SQL"),
        ("RTDAF LEY 1448", "RTDAF_VALOR_SQL_FINAL", "RTDAF_ANOT_SQL", None),
        ("OFERTA OTRAS ENTIDADES", "OFERTA_VALOR_SQL_FINAL", "OFERTA_ANOT_SQL", None),
        ("PROCESOS DE CLARIFICACIÓN", "PROC_CLARIF_VALOR_SQL_FINAL", "PROC_CLARIF_ANOT_SQL", None),
    ]

    rows = []
    for spec in specs:
        row = add(*spec)
        if row:
            rows.append(row)
    return rows

# =====================================================
# 6) MIGRACIÓN (transacción por fila + logs)
# =====================================================
oks = []
errs = []

for fila in FILAS_EXCEL:
    try:
        r = get_row_df(fila)
        stg_payload = payload_stg_from_dfrow(r)

        with engine.begin() as conn:
            # A) STG determinística: borrar-poner por fila
            conn.execute(sql_delete_stg_by_fila, {"Fila_Excel_aprox": stg_payload["Fila_Excel_aprox"]})
            conn.execute(sql_insert_stg, stg_payload)

            # B) Leer STG recién insertado
            stg = pd.read_sql(sql_get_stg, conn, params={"fila": fila})
            if stg.empty:
                raise ValueError("No se encontró STG recién insertado.")
            stg_row = stg.iloc[0].to_dict()

            idpost = str(stg_row["IdPostulacion"]).strip()

            # C) RegistroProceso (idempotente por IdPostulacion)
            rp_exist = conn.execute(sql_get_rp_by_idpost, {"IdPostulacion": idpost}).fetchone()
            if rp_exist:
                new_id_rp = int(rp_exist[0])
                rp_accion = "EXISTE"
            else:
                payload_reg = {
                    "IdPostulacion": idpost,
                    "FMI": str(stg_row["FMI"]).strip() if stg_row.get("FMI") is not None else None,
                    "NumeroExpediente": stg_row.get("NumeroExpediente"),
                    "IdFuenteProceso": stg_row.get("IdFuenteProceso"),
                    "IdTipoProceso": stg_row.get("IdTipoProceso"),
                    "IdEtapaProcesal": stg_row.get("IdEtapaProcesal"),
                    "Dependencia": stg_row.get("Dependencia"),
                    "RadicadoOrfeo": stg_row.get("RadicadoOrfeo"),
                }
                new_id_rp = conn.execute(sql_insert_rp, payload_reg).fetchone()[0]
                rp_accion = "INSERT"

            # D) EstudioTerreno (idempotente por IdRegistroProceso)
            et_exist = conn.execute(sql_get_et_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if et_exist:
                new_id_et = int(et_exist[0])
                et_accion = "EXISTE"
            else:
                payload_et = payload_et_from_dfrow(r, new_id_rp)
                new_id_et = conn.execute(sql_insert_et, payload_et).fetchone()[0]
                et_accion = "INSERT"

            # E) ConceptoFinal (idempotente por IdRegistroProceso)
            cf_exist = conn.execute(sql_get_cf_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if cf_exist:
                new_id_cf = int(cf_exist[0])
                cf_accion = "EXISTE"
            else:
                payload_cf = build_cf_payload_dfrow(r, new_id_rp)
                new_id_cf = insert_conceptofinal_dynamic(conn, payload_cf)
                cf_accion = "INSERT" if new_id_cf is not None else "SKIP_VACIO"

            # F) ConceptosPrevio (idempotente por IdRegistroProceso)
            cp_exist = conn.execute(sql_get_cp_by_rp, {"IdRegistroProceso": new_id_rp}).fetchone()
            if cp_exist:
                new_id_cp = int(cp_exist[0])
                cp_accion = "EXISTE"
            else:
                payload_cp = payload_cp_from_dfrow(r, new_id_rp)
                has_cp = (payload_cp["FechaInforme"] is not None) or (payload_cp["Concepto"] is not None)
                if has_cp:
                    new_id_cp = conn.execute(sql_insert_cp, payload_cp).fetchone()[0]
                    cp_accion = "INSERT"
                else:
                    new_id_cp = None
                    cp_accion = "SKIP_VACIO"

            # G) MedidaProcesal (idempotente simple)
            mp_already = conn.execute(sql_mp_exists, {"IdEstudioTerreno": new_id_et}).fetchone()
            if mp_already:
                mp_ins = 0
                mp_accion = "EXISTE"
            else:
                mp_rows = build_mp_rows(r, new_id_et)
                if mp_rows:
                    conn.execute(sql_insert_mp, mp_rows)  # executemany
                    mp_ins = len(mp_rows)
                    mp_accion = "INSERT"
                else:
                    mp_ins = 0
                    mp_accion = "SKIP_VACIO"

        oks.append({
            "Fila_Excel_aprox": fila,
            "IdPostulacion": idpost,
            "IdRegistroProceso": int(new_id_rp),
            "IdEstudioTerreno": int(new_id_et),
            "IdConceptoFinal": None if new_id_cf is None else int(new_id_cf),
            "IdConceptoPrevio": None if new_id_cp is None else int(new_id_cp),
            "MedidaProcesal_insertadas": int(mp_ins),
            "RP_accion": rp_accion,
            "ET_accion": et_accion,
            "CF_accion": cf_accion,
            "CP_accion": cp_accion,
            "MP_accion": mp_accion,
            "Estado": "OK",
        })

        if fila % 50 == 0:
            print(f"[OK] Hasta fila {fila} (última: RP={new_id_rp}, RP={rp_accion}, ET={et_accion}, CF={cf_accion}, CP={cp_accion}, MP={mp_accion})")

    except Exception as e:
        errs.append({
            "Fila_Excel_aprox": fila,
            "Estado": "ERROR",
            "Error": str(e)[:4000],
        })
        print(f"[ERROR] Fila {fila} -> {e}")

# =====================================================
# 7) Exportar logs a CSV
# =====================================================
df_ok = pd.DataFrame(oks)
df_err = pd.DataFrame(errs)

df_ok.to_csv(OUT_OK_CSV, index=False, encoding="utf-8-sig")
df_err.to_csv(OUT_ERR_CSV, index=False, encoding="utf-8-sig")

print("\n=== RESUMEN ===")
print("OK:", len(df_ok), "| ERRORES:", len(df_err))
print("Logs:", OUT_OK_CSV, "y", OUT_ERR_CSV)

if not df_err.empty:
    print("\n=== Primeros 10 ERRORES ===")
    print(df_err.head(10).to_string(index=False))


[OK] Hasta fila 10050 (última: RP=10050, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=SKIP_VACIO)
[OK] Hasta fila 10100 (última: RP=10100, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=INSERT)
[OK] Hasta fila 10150 (última: RP=10150, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=SKIP_VACIO)
[OK] Hasta fila 10200 (última: RP=10200, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=SKIP_VACIO)
[OK] Hasta fila 10250 (última: RP=10250, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=INSERT)
[OK] Hasta fila 10300 (última: RP=10300, RP=INSERT, ET=INSERT, CF=INSERT, CP=INSERT, MP=INSERT)
[OK] Hasta fila 10350 (última: RP=10350, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=SKIP_VACIO)
[OK] Hasta fila 10400 (última: RP=10400, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=INSERT)
[OK] Hasta fila 10450 (última: RP=10450, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO, MP=INSERT)
[OK] Hasta fila 10500 (última: RP=10500, RP=INSERT, ET=INSERT, CF=INSERT, CP=SKIP_VACIO,